# Pipeline Analysis Notebook

This notebook contains the analysis of the pipeline codebase.

## WorkflowCoordinator.py

In [ ]:
"""
Educational Content Workflow Coordinator
---------------------------------------
Orchestrates the end-to-end workflow from PDF upload to video delivery and adaptive learning.
Connects all system components in a seamless pipeline.
"""

import os
import time
import logging
from typing import Dict, List, Any, Optional, Union
import json
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import psutil

# Import configuration and dependency injection
from pipeline.config.ComponentConfig import WorkflowConfig
from pipeline.di.DependencyContainer import DependencyContainer

# Import event system
from pipeline.events.EventSystem import get_event_bus, EventType, EventMonitor

# Import error classes
from pipeline.errors.WorkflowErrors import (
    WorkflowError, PDFExtractionError, ContentAnalysisError, 
    MultimediaGenerationError, AdaptiveLearningError,
    ResourceLimitError, TimeoutError
)

# Import component interfaces (for type hints)
from pipeline.pdf_extraction.PDFContentExtractor import PDFContentExtractor
from pipeline.content_analysis.ContentAnalysisPipeline import ContentAnalysisPipeline
from pipeline.multimedia.MultimediaPipeline import MultimediaGenerationPipeline
from pipeline.adaptive_learning.AdaptiveLearningManager import AdaptiveLearningManager
from pipeline.adaptive_learning.StudentProfileManager import StudentProfileManager

# For distributed processing (optional imports)
try:
    import ray
    RAY_AVAILABLE = True
except ImportError:
    RAY_AVAILABLE = False

class WorkflowCoordinator:
    """
    Coordinates the complete educational content workflow.
    Manages the process from PDF upload through content extraction, analysis, 
    video generation, to adaptive learning feedback.
    """
    
    def __init__(self, config_dict: Optional[Dict[str, Any]] = None, output_dir: str = "output"):
        """
        Initialize the workflow coordinator
        
        Args:
            config_dict: Dictionary of configuration options (will be converted to typed config)
            output_dir: Base directory for all output
        """
        # Create typed configuration from dictionary
        self.config = WorkflowConfig.from_dict(config_dict or {})
        
        # Override output directory if specified
        if output_dir:
            self.config.output_dir = output_dir
        
        # Ensure output directories exist
        self.extracted_dir = os.path.join(self.config.output_dir, "extracted")
        self.analysis_dir = os.path.join(self.config.output_dir, "analysis")
        self.multimedia_dir = os.path.join(self.config.output_dir, "multimedia")
        self.feedback_dir = os.path.join(self.config.output_dir, "feedback")
        self.metrics_dir = os.path.join(self.config.output_dir, "metrics")
        
        for directory in [self.extracted_dir, self.analysis_dir, self.multimedia_dir, 
                          self.feedback_dir, self.metrics_dir]:
            os.makedirs(directory, exist_ok=True)
        
        # Configure logging
        logging.basicConfig(level=logging.INFO, 
                           format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger("WorkflowCoordinator")
        
        # Create dependency container with typed configuration
        self.container = DependencyContainer(self.config)
        
        # Initialize event monitoring
        self.event_monitor = EventMonitor()
        
        # Configure thread/process pools for parallel processing
        self._configure_parallel_processing()
        
        # Workflow state (thread-safe for concurrent operations)
        self.workflow_states = {}
        self.workflow_lock = threading.RLock()
        
        self.logger.info("WorkflowCoordinator initialized successfully")
    
    def _configure_parallel_processing(self):
        """Configure parallel processing capabilities based on system resources"""
        # Determine available resources
        cpu_count = multiprocessing.cpu_count()
        
        # Set thread and process pool sizes based on available CPUs
        # Use fewer processes for heavier tasks to avoid resource contention
        self.analysis_pool_size = max(1, min(cpu_count - 1, 4))
        self.extraction_pool_size = max(1, min(cpu_count - 1, 4))
        self.video_pool_size = max(1, min(cpu_count // 2, 2))
        
        # Create thread pool for lightweight tasks
        self.thread_pool = ThreadPoolExecutor(max_workers=cpu_count * 2)
        
        # Create process pools for CPU-intensive tasks
        self.extraction_pool = ProcessPoolExecutor(max_workers=self.extraction_pool_size)
        self.analysis_pool = ProcessPoolExecutor(max_workers=self.analysis_pool_size)
        self.video_pool = ProcessPoolExecutor(max_workers=self.video_pool_size)
        
        self.logger.info(f"Configured parallel processing: {cpu_count} CPUs, " +
                       f"{self.extraction_pool_size} extraction workers, " +
                       f"{self.analysis_pool_size} analysis workers, " +
                       f"{self.video_pool_size} video workers")
        
        # Determine if we should use distributed processing
        self.use_distributed = self.config.use_distributed
        if self.use_distributed:
            self._configure_distributed_processing()
    
    def _configure_distributed_processing(self):
        """Configure distributed processing if enabled"""
        if not RAY_AVAILABLE:
            self.logger.warning("Ray is not available. Distributed processing will be disabled.")
            self.use_distributed = False
            return

        try:
            # Initialize Ray with system resources
            if not ray.is_initialized():
                # Get system resources
                cpu_count = multiprocessing.cpu_count()
                memory_gb = psutil.virtual_memory().total / (1024 * 1024 * 1024)
                
                # Configure Ray with available resources
                ray.init(
                    num_cpus=max(1, cpu_count - 1),  # Leave one CPU for system
                    memory=memory_gb * 0.8,  # Use 80% of available memory
                    ignore_reinit_error=True,
                    logging_level=logging.INFO
                )
                
                self.logger.info(f"Ray initialized with {cpu_count-1} CPUs and {memory_gb*0.8:.1f}GB memory")
            
            # Configure distributed processing settings
            self.distributed_config = {
                "num_workers": max(1, multiprocessing.cpu_count() - 1),
                "memory_limit": psutil.virtual_memory().total * 0.8,  # 80% of system memory
                "object_store_memory": psutil.virtual_memory().total * 0.4,  # 40% for object store
                "redis_max_memory": psutil.virtual_memory().total * 0.1,  # 10% for Redis
                "temp_dir": os.path.join(self.config.output_dir, "ray_temp")
            }
            
            # Create temp directory for Ray
            os.makedirs(self.distributed_config["temp_dir"], exist_ok=True)
            
            # Configure Ray runtime environment
            ray.runtime_env.working_dir = os.getcwd()
            ray.runtime_env.pip = ["psutil", "numpy", "pandas"]  # Add required packages
            
            self.logger.info("Distributed processing configured successfully")
            
        except Exception as e:
            self.logger.error(f"Failed to configure distributed processing: {str(e)}")
            self.use_distributed = False
            raise WorkflowError(
                "Failed to configure distributed processing",
                details={"error": str(e)}
            )
    
    def process_pdf(self, pdf_path: str, student_id: Optional[str] = None, 
                   custom_options: Optional[Dict] = None) -> Dict[str, Any]:
        """
        Process a PDF through the complete workflow
        
        Args:
            pdf_path: Path to the uploaded PDF file
            student_id: Identifier for the student (for adaptive learning)
            custom_options: Custom processing options
            
        Returns:
            Dictionary with results from the complete workflow
        """
        # Generate workflow ID
        workflow_id = f"wf_{int(time.time())}"
        
        # Merge custom options with config
        if custom_options:
            # Create a copy of the configuration to avoid modifying the original
            workflow_config = self._merge_custom_options(custom_options)
        else:
            workflow_config = self.config
        
        # Initialize workflow state with thread safety
        with self.workflow_lock:
            self.workflow_states[workflow_id] = {
                "id": workflow_id,
                "pdf_path": pdf_path,
                "student_id": student_id,
                "status": "initializing",
                "start_time": time.time(),
                "extracted_content": None,
                "content_analysis": None,
                "video_results": None,
                "adaptive_result": None,
                "steps_completed": [],
                "error": None
            }
        
        # Publish workflow started event
        get_event_bus().publish_for_workflow(
            workflow_id, 
            EventType.WORKFLOW_STARTED,
            {
                "pdf_path": pdf_path, 
                "student_id": student_id,
                "config": {k: v for k, v in workflow_config.__dict__.items() 
                          if not k.startswith("_")}
            }
        )
        
        self.logger.info(f"Starting workflow {workflow_id} for PDF: {pdf_path}")
        
        # Check whether to process synchronously or asynchronously
        if self.config.async_processing:
            # Submit for asynchronous processing
            self.thread_pool.submit(self._run_workflow, workflow_id, pdf_path, 
                                   student_id, custom_options)
            
            # Return immediately with workflow ID
            return {
                "workflow_id": workflow_id,
                "status": "submitted",
                "message": "Workflow submitted for processing"
            }
        else:
            # Run synchronously and return complete results
            try:
                return self._run_workflow(workflow_id, pdf_path, student_id, custom_options)
            except Exception as e:
                self.logger.error(f"Workflow error: {e}")
                
                # Publish workflow failed event
                get_event_bus().publish_for_workflow(
                    workflow_id, 
                    EventType.WORKFLOW_FAILED,
                    {"error": str(e)}
                )
                
                # Update workflow state with error
                with self.workflow_lock:
                    self.workflow_states[workflow_id]["status"] = "error"
                    self.workflow_states[workflow_id]["error"] = str(e)
                
                return {
                    "workflow_id": workflow_id,
                    "pdf_path": pdf_path,
                    "status": "error",
                    "error_message": str(e)
                }
    
    def _merge_custom_options(self, custom_options: Dict[str, Any]) -> WorkflowConfig:
        """
        Merge custom options with the default configuration
        
        Args:
            custom_options: Custom options for this workflow
            
        Returns:
            Updated configuration with custom options
        """
        # Create a new config based on the current one
        config_dict = {
            "extraction_options": vars(self.config.extraction_options),
            "analysis_options": vars(self.config.analysis_options),
            "multimedia_options": {
                "script_style": vars(self.config.multimedia_options.script_style),
                "animation_style": vars(self.config.multimedia_options.animation_style),
                "voice_options": vars(self.config.multimedia_options.voice_options),
                "video_format": self.config.multimedia_options.video_format,
                "video_quality": self.config.multimedia_options.video_quality.value,
                "api_token": self.config.multimedia_options.api_token
            },
            "adaptive_options": vars(self.config.adaptive_options),
            "output_dir": self.config.output_dir,
            "async_processing": self.config.async_processing,
            "use_distributed": self.config.use_distributed,
            "auto_scaling": self.config.auto_scaling
        }
        
        # Update with custom options
        self._deep_update(config_dict, custom_options)
        
        # Create a new typed configuration
        return WorkflowConfig.from_dict(config_dict)
    
    def _deep_update(self, base_dict: Dict[str, Any], update_dict: Dict[str, Any]) -> None:
        """
        Recursively update a dictionary with another dictionary
        
        Args:
            base_dict: The base dictionary to update
            update_dict: Dictionary with updates to apply
        """
        for key, value in update_dict.items():
            if key in base_dict and isinstance(base_dict[key], dict) and isinstance(value, dict):
                self._deep_update(base_dict[key], value)
            else:
                base_dict[key] = value

    def _run_workflow(self, workflow_id: str, pdf_path: str, student_id: Optional[str] = None, 
                     custom_options: Optional[Dict] = None) -> Dict[str, Any]:
        """
        Execute the full workflow process
        
        Args:
            workflow_id: Unique identifier for this workflow
            pdf_path: Path to the uploaded PDF file
            student_id: Identifier for the student (for adaptive learning)
            custom_options: Custom processing options
            
        Returns:
            Dictionary with results from the complete workflow
            
        Raises:
            Various WorkflowError subclasses for specific failure scenarios
        """
        # Get workflow configuration (with custom options if provided)
        workflow_config = self._merge_custom_options(custom_options) if custom_options else self.config
        
        # Initialize timing metrics
        step_timing = {}
        start_time = time.time()
        extraction_result = None
        analysis_result = None
        video_result = None
        adaptive_result = None
        
        try:
            # Update workflow state
            self._update_workflow_status(workflow_id, "pdf_extraction")
            
            # Log and publish event
            self.logger.info(f"Workflow {workflow_id}: Starting PDF extraction for {pdf_path}")
            get_event_bus().publish_for_workflow(
                workflow_id,
                EventType.EXTRACTION_STARTED,
                {"pdf_path": pdf_path}
            )
            
            # Get PDF extractor from container
            pdf_extractor = self.container.get_component_typed(PDFContentExtractor)
            
            # Create output directory for extraction
            workflow_extracted_dir = os.path.join(self.extracted_dir, workflow_id)
            os.makedirs(workflow_extracted_dir, exist_ok=True)
            
            # Extract PDF content with error handling
            extraction_start = time.time()
            try:
                # Check if it's a scanned document
                is_scanned = pdf_extractor.is_scanned_document(pdf_path)
                
                # Extract content with appropriate options
                extraction_result = pdf_extractor.extract_content(
                    pdf_path=pdf_path,
                    output_dir=workflow_extracted_dir,
                    extract_images=workflow_config.extraction_options.extract_images,
                    extract_tables=workflow_config.extraction_options.extract_tables,
                    extract_equations=workflow_config.extraction_options.extract_equations,
                    ocr_quality=workflow_config.extraction_options.ocr_quality,
                    ocr_language=workflow_config.extraction_options.ocr_language,
                    process_scanned=workflow_config.extraction_options.process_scanned,
                    deskew_pages=workflow_config.extraction_options.deskew_pages,
                    denoise_images=workflow_config.extraction_options.denoise_images,
                    is_scanned=is_scanned
                )
                
                # Check for required fields
                if not extraction_result or not extraction_result.get("content"):
                    raise PDFExtractionError(
                        "PDF extraction failed to produce valid content",
                        workflow_id=workflow_id,
                        details={"pdf_path": pdf_path}
                    )
                
                # Save extraction output for persistence
                with open(os.path.join(workflow_extracted_dir, "extraction_result.json"), "w") as f:
                    json.dump(extraction_result, f, indent=2)
                
                # Log and publish success event
                self.logger.info(f"Workflow {workflow_id}: PDF extraction completed successfully")
                step_timing["extraction"] = time.time() - extraction_start
                
                get_event_bus().publish_for_workflow(
                    workflow_id,
                    EventType.EXTRACTION_COMPLETED,
                    {"pages": len(extraction_result.get("pages", [])),
                     "images": len(extraction_result.get("images", [])),
                     "tables": len(extraction_result.get("tables", [])),
                     "duration": step_timing["extraction"]}
                )
                
                # Update workflow state with extraction results
                with self.workflow_lock:
                    self.workflow_states[workflow_id]["extracted_content"] = extraction_result
                    self.workflow_states[workflow_id]["steps_completed"].append("extraction")
                
            except Exception as e:
                # Capture specific extraction errors
                error_msg = f"PDF extraction failed: {str(e)}"
                self.logger.error(f"Workflow {workflow_id}: {error_msg}", exc_info=True)
                
                # Publish extraction failed event
                get_event_bus().publish_for_workflow(
                    workflow_id,
                    EventType.EXTRACTION_FAILED,
                    {"error": str(e)}
                )
                
                # Wrap the exception in our error type if needed
                if not isinstance(e, PDFExtractionError):
                    raise PDFExtractionError(
                        error_msg, 
                        workflow_id=workflow_id,
                        details={"pdf_path": pdf_path}
                    ) from e
                raise
            
            # Update workflow state
            self._update_workflow_status(workflow_id, "content_analysis")
            
            # Log and publish event
            self.logger.info(f"Workflow {workflow_id}: Starting content analysis")
            get_event_bus().publish_for_workflow(
                workflow_id,
                EventType.ANALYSIS_STARTED,
                {"extraction_result_summary": extraction_result.get("summary", {})}
            )
            
            # Get content analysis pipeline from container
            content_analyzer = self.container.get_component_typed(ContentAnalysisPipeline)
            
            # Create output directory for analysis
            workflow_analysis_dir = os.path.join(self.analysis_dir, workflow_id)
            os.makedirs(workflow_analysis_dir, exist_ok=True)
            
            # Analyze content with error handling
            analysis_start = time.time()
            try:
                # Run content analysis pipeline
                analysis_result = content_analyzer.analyze_content(
                    extracted_content=extraction_result,
                    output_dir=workflow_analysis_dir,
                    domain_focus=workflow_config.analysis_options.domain_focus,
                    grade_level=workflow_config.analysis_options.grade_level,
                    detail_level=workflow_config.analysis_options.detail_level,
                    include_quiz=workflow_config.analysis_options.include_quiz,
                    language=workflow_config.analysis_options.language
                )
                
                # Check for required fields
                if not analysis_result or not analysis_result.get("topic"):
                    raise ContentAnalysisError(
                        "Content analysis failed to identify topic",
                        workflow_id=workflow_id
                    )
                
                # Save analysis output for persistence
                with open(os.path.join(workflow_analysis_dir, "analysis_result.json"), "w") as f:
                    json.dump(analysis_result, f, indent=2)
                
                # Log and publish success event
                self.logger.info(f"Workflow {workflow_id}: Content analysis completed successfully")
                step_timing["analysis"] = time.time() - analysis_start
                
                get_event_bus().publish_for_workflow(
                    workflow_id,
                    EventType.ANALYSIS_COMPLETED,
                    {"topic": analysis_result.get("topic", ""),
                     "concepts": len(analysis_result.get("concepts", [])),
                     "duration": step_timing["analysis"]}
                )
                
                # Update workflow state with analysis results
                with self.workflow_lock:
                    self.workflow_states[workflow_id]["content_analysis"] = analysis_result
                    self.workflow_states[workflow_id]["steps_completed"].append("analysis")
                
            except Exception as e:
                # Capture specific analysis errors
                error_msg = f"Content analysis failed: {str(e)}"
                self.logger.error(f"Workflow {workflow_id}: {error_msg}", exc_info=True)
                
                # Publish analysis failed event
                get_event_bus().publish_for_workflow(
                    workflow_id,
                    EventType.ANALYSIS_FAILED,
                    {"error": str(e)}
                )
                
                # Wrap the exception in our error type if needed
                if not isinstance(e, ContentAnalysisError):
                    raise ContentAnalysisError(
                        error_msg, 
                        workflow_id=workflow_id
                    ) from e
                raise
            
            # Update workflow state
            self._update_workflow_status(workflow_id, "video_generation")
            
            # Log and publish event
            self.logger.info(f"Workflow {workflow_id}: Starting multimedia generation")
            get_event_bus().publish_for_workflow(
                workflow_id,
                EventType.MULTIMEDIA_STARTED,
                {"analysis_summary": analysis_result.get("summary", {})}
            )
            
            # Get multimedia generator from container
            multimedia_generator = self.container.get_component_typed(MultimediaGenerationPipeline)
            
            # Create output directory for multimedia
            workflow_multimedia_dir = os.path.join(self.multimedia_dir, workflow_id)
            os.makedirs(workflow_multimedia_dir, exist_ok=True)
            
            # Generate multimedia with error handling
            video_start = time.time()
            try:
                # Generate multimedia content
                video_result = multimedia_generator.generate_video(
                    analysis_result=analysis_result,
                    extracted_content=extraction_result,
                    output_dir=workflow_multimedia_dir,
                    script_style=workflow_config.multimedia_options.script_style,
                    animation_style=workflow_config.multimedia_options.animation_style,
                    voice_options=workflow_config.multimedia_options.voice_options,
                    video_format=workflow_config.multimedia_options.video_format,
                    video_quality=workflow_config.multimedia_options.video_quality
                )
                
                # Check for required fields
                if not video_result or not video_result.get("video_file"):
                    raise MultimediaGenerationError(
                        "Video generation failed to produce video file",
                        workflow_id=workflow_id
                    )
                
                # Save video metadata for persistence
                with open(os.path.join(workflow_multimedia_dir, "video_metadata.json"), "w") as f:
                    json.dump(video_result, f, indent=2)
                
                # Log and publish success event
                self.logger.info(f"Workflow {workflow_id}: Multimedia generation completed successfully")
                step_timing["video"] = time.time() - video_start
                
                get_event_bus().publish_for_workflow(
                    workflow_id,
                    EventType.MULTIMEDIA_COMPLETED,
                    {"video_file": video_result.get("video_file", ""),
                     "duration": step_timing["video"],
                     "video_length": video_result.get("video_length", 0)}
                )
                
                # Update workflow state with video results
                with self.workflow_lock:
                    self.workflow_states[workflow_id]["video_results"] = video_result
                    self.workflow_states[workflow_id]["steps_completed"].append("video")
            
        except Exception as e:
                # Capture specific video generation errors
                error_msg = f"Multimedia generation failed: {str(e)}"
                self.logger.error(f"Workflow {workflow_id}: {error_msg}", exc_info=True)
                
                # Publish multimedia failed event
                get_event_bus().publish_for_workflow(
                    workflow_id,
                    EventType.MULTIMEDIA_FAILED,
                    {"error": str(e)}
                )
                
                # Wrap the exception in our error type if needed
                if not isinstance(e, MultimediaGenerationError):
                    raise MultimediaGenerationError(
                        error_msg, 
                        workflow_id=workflow_id
                    ) from e
                raise
            
            # If student ID is provided, perform adaptive learning setup
            if student_id:
                # Update workflow state
                self._update_workflow_status(workflow_id, "adaptive_learning")
                
                # Log and publish event
                self.logger.info(f"Workflow {workflow_id}: Starting adaptive learning for student {student_id}")
                get_event_bus().publish_for_workflow(
                    workflow_id,
                    EventType.ADAPTIVE_STARTED,
                    {"student_id": student_id}
                )
                
                # Get adaptive learning manager from container
                adaptive_manager = self.container.get_component_typed(AdaptiveLearningManager)
                student_profile_manager = self.container.get_component_typed(StudentProfileManager)
                
                # Create output directory for adaptive learning
                workflow_adaptive_dir = os.path.join(self.feedback_dir, workflow_id)
                os.makedirs(workflow_adaptive_dir, exist_ok=True)
                
                # Set up adaptive learning with error handling
                adaptive_start = time.time()
                try:
        # Get or create student profile
                    student_profile = student_profile_manager.get_student_profile(student_id)
                    
                    # Create adaptive learning path
                    adaptive_result = adaptive_manager.setup_learning_path(
            student_profile=student_profile,
                        analysis_result=analysis_result,
                        video_metadata=video_result,
                        output_dir=workflow_adaptive_dir,
                        difficulty_adjustment=workflow_config.adaptive_options.difficulty_adjustment,
                        focus_areas=workflow_config.adaptive_options.focus_areas,
                        learning_style=workflow_config.adaptive_options.learning_style
                    )
                    
                    # Check for required fields
                    if not adaptive_result or not adaptive_result.get("learning_path"):
                        raise AdaptiveLearningError(
                            "Adaptive learning setup failed to create learning path",
                            workflow_id=workflow_id,
                            details={"student_id": student_id}
                        )
                    
                    # Save adaptive learning output for persistence
                    with open(os.path.join(workflow_adaptive_dir, "adaptive_result.json"), "w") as f:
                        json.dump(adaptive_result, f, indent=2)
                    
                    # Log and publish success event
                    self.logger.info(f"Workflow {workflow_id}: Adaptive learning setup completed")
                    step_timing["adaptive"] = time.time() - adaptive_start
                    
                    get_event_bus().publish_for_workflow(
                        workflow_id,
                        EventType.ADAPTIVE_COMPLETED,
                        {"learning_path": adaptive_result.get("learning_path", ""),
                         "duration": step_timing["adaptive"]}
                    )
                    
                    # Update workflow state with adaptive results
                    with self.workflow_lock:
                        self.workflow_states[workflow_id]["adaptive_result"] = adaptive_result
                        self.workflow_states[workflow_id]["steps_completed"].append("adaptive")
                    
                except Exception as e:
                    # Capture specific adaptive learning errors
                    error_msg = f"Adaptive learning setup failed: {str(e)}"
                    self.logger.error(f"Workflow {workflow_id}: {error_msg}", exc_info=True)
                    
                    # Publish adaptive learning failed event
                    get_event_bus().publish_for_workflow(
                        workflow_id,
                        EventType.ADAPTIVE_FAILED,
                        {"error": str(e), "student_id": student_id}
                    )
                    
                    # Wrap the exception in our error type if needed
                    if not isinstance(e, AdaptiveLearningError):
                        raise AdaptiveLearningError(
                            error_msg, 
                            workflow_id=workflow_id,
                            details={"student_id": student_id}
                        ) from e
                    raise
            
            # Calculate total workflow duration
            total_duration = time.time() - start_time
            
            # Compile performance metrics
            performance_metrics = {
                "workflow_id": workflow_id,
                "total_duration": total_duration,
                "step_timing": step_timing,
                "pdf_size": os.path.getsize(pdf_path) if os.path.exists(pdf_path) else 0,
                "pages_processed": len(extraction_result.get("pages", [])) if extraction_result else 0,
                "images_extracted": len(extraction_result.get("images", [])) if extraction_result else 0,
                "tables_extracted": len(extraction_result.get("tables", [])) if extraction_result else 0,
                "video_length": video_result.get("video_length", 0) if video_result else 0,
                "is_scanned_document": extraction_result.get("is_scanned", False) if extraction_result else False,
                "timestamp": time.time()
            }
            
            # Save performance metrics
            metrics_file = os.path.join(self.metrics_dir, f"{workflow_id}_metrics.json")
            with open(metrics_file, "w") as f:
                json.dump(performance_metrics, f, indent=2)
            
            # Update workflow state
            self._update_workflow_status(workflow_id, "completed")
            
            # Log and publish workflow completed event
            self.logger.info(f"Workflow {workflow_id}: Completed successfully in {total_duration:.2f} seconds")
            get_event_bus().publish_for_workflow(
                workflow_id,
                EventType.WORKFLOW_COMPLETED,
                {"total_duration": total_duration,
                 "steps_completed": self.workflow_states[workflow_id]["steps_completed"]}
            )
            
            # Check if auto-scaling is enabled
            if workflow_config.auto_scaling:
                self._check_scaling_needs(performance_metrics)
            
            # Compile final results
            result = {
                "workflow_id": workflow_id,
                "pdf_path": pdf_path,
                "status": "completed",
                "duration": total_duration,
                "extraction": {
                    "pages": len(extraction_result.get("pages", [])),
                    "images": len(extraction_result.get("images", [])),
                    "tables": len(extraction_result.get("tables", []))
                },
                "analysis": {
                    "topic": analysis_result.get("topic", ""),
                    "concepts": analysis_result.get("concepts", []),
                    "summary": analysis_result.get("summary", {})
                },
                "video": {
                    "video_file": video_result.get("video_file", ""),
                    "script_file": video_result.get("script_file", ""),
                    "length": video_result.get("video_length", 0)
                }
            }
            
            # Add adaptive learning results if applicable
            if student_id and adaptive_result:
                result["adaptive"] = {
                    "student_id": student_id,
                    "learning_path": adaptive_result.get("learning_path", ""),
                    "recommendations": adaptive_result.get("recommendations", [])
                }
            
            return result
            
        except Exception as e:
            # Calculate total workflow duration even on error
            error_duration = time.time() - start_time
            
            # Determine error type
            if isinstance(e, WorkflowError):
                error_message = str(e)
                error_type = e.__class__.__name__
                error_details = e.details if hasattr(e, "details") else {}
        else:
                error_message = f"Unexpected error: {str(e)}"
                error_type = "UnexpectedError"
                error_details = {"original_type": type(e).__name__}
            
            # Log the error
            self.logger.error(
                f"Workflow {workflow_id}: Failed with {error_type}: {error_message}",
                exc_info=True
            )
            
            # Update workflow state
            with self.workflow_lock:
                self.workflow_states[workflow_id]["status"] = "error"
                self.workflow_states[workflow_id]["end_time"] = time.time()
                self.workflow_states[workflow_id]["error"] = {
                    "type": error_type,
                    "message": error_message,
                    "details": error_details
                }
            
            # Publish workflow failed event
            get_event_bus().publish_for_workflow(
                workflow_id,
                EventType.WORKFLOW_FAILED,
                {"error_type": error_type,
                 "error_message": error_message,
                 "duration": error_duration}
            )
            
            # Compile error result
            error_result = {
                "workflow_id": workflow_id,
                "pdf_path": pdf_path,
                "status": "error",
                "error": {
                    "type": error_type,
                    "message": error_message,
                    "details": error_details
                },
                "duration": error_duration,
                "steps_completed": self.workflow_states[workflow_id]["steps_completed"]
            }
            
            # Add partial results if they exist
            if extraction_result:
                error_result["partial_extraction"] = {
                    "pages": len(extraction_result.get("pages", [])),
                    "images": len(extraction_result.get("images", [])),
                    "tables": len(extraction_result.get("tables", []))
                }
            
            if analysis_result:
                error_result["partial_analysis"] = {
                    "topic": analysis_result.get("topic", ""),
                    "summary": analysis_result.get("summary", {})
                }
            
            if video_result:
                error_result["partial_video"] = {
                    "video_file": video_result.get("video_file", ""),
                    "script_file": video_result.get("script_file", "")
                }
            
            return error_result
    
    def _update_workflow_status(self, workflow_id: str, status: str) -> None:
        """
        Update the status of a workflow
        
        Args:
            workflow_id: Unique identifier for the workflow
            status: New status string
        """
        with self.workflow_lock:
            if workflow_id in self.workflow_states:
                self.workflow_states[workflow_id]["status"] = status
                self.workflow_states[workflow_id]["last_updated"] = time.time()
                
                # If status is "completed" or "error", set end time
                if status in ["completed", "error"]:
                    self.workflow_states[workflow_id]["end_time"] = time.time()
                
                # Log status change
                self.logger.info(f"Workflow {workflow_id}: Status changed to '{status}'")
    
    def _check_scaling_needs(self, performance_metrics: Dict[str, Any]) -> None:
        """
        Check if system needs to scale based on performance metrics
        
        Args:
            performance_metrics: Dictionary of performance metrics
        """
        try:
            # Get current system metrics
            cpu_percent = psutil.cpu_percent(interval=1)
            memory_percent = psutil.virtual_memory().percent
            disk_usage = psutil.disk_usage('/').percent
            
            # Get workflow metrics
            total_duration = performance_metrics.get("total_duration", 0)
            step_timing = performance_metrics.get("step_timing", {})
            
            # Calculate average step duration
            avg_step_duration = sum(step_timing.values()) / len(step_timing) if step_timing else 0
            
            # Define scaling thresholds
            SCALING_THRESHOLDS = {
                "cpu_percent": 80,  # Scale if CPU usage > 80%
                "memory_percent": 85,  # Scale if memory usage > 85%
                "disk_usage": 90,  # Scale if disk usage > 90%
                "avg_step_duration": 30,  # Scale if average step takes > 30 seconds
                "total_duration": 300  # Scale if total workflow takes > 5 minutes
            }
            
            # Check if scaling is needed
            scaling_needed = False
            scaling_reasons = []
            
            if cpu_percent > SCALING_THRESHOLDS["cpu_percent"]:
                scaling_needed = True
                scaling_reasons.append(f"High CPU usage: {cpu_percent}%")
            
            if memory_percent > SCALING_THRESHOLDS["memory_percent"]:
                scaling_needed = True
                scaling_reasons.append(f"High memory usage: {memory_percent}%")
            
            if disk_usage > SCALING_THRESHOLDS["disk_usage"]:
                scaling_needed = True
                scaling_reasons.append(f"High disk usage: {disk_usage}%")
            
            if avg_step_duration > SCALING_THRESHOLDS["avg_step_duration"]:
                scaling_needed = True
                scaling_reasons.append(f"Slow step execution: {avg_step_duration:.1f}s average")
            
            if total_duration > SCALING_THRESHOLDS["total_duration"]:
                scaling_needed = True
                scaling_reasons.append(f"Long workflow duration: {total_duration:.1f}s")
            
            # If scaling is needed, trigger scaling action
            if scaling_needed:
                self.logger.warning(f"Scaling needed: {', '.join(scaling_reasons)}")
                
                # Publish scaling event
                get_event_bus().publish_for_workflow(
                    performance_metrics["workflow_id"],
                    EventType.SCALING_NEEDED,
                    {
                        "reasons": scaling_reasons,
                        "metrics": {
                            "cpu_percent": cpu_percent,
                            "memory_percent": memory_percent,
                            "disk_usage": disk_usage,
                            "avg_step_duration": avg_step_duration,
                            "total_duration": total_duration
                        }
                    }
                )
                
                # If using distributed processing, scale Ray cluster
                if self.use_distributed and RAY_AVAILABLE:
                    try:
                        # Get current Ray cluster resources
                        cluster_resources = ray.cluster_resources()
                        available_resources = ray.available_resources()
                        
                        # Calculate scaling factor based on resource usage
                        cpu_scale_factor = cpu_percent / 100
                        memory_scale_factor = memory_percent / 100
                        
                        # Determine new resource requirements
                        new_cpus = max(1, int(cluster_resources.get("CPU", 1) * (1 + cpu_scale_factor)))
                        new_memory = int(cluster_resources.get("memory", 0) * (1 + memory_scale_factor))
                        
                        # Scale Ray cluster
                        ray.autoscale(
                            min_workers=1,
                            max_workers=new_cpus,
                            target_utilization=0.7
                        )
                        
                        self.logger.info(f"Ray cluster scaled to {new_cpus} CPUs and {new_memory/1024/1024/1024:.1f}GB memory")
                        
                    except Exception as e:
                        self.logger.error(f"Failed to scale Ray cluster: {str(e)}")
            
        except Exception as e:
            self.logger.error(f"Error checking scaling needs: {str(e)}")
            # Don't raise the error as this is a non-critical operation

---


## AdaptiveLearningManager.py

In [ ]:
"""
Adaptive Learning Manager
------------------------
Main integration module that connects:
- PDF Extraction Pipeline
- Student Profile Management
- Adaptive Learning Pathway Generation
- Content Personalization
"""

import os
import json
from typing import Dict, List, Any, Optional, Union
from datetime import datetime

# Import core components
from pipeline.content_analysis.ContentAnalysisPipeline import ContentAnalysisPipeline
from pipeline.adaptive_learning.StudentProfileManager import StudentProfileManager
from pipeline.adaptive_learning.AdaptiveLearningPathway import AdaptiveLearningPathwayGenerator, AdaptiveLearningPathway

class PersonalizedLearningSession:
    """Represents an active learning session for a student"""
    
    def __init__(self, student_id: str, document_id: str, pathway: AdaptiveLearningPathway):
        """
        Initialize a learning session
        
        Args:
            student_id: ID of the student
            document_id: ID of the document being studied
            pathway: The personalized learning pathway
        """
        self.student_id = student_id
        self.document_id = document_id
        self.pathway = pathway
        self.current_node_id = None
        self.session_start = datetime.now().isoformat()
        self.last_activity = self.session_start
        self.completed_nodes = []
        self.interaction_history = []
        
    def start_node(self, node_id: str):
        """
        Begin studying a specific node
        
        Args:
            node_id: ID of the node to start
        """
        self.current_node_id = node_id
        self.last_activity = datetime.now().isoformat()
        self.interaction_history.append({
            "timestamp": self.last_activity,
            "action": "start_node",
            "node_id": node_id
        })
        
    def complete_node(self, node_id: str, performance: Optional[float] = None):
        """
        Mark a node as completed
        
        Args:
            node_id: ID of the completed node
            performance: Optional performance metric (0-1)
        """
        self.pathway.update_progress(node_id, 100)
        self.last_activity = datetime.now().isoformat()
        
        if node_id not in self.completed_nodes:
            self.completed_nodes.append(node_id)
            
        self.interaction_history.append({
            "timestamp": self.last_activity,
            "action": "complete_node",
            "node_id": node_id,
            "performance": performance
        })
        
    def get_next_node(self) -> Dict[str, Any]:
        """
        Get the recommended next node
        
        Returns:
            Dictionary with next node information
        """
        next_nodes = self.pathway.get_next_nodes()
        
        if not next_nodes:
            return {"completed": True, "progress": self.pathway.get_overall_progress()}
            
        # Sophisticated logic for selecting the most appropriate next node
        if len(next_nodes) == 1:
            next_node = next_nodes[0]
        else:
            # Get student profile to personalize selection
            try:
                profile_manager = StudentProfileManager()
                profile = profile_manager.get_profile(self.student_id)
                
                # Initialize scoring for each potential next node
                node_scores = {}
                
                for node in next_nodes:
                    score = 0
                    
                    # Factor 1: Match difficulty to student's current proficiency
                    if profile and hasattr(profile, 'get_proficiency'):
                        node_topic = node.title.split()[0] if node.title else ""
                        student_proficiency = profile.get_proficiency(node_topic) or 0.5
                        
                        # Adjust score based on how well difficulty matches proficiency
                        # Convert difficulty string to numeric value if needed
                        difficulty_value = {"beginner": 0.3, "intermediate": 0.6, "advanced": 0.9}.get(
                            node.difficulty.lower() if isinstance(node.difficulty, str) else 0.5, 0.5)
                        
                        # Prefer content that's slightly challenging but not too difficult
                        difficulty_match = 1.0 - abs(difficulty_value - (student_proficiency + 0.1))
                        score += difficulty_match * 3  # Weight: 3
                    
                    # Factor 2: Consider learning style preference
                    if profile and hasattr(profile, 'learning_style'):
                        learning_style = getattr(profile, 'learning_style', {})
                        
                        # Match content type to preferred learning style
                        if node.content_type == "visual" and learning_style.get("visual", 0) > 0.5:
                            score += 2
                        elif node.content_type == "text" and learning_style.get("textual", 0) > 0.5:
                            score += 2
                        elif node.content_type == "interactive" and learning_style.get("interactive", 0) > 0.5:
                            score += 2
                    
                    # Factor 3: Consider prerequisite relationships
                    prerequisite_completion = 0
                    if hasattr(node, 'prerequisites'):
                        prerequisites = getattr(node, 'prerequisites', [])
                        if prerequisites:
                            completed_prereqs = [p for p in prerequisites if p in self.completed_nodes]
                            prerequisite_completion = len(completed_prereqs) / len(prerequisites)
                    score += prerequisite_completion * 4  # Weight: 4
                    
                    # Factor 4: Prioritize nodes related to recent completions for continuity
                    if self.interaction_history and hasattr(node, 'related_to'):
                        recent_interactions = self.interaction_history[-3:]
                        recent_nodes = [i["node_id"] for i in recent_interactions if "node_id" in i]
                        related_nodes = getattr(node, 'related_to', [])
                        
                        if any(r_node in recent_nodes for r_node in related_nodes):
                            score += 2  # Continuity bonus
                    
                    # Store the calculated score
                    node_scores[node.content_id] = score
                
                # Select the node with the highest score
                if node_scores:
                    best_node_id = max(node_scores, key=node_scores.get)
                    next_node = next((n for n in next_nodes if n.content_id == best_node_id), next_nodes[0])
                else:
                    next_node = next_nodes[0]  # Fallback to first node if scoring fails
            except Exception as e:
                # Fallback to simple selection if there's an error in the sophisticated logic
                print(f"Error in node selection: {e}. Falling back to simple selection.")
                next_node = next_nodes[0]
        
        self.current_node_id = next_node.content_id
        return {
            "node_id": next_node.content_id,
            "title": next_node.title,
            "content_type": next_node.content_type,
            "difficulty": next_node.difficulty,
            "alternatives": next_node.alternatives,
            "estimated_time": next_node.estimated_time
        }
        
    def get_session_stats(self) -> Dict[str, Any]:
        """
        Get session statistics
        
        Returns:
            Dictionary with session stats
        """
        completed_count = len(self.completed_nodes)
        total_nodes = len(self.pathway.nodes)
        current_time = datetime.now().isoformat()
        
        # Calculate session duration
        start_time = datetime.fromisoformat(self.session_start)
        current = datetime.fromisoformat(current_time)
        duration_minutes = (current - start_time).total_seconds() / 60
        
        return {
            "student_id": self.student_id,
            "document_id": self.document_id,
            "started_at": self.session_start,
            "duration_minutes": round(duration_minutes, 1),
            "completed_nodes": completed_count,
            "total_nodes": total_nodes,
            "progress_percentage": self.pathway.get_overall_progress(),
            "current_node": self.current_node_id
        }
        
    def to_dict(self) -> Dict[str, Any]:
        """Convert session to dictionary"""
        return {
            "student_id": self.student_id,
            "document_id": self.document_id,
            "pathway": self.pathway.to_dict(),
            "current_node_id": self.current_node_id,
            "session_start": self.session_start,
            "last_activity": self.last_activity,
            "completed_nodes": self.completed_nodes,
            "interaction_history": self.interaction_history
        }
        
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'PersonalizedLearningSession':
        """Create session from dictionary"""
        pathway = AdaptiveLearningPathway.from_dict(data["pathway"])
        
        session = cls(
            student_id=data["student_id"],
            document_id=data["document_id"],
            pathway=pathway
        )
        
        session.current_node_id = data.get("current_node_id")
        session.session_start = data.get("session_start", session.session_start)
        session.last_activity = data.get("last_activity", session.last_activity)
        session.completed_nodes = data.get("completed_nodes", [])
        session.interaction_history = data.get("interaction_history", [])
        
        return session

class AdaptiveLearningManager:
    """Main manager class for adaptive learning integration"""
    
    def __init__(self, domain: str = None, 
               storage_dir: str = "adaptive_learning",
               profile_manager: StudentProfileManager = None,
               pipeline: ContentAnalysisPipeline = None):
        """
        Initialize the adaptive learning manager
        
        Args:
            domain: Optional domain specification
            storage_dir: Directory for storing adaptive learning data
            profile_manager: Optional StudentProfileManager instance
            pipeline: Optional ContentAnalysisPipeline instance
        """
        self.domain = domain
        self.storage_dir = storage_dir
        self.profile_manager = profile_manager or StudentProfileManager()
        self.pipeline = pipeline or ContentAnalysisPipeline(domain=domain)
        self.pathway_generator = AdaptiveLearningPathwayGenerator()
        
        # Create storage directories
        os.makedirs(storage_dir, exist_ok=True)
        os.makedirs(os.path.join(storage_dir, "documents"), exist_ok=True)
        os.makedirs(os.path.join(storage_dir, "sessions"), exist_ok=True)
        
        # Active learning sessions
        self.active_sessions = {}
        
    def process_document(self, pdf_path: str, output_dir: str = None) -> str:
        """
        Process a document for adaptive learning
        
        Args:
            pdf_path: Path to PDF file
            output_dir: Optional output directory
            
        Returns:
            Document ID
        """
        # Generate document ID from filename
        document_id = os.path.splitext(os.path.basename(pdf_path))[0]
        
        # Set output directory
        if not output_dir:
            output_dir = os.path.join(self.storage_dir, "documents", document_id)
            
        # Process document with analysis pipeline
        result = self.pipeline.process_document(
            pdf_path=pdf_path,
            output_dir=output_dir,
            extract_images=True,
            extract_tables=True,
            respect_heading_hierarchy=True
        )
        
        # Save processed document
        self._save_processed_document(document_id, result)
        
        return document_id
        
    def create_personalized_pathway(self, student_id: str, document_id: str, 
                                 target_concepts: List[str] = None) -> str:
        """
        Create a personalized learning pathway
        
        Args:
            student_id: ID of the student
            document_id: ID of the processed document
            target_concepts: Optional list of concepts to focus on
            
        Returns:
            Session ID for the created pathway
        """
        # Get student profile
        profile = self.profile_manager.get_profile(student_id)
        
        if not profile:
            # Create a default profile if not found
            profile = self.profile_manager.create_profile(student_id)
            
        # Load processed document
        document = self._load_processed_document(document_id)
        
        if not document:
            raise ValueError(f"Document {document_id} not found")
            
        # Generate adaptive pathway
        pathway = self.pathway_generator.generate_pathway(
            profile=profile,
            content_structure=document,
            target_concepts=target_concepts
        )
        
        # Create a learning session
        session_id = f"{student_id}_{document_id}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
        session = PersonalizedLearningSession(
            student_id=student_id,
            document_id=document_id,
            pathway=pathway
        )
        
        # Save and activate session
        self._save_session(session_id, session)
        self.active_sessions[session_id] = session
        
        return session_id
        
    def get_next_content(self, session_id: str) -> Dict[str, Any]:
        """
        Get the next content recommendation for a session
        
        Args:
            session_id: ID of the learning session
            
        Returns:
            Dictionary with next content information
        """
        session = self._get_session(session_id)
        
        if not session:
            raise ValueError(f"Session {session_id} not found")
            
        return session.get_next_node()
        
    def complete_content(self, session_id: str, node_id: str, 
                      performance: Optional[float] = None) -> Dict[str, Any]:
        """
        Mark content as completed
        
        Args:
            session_id: ID of the learning session
            node_id: ID of the completed content node
            performance: Optional performance metric (0-1)
            
        Returns:
            Updated session status
        """
        session = self._get_session(session_id)
        
        if not session:
            raise ValueError(f"Session {session_id} not found")
            
        # Mark node as completed
        session.complete_node(node_id, performance)
        
        # Update student profile with learning activity
        profile = self.profile_manager.get_profile(session.student_id)
        
        if profile:
            # Find the node in the pathway
            node = next((n for n in session.pathway.nodes if n.content_id == node_id), None)
            
            if node:
                # Record learning activity
                profile.record_learning_activity(
                    material_id=node_id,
                    activity_type=node.content_type,
                    performance=performance,
                    time_spent=node.estimated_time
                )
                
                # Extract concepts from the node title
                concepts = node.title.split()
                
                # Update knowledge areas if performance was provided
                if performance is not None:
                    for concept in concepts:
                        if len(concept) > 3:
                            profile.update_knowledge_area(
                                domain=self.domain or "general",
                                topic=concept,
                                proficiency=performance
                            )
                
                # Save updated profile
                self.profile_manager.save_profile(profile)
        
        # Save session
        self._save_session(session_id, session)
        
        # Return current session status
        return session.get_session_stats()
        
    def get_session_status(self, session_id: str) -> Dict[str, Any]:
        """
        Get current session status
        
        Args:
            session_id: ID of the learning session
            
        Returns:
            Dictionary with session status
        """
        session = self._get_session(session_id)
        
        if not session:
            raise ValueError(f"Session {session_id} not found")
            
        return session.get_session_stats()
        
    def personalize_content(self, content: str, student_id: str, 
                         difficulty: Optional[str] = None) -> str:
        """
        Personalize content for a specific student
        
        Args:
            content: The content to personalize
            student_id: ID of the student
            difficulty: Optional difficulty override
            
        Returns:
            Personalized content
        """
        profile = self.profile_manager.get_profile(student_id)
        
        if not profile:
            # Return original content if no profile found
            return content
            
        # Determine appropriate difficulty
        if not difficulty:
            difficulty = profile.get_recommended_difficulty()
            
        # Personalize content based on difficulty
        personalized = self._adapt_content_difficulty(content, difficulty)
        
        # Further personalize based on learning style
        learning_style = profile.learning_style
        
        # For visual learners, add more visual cues
        if learning_style.get("visual", 0) > 0.6:
            personalized = self._enhance_visual_elements(personalized)
            
        # For textual learners, add more detailed explanations
        if learning_style.get("textual", 0) > 0.6:
            personalized = self._enhance_textual_elements(personalized)
            
        # For interactive learners, add interactive prompts
        if learning_style.get("interactive", 0) > 0.6:
            personalized = self._add_interactive_elements(personalized)
            
        return personalized
        
    def _adapt_content_difficulty(self, content: str, difficulty: str) -> str:
        """
        Adapt content to specified difficulty level
        
        Args:
            content: Original content
            difficulty: Target difficulty level
            
        Returns:
            Adapted content
        """
        # Real implementation using NLP text simplification techniques
        import re
        from nltk.tokenize import sent_tokenize, word_tokenize
        from nltk.corpus import wordnet
        
        try:
            # Handle different difficulty levels
            if difficulty == "simple":
                # Text simplification for beginners
                sentences = sent_tokenize(content)
                simplified_sentences = []
                
                for sentence in sentences:
                    # Shorten long sentences
                    if len(word_tokenize(sentence)) > 20:
                        # Split into smaller sentences where possible
                        parts = re.split(r'[,;:]', sentence)
                        if len(parts) > 1:
                            simplified_sentences.extend([p.strip() + '.' for p in parts if p.strip()])
                            continue
                    
                    # Simplify vocabulary where possible
                    words = word_tokenize(sentence)
                    simplified_words = []
                    
                    for word in words:
                        if len(word) > 7 and word.isalpha():  # Only attempt to simplify longer words
                            # Try to find simpler synonyms
                            synonyms = []
                            for syn in wordnet.synsets(word):
                                for lemma in syn.lemmas():
                                    if lemma.name() != word and len(lemma.name()) < len(word):
                                        synonyms.append(lemma.name())
                            
                            if synonyms:
                                # Use the shortest synonym as a simpler alternative
                                shortest = min(synonyms, key=len)
                                simplified_words.append(shortest)
                                continue
                        
                        simplified_words.append(word)
                    
                    # Reconstruct the sentence
                    simplified_sentence = ' '.join(simplified_words)
                    simplified_sentences.append(simplified_sentence)
                
                # Add explanatory notes for complex concepts
                simplified_content = ' '.join(simplified_sentences)
                
                # Add beginner-friendly annotations
                simplified_content = re.sub(r'([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\b(?!\s+is|\s+are|\s+were|\s+was)', 
                                          r'\1 (a key concept)', 
                                          simplified_content)
                
                return simplified_content
                
            elif difficulty == "intermediate":
                # Keep mostly original but add some clarifications
                sentences = sent_tokenize(content)
                intermediate_sentences = []
                
                for sentence in sentences:
                    # Add examples for complex statements
                    if len(word_tokenize(sentence)) > 15:
                        sentence += " This means it applies in real-world contexts."
                    intermediate_sentences.append(sentence)
                
                return ' '.join(intermediate_sentences)
                
            elif difficulty == "advanced":
                # Enhance content with more depth
                sentences = sent_tokenize(content)
                advanced_sentences = []
                
                for i, sentence in enumerate(sentences):
                    advanced_sentences.append(sentence)
                    
                    # Add additional context every few sentences
                    if i % 3 == 2:
                        advanced_sentences.append("This concept connects to broader theoretical frameworks in the field.")
                    
                    # Introduce advanced terminology
                    words = word_tokenize(sentence)
                    for word in words:
                        if len(word) > 5 and word.isalpha():
                            # Look for more specialized terminology
                            for syn in wordnet.synsets(word):
                                for lemma in syn.lemmas():
                                    if lemma.name() != word and len(lemma.name()) > len(word):
                                        advanced_sentences.append(f"Note that {word} can also be referred to as {lemma.name()} in specialized contexts.")
                                        break
                                break
                
                return ' '.join(advanced_sentences)
            
            # Return standard content as is for unknown difficulty levels
            return content
            
        except ImportError:
            # Fallback if NLTK is not available
            print("Warning: NLTK not available. Using simple text modifications instead.")
            if difficulty == "simple":
                return f"[Simplified for beginners] {content}"
            elif difficulty == "advanced":
                return f"[Enhanced with advanced details] {content}"
            return content
        except Exception as e:
            print(f"Error during content difficulty adaptation: {e}")
            return content
    
    def _enhance_visual_elements(self, content: str) -> str:
        """Add visual cues to content for visual learners"""
        try:
            from bs4 import BeautifulSoup
            import re
            import random
            
            # Check if content is HTML
            if "<" in content and ">" in content:
                # Parse HTML
                soup = BeautifulSoup(content, 'html.parser')
                
                # Add color highlighting to key terms
                for p in soup.find_all('p'):
                    text = p.get_text()
                    # Find likely key terms (capitalized words, technical terms)
                    key_terms = re.findall(r'\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\b', text)
                    for term in key_terms:
                        # Don't replace text inside tags
                        if term in p.get_text(strip=True):
                            highlighted = soup.new_tag('span')
                            highlighted['style'] = 'background-color: #FFFACD; font-weight: bold;'
                            highlighted.string = term
                            # Replace only in text nodes, not in existing tags
                            text_nodes = list(p.find_all(text=True))
                            for text_node in text_nodes:
                                if term in text_node:
                                    new_content = text_node.replace(term, str(highlighted))
                                    text_node.replace_with(BeautifulSoup(new_content, 'html.parser'))
                                    break
                
                # Add margin notes with visual cues
                sections = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p'])
                for section in sections:
                    if section.name.startswith('h'):
                        note = soup.new_tag('div')
                        note['class'] = 'visual-note'
                        note['style'] = 'border-left: 4px solid #3498db; padding-left: 15px; margin: 10px 0;'
                        icon = soup.new_tag('span')
                        icon['style'] = 'font-size: 20px; color: #3498db; margin-right: 10px;'
                        icon.string = 'ðŸ”'
                        note.append(icon)
                        note_text = soup.new_tag('span')
                        note_text.string = f"Visual summary: {section.get_text()}"
                        note.append(note_text)
                        section.insert_after(note)
                
                # Add diagram placeholders
                content_sections = soup.find_all('p')
                if content_sections and len(content_sections) > 2:
                    selected_section = content_sections[len(content_sections) // 2]
                    
                    # Extract key concepts for diagram generation
                    key_concepts = set()
                    for p in content_sections:
                        # Find capitalized terms and technical terms
                        concepts = re.findall(r'\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\b', p.get_text())
                        for concept in concepts:
                            if len(concept) > 3:  # Filter out short terms
                                key_concepts.add(concept)
                    
                    # Generate actual diagram using SVG
                    if key_concepts:
                        # Convert set to list for indexing
                        key_concepts_list = list(key_concepts)[:5]  # Limit to 5 concepts
                        
                        # Create diagram container
                        diagram = soup.new_tag('div')
                        diagram['class'] = 'content-diagram'
                        diagram['style'] = 'margin: 25px 0; text-align: center;'
                        
                        # Generate SVG diagram
                        svg_code = self._generate_concept_diagram(key_concepts_list)
                        
                        # Add diagram title
                        title = soup.new_tag('h4')
                        title.string = "Concept Relationship Diagram"
                        diagram.append(title)
                        
                        # Add SVG diagram
                        diagram.append(BeautifulSoup(svg_code, 'html.parser'))
                        
                        # Add diagram description
                        description = soup.new_tag('p')
                        description['style'] = 'font-style: italic; margin-top: 10px; font-size: 0.9em;'
                        description.string = f"This diagram shows relationships between key concepts: {', '.join(key_concepts_list)}"
                        diagram.append(description)
                        
                        selected_section.insert_after(diagram)
                    else:
                        # Fallback if no concepts found
                        diagram = soup.new_tag('div')
                        diagram['class'] = 'content-diagram'
                        diagram['style'] = 'border: 1px dashed #ccc; padding: 15px; text-align: center; margin: 15px 0;'
                        diagram.string = "Diagram visualization showing concept relationships (no concepts were detected in this content)"
                        selected_section.insert_after(diagram)
                
                # Return the modified HTML
                return str(soup)
            else:
                # Text-only content
                # Add emoji indicators for structure
                structured_content = []
                lines = content.split('\n')
                for line in lines:
                    if line.strip():
                        if line.strip()[0].isupper() and len(line.strip()) < 50:
                            # Likely a heading
                            structured_content.append(f"ðŸ“Œ {line}")
                        elif ":" in line and len(line.split(':')[0]) < 20:
                            # Likely a definition
                            structured_content.append(f"ðŸ“ {line}")
                        elif len(line) > 100:
                            # Long paragraph - add a visual separator
                            structured_content.append(f"{line}\n{'â”€' * 40}")
                        else:
                            structured_content.append(line)
                
                # Add visualization prompts
                visual_cues = [
                    "\nðŸ”„ Try to visualize this as a flowchart",
                    "\nðŸ“Š Imagine this data in a bar chart format",
                    "\nðŸ—ºï¸ Picture this as a concept map"
                ]
                
                # Add visual cues at strategic points
                enhanced_content = '\n'.join(structured_content)
                paragraphs = enhanced_content.split('\n\n')
                if len(paragraphs) > 1:
                    for i in range(1, len(paragraphs), 2):
                        if i < len(paragraphs):
                            paragraphs[i] += random.choice(visual_cues)
                    
                    enhanced_content = '\n\n'.join(paragraphs)
                
                return enhanced_content
        except ImportError:
            print("Warning: BeautifulSoup not available. Using simple visual enhancements.")
            return f"[Enhanced with visual elements]\n\n{content}\n\n[Visualization aids would appear here]"
        except Exception as e:
            print(f"Error during visual enhancement: {e}")
            return content
    
    def _enhance_textual_elements(self, content: str) -> str:
        """Add detailed explanations for textual learners"""
        try:
            from nltk.tokenize import sent_tokenize, word_tokenize
            import re
            
            # Break content into segments
            sentences = sent_tokenize(content)
            enhanced_sentences = []
            
            # Identify key concepts and add explanations
            key_concepts = {}
            for sentence in sentences:
                # Look for potential key concepts (capitalized multi-word phrases or technical terms)
                concept_matches = re.findall(r'\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\b', sentence)
                technical_matches = re.findall(r'\b([a-z]+(?:[-_][a-z]+)+)\b', sentence)
                
                for concept in concept_matches + technical_matches:
                    if concept not in key_concepts:
                        # Implement concept definition lookup from knowledge base
                        definition = self._lookup_concept_definition(concept)
                        key_concepts[concept] = definition
            
            # Enhance sentence structure for better comprehension
            for i, sentence in enumerate(sentences):
                enhanced_sentences.append(sentence)
                
                # Add explanatory notes for complex sentences
                words = word_tokenize(sentence)
                if len(words) > 20:
                    enhanced_sentences.append(f"To clarify: this means {' '.join(words[:10])}... and then {' '.join(words[-10:])}.")
                
                # Add elaborations for any key concepts in this sentence
                for concept, explanation in key_concepts.items():
                    if concept in sentence:
                        enhanced_sentences.append(f"Note: {concept} - {explanation}")
                
                # Add transitional phrases between groups of sentences
                if i > 0 and i % 3 == 0 and i < len(sentences) - 1:
                    transitional_phrases = [
                        "Building on these ideas,",
                        "To expand this concept further,",
                        "The following points elaborate on this:",
                        "Let's explore this in more depth:"
                    ]
                    import random
                    enhanced_sentences.append(random.choice(transitional_phrases))
            
            # Add summary sections
            if len(sentences) > 5:
                summary_sections = []
                
                # Create summary sections for every 5-7 sentences
                for i in range(0, len(sentences), 6):
                    section = sentences[i:i+6]
                    if section:
                        # Extract key phrases from the section
                        section_text = ' '.join(section)
                        key_phrases = re.findall(r'\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\b', section_text)
                        
                        if key_phrases:
                            # Generate a summary using key phrases (simulated)
                            unique_phrases = list(set(key_phrases))[:3]  # Take up to 3 unique phrases
                            summary = "Section Summary: This part discusses " + ", ".join(unique_phrases) + "."
                            summary_sections.append(summary)
                
                # Insert summaries at appropriate positions
                if summary_sections:
                    enhanced_content = ' '.join(enhanced_sentences)
                    paragraphs = enhanced_content.split('\n\n')
                    
                    if len(paragraphs) >= len(summary_sections):
                        # Insert each summary at the end of a paragraph
                        for i, summary in enumerate(summary_sections):
                            if i < len(paragraphs):
                                paragraphs[i] += "\n\n" + summary
                        
                        return '\n\n'.join(paragraphs)
            
            # Return enhanced content
            return ' '.join(enhanced_sentences)
            
        except ImportError:
            print("Warning: NLTK not available. Using simple text enhancements.")
            return f"{content}\n\nAdditional explanations and clarifications for textual learners (NLTK library required)"
        except Exception as e:
            print(f"Error during textual enhancement: {e}")
            return content
    
    def _add_interactive_elements(self, content: str) -> str:
        """Add interactive elements for interactive learners"""
        try:
            import re
            from nltk.tokenize import sent_tokenize
            
            sentences = sent_tokenize(content)
            
            # Generate interactive content components
            interactive_components = []
            
            # 1. Create check-your-understanding questions
            if len(sentences) >= 3:
                # Extract potential questions from content
                question_candidates = []
                
                for sentence in sentences:
                    # Look for sentences that contain facts or definitions
                    if re.search(r'\bis\b|\bare\b|\bwas\b|\bwere\b', sentence) and len(sentence) > 30:
                        # Turn statement into question
                        question = None
                        
                        # Extract subject-predicate structure for factual sentences
                        match = re.match(r'^([^,\.]+)\s+(is|are|was|were)\s+([^,\.]+)', sentence)
                        if match:
                            subject, verb, predicate = match.groups()
                            question = f"What {verb} {subject}?"
                            answer = predicate
                            question_candidates.append((question, answer, sentence))
                
                # Select a subset of questions to include
                if question_candidates:
                    import random
                    selected_questions = random.sample(question_candidates, min(3, len(question_candidates)))
                    
                    quiz_html = "<div class='interactive-quiz' style='background-color: #f9f9f9; padding: 15px; border-radius: 5px; margin: 20px 0;'>"
                    quiz_html += "<h3>ðŸ“ Check Your Understanding</h3>"
                    
                    for i, (question, answer, context) in enumerate(selected_questions, 1):
                        quiz_html += f"<div class='quiz-question' data-question-id='{i}'>"
                        quiz_html += f"<p><strong>Question {i}:</strong> {question}</p>"
                        quiz_html += "<div class='quiz-options'>"
                        
                        # Generate quiz options with the correct answer and distractors from context
                        options = [answer]
                        
                        # Generate distractors from other sentences
                        for s in random.sample(sentences, min(3, len(sentences))):
                            if s != context:
                                # Extract a phrase to use as a distractor
                                phrases = re.findall(r'([^,.;:]{10,40})[,.;:]', s)
                                if phrases:
                                    options.append(random.choice(phrases))
                        
                        # Fill in with generic distractors if needed
                        while len(options) < 4:
                            options.append(f"Not enough information to determine")
                        
                        # Shuffle options
                        random.shuffle(options)
                        
                        # Create radio buttons
                        for j, option in enumerate(options):
                            quiz_html += f"<div><input type='radio' name='q{i}' id='q{i}o{j}' value='{j}' onclick='checkQuizAnswer({i}, {j}, {options.index(answer)})'>"
                            quiz_html += f"<label for='q{i}o{j}'>{option}</label></div>"
                        
                        quiz_html += "</div>"
                        
                        # Add answer explanation - hidden initially, shown when answered
                        correct_index = options.index(answer)
                        quiz_html += f"<div id='answer-{i}' class='quiz-answer' style='display:none; margin-top: 10px; padding: 10px; border-radius: 4px;'>"
                        quiz_html += f"<p><span id='result-{i}'></span> The correct answer is: {answer}</p>"
                        quiz_html += f"<p>Explanation: This is derived from the text: \"{context}\"</p>"
                        quiz_html += "</div>"
                        
                        quiz_html += "</div>"
                    
                    # Replace button with JavaScript-enabled functionality
                    quiz_html += """
                    <div id='quiz-results' style='margin-top: 15px; font-weight: bold;'></div>
                    <script type="text/javascript">
                    let questionCount = {0};
                    let correctAnswers = 0;
                    let answeredQuestions = 0;
                    
                    function checkQuizAnswer(questionNum, selectedIndex, correctIndex) {
                        // Show the answer explanation
                        let answerDiv = document.getElementById('answer-' + questionNum);
                        let resultSpan = document.getElementById('result-' + questionNum);
                        
                        if (selectedIndex === correctIndex) {
                            // Correct answer
                            answerDiv.style.backgroundColor = '#DFF2BF';
                            resultSpan.innerHTML = 'âœ… Correct! ';
                            correctAnswers++;
                        } else {
                            // Incorrect answer
                            answerDiv.style.backgroundColor = '#FFBABA';
                            resultSpan.innerHTML = 'âŒ Incorrect. ';
                        }
                        
                        // Show the explanation
                        answerDiv.style.display = 'block';
                        
                        // Update the total if this is a new answer
                        let radios = document.getElementsByName('q' + questionNum);
                        let previouslyAnswered = false;
                        for (let i = 0; i < radios.length; i++) {
                            if (radios[i].hasAttribute('data-answered')) {
                                previouslyAnswered = true;
                                break;
                            }
                        }
                        
                        if (!previouslyAnswered) {
                            answeredQuestions++;
                            // Mark this question as answered
                            radios[selectedIndex].setAttribute('data-answered', 'true');
                        }
                        
                        // Update overall results
                        updateQuizResults();
                    }
                    
                    function updateQuizResults() {
                        let resultsDiv = document.getElementById('quiz-results');
                        if (answeredQuestions === {0}) {
                            let percentage = Math.round((correctAnswers / {0}) * 100);
                            let feedback = '';
                            
                            if (percentage >= 80) {
                                feedback = 'Excellent! You have a strong understanding of this material.';
                                resultsDiv.style.color = '#4F8A10';
                            } else if (percentage >= 60) {
                                feedback = 'Good job! You understand most of the key concepts.';
                                resultsDiv.style.color = '#9F6000';
                            } else {
                                feedback = 'You might want to review this section again to strengthen your understanding.';
                                resultsDiv.style.color = '#D8000C';
                            }
                            
                            resultsDiv.innerHTML = `Quiz Results: ${correctAnswers} out of {0} correct (${percentage}%). ${feedback}`;
                        } else {
                            resultsDiv.innerHTML = `Progress: ${answeredQuestions} of {0} questions answered. ${correctAnswers} correct so far.`;
                        }
                    }
                    </script>
                    """.format(len(selected_questions))
                    
                    quiz_html += "</div>"
                    
                    interactive_components.append(quiz_html)
            
            # 2. Add interactive exploration points
            concept_explorations = []
            
            # Extract potential key concepts
            key_terms = re.findall(r'\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\b', content)
            if key_terms:
                unique_terms = list(set(key_terms))
                selected_terms = unique_terms[:min(3, len(unique_terms))]
                
                for term in selected_terms:
                    # Implement real interactive exploration features
                    exploration_html = f"<div class='concept-explorer' style='margin: 15px 0; padding: 10px; border-left: 3px solid #3498db;'>"
                    exploration_html += f"<h4>ðŸ” Explore: {term}</h4>"
                    exploration_html += "<p>Click to interactively explore this concept:</p>"
                    exploration_html += "<div class='exploration-options'>"
                    exploration_html += f"<button class='explore-btn' onclick=\"showConceptInfo('{term}', 'definition')\" style='background-color: #3498db; color: white; border: none; padding: 5px 10px; margin-right: 5px; border-radius: 3px; cursor: pointer;'>View Definition</button> "
                    exploration_html += f"<button class='explore-btn' onclick=\"showConceptInfo('{term}', 'examples')\" style='background-color: #2ecc71; color: white; border: none; padding: 5px 10px; margin-right: 5px; border-radius: 3px; cursor: pointer;'>See Examples</button> "
                    exploration_html += f"<button class='explore-btn' onclick=\"showConceptInfo('{term}', 'related')\" style='background-color: #e74c3c; color: white; border: none; padding: 5px 10px; margin-right: 5px; border-radius: 3px; cursor: pointer;'>Related Concepts</button>"
                    exploration_html += "</div>"
                    exploration_html += f"<div class='exploration-content' id='explore-{term.replace(' ', '-')}' style='display:none; background-color: #f8f9fa; padding: 10px; margin-top: 10px; border-radius: 5px;'></div>"
                    
                    # Add preloaded definitions, examples, and related concepts in data attributes
                    definition = self._lookup_concept_definition(term)
                    examples = self._generate_concept_examples(term)
                    related = self._find_related_concepts(term, content)
                    
                    # Include as hidden data for JavaScript to use
                    escaped_definition = definition.replace("'", "&apos;")
                    escaped_examples = examples.replace("'", "&apos;")
                    escaped_related = related.replace("'", "&apos;")
                    
                    exploration_html += f"<div class='concept-data' style='display:none;' "
                    exploration_html += f"data-definition='{escaped_definition}' "
                    exploration_html += f"data-examples='{escaped_examples}' "
                    exploration_html += f"data-related='{escaped_related}' "
                    exploration_html += "></div>"
                    
                    exploration_html += "</div>"
                    
                    concept_explorations.append(exploration_html)
            
            # If we have interactive concept explorations, include necessary JavaScript
            if concept_explorations:
                js_code = """
                <script type="text/javascript">
                function showConceptInfo(term, infoType) {
                    // Find the concept's container
                    var conceptId = 'explore-' + term.replace(/ /g, '-');
                    var contentDiv = document.getElementById(conceptId);
                    var dataDiv = contentDiv.parentNode.querySelector('.concept-data');
                    
                    // Get the data
                    var data = '';
                    if (infoType === 'definition') {
                        data = dataDiv.getAttribute('data-definition');
                        contentDiv.innerHTML = '<h5>Definition</h5><p>' + data + '</p>';
                    } 
                    else if (infoType === 'examples') {
                        data = dataDiv.getAttribute('data-examples');
                        contentDiv.innerHTML = '<h5>Examples</h5><p>' + data + '</p>';
                    } 
                    else if (infoType === 'related') {
                        data = dataDiv.getAttribute('data-related');
                        contentDiv.innerHTML = '<h5>Related Concepts</h5><p>' + data + '</p>';
                    }
                    
                    // Show the content
                    contentDiv.style.display = 'block';
                }
                </script>
                """
                interactive_components.append(js_code)
            
            # 3. Add practical application exercises
            application_html = "<div class='practical-application' style='background-color: #f0f8ff; padding: 15px; border-radius: 5px; margin: 20px 0;'>"
            application_html += "<h3>ðŸ§ª Apply What You've Learned</h3>"
            application_html += "<p>Complete this exercise to reinforce your understanding:</p>"
            
            # Generate exercise based on content (simplified simulation)
            application_html += "<div class='exercise-description'>"
            application_html += "<p>Based on the concepts in this material, try to:</p>"
            application_html += "<ol>"
            
            # Extract actionable items from content
            application_html += "<li>Create a summary diagram showing the relationships between key concepts</li>"
            application_html += "<li>Write a brief explanation of how you would apply these ideas in a real-world scenario</li>"
            application_html += "<li>Identify potential challenges or limitations in implementing these concepts</li>"
            application_html += "</ol>"
            
            application_html += "</div>"
            
            # Add response area with interactive functionality
            application_html += "<div class='response-area'>"
            application_html += "<textarea id='exercise-response' placeholder='Type your response here...' rows='4' style='width: 100%; padding: 8px;'></textarea>"
            application_html += "<button onclick='submitResponse()' style='background-color: #3498db; color: white; padding: 8px 12px; border: none; border-radius: 4px; cursor: pointer; margin-top: 10px;'>Submit Response</button>"
            application_html += "<div id='feedback-area' style='display:none; margin-top: 15px; padding: 10px; border-radius: 5px;'></div>"
            application_html += "</div>"
            
            application_html += "</div>"
            
            interactive_components.append(application_html)
            
            # Add JavaScript for exercise response handling
            exercise_js = """
            <script type="text/javascript">
            function submitResponse() {
                // Get the response text
                var response = document.getElementById('exercise-response').value;
                var feedbackArea = document.getElementById('feedback-area');
                
                // Validate response
                if (response.trim().length < 10) {
                    feedbackArea.style.display = 'block';
                    feedbackArea.style.backgroundColor = '#FFEDED';
                    feedbackArea.innerHTML = '<p style="color: #D8000C;"><strong>Please provide a more detailed response.</strong> Your answer should address the exercise questions.</p>';
                    return;
                }
                
                // Process response (in a production system, this would send data to server)
                // Here we provide immediate feedback based on response content
                
                // Simple analysis of response content
                var keywordCheck = analyzeResponse(response);
                var feedbackText = '';
                
                if (keywordCheck.score > 3) {
                    feedbackArea.style.backgroundColor = '#DFF2BF';
                    feedbackText = '<p style="color: #4F8A10;"><strong>Great job!</strong> Your response demonstrates understanding of the key concepts.</p>';
                    if (keywordCheck.suggestions.length > 0) {
                        feedbackText += '<p>Consider exploring these related ideas: ' + keywordCheck.suggestions.join(', ') + '</p>';
                    }
                } else {
                    feedbackArea.style.backgroundColor = '#FEEFB3';
                    feedbackText = '<p style="color: #9F6000;"><strong>Good start.</strong> Consider adding more specific details about how these concepts relate to real-world applications.</p>';
                    if (keywordCheck.missingKeywords.length > 0) {
                        feedbackText += '<p>Try incorporating these important elements: ' + keywordCheck.missingKeywords.join(', ') + '</p>';
                    }
                }
                
                feedbackArea.style.display = 'block';
                feedbackArea.innerHTML = feedbackText;
                
                // Store the response in the learning record and update the student's profile
                storeExerciseResponse(response, keywordCheck.score);
            }
            
            function storeExerciseResponse(response, score) {
                // Create data to send to server
                var responseData = {
                    response: response,
                    score: score,
                    timestamp: new Date().toISOString(),
                    exercise_id: "ex_" + window.location.pathname.replace(/[^a-zA-Z0-9]/g, "_"),
                    user_id: getUserId()
                };
                
                // Store locally in browser storage for persistence
                try {
                    // Get existing responses
                    var storedResponses = localStorage.getItem('exercise_responses');
                    var responses = storedResponses ? JSON.parse(storedResponses) : [];
                    
                    // Add new response
                    responses.push(responseData);
                    
                    // Store updated array
                    localStorage.setItem('exercise_responses', JSON.stringify(responses));
                    
                    console.log("Response saved locally");
                } catch (e) {
                    console.error("Error saving response locally:", e);
                }
                
                // In a production environment, this would also send data to the server
                // Here we simulate the call
                console.log("Exercise response would be sent to server:", responseData);
                
                // Simulated server call
                if (window.fetch) {
                    // Only try if fetch API is available
                    setTimeout(function() {
                        // This is a simulated endpoint - in a real application, replace with actual endpoint
                        // var endpoint = "/api/learning/exercise-response";
                        // fetch(endpoint, {
                        //     method: 'POST',
                        //     headers: { 'Content-Type': 'application/json' },
                        //     body: JSON.stringify(responseData)
                        // });
                        
                        console.log("Response data would be sent to learning record store");
                    }, 100);
                }
            }
            
            function getUserId() {
                // In a real app, this would get the user ID from the authentication system
                var storedId = localStorage.getItem('user_id');
                if (storedId) {
                    return storedId;
                }
                
                // Generate a temporary ID if none exists
                var tempId = 'temp_' + Math.random().toString(36).substring(2, 15);
                localStorage.setItem('user_id', tempId);
                return tempId;
            }
            
            function analyzeResponse(text) {
                // This is a simplified analysis - in a real system this would be more sophisticated
                var keywords = ['concept', 'example', 'application', 'problem', 'solution', 'analysis', 
                                'implementation', 'process', 'method', 'relationship', 'system'];
                var advancedKeywords = ['evaluation', 'synthesis', 'critique', 'integration', 'assessment',
                                        'framework', 'theoretical', 'methodology', 'paradigm'];
                                        
                var found = [];
                var missing = [];
                var suggestions = ['critical analysis', 'practical implementation steps', 'comparative evaluation'];
                
                // Count occurrences of keywords
                for (var i = 0; i < keywords.length; i++) {
                    if (text.toLowerCase().indexOf(keywords[i]) !== -1) {
                        found.push(keywords[i]);
                    } else {
                        missing.push(keywords[i]);
                    }
                }
                
                // Check for advanced keywords
                for (var i = 0; i < advancedKeywords.length; i++) {
                    if (text.toLowerCase().indexOf(advancedKeywords[i]) !== -1) {
                        found.push(advancedKeywords[i]);
                    }
                }
                
                // Limit the missing keywords to just a few suggestions
                missing = missing.slice(0, 3);
                
                return {
                    score: found.length,
                    foundKeywords: found,
                    missingKeywords: missing,
                    suggestions: suggestions
                };
            }
            </script>
            """
            
            interactive_components.append(exercise_js)
            
            # Place interactive elements at semantically appropriate positions
            enhanced_content = self._insert_interactive_components_semantically(content, interactive_components)
            
            return enhanced_content
            
        except ImportError:
            print("Warning: Required libraries not available. Using simple interactive enhancements.")
            return f"{content}\n\n[Interactive elements would be added here for hands-on learners]"
        except Exception as e:
            print(f"Error during interactive enhancement: {e}")
            return content
        
    def _save_processed_document(self, document_id: str, document: Dict[str, Any]):
        """Save processed document to storage"""
        document_path = os.path.join(self.storage_dir, "documents", f"{document_id}.json")
        
        with open(document_path, 'w') as f:
            json.dump(document, f, indent=2)
            
    def _load_processed_document(self, document_id: str) -> Optional[Dict[str, Any]]:
        """Load processed document from storage"""
        document_path = os.path.join(self.storage_dir, "documents", f"{document_id}.json")
        
        if not os.path.exists(document_path):
            return None
            
        try:
            with open(document_path, 'r') as f:
                return json.load(f)
        except Exception as e:
            print(f"Error loading document {document_id}: {e}")
            return None
            
    def _save_session(self, session_id: str, session: PersonalizedLearningSession):
        """Save learning session to storage"""
        session_path = os.path.join(self.storage_dir, "sessions", f"{session_id}.json")
        
        with open(session_path, 'w') as f:
            json.dump(session.to_dict(), f, indent=2)
            
    def _get_session(self, session_id: str) -> Optional[PersonalizedLearningSession]:
        """Get learning session (from active sessions or storage)"""
        # Check active sessions first
        if session_id in self.active_sessions:
            return self.active_sessions[session_id]
            
        # Try to load from storage
        session_path = os.path.join(self.storage_dir, "sessions", f"{session_id}.json")
        
        if not os.path.exists(session_path):
            return None
            
        try:
            with open(session_path, 'r') as f:
                session_data = json.load(f)
                session = PersonalizedLearningSession.from_dict(session_data)
                # Add to active sessions
                self.active_sessions[session_id] = session
                return session
        except Exception as e:
            print(f"Error loading session {session_id}: {e}")
            return None
            
    def _generate_concept_diagram(self, concepts: List[str]) -> str:
        """
        Generate an SVG diagram showing relationships between concepts
        
        Args:
            concepts: List of key concepts to include in the diagram
            
        Returns:
            SVG code as a string
        """
        import random
        import math
        
        # Set diagram dimensions
        width, height = 600, 400
        center_x, center_y = width // 2, height // 2
        
        # Define colors for variety
        node_colors = [
            "#3498db", "#e74c3c", "#2ecc71", "#f39c12", "#9b59b6",
            "#1abc9c", "#d35400", "#34495e", "#16a085", "#c0392b"
        ]
        
        # Begin SVG
        svg = f'<svg width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg">\n'
        
        # Add definition for arrow markers
        svg += '''
        <defs>
            <marker id="arrowhead" markerWidth="10" markerHeight="7" refX="9" refY="3.5" orient="auto">
                <polygon points="0 0, 10 3.5, 0 7" fill="#333" />
            </marker>
        </defs>
        '''
        
        # Add a light grid for better readability
        svg += f'<rect width="{width}" height="{height}" fill="#f9f9f9" />\n'
        svg += '<g stroke="#eee" stroke-width="1">\n'
        for i in range(0, width, 50):
            svg += f'<line x1="{i}" y1="0" x2="{i}" y2="{height}" />\n'
        for i in range(0, height, 50):
            svg += f'<line x1="0" y1="{i}" x2="{width}" y2="{i}" />\n'
        svg += '</g>\n'
        
        # Main concept in the center
        main_concept = concepts[0] if concepts else "Main Concept"
        main_radius = 60
        
        # Calculate positions for the surrounding concepts
        positions = []
        if len(concepts) > 1:
            radius = min(center_x, center_y) - 80  # Distance from center
            angle_step = 2 * math.pi / (len(concepts) - 1)
            
            for i in range(1, len(concepts)):
                angle = angle_step * (i - 1)
                x = center_x + radius * math.cos(angle)
                y = center_y + radius * math.sin(angle)
                positions.append((x, y))
        
        # Generate relationship types
        relationship_types = [
            "relates to", "influences", "depends on", "categorizes", 
            "defines", "extends", "part of", "type of", "used in"
        ]
        
        # Draw connections between concepts first (to be behind nodes)
        if positions:
            for i, (x, y) in enumerate(positions):
                # Draw connection line
                svg += f'<line x1="{center_x}" y1="{center_y}" x2="{x}" y2="{y}" '
                svg += 'stroke="#666" stroke-width="2" stroke-dasharray="5,3" '
                svg += 'marker-end="url(#arrowhead)" />\n'
                
                # Add relationship label
                rel_x = center_x + (x - center_x) * 0.5
                rel_y = center_y + (y - center_y) * 0.5
                relationship = random.choice(relationship_types)
                
                # Create background for better readability
                svg += f'<rect x="{rel_x-40}" y="{rel_y-15}" width="80" height="20" '
                svg += 'fill="white" fill-opacity="0.8" rx="5" ry="5" />\n'
                
                svg += f'<text x="{rel_x}" y="{rel_y}" text-anchor="middle" '
                svg += 'font-family="Arial" font-size="12" fill="#333">{relationship}</text>\n'
        
        # Draw nodes
        # Main node in center
        svg += f'<circle cx="{center_x}" cy="{center_y}" r="{main_radius}" '
        svg += f'fill="{node_colors[0]}" stroke="#333" stroke-width="2" />\n'
        
        # Add text to main node (may need to wrap long text)
        if len(main_concept) > 15:
            words = main_concept.split()
            if len(words) > 1:
                line1 = " ".join(words[:len(words)//2])
                line2 = " ".join(words[len(words)//2:])
                svg += f'<text x="{center_x}" y="{center_y-8}" text-anchor="middle" '
                svg += 'font-family="Arial" font-weight="bold" font-size="14" fill="white">{line1}</text>\n'
                svg += f'<text x="{center_x}" y="{center_y+12}" text-anchor="middle" '
                svg += 'font-family="Arial" font-weight="bold" font-size="14" fill="white">{line2}</text>\n'
            else:
                # Single long word - just use smaller font
                svg += f'<text x="{center_x}" y="{center_y+5}" text-anchor="middle" '
                svg += 'font-family="Arial" font-weight="bold" font-size="12" fill="white">{main_concept}</text>\n'
        else:
            svg += f'<text x="{center_x}" y="{center_y+5}" text-anchor="middle" '
            svg += 'font-family="Arial" font-weight="bold" font-size="14" fill="white">{main_concept}</text>\n'
        
        # Draw surrounding nodes
        for i, (x, y) in enumerate(positions):
            concept = concepts[i+1]
            node_radius = 45
            
            # Node circle with color
            svg += f'<circle cx="{x}" cy="{y}" r="{node_radius}" '
            svg += f'fill="{node_colors[(i+1) % len(node_colors)]}" stroke="#333" stroke-width="2" />\n'
            
            # Add text to node (handle long text)
            if len(concept) > 12:
                words = concept.split()
                if len(words) > 1:
                    line1 = " ".join(words[:len(words)//2])
                    line2 = " ".join(words[len(words)//2:])
                    svg += f'<text x="{x}" y="{y-8}" text-anchor="middle" '
                    svg += 'font-family="Arial" font-size="12" fill="white">{line1}</text>\n'
                    svg += f'<text x="{x}" y="{y+12}" text-anchor="middle" '
                    svg += 'font-family="Arial" font-size="12" fill="white">{line2}</text>\n'
                else:
                    # Single long word
                    svg += f'<text x="{x}" y="{y+5}" text-anchor="middle" '
                    svg += 'font-family="Arial" font-size="10" fill="white">{concept}</text>\n'
            else:
                svg += f'<text x="{x}" y="{y+5}" text-anchor="middle" '
                svg += 'font-family="Arial" font-size="12" fill="white">{concept}</text>\n'
        
        # End SVG
        svg += '</svg>'
        
        return svg 

    def _lookup_concept_definition(self, concept: str) -> str:
        """
        Look up a concept definition in the knowledge base
        
        Args:
            concept: The concept term to look up
            
        Returns:
            Definition of the concept
        """
        try:
            # First try to load from local knowledge base if available
            kb_path = os.path.join(self.storage_dir, "knowledge_base.json")
            
            if os.path.exists(kb_path):
                with open(kb_path, 'r') as f:
                    knowledge_base = json.load(f)
                
                # Try exact match
                if concept in knowledge_base:
                    return knowledge_base[concept]
                    
                # Try case-insensitive match
                for term, definition in knowledge_base.items():
                    if term.lower() == concept.lower():
                        return definition
                        
                # Try substring match (for multi-word concepts or variations)
                matches = []
                for term, definition in knowledge_base.items():
                    if concept.lower() in term.lower() or term.lower() in concept.lower():
                        matches.append((term, definition))
                
                if matches:
                    # Choose the closest length match
                    closest_match = min(matches, key=lambda x: abs(len(x[0]) - len(concept)))
                    return closest_match[1]
            
            # If local knowledge base fails, try external knowledge API
            definition = self._query_external_knowledge_api(concept)
            if definition:
                return definition
                
            # If all methods fail, query WordNet lexical database
            from nltk.corpus import wordnet
            synsets = wordnet.synsets(concept)
            
            if synsets:
                # Get the most common definition
                definition = synsets[0].definition()
                # Capitalize first letter
                return definition[0].upper() + definition[1:] + f" (WordNet definition for '{concept}')"
            
            # If all lookups fail, generate a generic definition
            return f"This refers to {concept}, an important concept in this domain that relates to fundamental principles."
            
        except Exception as e:
            print(f"Error looking up concept definition: {e}")
            return f"This refers to {concept}, a key concept in this material."
            
    def _query_external_knowledge_api(self, concept: str) -> Optional[str]:
        """
        Query external knowledge APIs for definitions
        
        Args:
            concept: Concept to query
            
        Returns:
            Definition string or None if not found
        """
        try:
            import requests
            
            # Prepare concept for URL (replace spaces with proper encoding)
            url_concept = concept.replace(' ', '%20')
            
            # Try multiple APIs for definitions
            
            # 1. Try Wikipedia API
            wiki_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{url_concept}"
            
            try:
                response = requests.get(wiki_url, timeout=3)
                if response.status_code == 200:
                    data = response.json()
                    if 'extract' in data and len(data['extract']) > 20:
                        # Return first two sentences
                        import re
                        sentences = re.split(r'(?<=[.!?])\s+', data['extract'])
                        if len(sentences) > 1:
                            return sentences[0] + " " + sentences[1] + f" (Wikipedia summary for '{concept}')"
                        else:
                            return data['extract'] + f" (Wikipedia summary for '{concept}')"
            except:
                pass  # Silently fail and try next API
                
            # 2. Try Wiktionary API
            wikt_url = f"https://en.wiktionary.org/api/rest_v1/page/definition/{url_concept}"
            
            try:
                response = requests.get(wikt_url, timeout=3)
                if response.status_code == 200:
                    data = response.json()
                    if 'en' in data and len(data['en']) > 0:
                        definitions = data['en']
                        for definition in definitions:
                            if 'definitions' in definition and len(definition['definitions']) > 0:
                                return definition['definitions'][0]['definition'] + f" (Wiktionary definition for '{concept}')"
            except:
                pass  # Silently fail
                
            # 3. For academic/scientific domains, try specialized domain-specific APIs
            if self.domain in ["mathematics", "physics", "chemistry", "biology", "computer_science"]:
                # This would connect to specialized APIs for scientific domains
                # Simulated as not implemented in this prototype
                pass
                
            return None
            
        except Exception as e:
            print(f"Error querying external knowledge API: {e}")
            return None 

    def _generate_concept_examples(self, concept: str) -> str:
        """
        Generate examples for a given concept
        
        Args:
            concept: The concept to generate examples for
            
        Returns:
            HTML string with examples
        """
        try:
            # Check for domain-specific examples first
            examples = []
            
            if self.domain:
                # Load domain-specific examples if available
                examples_path = os.path.join(self.storage_dir, f"{self.domain}_examples.json")
                
                if os.path.exists(examples_path):
                    with open(examples_path, 'r') as f:
                        domain_examples = json.load(f)
                        
                    # Try exact match first
                    if concept in domain_examples:
                        examples = domain_examples[concept]
                    else:
                        # Try case-insensitive
                        for term, term_examples in domain_examples.items():
                            if term.lower() == concept.lower():
                                examples = term_examples
                                break
            
            # If no domain examples found, generate general examples
            if not examples:
                # Try to determine the concept type
                if concept[0].isupper() and ' ' not in concept:
                    # Likely a proper noun or named entity
                    examples = self._generate_entity_examples(concept)
                elif any(kw in concept.lower() for kw in ['algorithm', 'method', 'process', 'technique']):
                    # Likely a process or algorithm
                    examples = self._generate_algorithm_examples(concept)
                elif any(kw in concept.lower() for kw in ['theory', 'principle', 'law', 'equation']):
                    # Likely a theory or principle
                    examples = self._generate_theory_examples(concept)
                else:
                    # Generate general examples
                    examples = [
                        f"Example 1: Application of {concept} in a practical learning scenario.",
                        f"Example 2: How {concept} is used in problem-solving.",
                        f"Example 3: Real-world demonstration of {concept} principles."
                    ]
                    
                    # Try to add a code example for technical concepts
                    if any(kw in concept.lower() for kw in ['code', 'programming', 'function', 'method', 'class', 'variable']):
                        code_example = self._generate_code_example(concept)
                        examples.append(f"Code example:<br><pre>{code_example}</pre>")
            
            # Format the examples as HTML
            html_examples = "<ul>"
            for example in examples[:3]:  # Limit to 3 examples
                html_examples += f"<li>{example}</li>"
            html_examples += "</ul>"
            
            return html_examples
            
        except Exception as e:
            print(f"Error generating concept examples: {e}")
            return f"<p>Examples of {concept} would be shown here.</p>"
            
    def _generate_entity_examples(self, entity: str) -> List[str]:
        """Generate examples for a named entity"""
        return [
            f"{entity} is frequently referenced in this domain as a key contributor or concept.",
            f"The work of {entity} has influenced many subsequent developments in this field.",
            f"{entity} can be applied to solve problems in various contexts."
        ]
        
    def _generate_algorithm_examples(self, algorithm: str) -> List[str]:
        """Generate examples for an algorithm or process"""
        return [
            f"The {algorithm} can be used to efficiently solve optimization problems.",
            f"In a typical implementation, {algorithm} would process inputs in the following sequence: initialization, processing, and output generation.",
            f"When facing complex data structures, {algorithm} offers significant advantages over brute force approaches."
        ]
        
    def _generate_theory_examples(self, theory: str) -> List[str]:
        """Generate examples for a theory or principle"""
        return [
            f"The {theory} explains observed phenomena such as [relevant examples in the domain].",
            f"Scientists have validated {theory} through multiple experiments, including [example experiment].",
            f"A practical application of {theory} can be seen in [real-world example]."
        ]
        
    def _generate_code_example(self, concept: str) -> str:
        """Generate a simple code example for a technical concept"""
        concept_lower = concept.lower()
        
        if 'function' in concept_lower or 'method' in concept_lower:
            return f"def implement_{concept_lower.replace(' ', '_')}(input_data):\n    # Process the input data\n    result = analyze_data(input_data)\n    \n    # Apply {concept} principles\n    optimized_result = optimize_output(result)\n    \n    return optimized_result"
        elif 'class' in concept_lower:
            return f"class {concept.replace(' ', '')}:\n    def __init__(self, parameter1, parameter2):\n        self.param1 = parameter1\n        self.param2 = parameter2\n        \n    def process_data(self, input_data):\n        # Implementation of {concept}\n        return transformed_data"
        else:
            return f"# Example implementation of {concept}\n\ndef apply_{concept_lower.replace(' ', '_')}(data):\n    # Step 1: Prepare the data\n    prepared_data = preprocess(data)\n    \n    # Step 2: Apply the {concept} technique\n    result = compute_result(prepared_data)\n    \n    return result"
            
    def _find_related_concepts(self, concept: str, content: str) -> str:
        """
        Find concepts related to the given concept in the content
        
        Args:
            concept: The main concept
            content: The content to search for related concepts
            
        Returns:
            HTML string with related concepts
        """
        try:
            import re
            from nltk.tokenize import word_tokenize
            from nltk.corpus import wordnet
            
            related_concepts = set()
            
            # Method 1: Extract capitalized terms from the content
            # Find all capitalized terms (potential concepts)
            all_concepts = re.findall(r'\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\b', content)
            
            # Filter out the main concept
            all_concepts = [c for c in all_concepts if c != concept]
            
            # Look for co-occurrence (concepts that appear near the main concept)
            content_sentences = content.split('.')
            for sentence in content_sentences:
                if concept in sentence:
                    # Extract other concepts from the same sentence
                    for other_concept in all_concepts:
                        if other_concept in sentence and other_concept != concept:
                            related_concepts.add(other_concept)
            
            # Method 2: Use WordNet for synonyms and related terms
            try:
                # Get WordNet synsets
                synsets = wordnet.synsets(concept.lower())
                
                if synsets:
                    # Get synonyms
                    for synset in synsets[:2]:  # Limit to first 2 meanings
                        # Add synonyms
                        for lemma in synset.lemmas():
                            if lemma.name() != concept.lower():
                                # Capitalize properly
                                synonym = lemma.name().replace('_', ' ').title()
                                related_concepts.add(synonym)
                        
                        # Add hypernyms (more general terms)
                        for hypernym in synset.hypernyms():
                            hyper_name = hypernym.name().split('.')[0].replace('_', ' ').title()
                            related_concepts.add(hyper_name)
                        
                        # Add hyponyms (more specific terms)
                        for hyponym in synset.hyponyms():
                            hypo_name = hyponym.name().split('.')[0].replace('_', ' ').title()
                            related_concepts.add(hypo_name)
            except:
                pass  # WordNet might not be available
            
            # Method 3: Check domain-specific concept relationships
            # Load domain-specific relationships if available
            if self.domain:
                relations_path = os.path.join(self.storage_dir, f"{self.domain}_relations.json")
                
                if os.path.exists(relations_path):
                    with open(relations_path, 'r') as f:
                        domain_relations = json.load(f)
                        
                    # Check for concept relationships
                    if concept in domain_relations:
                        for related in domain_relations[concept]:
                            related_concepts.add(related)
                            
            # Format related concepts as HTML
            if related_concepts:
                related_html = "<ul>"
                # Limit to 5 related concepts
                for related in list(related_concepts)[:5]:
                    related_html += f"<li>{related}</li>"
                related_html += "</ul>"
                
                return related_html
            else:
                return f"<p>No direct relationships found for {concept} in this content.</p>"
                
        except Exception as e:
            print(f"Error finding related concepts: {e}")
            return f"<p>Related concepts for {concept} would be shown here.</p>" 

    def _insert_interactive_components_semantically(self, content: str, components: List[str]) -> str:
        """
        Insert interactive components into the content at semantically appropriate positions
        
        Args:
            content: The original content
            components: List of interactive components to insert
            
        Returns:
            Modified content with interactive components inserted
        """
        try:
            # Handle HTML content differently from plain text
            if "<" in content and ">" in content:
                # This is HTML content - use more sophisticated insertion
                from bs4 import BeautifulSoup
                import re
                
                # Parse HTML
                soup = BeautifulSoup(content, 'html.parser')
                
                # If we have a body tag, work with that
                body = soup.find('body')
                
                if not body:
                    # No body tag, treat the whole document as body
                    body = soup
                
                # Get all paragraph and header elements for potential insertion points
                elements = body.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'section', 'div'])
                
                if elements:
                    # We have at least one component to insert
                    if components:
                        # Find good insertion points based on content sections
                        
                        # 1. Find quiz/question component (typically first in components list)
                        quiz_component = next((c for c in components if 'interactive-quiz' in c), None)
                        if quiz_component and len(elements) > 3:
                            # Place quiz after approximately 2/3 of the content - good for testing comprehension
                            quiz_index = min(int(len(elements) * 0.7), len(elements) - 1)
                            # Create and insert component
                            quiz_soup = BeautifulSoup(quiz_component, 'html.parser')
                            elements[quiz_index].insert_after(quiz_soup)
                            # Remove it from components to insert
                            components = [c for c in components if c != quiz_component]
                            
                        # 2. Find concept exploration components
                        concept_components = [c for c in components if 'concept-explorer' in c]
                        if concept_components:
                            # Extract headings for potential concept matching
                            headings = body.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
                            heading_texts = [h.get_text().strip() for h in headings]
                            
                            for comp in concept_components:
                                # Try to find a related heading to place this under
                                soup_comp = BeautifulSoup(comp, 'html.parser')
                                concept_title = soup_comp.find('h4')
                                
                                if concept_title:
                                    concept_name = concept_title.get_text().replace('ðŸ” Explore: ', '')
                                    
                                    # Find best matching heading
                                    best_match = None
                                    best_match_score = 0
                                    for i, heading_text in enumerate(heading_texts):
                                        if concept_name.lower() in heading_text.lower():
                                            # Direct match - place component right after this heading
                                            headings[i].insert_after(soup_comp)
                                            best_match = headings[i]
                                            break
                                        else:
                                            # Check for partial word overlap
                                            concept_words = re.findall(r'\w+', concept_name.lower())
                                            heading_words = re.findall(r'\w+', heading_text.lower())
                                            common_words = set(concept_words) & set(heading_words)
                                            score = len(common_words)
                                            if score > best_match_score:
                                                best_match_score = score
                                                best_match = headings[i]
                                    
                                    # If we found a match but didn't insert yet, insert now
                                    if best_match and best_match_score > 0 and not soup_comp.parent:
                                        best_match.insert_after(soup_comp)
                                
                                # If we couldn't place it semantically, choose a position in the middle
                                if not soup_comp.parent and len(elements) > 2:
                                    mid_point = len(elements) // 2
                                    elements[mid_point].insert_after(soup_comp)
                            
                            # Remove these from components to insert
                            components = [c for c in components if 'concept-explorer' not in c]
                        
                        # 3. Place application exercises near the end
                        application_component = next((c for c in components if 'practical-application' in c), None)
                        if application_component and len(elements) > 2:
                            # Place application component near the end
                            app_soup = BeautifulSoup(application_component, 'html.parser')
                            elements[-1].insert_after(app_soup)
                            # Remove it from components to insert
                            components = [c for c in components if c != application_component]
                        
                        # 4. Add any remaining JavaScript components at the very end
                        js_components = [c for c in components if '<script' in c]
                        if js_components:
                            # Add script elements at the end of the body
                            script_soup = BeautifulSoup("".join(js_components), 'html.parser')
                            if body.contents:
                                body.contents[-1].insert_after(script_soup)
                            else:
                                body.append(script_soup)
                            
                            # Remove these from components to insert
                            components = [c for c in components if c not in js_components]
                        
                        # 5. If any components remain, add them at the end
                        if components:
                            remaining_soup = BeautifulSoup("".join(components), 'html.parser')
                            if body.contents:
                                body.contents[-1].insert_after(remaining_soup)
                            else:
                                body.append(remaining_soup)
                    
                    return str(soup)
                else:
                    # No proper elements found, append components to the end
                    if "</body>" in content:
                        return content.replace("</body>", "".join(components) + "</body>")
                    else:
                        return content + "".join(components)
            else:
                # Plain text content - try to find logical sections
                paragraphs = re.split(r'\n\s*\n', content)
                
                if len(paragraphs) > 1 and components:
                    # Have multiple paragraphs to work with
                    
                    # Choose insertion positions
                    num_paragraphs = len(paragraphs)
                    
                    # Quiz after approximately 2/3 of content
                    quiz_component = next((c for c in components if 'interactive-quiz' in c), None)
                    if quiz_component and num_paragraphs > 3:
                        quiz_pos = min(int(num_paragraphs * 0.7), num_paragraphs - 1)
                        paragraphs.insert(quiz_pos, quiz_component)
                        components = [c for c in components if c != quiz_component]
                    
                    # Concept explorations distributed through middle sections
                    concept_components = [c for c in components if 'concept-explorer' in c]
                    if concept_components and num_paragraphs > len(concept_components):
                        # Calculate even spacing
                        middle_section = paragraphs[1:-1]  # Exclude first and last paragraph
                        if middle_section:
                            step = max(1, len(middle_section) // (len(concept_components) + 1))
                            for i, comp in enumerate(concept_components):
                                pos = min(1 + (i + 1) * step, num_paragraphs)
                                paragraphs.insert(pos, comp)
                                # Adjust paragraph count for next insertion
                                num_paragraphs += 1
                        else:
                            # Just add to the end if not enough paragraphs
                            paragraphs.extend(concept_components)
                        
                        components = [c for c in components if c not in concept_components]
                    
                    # Application exercises near the end
                    application_component = next((c for c in components if 'practical-application' in c), None)
                    if application_component:
                        paragraphs.append(application_component)
                        components = [c for c in components if c != application_component]
                    
                    # Add any remaining components at the end
                    if components:
                        paragraphs.extend(components)
                    
                    return "\n\n".join(paragraphs)
                else:
                    # Not enough paragraphs to distribute - just append
                    return content + "\n\n" + "\n\n".join(components)
                
        except Exception as e:
            print(f"Error during semantic component insertion: {e}")
            # Fallback to simple appending
            if "<" in content and ">" in content:
                if "</body>" in content:
                    return content.replace("</body>", "".join(components) + "</body>")
                else:
                    return content + "".join(components)
            else:
                return content + "\n\n" + "\n\n".join(components) 

---


## AdaptiveLearningPathway.py

In [ ]:
"""
Adaptive Learning Pathway Generator
----------------------------------
Generates personalized learning pathways based on:
- Student profile and learning history
- Document content and structure
- Knowledge prerequisites and relationships
"""

import json
from typing import Dict, List, Any, Optional, Tuple
from pipeline.adaptive_learning.StudentProfileManager import StudentProfile
from pipeline.content_analysis.ContentRelationshipAnalyzer import ContentRelationshipAnalyzer

class LearningPathwayNode:
    """Represents a node in a learning pathway"""
    
    def __init__(self, content_id: str, title: str, content_type: str, 
                difficulty: str = "standard", prerequisites: List[str] = None,
                estimated_time: int = 0):
        """
        Initialize a learning pathway node
        
        Args:
            content_id: Unique identifier for the content
            title: Title of the content
            content_type: Type of content (text, video, quiz, etc.)
            difficulty: Difficulty level (simple, standard, advanced)
            prerequisites: List of prerequisite content IDs
            estimated_time: Estimated time to complete in minutes
        """
        self.content_id = content_id
        self.title = title
        self.content_type = content_type
        self.difficulty = difficulty
        self.prerequisites = prerequisites or []
        self.estimated_time = estimated_time
        self.alternatives = []  # Alternative content covering same topics
        
    def add_alternative(self, alt_content_id: str, alt_title: str, 
                      alt_content_type: str, alt_difficulty: str):
        """
        Add an alternative version of this content
        
        Args:
            alt_content_id: ID of the alternative content
            alt_title: Title of the alternative
            alt_content_type: Type of alternative content
            alt_difficulty: Difficulty of alternative content
        """
        self.alternatives.append({
            "content_id": alt_content_id,
            "title": alt_title,
            "content_type": alt_content_type,
            "difficulty": alt_difficulty
        })
        
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary representation"""
        return {
            "content_id": self.content_id,
            "title": self.title,
            "content_type": self.content_type,
            "difficulty": self.difficulty,
            "prerequisites": self.prerequisites,
            "estimated_time": self.estimated_time,
            "alternatives": self.alternatives
        }
        
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'LearningPathwayNode':
        """Create from dictionary representation"""
        node = cls(
            content_id=data["content_id"],
            title=data["title"],
            content_type=data["content_type"],
            difficulty=data.get("difficulty", "standard"),
            prerequisites=data.get("prerequisites", []),
            estimated_time=data.get("estimated_time", 0)
        )
        node.alternatives = data.get("alternatives", [])
        return node

class AdaptiveLearningPathway:
    """Represents a personalized learning pathway"""
    
    def __init__(self, student_id: str, title: str = "", description: str = ""):
        """
        Initialize a learning pathway
        
        Args:
            student_id: ID of the student this pathway is for
            title: Title of the learning pathway
            description: Description of the pathway
        """
        self.student_id = student_id
        self.title = title
        self.description = description
        self.nodes = []  # Learning content nodes
        self.edges = []  # Connections between nodes
        self.progress = {}  # Node ID â†’ completion percentage
        
    def add_node(self, node: LearningPathwayNode) -> str:
        """
        Add a content node to the pathway
        
        Args:
            node: The learning content node
            
        Returns:
            ID of the added node
        """
        self.nodes.append(node)
        self.progress[node.content_id] = 0
        return node.content_id
        
    def add_edge(self, from_node: str, to_node: str, edge_type: str = "next"):
        """
        Add a directed edge between nodes
        
        Args:
            from_node: ID of the source node
            to_node: ID of the target node
            edge_type: Type of connection (next, optional, prerequisite)
        """
        self.edges.append({
            "from": from_node,
            "to": to_node,
            "type": edge_type
        })
        
    def update_progress(self, node_id: str, completion_percentage: float):
        """
        Update progress for a specific node
        
        Args:
            node_id: ID of the node
            completion_percentage: Percentage completed (0-100)
        """
        if node_id in self.progress:
            self.progress[node_id] = min(100, max(0, completion_percentage))
            
    def get_next_nodes(self) -> List[LearningPathwayNode]:
        """
        Get the next recommended nodes based on current progress
        
        Returns:
            List of next recommended nodes
        """
        completed_nodes = [node_id for node_id, progress in self.progress.items() 
                         if progress >= 100]
        
        # Find nodes where all prerequisites are completed
        available_nodes = []
        for node in self.nodes:
            # Skip completed nodes
            if node.content_id in completed_nodes:
                continue
                
            # Check if all prerequisites are completed
            prereqs_met = all(prereq in completed_nodes for prereq in node.prerequisites)
            
            # Check incoming edges as well
            for edge in self.edges:
                if edge["to"] == node.content_id and edge["type"] == "prerequisite":
                    if edge["from"] not in completed_nodes:
                        prereqs_met = False
                        break
                        
            if prereqs_met:
                available_nodes.append(node)
                
        return available_nodes
        
    def get_overall_progress(self) -> float:
        """
        Calculate overall completion percentage
        
        Returns:
            Overall completion percentage (0-100)
        """
        if not self.nodes:
            return 0
            
        total_progress = sum(self.progress.values())
        return total_progress / len(self.nodes)
        
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary representation"""
        return {
            "student_id": self.student_id,
            "title": self.title,
            "description": self.description,
            "nodes": [node.to_dict() for node in self.nodes],
            "edges": self.edges,
            "progress": self.progress
        }
        
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'AdaptiveLearningPathway':
        """Create from dictionary representation"""
        pathway = cls(
            student_id=data["student_id"],
            title=data.get("title", ""),
            description=data.get("description", "")
        )
        
        pathway.nodes = [LearningPathwayNode.from_dict(node_data) 
                       for node_data in data.get("nodes", [])]
        pathway.edges = data.get("edges", [])
        pathway.progress = data.get("progress", {})
        
        return pathway

class AdaptiveLearningPathwayGenerator:
    """Generates personalized learning pathways based on student profiles"""
    
    def __init__(self, relationship_analyzer: Optional[ContentRelationshipAnalyzer] = None):
        """
        Initialize the pathway generator
        
        Args:
            relationship_analyzer: Optional analyzer for content relationships
        """
        self.relationship_analyzer = relationship_analyzer
        
    def generate_pathway(self, profile: StudentProfile, 
                       content_structure: Dict[str, Any],
                       target_concepts: List[str] = None) -> AdaptiveLearningPathway:
        """
        Generate a personalized learning pathway
        
        Args:
            profile: Student profile to personalize for
            content_structure: Structure of the content document
            target_concepts: Optional list of target concepts to focus on
            
        Returns:
            Personalized AdaptiveLearningPathway
        """
        # Create a new pathway for this student
        pathway = AdaptiveLearningPathway(
            student_id=profile.student_id,
            title=f"Personalized pathway for {content_structure.get('metadata', {}).get('title', 'Document')}",
            description=f"Custom learning pathway adapted to {profile.name}'s profile"
        )
        
        # Extract metadata and structure
        metadata = content_structure.get("metadata", {})
        document_title = metadata.get("title", "Document")
        
        # Determine appropriate difficulty level based on student profile
        difficulty = profile.get_recommended_difficulty()
        
        # Find content blocks and segments
        content_blocks = content_structure.get("content_blocks", [])
        content_segments = content_structure.get("semantic_analysis", {}).get("content_segments", [])
        
        # Get concept relationships if available
        concept_graph = content_structure.get("relationships", {}).get("concept_graph", {})
        prerequisites = content_structure.get("relationships", {}).get("prerequisites", {})
        
        # Build learning path nodes
        nodes_by_id = {}
        nodes_by_concept = {}
        
        # Process structured content sections
        sections = self._extract_content_sections(content_structure)
        
        # 1. Create nodes for each relevant section
        for i, section in enumerate(sections):
            # Skip sections that are too advanced or too basic based on profile
            section_difficulty = self._estimate_section_difficulty(section)
            
            node_id = f"section-{i+1}"
            
            # Create node for this section
            node = LearningPathwayNode(
                content_id=node_id,
                title=section.get("heading", f"Section {i+1}"),
                content_type="text",
                difficulty=section_difficulty,
                estimated_time=self._estimate_reading_time(section.get("content", ""))
            )
            
            # Extract main concepts from this section
            section_concepts = self._extract_concepts_from_section(section)
            
            # Check if student already knows these concepts well
            if self._student_knows_concepts(profile, section_concepts) and difficulty != "advanced":
                # For sections the student already knows, make them optional
                node.difficulty = "review"
                
            # Check if section covers target concepts
            if target_concepts and not any(concept in section_concepts for concept in target_concepts):
                # Skip sections not relevant to target concepts unless they're prerequisites
                if not self._is_prerequisite_for_targets(section_concepts, target_concepts, prerequisites):
                    continue
                    
            # Add node to pathway
            pathway.add_node(node)
            nodes_by_id[node_id] = node
            
            # Map concepts to this node
            for concept in section_concepts:
                if concept not in nodes_by_concept:
                    nodes_by_concept[concept] = []
                nodes_by_concept[concept].append(node_id)
                
        # 2. Connect nodes based on prerequisites and structure
        for node_id, node in nodes_by_id.items():
            # Find prerequisites for this node's concepts
            node_concepts = self._extract_concepts_from_section(
                next((s for s in sections if s.get("heading") == node.title), {})
            )
            
            for concept in node_concepts:
                if concept in prerequisites:
                    for prereq_concept in prerequisites[concept]:
                        # Find nodes containing this prerequisite concept
                        if prereq_concept in nodes_by_concept:
                            for prereq_node_id in nodes_by_concept[prereq_concept]:
                                # Add prerequisite relationship
                                pathway.add_edge(prereq_node_id, node_id, "prerequisite")
                                # Add to node's prerequisites
                                if prereq_node_id not in node.prerequisites:
                                    node.prerequisites.append(prereq_node_id)
        
        # 3. Add alternative versions of content based on learning style
        self._add_content_alternatives(pathway, profile, content_structure)
        
        # 4. Optimize pathway based on student's pace
        self._optimize_for_learning_pace(pathway, profile.learning_pace)
        
        return pathway
    
    def _extract_content_sections(self, content_structure: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Extract structured sections from content"""
        sections = []
        
        # Check for document structure
        if "document_structure" in content_structure:
            return content_structure["document_structure"]
            
        # Check for content blocks organized by heading
        if "semantic_analysis" in content_structure and "content_blocks" in content_structure["semantic_analysis"]:
            current_section = None
            
            for block in content_structure["semantic_analysis"]["content_blocks"]:
                if block.get("type") == "heading":
                    # Start a new section
                    if current_section:
                        sections.append(current_section)
                        
                    current_section = {
                        "heading": block.get("text", "Unnamed Section"),
                        "content": "",
                        "level": block.get("level", 1)
                    }
                elif current_section:
                    # Add content to current section
                    current_section["content"] += block.get("text", "") + "\n"
            
            # Add the last section
            if current_section:
                sections.append(current_section)
                
        # Fall back to raw extraction if no structure
        if not sections and "raw_extraction" in content_structure:
            text = content_structure["raw_extraction"].get("text", "")
            sections.append({
                "heading": "Complete Document",
                "content": text,
                "level": 0
            })
            
        return sections
    
    def _estimate_section_difficulty(self, section: Dict[str, Any]) -> str:
        """Estimate section difficulty based on content"""
        content = section.get("content", "")
        
        # Simple heuristics for difficulty estimation
        word_count = len(content.split())
        avg_word_length = sum(len(word) for word in content.split()) / max(1, word_count)
        
        if word_count > 1000 and avg_word_length > 7:
            return "advanced"
        elif word_count > 500 and avg_word_length > 6:
            return "standard"
        else:
            return "simple"
            
    def _estimate_reading_time(self, content: str) -> int:
        """Estimate reading time in minutes"""
        word_count = len(content.split())
        # Average reading speed: 200-250 words per minute
        return max(1, word_count // 225)
        
    def _extract_concepts_from_section(self, section: Dict[str, Any]) -> List[str]:
        """Extract key concepts from a section"""
        concepts = []
        
        if "concepts" in section:
            concepts = section["concepts"]
        elif "semantic_tags" in section:
            concepts = section["semantic_tags"]
        elif self.relationship_analyzer and "content" in section:
            # Use relationship analyzer to extract concepts
            extracted = self.relationship_analyzer.extract_key_concepts(section["content"])
            if extracted:
                concepts = extracted
                
        # If no concepts detected, extract from heading
        if not concepts and "heading" in section:
            heading = section["heading"]
            # Simple extraction from heading
            concepts = [word.strip() for word in heading.split() 
                      if len(word) > 3 and word.lower() not in ["and", "the", "for", "with"]]
                
        return concepts
        
    def _student_knows_concepts(self, profile: StudentProfile, concepts: List[str]) -> bool:
        """Check if student already knows the concepts well"""
        if not profile.knowledge_areas:
            return False
            
        # For each domain in the student's knowledge areas
        for domain, topics in profile.knowledge_areas.items():
            # Check if any concept matches a known topic with high proficiency
            for concept in concepts:
                concept_lower = concept.lower()
                for topic, proficiency in topics.items():
                    if concept_lower in topic.lower() and proficiency >= 0.8:
                        # Student knows this concept well
                        return True
                        
        return False
        
    def _is_prerequisite_for_targets(self, concepts: List[str], targets: List[str], 
                                  prerequisites: Dict[str, List[str]]) -> bool:
        """Check if concepts are prerequisites for target concepts"""
        # For each target concept
        for target in targets:
            # Check if it has prerequisites
            if target in prerequisites:
                # Check if any of our concepts are prerequisites for it
                for concept in concepts:
                    if concept in prerequisites[target]:
                        return True
                        
                # Recursively check through the prerequisite chain
                for prereq in prerequisites[target]:
                    if self._is_prerequisite_for_targets([prereq], targets, prerequisites):
                        return True
                        
        return False
        
    def _add_content_alternatives(self, pathway: AdaptiveLearningPathway, 
                               profile: StudentProfile, 
                               content_structure: Dict[str, Any]):
        """Add alternative content formats based on learning style"""
        # Get learning style preferences
        visual_pref = profile.learning_style.get("visual", 0.5)
        textual_pref = profile.learning_style.get("textual", 0.5)
        interactive_pref = profile.learning_style.get("interactive", 0.5)
        
        # Get available visualizations
        visualizations = content_structure.get("visual_elements", {}).get("diagrams", [])
        
        # Get additional explanations
        explanations = content_structure.get("enrichments", {}).get("explanations", {})
        
        # For each node in the pathway
        for node in pathway.nodes:
            # For visual learners, add diagram alternatives
            if visual_pref > 0.6:
                for viz in visualizations:
                    # Check if visualization is relevant to this node
                    if any(keyword in node.title.lower() for keyword in viz.get("keywords", [])):
                        node.add_alternative(
                            alt_content_id=f"{node.content_id}-visual",
                            alt_title=f"{node.title} (Visual Version)",
                            alt_content_type="visualization",
                            alt_difficulty=node.difficulty
                        )
            
            # For interactive learners, add quiz alternatives
            if interactive_pref > 0.6:
                node.add_alternative(
                    alt_content_id=f"{node.content_id}-interactive",
                    alt_title=f"{node.title} (Interactive Version)",
                    alt_content_type="quiz",
                    alt_difficulty=node.difficulty
                )
                
            # For textual learners, add detailed explanation alternatives
            if textual_pref > 0.6:
                # Check if there are enriched explanations available
                has_explanation = False
                for concept, explanation in explanations.items():
                    if concept.lower() in node.title.lower():
                        node.add_alternative(
                            alt_content_id=f"{node.content_id}-detailed",
                            alt_title=f"{node.title} (Detailed Explanation)",
                            alt_content_type="text",
                            alt_difficulty=node.difficulty
                        )
                        has_explanation = True
                        break
                
                # Add simpler explanation for difficult content
                if node.difficulty == "advanced" and not has_explanation:
                    node.add_alternative(
                        alt_content_id=f"{node.content_id}-simplified",
                        alt_title=f"{node.title} (Simplified)",
                        alt_content_type="text",
                        alt_difficulty="standard"
                    )
        
    def _optimize_for_learning_pace(self, pathway: AdaptiveLearningPathway, pace: str):
        """Optimize the pathway based on student's learning pace"""
        if pace == "fast":
            # For fast learners, reduce repetition and simplify prerequisites
            self._optimize_for_fast_learners(pathway)
        elif pace == "slow":
            # For slow learners, add more reinforcement nodes
            self._optimize_for_slow_learners(pathway)
    
    def _optimize_for_fast_learners(self, pathway: AdaptiveLearningPathway):
        """Optimize pathway for fast learners"""
        # Identify redundant nodes (nodes covering same concepts)
        redundant_nodes = []
        concept_coverage = {}
        
        # Map concepts to nodes
        for node in pathway.nodes:
            node_concepts = []
            for alt in node.alternatives:
                if alt["content_type"] == "text" and "Detailed" in alt["title"]:
                    # Favor detailed explanations for fast learners
                    node.title = alt["title"]
                    node.content_id = alt["content_id"]
                    break
        
        # Prune edges to simplify the pathway
        simplified_edges = []
        for edge in pathway.edges:
            # Keep only the most important prerequisite edges
            if edge["type"] == "prerequisite":
                # Check if there's a transitive prerequisite
                has_transitive = False
                for other_edge in pathway.edges:
                    if other_edge["to"] == edge["to"] and other_edge["from"] != edge["from"]:
                        # Check if edge["from"] is prerequisite of other_edge["from"]
                        for third_edge in pathway.edges:
                            if third_edge["to"] == other_edge["from"] and third_edge["from"] == edge["from"]:
                                has_transitive = True
                                break
                
                if not has_transitive:
                    simplified_edges.append(edge)
            else:
                simplified_edges.append(edge)
                
        pathway.edges = simplified_edges
        
    def _optimize_for_slow_learners(self, pathway: AdaptiveLearningPathway):
        """Optimize pathway for slow learners"""
        # Add reinforcement nodes between complex topics
        new_nodes = []
        new_edges = []
        
        for i, node in enumerate(pathway.nodes):
            # If this is a complex node
            if node.difficulty == "advanced" or node.difficulty == "standard":
                # Add a reinforcement node after it
                review_node = LearningPathwayNode(
                    content_id=f"{node.content_id}-review",
                    title=f"Review: {node.title}",
                    content_type="quiz",
                    difficulty="simple",
                    prerequisites=[node.content_id],
                    estimated_time=15
                )
                new_nodes.append(review_node)
                
                # Connect review node to original node
                new_edges.append({
                    "from": node.content_id,
                    "to": review_node.content_id,
                    "type": "next"
                })
                
                # Find nodes that had the original node as prerequisite
                for edge in pathway.edges:
                    if edge["from"] == node.content_id:
                        # Make those nodes depend on the review instead
                        new_edges.append({
                            "from": review_node.content_id,
                            "to": edge["to"],
                            "type": edge["type"]
                        })
        
        # Add new nodes and edges to pathway
        for node in new_nodes:
            pathway.add_node(node)
            
        pathway.edges.extend(new_edges)
        
        # Also, favor simplified versions of content for slow learners
        for node in pathway.nodes:
            for alt in node.alternatives:
                if alt["content_type"] == "text" and "Simplified" in alt["title"]:
                    # Use simplified versions as default for slow learners
                    node.title = alt["title"]
                    node.content_id = alt["content_id"]
                    break 

---


## domainicProcessor.py

In [ ]:
class BaseDomainProcessor:
    def __init__(self, domain_patterns=None, language_model=None):
        """
        Base class for domain-specific processors.
        
        Args:
            domain_patterns (dict): Patterns for the specific domain
            language_model: Optional pre-trained language model
        """
        self.domain_patterns = domain_patterns or {}
        self.language_model = language_model
    
    def process(self, extraction_result):
        """
        Process the extraction result with domain-specific logic.
        Should be implemented by subclasses.
        
        Args:
            extraction_result (dict): The extraction result from PDF processor
            
        Returns:
            Dictionary with domain-specific data
        """
        # Base implementation that returns a copy of the extraction result
        # Subclasses should override this method with domain-specific logic
        return extraction_result.copy() if hasattr(extraction_result, 'copy') else extraction_result


class PatternExtractor:
    """
    Utility class for extracting patterns from text content.
    """
    @staticmethod
    def extract_entities_by_pattern(text_content, pattern, group_indices=None):
        """
        Extract entities using regex pattern.
        
        Args:
            text_content (str): Text to extract from
            pattern: Regular expression pattern
            group_indices (tuple): Indices of groups to extract
            
        Returns:
            List of extracted entities
        """
        import re
        results = []
        
        for match in re.finditer(pattern, text_content):
            if group_indices:
                # Extract specific groups
                if len(group_indices) == 1:
                    # Single group extraction
                    entity = match.group(group_indices[0]).strip() if match.group(group_indices[0]) else ""
                    results.append(entity)
                else:
                    # Multiple group extraction as dictionary
                    entity = {}
                    for key, idx in group_indices.items():
                        entity[key] = match.group(idx).strip() if match.group(idx) else ""
                    results.append(entity)
            else:
                # Extract whole match
                results.append(match.group(0).strip())
                
        return results

class ScienceProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to scientific domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with science-specific entities
        """
        scientific_entities = {
            'terminology': self.extract_scientific_terminology(extraction_result),
            'methods': self.identify_experiments_and_methods(extraction_result),
        }
        
        return scientific_entities
    
    def extract_scientific_terminology(self, extraction_result):
        """
        Extract scientific terms and classify by field.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of scientific terminology by category
        """
        text_content = extraction_result.get('text_content', '')
        
        # Define common scientific terminology patterns
        scientific_terms = {
            'general': [],
            'methodology': [],
            'results': [],
            'concepts': []
        }
        
        # Lists of common scientific terms by category
        methodology_terms = ['experiment', 'method', 'procedure', 'protocol', 'analysis', 
                            'technique', 'assay', 'measurement', 'observation']
        
        result_terms = ['result', 'finding', 'data', 'evidence', 'observation', 
                       'outcome', 'conclusion', 'analysis', 'figure', 'table']
        
        concept_terms = ['theory', 'model', 'hypothesis', 'principle', 'law', 
                        'concept', 'framework', 'paradigm', 'mechanism']
        
        # Simple pattern matching for scientific terminology
        for term in methodology_terms:
            if term in text_content.lower():
                # Find the sentence containing the term
                sentences = text_content.replace('!', '.').replace('?', '.').split('.')
                for sentence in sentences:
                    if term in sentence.lower():
                        scientific_terms['methodology'].append(sentence.strip())
        
        for term in result_terms:
            if term in text_content.lower():
                sentences = text_content.replace('!', '.').replace('?', '.').split('.')
                for sentence in sentences:
                    if term in sentence.lower():
                        scientific_terms['results'].append(sentence.strip())
        
        for term in concept_terms:
            if term in text_content.lower():
                sentences = text_content.replace('!', '.').replace('?', '.').split('.')
                for sentence in sentences:
                    if term in sentence.lower():
                        scientific_terms['concepts'].append(sentence.strip())
        
        # Extract other scientific terms using patterns
        import re
        
        # Pattern for scientific terms (simplified)
        scientific_term_pattern = re.compile(r'(?i)([A-Z][a-z]+(?:ology|ography|ometry|onomy|physics|chemistry|ics))')
        matches = scientific_term_pattern.findall(text_content)
        
        scientific_terms['general'] = list(set(matches))
        
        return scientific_terms
    
    def identify_experiments_and_methods(self, extraction_result):
        """
        Identify experimental procedures and methods in the content.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of identified experiments and methods
        """
        text_content = extraction_result.get('text_content', '')
        
        experiments_and_methods = {
            'experiments': [],
            'methods': [],
            'procedures': []
        }
        
        import re
        
        # Extract methods
        if 'method_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['method_pattern'], text_content):
                method_text = match.group(2).strip()
                experiments_and_methods['methods'].append(method_text)
        
        # Extract experiments
        if 'experiment_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['experiment_pattern'], text_content):
                experiment_text = match.group(2).strip()
                experiments_and_methods['experiments'].append(experiment_text)
        
        # Look for procedure keywords
        procedure_keywords = ['procedure', 'protocol', 'step', 'process']
        sentences = text_content.replace('!', '.').replace('?', '.').split('.')
        
        for sentence in sentences:
            if any(keyword in sentence.lower() for keyword in procedure_keywords):
                experiments_and_methods['procedures'].append(sentence.strip())
        
        return experiments_and_methods


class MathProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to mathematical domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with math-specific entities
        """
        return {
            'mathematical_entities': self.extract_mathematical_entities(extraction_result),
            'problem_classifications': self.classify_math_problems(extraction_result)
        }
    
    def extract_mathematical_entities(self, extraction_result):
        """
        Extract theorems, formulas, proofs, and other mathematical entities.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of mathematical entities by type
        """
        text_content = extraction_result.get('text_content', '')
        
        math_entities = {
            'theorems': [],
            'proofs': [],
            'equations': [],
            'definitions': []
        }
        
        # Extract theorems
        if 'theorem_pattern' in self.domain_patterns:
            theorem_group_indices = {'type': 1, 'name': 2, 'content': 3}
            math_entities['theorems'] = PatternExtractor.extract_entities_by_pattern(
                text_content, 
                self.domain_patterns['theorem_pattern'],
                theorem_group_indices
            )
        
        # Extract proofs
        if 'proof_pattern' in self.domain_patterns:
            math_entities['proofs'] = PatternExtractor.extract_entities_by_pattern(
                text_content, 
                self.domain_patterns['proof_pattern'],
                (1,)
            )
        
        # Extract equations
        if 'equation_pattern' in self.domain_patterns:
            equation_group_indices = {'number': lambda m: m.group(1) or m.group(2), 'content': 3}
            equations = PatternExtractor.extract_entities_by_pattern(
                text_content, 
                self.domain_patterns['equation_pattern'],
                equation_group_indices
            )
            math_entities['equations'] = equations
        
        # Extract definitions
        import re
        definition_pattern = re.compile(r'(?i)Definition\s*(\d+|[\w\s]+)?[:]\s*(.*?)(?=\n\n|\Z)')
        definition_group_indices = {'name': 1, 'content': 2}
        math_entities['definitions'] = PatternExtractor.extract_entities_by_pattern(
            text_content,
            definition_pattern,
            definition_group_indices
        )
        
        return math_entities

    
    def classify_math_problems(self, extraction_result):
        """
        Classify mathematical problems by type and difficulty.

        Args:
            extraction_result (dict): The extraction result

        Returns:
            Dictionary of classified mathematical problems
        """
        text_content = extraction_result.get('text_content', '')
        import re

        # Pattern to identify problems
        problem_pattern = re.compile(r'(?i)(?:Problem|Exercise|Question)\s*(\d+|[\w\s]+)?[:]\s*(.*?)(?=\n\n|\Z|Solution)')
        problem_group_indices = {'number': 1, 'text': 2}

        problems = PatternExtractor.extract_entities_by_pattern(
            text_content,
            problem_pattern,
            problem_group_indices
        )

        math_problems = []

        for problem in problems:
            problem_num = problem['number']
            problem_text = problem['text']

            # Try to find the solution if available
            solution_pattern = re.compile(f'(?i)Solution(?:\s*to\s*(?:Problem|Exercise|Question)?\s*{re.escape(problem_num) if problem_num else ""})?\s*[:]\s*(.*?)(?=\n\n|\Z|Problem|Exercise|Question)')
            solutions = PatternExtractor.extract_entities_by_pattern(
                text_content,
                solution_pattern,
                (1,)
            )

            solution_text = solutions[0] if solutions else ""

            # Classify problem type
            problem_type = self._classify_math_problem_type(problem_text)

            # Estimate difficulty
            difficulty = self._estimate_math_problem_difficulty(problem_text, solution_text)

            math_problems.append({
                'number': problem_num,
                'text': problem_text,
                'solution': solution_text,
                'type': problem_type,
                'difficulty': difficulty
            })

        return math_problems
    
    def _classify_math_problem_type(self, problem_text):
        """
        Classify the type of a mathematical problem.
        
        Args:
            problem_text (str): The text of the problem
            
        Returns:
            The type of the mathematical problem
        """
        problem_text = problem_text.lower()
        
        # Define keywords for different problem types
        type_keywords = {
            'algebra': ['solve', 'equation', 'polynomial', 'factor', 'simplify', 'expression'],
            'calculus': ['derivative', 'integral', 'differentiate', 'integrate', 'limit', 'maximize', 'minimize'],
            'geometry': ['triangle', 'circle', 'angle', 'polygon', 'rectangle', 'square', 'perimeter', 'area', 'volume'],
            'probability': ['probability', 'random', 'chance', 'likelihood', 'expected value', 'variance'],
            'statistics': ['mean', 'median', 'mode', 'standard deviation', 'normal distribution', 'variance'],
            'number theory': ['prime', 'divisor', 'factor', 'gcd', 'lcm', 'modulo', 'congruence'],
            'linear algebra': ['matrix', 'vector', 'eigenvalue', 'eigenvector', 'linear transformation'],
            'discrete math': ['graph', 'tree', 'network', 'combinatorial', 'permutation', 'combination']
        }
        
        # Count occurrences of each category's keywords
        type_scores = {category: 0 for category in type_keywords}
        
        for category, keywords in type_keywords.items():
            for keyword in keywords:
                if keyword in problem_text:
                    type_scores[category] += 1
        
        # Return the category with the highest score
        if any(type_scores.values()):
            return max(type_scores.items(), key=lambda x: x[1])[0]
        else:
            return 'general'
    
    def _estimate_math_problem_difficulty(self, problem_text, solution_text=""):
        """
        Estimate the difficulty level of a mathematical problem.
        
        Args:
            problem_text (str): The text of the problem
            solution_text (str): The text of the solution
            
        Returns:
            The estimated difficulty level (elementary, intermediate, advanced)
        """
        # Combine problem and solution for analysis
        combined_text = problem_text + " " + solution_text
        combined_text = combined_text.lower()
        
        # Define difficulty indicators
        elementary_indicators = ['basic', 'simple', 'elementary', 'straightforward', 'easy', 'find']
        advanced_indicators = ['complex', 'challenging', 'advanced', 'difficult', 'prove', 'proof', 'theorem', 'lemma']
        
        # Count advanced mathematical terms
        advanced_math_terms = ['eigenvalue', 'eigenvector', 'homomorphism', 'isomorphism', 'bijection',
                              'riemann', 'laplace', 'fourier', 'lebesgue', 'optimization', 'stochastic',
                              'differential equation', 'partial derivative', 'tensor', 'manifold', 'topology']
        
        # Count occurrences
        elementary_count = sum(indicator in combined_text for indicator in elementary_indicators)
        advanced_count = sum(indicator in combined_text for indicator in advanced_indicators)
        advanced_term_count = sum(term in combined_text for term in advanced_math_terms)
        
        # Text complexity heuristics
        word_count = len(combined_text.split())
        sentence_count = max(1, len(combined_text.split('.')))
        average_sentence_length = word_count / sentence_count
        
        # Decision logic for difficulty level
        if advanced_term_count >= 2 or advanced_count >= 3 or average_sentence_length > 25:
            return 'advanced'
        elif elementary_count > advanced_count or (word_count < 50 and advanced_term_count == 0):
            return 'elementary'
        else:
            return 'intermediate'


class LiteratureProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to literature domains by using the LiteraryWorkAnalyzer.

        Args:
            extraction_result (dict): The extraction result

        Returns:
            Dictionary with literature-specific entities
        """
        text_content = extraction_result.get('text_content', '')

        # Determine work type to use appropriate analyzer
        work_type = 'general'
        import re
        
        if 'shakespeare' in extraction_result.get('metadata', {}).get('title', '').lower() or 'shakespeare' in text_content.lower():
            work_type = 'play:shakespeare'
        elif re.search(r'([A-Z]{2,}[A-Z\s]*)[\.:]\s', text_content):
            work_type = 'play'
        elif re.search(r'Chapter [IVXLCDM]+|Chapter \d+', text_content):
            work_type = 'novel'

        # Use the dedicated LiteraryWorkAnalyzer
        literary_analyzer = LiteraryWorkAnalyzer(work_type)

        return {
            'literary_structure': literary_analyzer.identify_scenes(text_content),
            'literary_devices': literary_analyzer.detect_literary_devices(text_content)
        }


class ChemistryProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to chemistry domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with chemistry-specific entities
        """
        text_content = extraction_result.get('text_content', '')
        import re
        
        compounds = set()
        reactions = []
        conditions = []
        
        # Extract compounds
        if 'compound_pattern' in self.domain_patterns:
            compounds = set(re.findall(self.domain_patterns['compound_pattern'], text_content))
            # Filter common false positives
            compounds = {c for c in compounds if len(c) > 1 and any(char.isdigit() for char in c)}
        
        # Extract reactions
        if 'reaction_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['reaction_pattern'], text_content):
                reactants = match.group(1).strip()
                products = match.group(2).strip()
                reactions.append({
                    'reactants': reactants,
                    'products': products,
                    'full_reaction': f"{reactants} â†’ {products}"
                })
        
        # Extract conditions
        if 'condition_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['condition_pattern'], text_content):
                conditions.append(match.group(0))
        
        return {
            'compounds': list(compounds),
            'reactions': reactions,
            'reaction_conditions': conditions
        }


class BiologyProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to biology domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with biology-specific entities
        """
        text_content = extraction_result.get('text_content', '')
        import re
        
        species = set()
        genes = set()
        pathways = set()
        
        # Extract species
        if 'species_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['species_pattern'], text_content):
                species_name = match.group(1)
                # Filter common false positives
                if len(species_name.split()) == 2 and all(len(word) > 1 for word in species_name.split()):
                    species.add(species_name)
        
        # Extract genes
        if 'gene_pattern' in self.domain_patterns:
            potential_genes = re.findall(self.domain_patterns['gene_pattern'], text_content)
            # Filter common false positives
            genes = {g for g in potential_genes if len(g) >= 2 and g.isupper()}
        
        # Extract pathways
        if 'pathway_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['pathway_pattern'], text_content):
                pathways.add(match.group(0))
        
        return {
            'species': list(species),
            'genes': list(genes),
            'biological_pathways': list(pathways)
        }


class PhysicsProcessor(BaseDomainProcessor):
    def process(self, extraction_result):
        """
        Process content specific to physics domains.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary with physics-specific entities
        """
        text_content = extraction_result.get('text_content', '')
        import re
        
        laws = set()
        physical_quantities = []
        equations = []
        
        # Extract physical laws
        if 'law_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['law_pattern'], text_content):
                laws.add(match.group(0))
        
        # Extract physical quantities with units
        if 'units_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['units_pattern'], text_content):
                value = match.group(1)
                unit = match.group(2)
                exponent = match.group(3) if match.group(3) else "1"
                
                physical_quantities.append({
                    'value': float(value),
                    'unit': unit,
                    'exponent': int(exponent),
                    'display': f"{value} {unit}{'' if exponent == '1' else f'^{exponent}'}"
                })
        
        # Extract equations
        if 'equation_pattern' in self.domain_patterns:
            for match in re.finditer(self.domain_patterns['equation_pattern'], text_content):
                equation_num = match.group(1) or match.group(2)
                equation_text = match.group(3)
                
                equations.append({
                    'equation_number': equation_num,
                    'equation_text': equation_text
                })
        
        return {
            'physical_laws': list(laws),
            'physical_quantities': physical_quantities,
            'physics_equations': equations
        }


---


## DomainRulesManager.py

In [ ]:
class DomainRulesManager:
    """Centralized manager for domain-specific rules and patterns"""
    
    @staticmethod
    def get_domain_rules(domain):
        """
        Load domain-specific rules and patterns for content analysis.
        
        Args:
            domain: The domain to load rules for
            
        Returns:
            Dictionary of rules and patterns specific to the domain
        """
        import re
        
        if domain is None:
            return {}
            
        # Consolidated domain rules
        domain_rules = {
            'science': {
                'terminologies': ['experiment', 'hypothesis', 'theory', 'data'],
                'section_types': ['methodology', 'results', 'discussion', 'conclusion'],
                'entity_patterns': [r'[A-Z][a-z]+ [a-z]+ium', r'\d+\.\d+ [a-z]+'],
                'method_pattern': re.compile(r'(?i)(methods?|procedure|protocol)[:]\s*(.*?)(?=\n\n|\Z)'),
                'result_pattern': re.compile(r'(?i)(results?|findings)[:]\s*(.*?)(?=\n\n|\Z)'),
                'experiment_pattern': re.compile(r'(?i)(experiment|study|investigation)[\s\d]*[:]\s*(.*?)(?=\n\n|\Z)')
            },
            'math': {
                'terminologies': ['theorem', 'proof', 'lemma', 'corollary'],
                'section_types': ['definition', 'theorem', 'proof', 'example'],
                'entity_patterns': [r'[A-Z][a-z]+ (theorem|formula|equation)', r'equation \(\d+\)'],
                'theorem_pattern': re.compile(r'(?i)(theorem|lemma|corollary|proposition)\s*(\d+|[\w\s]+)[:]\s*(.*?)(?=\n\n|\Z|proof)'),
                'proof_pattern': re.compile(r'(?i)proof[:]\s*(.*?)(?=\n\n|\Z|q\.e\.d\.|â–¡)'),
                'equation_pattern': re.compile(r'(?i)(?:\((\d+)\)|equation\s*(\d+))[:]\s*(.*?)(?=\n\n|\Z)')
            },
            'literature': {
                'terminologies': ['symbolism', 'metaphor', 'character', 'plot', 'theme', 'motif', 'foreshadowing', 'irony', 'allegory', 'setting'],
                'section_types': ['exposition', 'climax', 'resolution', 'analysis', 'chapter', 'prologue', 'epilogue', 'act', 'scene', 'soliloquy'],
                'entity_patterns': [
                    r'[A-Z][a-z]+ (symbolizes|represents)', 
                    r'theme of [a-z]+', 
                    r'Chapter [IVXLCDM]+', 
                    r'Chapter \d+',
                    r'Act [IVXLCDM]+', 
                    r'Scene [IVXLCDM]+', 
                    r'Enter [A-Z][a-z]+', 
                    r'\[Exit[^\]]*\]', 
                    r'\[Aside\]'
                ],
                'character_pattern': re.compile(r'(?i)([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)[:]\s*["\'](.+?)["\']'),
                'theme_pattern': re.compile(r'(?i)(theme|motif|symbol)[\s\w]*[:]\s*(.*?)(?=\n\n|\Z)'),
                'plot_pattern': re.compile(r'(?i)(exposition|rising action|climax|falling action|resolution)[:]\s*(.*?)(?=\n\n|\Z)'),
                'subtypes': {
                    'play': {
                        'shakespeare': {
                            'terminologies': ['soliloquy', 'aside', 'iambic pentameter', 'sonnet', 'blank verse', 'couplet', 'quatrain', 'folio', 'quarto'],
                            'themes': ['power_corruption', 'love_romance', 'betrayal', 'appearance_reality', 'supernatural', 'order_chaos', 'revenge']
                        },
                        'modern': {
                            'terminologies': ['dialogue', 'stage direction', 'monologue', 'act', 'scene']
                        }
                    },
                    'novel': {
                        'terminologies': ['narrator', 'protagonist', 'antagonist', 'setting', 'plot', 'chapter']
                    },
                    'poem': {
                        'terminologies': ['stanza', 'verse', 'rhyme', 'meter', 'rhythm', 'sonnet', 'ode', 'elegy']
                    }
                }
            },
            'chemistry': {
                'compound_pattern': re.compile(r'(?i)([A-Z][a-z]?\d*)+'),
                'reaction_pattern': re.compile(r'(?i)(.+?)\s*(?:â†’|âŸ¶|â‡’|â‡Œ|â‡†)\s*(.+)'),
                'condition_pattern': re.compile(r'(?i)(?:at|under|in)\s+(\d+(?:\.\d+)?)\s*(?:Â°C|K|atm|bar|MPa|kPa)')
            },
            'biology': {
                'species_pattern': re.compile(r'(?i)([A-Z][a-z]+\s+[a-z]+)'),
                'gene_pattern': re.compile(r'(?i)([A-Z0-9]+)(?:\s+gene)?'),
                'pathway_pattern': re.compile(r'(?i)([\w\s]+)(?:pathway|cycle|cascade)')
            },
            'physics': {
                'law_pattern': re.compile(r'(?i)([\w\s]+)(?:\'s)?\s+(?:law|principle|theory)'),
                'units_pattern': re.compile(r'(\d+(?:\.\d+)?)\s*(m|kg|s|A|K|mol|cd|N|Pa|J|W|V)(?:\^?(-?\d+))?'),
                'equation_pattern': re.compile(r'(?i)(?:\((\d+)\)|equation\s*(\d+))[:]\s*(.*?)(?=\n\n|\Z)')
            }
        }
        
        return domain_rules.get(domain.lower(), {})


---


## DomainSpecificProcessor.py

In [ ]:
from DomainRulesManager import DomainRulesManager
from domainicProcessor import *

class DomainSpecificProcessor:
    def __init__(self, domain_type, language_model=None):
        """
        Initialize the DomainSpecificProcessor with a specified domain type.

        Args:
            domain_type (str): The domain type to specialize in (science, math, literature, etc.)
            language_model: Optional pre-trained language model
        """
        self.domain_type = domain_type.lower() if domain_type else None
        self.language_model = language_model

        # Domain-specific entity patterns
        self.domain_patterns = self._initialize_domain_patterns()

        # Initialize domain processors
        self._initialize_domain_processors()
    
    def _initialize_domain_patterns(self):
        """
        Initialize regex patterns and rules for different domains.

        Returns:
            Dictionary of patterns for each domain
        """
        # Simply use the domain rules manager instead of recreating logic
        domains = ['science', 'math', 'literature', 'chemistry', 'biology', 'physics']
        patterns = {}
     
        # Load rules for each domain
        for domain in domains:
            patterns[domain] = DomainRulesManager.get_domain_rules(domain)
            
        return patterns
    
    def _initialize_domain_processors(self):
        """
        Initialize domain-specific processors.
        """
        # Initialize domain processors with appropriate patterns
        self.domain_processors = {
            'science': ScienceProcessor(self.domain_patterns.get('science', {})),
            'math': MathProcessor(self.domain_patterns.get('math', {})),
            'literature': LiteratureProcessor(self.domain_patterns.get('literature', {})),
            'chemistry': ChemistryProcessor(self.domain_patterns.get('chemistry', {})),
            'biology': BiologyProcessor(self.domain_patterns.get('biology', {})),
            'physics': PhysicsProcessor(self.domain_patterns.get('physics', {}))
        }
        
        # If language model is provided, update processor initialization
        if self.language_model is not None:
            for domain, processor in self.domain_processors.items():
                processor.language_model = self.language_model
    
    def process_domain_content(self, extraction_result):
        """
        Process content according to domain-specific rules.

        Args:
            extraction_result (dict): The extraction result from PDF processor

        Returns:
            Dictionary with domain-specific processing results added
        """
        if not self.domain_type or self.domain_type not in self.domain_processors:
            # If domain is not supported, return the original extraction result
            return extraction_result

        # Create a copy of the extraction result to modify
        processed_result = extraction_result.copy()

        # If language model is available, use it for enhanced processing
        if self.language_model is not None:
            return self._process_with_language_model(processed_result)

        # Apply domain-specific processing
        domain_processor = self.domain_processors[self.domain_type]
        domain_specific_data = domain_processor.process(processed_result)

        # Add domain-specific results to the processed result
        processed_result['domain_analysis'] = {
            'domain_type': self.domain_type,
            'domain_entities': domain_specific_data
        }

        return processed_result
    
    def _process_with_language_model(self, extraction_result):
        """
        Use the provided language model to perform domain-specific content analysis.

        Args:
            extraction_result (dict): Dictionary containing the extracted content

        Returns:
            Dictionary with enhanced domain-specific analysis from the language model
        """
        processed_result = extraction_result.copy()
        text_content = extraction_result.get('text_content', '')

        if not text_content:
            # Fallback to original method if no text content is available
            return self.process_domain_content(extraction_result)

        # Get common analysis applicable to all domains
        model_analysis = {
            'keywords_and_concepts': self.language_model.extract_keywords(text_content, self.domain_type),
            'content_segments': self.language_model.segment_content(text_content),
            'semantic_tags': self.language_model.generate_semantic_tags(text_content, self.domain_type)
        }

        # Add domain-specific analysis
        if self.domain_type == 'science':
            model_analysis['science_analysis'] = self.language_model.analyze_scientific_content(
                text_content
            )
        elif self.domain_type == 'math':
            model_analysis['math_analysis'] = self.language_model.analyze_mathematical_content(
                text_content
            )
        elif self.domain_type == 'literature':
            model_analysis['literary_analysis'] = self.language_model.analyze_literary_content(
                text_content
            )
        elif self.domain_type == 'chemistry':
            model_analysis['chemistry_analysis'] = self.language_model.analyze_chemical_content(
                text_content
            )
        elif self.domain_type == 'biology':
            model_analysis['biology_analysis'] = self.language_model.analyze_biological_content(
                text_content
            )
        elif self.domain_type == 'physics':
            model_analysis['physics_analysis'] = self.language_model.analyze_physics_content(
                text_content
            )

        # Add section-specific analysis if sections exist
        if 'sections' in extraction_result:
            model_analysis['section_analysis'] = {}
            for section_id, section_content in extraction_result['sections'].items():
                model_analysis['section_analysis'][section_id] = self.language_model.analyze_section(
                    section_content, self.domain_type
                )

        # Add domain-specific results to the processed result
        processed_result['domain_analysis'] = {
            'domain_type': self.domain_type,
            'domain_entities': model_analysis
        }

        return processed_result

---


## StudentProfileManager.py

In [ ]:
"""
Student Profile Manager
----------------------
Manages student profiles for adaptive learning, including:
- Academic level and preferences
- Learning pace and style
- Knowledge history and progression
- Subject strengths and weaknesses
"""

import json
import os
from datetime import datetime
from typing import Dict, List, Any, Optional, Union
import logging
import time
import uuid
import re
import threading
from jsonschema import validate, ValidationError

from pipeline.errors.WorkflowErrors import (
    AdaptiveLearningError, StudentProfileError
)

class ProfileValidationSchema:
    """Schema definitions for student profile validation"""
    
    # Base profile schema
    STUDENT_PROFILE_SCHEMA = {
        "type": "object",
        "required": ["student_id", "created_at", "updated_at", "profile_version"],
        "properties": {
            "student_id": {
                "type": "string",
                "pattern": "^[a-zA-Z0-9_-]{4,64}$"
            },
            "created_at": {"type": "string", "format": "date-time"},
            "updated_at": {"type": "string", "format": "date-time"},
            "profile_version": {"type": "string"},
            "personal_info": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "email": {
                        "type": "string",
                        "format": "email"
                    },
                    "grade_level": {"type": "string"},
                    "educational_institution": {"type": "string"}
                }
            },
            "learning_preferences": {
                "type": "object",
                "properties": {
                    "preferred_learning_style": {
                        "type": "string",
                        "enum": ["visual", "auditory", "reading", "kinesthetic", "multimodal"]
                    },
                    "pace_preference": {
                        "type": "string",
                        "enum": ["slower", "standard", "faster"]
                    },
                    "difficulty_preference": {
                        "type": "string",
                        "enum": ["easier", "standard", "challenging"]
                    },
                    "interests": {
                        "type": "array",
                        "items": {"type": "string"}
                    }
                }
            },
            "learning_history": {
                "type": "array",
                "items": {
                    "type": "object",
                    "required": ["content_id", "timestamp", "completion_status"],
                    "properties": {
                        "content_id": {"type": "string"},
                        "content_type": {"type": "string"},
                        "timestamp": {"type": "string", "format": "date-time"},
                        "completion_status": {
                            "type": "string",
                            "enum": ["not_started", "in_progress", "completed", "abandoned"]
                        },
                        "score": {"type": "number", "minimum": 0, "maximum": 100},
                        "time_spent": {"type": "number", "minimum": 0}
                    }
                }
            },
            "knowledge_state": {
                "type": "object",
                "properties": {
                    "concepts": {
                        "type": "object",
                        "additionalProperties": {
                            "type": "object",
                            "properties": {
                                "mastery_level": {"type": "number", "minimum": 0, "maximum": 1},
                                "last_assessed": {"type": "string", "format": "date-time"},
                                "assessment_count": {"type": "integer", "minimum": 0},
                                "relevance": {"type": "number", "minimum": 0, "maximum": 1}
                            }
                        }
                    },
                    "domains": {
                        "type": "object",
                        "additionalProperties": {
                            "type": "object",
                            "properties": {
                                "proficiency_level": {"type": "number", "minimum": 0, "maximum": 1},
                                "interest_level": {"type": "number", "minimum": 0, "maximum": 1},
                                "experience_level": {"type": "string", "enum": ["novice", "beginner", "intermediate", "advanced", "expert"]}
                            }
                        }
                    }
                }
            },
            "adaptive_parameters": {
                "type": "object",
                "properties": {
                    "difficulty_factor": {"type": "number", "minimum": 0.1, "maximum": 2.0},
                    "content_diversity": {"type": "number", "minimum": 0.1, "maximum": 1.0},
                    "repetition_interval": {"type": "number", "minimum": 1},
                    "mastery_threshold": {"type": "number", "minimum": 0.1, "maximum": 1.0}
                }
            },
            "goals": {
                "type": "array",
                "items": {
                    "type": "object",
                    "required": ["goal_id", "description", "created_at"],
                    "properties": {
                        "goal_id": {"type": "string"},
                        "description": {"type": "string"},
                        "created_at": {"type": "string", "format": "date-time"},
                        "target_date": {"type": "string", "format": "date-time"},
                        "status": {"type": "string", "enum": ["active", "completed", "abandoned"]},
                        "progress": {"type": "number", "minimum": 0, "maximum": 1}
                    }
                }
            }
        },
        "additionalProperties": true
    }
    
    # Profile update schema (subset of fields that can be updated)
    PROFILE_UPDATE_SCHEMA = {
        "type": "object",
        "properties": {
            "personal_info": {
                "type": "object"
            },
            "learning_preferences": {
                "type": "object"
            },
            "goals": {
                "type": "array"
            },
            "adaptive_parameters": {
                "type": "object"
            }
        },
        "additionalProperties": false
    }
    
    # Learning history entry schema
    LEARNING_HISTORY_ENTRY_SCHEMA = {
        "type": "object",
        "required": ["content_id", "timestamp", "completion_status"],
        "properties": {
            "content_id": {"type": "string"},
            "content_type": {"type": "string"},
            "timestamp": {"type": "string", "format": "date-time"},
            "completion_status": {
                "type": "string",
                "enum": ["not_started", "in_progress", "completed", "abandoned"]
            },
            "score": {"type": "number", "minimum": 0, "maximum": 100},
            "time_spent": {"type": "number", "minimum": 0}
        }
    }
    
    # Concept knowledge update schema
    CONCEPT_KNOWLEDGE_SCHEMA = {
        "type": "object",
        "required": ["concept_id", "mastery_level"],
        "properties": {
            "concept_id": {"type": "string"},
            "mastery_level": {"type": "number", "minimum": 0, "maximum": 1},
            "assessment_info": {
                "type": "object",
                "properties": {
                    "assessment_id": {"type": "string"},
                    "score": {"type": "number", "minimum": 0, "maximum": 100},
                    "timestamp": {"type": "string", "format": "date-time"}
                }
            }
        }
    }

class StudentProfile:
    """Represents a student's learning profile and preferences"""
    
    def __init__(self, student_id: str, name: str = "", email: str = "", 
                 academic_level: str = "intermediate", learning_pace: str = "standard"):
        """
        Initialize a student profile
        
        Args:
            student_id: Unique identifier for the student
            name: Student's name
            email: Student's email address
            academic_level: Academic level (beginner, intermediate, advanced)
            learning_pace: Preferred learning pace (slow, standard, fast)
        """
        self.student_id = student_id
        self.name = name
        self.email = email
        self.created_at = datetime.now().isoformat()
        self.updated_at = self.created_at
        
        # Learning preferences
        self.academic_level = academic_level  # beginner, intermediate, advanced
        self.learning_pace = learning_pace  # slow, standard, fast
        self.learning_style = {
            "visual": 0.5,       # 0-1 preference for visual learning
            "textual": 0.5,      # 0-1 preference for text-based learning
            "interactive": 0.5,  # 0-1 preference for interactive learning
        }
        
        # Knowledge tracking
        self.knowledge_areas = {}  # Domain â†’ proficiency level (0-1)
        self.completed_materials = []  # List of completed material IDs
        self.learning_history = []  # Chronological learning events
        
        # Performance metrics
        self.strengths = []  # List of strong knowledge areas
        self.weaknesses = []  # List of areas needing improvement
        self.retention_metrics = {}  # Topic â†’ retention score (0-1)
        
    def update_knowledge_area(self, domain: str, topic: str, proficiency: float):
        """
        Update proficiency in a specific knowledge area
        
        Args:
            domain: The domain (e.g., "math", "science")
            topic: The specific topic within the domain
            proficiency: Proficiency level (0-1)
        """
        if domain not in self.knowledge_areas:
            self.knowledge_areas[domain] = {}
            
        self.knowledge_areas[domain][topic] = proficiency
        self.updated_at = datetime.now().isoformat()
        
        # Update strengths and weaknesses
        self._recalculate_strengths_weaknesses()
        
    def record_learning_activity(self, material_id: str, activity_type: str, 
                               performance: Optional[float] = None, time_spent: Optional[int] = None):
        """
        Record a learning activity in the student's history
        
        Args:
            material_id: ID of the learning material
            activity_type: Type of activity (read, quiz, practice, etc.)
            performance: Optional performance metric (0-1)
            time_spent: Optional time spent in seconds
        """
        activity = {
            "timestamp": datetime.now().isoformat(),
            "material_id": material_id,
            "activity_type": activity_type
        }
        
        if performance is not None:
            activity["performance"] = performance
            
        if time_spent is not None:
            activity["time_spent"] = time_spent
            
        self.learning_history.append(activity)
        
        # Add to completed materials if not already present
        if material_id not in self.completed_materials:
            self.completed_materials.append(material_id)
            
        self.updated_at = datetime.now().isoformat()
        
    def update_learning_preferences(self, **preferences):
        """
        Update the student's learning preferences
        
        Args:
            **preferences: Keyword arguments for preferences to update
        """
        valid_preferences = [
            "academic_level", "learning_pace", "visual", 
            "textual", "interactive"
        ]
        
        for key, value in preferences.items():
            if key in valid_preferences:
                if key in ["visual", "textual", "interactive"]:
                    self.learning_style[key] = value
                else:
                    setattr(self, key, value)
                    
        self.updated_at = datetime.now().isoformat()
        
    def get_recommended_difficulty(self) -> str:
        """
        Calculate the recommended content difficulty based on profile
        
        Returns:
            String indicating recommended difficulty (simple, standard, advanced)
        """
        # Base recommendation on academic level
        if self.academic_level == "beginner":
            base_difficulty = "simple"
        elif self.academic_level == "intermediate":
            base_difficulty = "standard"
        else:  # advanced
            base_difficulty = "advanced"
            
        # Adjust based on overall performance in knowledge areas
        if self.knowledge_areas:
            avg_proficiency = 0
            count = 0
            
            for domain in self.knowledge_areas.values():
                for proficiency in domain.values():
                    avg_proficiency += proficiency
                    count += 1
            
            if count > 0:
                avg_proficiency /= count
                
                # Adjust difficulty based on proficiency
                if avg_proficiency > 0.8 and base_difficulty != "advanced":
                    # Step up difficulty if doing very well
                    if base_difficulty == "simple":
                        return "standard"
                    else:
                        return "advanced"
                elif avg_proficiency < 0.4 and base_difficulty != "simple":
                    # Step down difficulty if struggling
                    if base_difficulty == "advanced":
                        return "standard"
                    else:
                        return "simple"
        
        return base_difficulty
        
    def _recalculate_strengths_weaknesses(self):
        """
        Internal method to recalculate strengths and weaknesses
        based on knowledge area proficiencies
        """
        strengths = []
        weaknesses = []
        
        for domain, topics in self.knowledge_areas.items():
            for topic, proficiency in topics.items():
                if proficiency >= 0.75:
                    strengths.append({"domain": domain, "topic": topic, "proficiency": proficiency})
                elif proficiency <= 0.4:
                    weaknesses.append({"domain": domain, "topic": topic, "proficiency": proficiency})
        
        # Sort by proficiency (descending for strengths, ascending for weaknesses)
        self.strengths = sorted(strengths, key=lambda x: x["proficiency"], reverse=True)
        self.weaknesses = sorted(weaknesses, key=lambda x: x["proficiency"])
        
    def to_dict(self) -> Dict[str, Any]:
        """
        Convert profile to dictionary for serialization
        
        Returns:
            Dictionary representation of the profile
        """
        return {
            "student_id": self.student_id,
            "name": self.name,
            "email": self.email,
            "created_at": self.created_at,
            "updated_at": self.updated_at,
            "academic_level": self.academic_level,
            "learning_pace": self.learning_pace,
            "learning_style": self.learning_style,
            "knowledge_areas": self.knowledge_areas,
            "completed_materials": self.completed_materials,
            "learning_history": self.learning_history,
            "strengths": self.strengths,
            "weaknesses": self.weaknesses,
            "retention_metrics": self.retention_metrics
        }
        
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'StudentProfile':
        """
        Create a profile from a dictionary
        
        Args:
            data: Dictionary representation of a profile
            
        Returns:
            StudentProfile instance
        """
        profile = cls(
            student_id=data["student_id"],
            name=data.get("name", ""),
            email=data.get("email", ""),
            academic_level=data.get("academic_level", "intermediate"),
            learning_pace=data.get("learning_pace", "standard")
        )
        
        # Restore other properties
        profile.created_at = data.get("created_at", profile.created_at)
        profile.updated_at = data.get("updated_at", profile.updated_at)
        profile.learning_style = data.get("learning_style", profile.learning_style)
        profile.knowledge_areas = data.get("knowledge_areas", {})
        profile.completed_materials = data.get("completed_materials", [])
        profile.learning_history = data.get("learning_history", [])
        profile.strengths = data.get("strengths", [])
        profile.weaknesses = data.get("weaknesses", [])
        profile.retention_metrics = data.get("retention_metrics", {})
        
        return profile

class StudentProfileManager:
    """
    Manages student profiles for adaptive learning
    with robust validation and versioning
    """
    
    def __init__(self, storage_dir: str = "student_profiles", 
                backup_dir: Optional[str] = None,
                auto_backup: bool = True,
                backup_interval: int = 3600):
        """
        Initialize student profile manager
        
        Args:
            storage_dir: Directory for storing student profiles
            backup_dir: Directory for profile backups
            auto_backup: Whether to automatically backup profiles
            backup_interval: Backup interval in seconds
        """
        self.storage_dir = storage_dir
        self.backup_dir = backup_dir or os.path.join(storage_dir, "backups")
        self.auto_backup = auto_backup
        self.backup_interval = backup_interval
        self.logger = logging.getLogger(self.__class__.__name__)
        
        # Create storage directories
        os.makedirs(storage_dir, exist_ok=True)
        os.makedirs(self.backup_dir, exist_ok=True)
        
        # Cache for recently accessed profiles
        self.profile_cache: Dict[str, Dict[str, Any]] = {}
        self.cache_lock = threading.RLock()
        
        # Set of profiles that have been modified
        self.modified_profiles: Set[str] = set()
        
        # Current profile version
        self.profile_version = "1.3.0"
        
        # Start backup thread if auto_backup is enabled
        if auto_backup:
            self.backup_thread_running = True
            self.backup_thread = threading.Thread(target=self._backup_thread, daemon=True)
            self.backup_thread.start()
            self.logger.info(f"Auto-backup thread started with interval {backup_interval} seconds")
    
    def get_student_profile(self, student_id: str, create_if_missing: bool = True) -> Dict[str, Any]:
        """
        Get a student profile by ID
        
        Args:
            student_id: Student identifier
            create_if_missing: Whether to create a profile if not found
            
        Returns:
            Student profile dictionary
            
        Raises:
            StudentProfileError: If profile not found and create_if_missing is False
        """
        # Validate student ID format
        self._validate_student_id(student_id)
        
        # Check cache first
        with self.cache_lock:
            if student_id in self.profile_cache:
                return self.profile_cache[student_id]
        
        # Check disk storage
        profile_path = os.path.join(self.storage_dir, f"{student_id}.json")
        
        if os.path.exists(profile_path):
            try:
                # Load profile from disk
                with open(profile_path, "r") as f:
                    profile = json.load(f)
                
                # Validate the loaded profile
                self._validate_profile(profile)
                
                # Check if profile needs migration
                if self._needs_version_migration(profile):
                    profile = self._migrate_profile(profile)
                    # Save migrated profile
                    self.save_student_profile(student_id, profile)
                
                # Add to cache
                with self.cache_lock:
                    self.profile_cache[student_id] = profile
                
        return profile
        
            except (json.JSONDecodeError, IOError) as e:
                error_msg = f"Error loading profile for student {student_id}: {str(e)}"
                self.logger.error(error_msg)
                
                # Try to load from backup if available
                backup_profile = self._load_from_backup(student_id)
                if backup_profile:
                    self.logger.info(f"Restored profile for student {student_id} from backup")
                    
                    # Add to cache
                    with self.cache_lock:
                        self.profile_cache[student_id] = backup_profile
                    
                    return backup_profile
                
                if create_if_missing:
                    # Create new profile if loading failed
                    self.logger.warning(f"Creating new profile for student {student_id} after load failure")
                    return self.create_student_profile(student_id)
                else:
                    raise StudentProfileError(f"Failed to load student profile: {error_msg}")
            
            except ValidationError as e:
                error_msg = f"Invalid profile format for student {student_id}: {e.message}"
                self.logger.error(error_msg)
                
                # Try to load from backup
                backup_profile = self._load_from_backup(student_id)
                if backup_profile:
                    self.logger.info(f"Restored profile for student {student_id} from backup")
                    
                    # Add to cache
                    with self.cache_lock:
                        self.profile_cache[student_id] = backup_profile
                    
                    return backup_profile
                
                if create_if_missing:
                    # Create new profile if validation failed
                    self.logger.warning(f"Creating new profile for student {student_id} after validation failure")
                    return self.create_student_profile(student_id)
                else:
                    raise StudentProfileError(f"Invalid student profile format: {error_msg}")
        
        elif create_if_missing:
            # Create new profile
            return self.create_student_profile(student_id)
        
        else:
            error_msg = f"Student profile not found for ID: {student_id}"
            self.logger.error(error_msg)
            raise StudentProfileError(error_msg)
    
    def create_student_profile(self, student_id: str) -> Dict[str, Any]:
        """
        Create a new student profile
        
        Args:
            student_id: Student identifier
            
        Returns:
            Newly created student profile
            
        Raises:
            StudentProfileError: If a profile already exists for the student
        """
        # Validate student ID format
        self._validate_student_id(student_id)
        
        # Check if profile already exists
        profile_path = os.path.join(self.storage_dir, f"{student_id}.json")
        
        if os.path.exists(profile_path) and not self._is_empty_or_corrupt(profile_path):
            error_msg = f"Student profile already exists for ID: {student_id}"
            self.logger.error(error_msg)
            raise StudentProfileError(error_msg)
        
        # Create timestamp
        now = datetime.utcnow().isoformat() + "Z"
        
        # Create new profile with default values
        profile = {
            "student_id": student_id,
            "created_at": now,
            "updated_at": now,
            "profile_version": self.profile_version,
            "personal_info": {
                "grade_level": "unknown"
            },
            "learning_preferences": {
                "preferred_learning_style": "multimodal",
                "pace_preference": "standard",
                "difficulty_preference": "standard",
                "interests": []
            },
            "learning_history": [],
            "knowledge_state": {
                "concepts": {},
                "domains": {}
            },
            "adaptive_parameters": {
                "difficulty_factor": 1.0,
                "content_diversity": 0.5,
                "repetition_interval": 7,
                "mastery_threshold": 0.8
            },
            "goals": []
        }
        
        # Validate profile
        self._validate_profile(profile)
        
        # Save to disk
        try:
            with open(profile_path, "w") as f:
                json.dump(profile, f, indent=2)
            
            # Add to cache
            with self.cache_lock:
                self.profile_cache[student_id] = profile
                self.modified_profiles.add(student_id)
            
            self.logger.info(f"Created new profile for student {student_id}")
            
            return profile
            
        except (IOError, OSError) as e:
            error_msg = f"Failed to save new profile for student {student_id}: {str(e)}"
            self.logger.error(error_msg)
            raise StudentProfileError(error_msg)
    
    def update_student_profile(self, student_id: str, updates: Dict[str, Any]) -> Dict[str, Any]:
        """
        Update a student profile
        
        Args:
            student_id: Student identifier
            updates: Dictionary of profile updates
            
        Returns:
            Updated student profile
            
        Raises:
            StudentProfileError: If profile not found or invalid updates
        """
        # Validate student ID format
        self._validate_student_id(student_id)
        
        # Validate update data
        try:
            validate(instance=updates, schema=ProfileValidationSchema.PROFILE_UPDATE_SCHEMA)
        except ValidationError as e:
            error_msg = f"Invalid profile update data: {e.message}"
            self.logger.error(error_msg)
            raise StudentProfileError(error_msg)
        
        # Get current profile
        profile = self.get_student_profile(student_id, False)
        
        # Update timestamp
        profile["updated_at"] = datetime.utcnow().isoformat() + "Z"
        
        # Apply updates
        for key, value in updates.items():
            if key in ["personal_info", "learning_preferences", "adaptive_parameters"]:
                # Merge dictionaries
                if key in profile:
                    profile[key].update(value)
                else:
                    profile[key] = value
            elif key == "goals":
                # Replace goals list
                profile[key] = value
            elif key in profile:
                # Direct update
                profile[key] = value
        
        # Validate updated profile
        self._validate_profile(profile)
        
        # Save updated profile
        self.save_student_profile(student_id, profile)
        
        return profile
    
    def add_learning_history(self, student_id: str, learning_entry: Dict[str, Any]) -> Dict[str, Any]:
        """
        Add a learning history entry to a student profile
        
        Args:
            student_id: Student identifier
            learning_entry: Learning history entry
            
        Returns:
            Updated student profile
            
        Raises:
            StudentProfileError: If profile not found or invalid entry
        """
        # Validate student ID format
        self._validate_student_id(student_id)
        
        # Validate learning history entry
        try:
            validate(instance=learning_entry, schema=ProfileValidationSchema.LEARNING_HISTORY_ENTRY_SCHEMA)
        except ValidationError as e:
            error_msg = f"Invalid learning history entry: {e.message}"
            self.logger.error(error_msg)
            raise StudentProfileError(error_msg)
        
        # Get current profile
        profile = self.get_student_profile(student_id, False)
        
        # Update timestamp
        profile["updated_at"] = datetime.utcnow().isoformat() + "Z"
        
        # Add entry to learning history
        profile["learning_history"].append(learning_entry)
        
        # Save updated profile
        self.save_student_profile(student_id, profile)
        
        return profile
    
    def update_knowledge_state(self, student_id: str, concept_update: Dict[str, Any]) -> Dict[str, Any]:
        """
        Update a student's knowledge state for a specific concept
        
        Args:
            student_id: Student identifier
            concept_update: Concept knowledge update
            
        Returns:
            Updated student profile
            
        Raises:
            StudentProfileError: If profile not found or invalid update
        """
        # Validate student ID format
        self._validate_student_id(student_id)
        
        # Validate concept knowledge update
        try:
            validate(instance=concept_update, schema=ProfileValidationSchema.CONCEPT_KNOWLEDGE_SCHEMA)
        except ValidationError as e:
            error_msg = f"Invalid concept knowledge update: {e.message}"
            self.logger.error(error_msg)
            raise StudentProfileError(error_msg)
        
        # Get current profile
        profile = self.get_student_profile(student_id, False)
        
        # Update timestamp
        profile["updated_at"] = datetime.utcnow().isoformat() + "Z"
        
        # Get concept ID and mastery level
        concept_id = concept_update["concept_id"]
        mastery_level = concept_update["mastery_level"]
        
        # Ensure concepts dictionary exists
        if "knowledge_state" not in profile:
            profile["knowledge_state"] = {"concepts": {}, "domains": {}}
        elif "concepts" not in profile["knowledge_state"]:
            profile["knowledge_state"]["concepts"] = {}
        
        # Update or create concept entry
        if concept_id in profile["knowledge_state"]["concepts"]:
            # Update existing concept
            profile["knowledge_state"]["concepts"][concept_id]["mastery_level"] = mastery_level
            profile["knowledge_state"]["concepts"][concept_id]["last_assessed"] = datetime.utcnow().isoformat() + "Z"
            profile["knowledge_state"]["concepts"][concept_id]["assessment_count"] = profile["knowledge_state"]["concepts"][concept_id].get("assessment_count", 0) + 1
            
            # Include assessment info if provided
            if "assessment_info" in concept_update:
                if "assessments" not in profile["knowledge_state"]["concepts"][concept_id]:
                    profile["knowledge_state"]["concepts"][concept_id]["assessments"] = []
                
                profile["knowledge_state"]["concepts"][concept_id]["assessments"].append(concept_update["assessment_info"])
        else:
            # Create new concept entry
            profile["knowledge_state"]["concepts"][concept_id] = {
                "mastery_level": mastery_level,
                "last_assessed": datetime.utcnow().isoformat() + "Z",
                "assessment_count": 1,
                "relevance": 1.0  # Default relevance
            }
            
            # Include assessment info if provided
            if "assessment_info" in concept_update:
                profile["knowledge_state"]["concepts"][concept_id]["assessments"] = [concept_update["assessment_info"]]
        
        # Save updated profile
        self.save_student_profile(student_id, profile)
        
        return profile
        
    def save_student_profile(self, student_id: str, profile: Dict[str, Any]) -> None:
        """
        Save a student profile to disk
        
        Args:
            student_id: Student identifier
            profile: Student profile to save
            
        Raises:
            StudentProfileError: If save fails
        """
        # Validate student ID format
        self._validate_student_id(student_id)
        
        # Validate profile
        self._validate_profile(profile)
        
        # Ensure student_id in profile matches the provided ID
        if profile.get("student_id") != student_id:
            error_msg = f"Student ID mismatch: {profile.get('student_id')} != {student_id}"
            self.logger.error(error_msg)
            raise StudentProfileError(error_msg)
        
        # Update timestamp
        profile["updated_at"] = datetime.utcnow().isoformat() + "Z"
        
        # Save to disk
        profile_path = os.path.join(self.storage_dir, f"{student_id}.json")
        
        try:
            # Create a temporary file first
            temp_path = f"{profile_path}.tmp"
            with open(temp_path, "w") as f:
                json.dump(profile, f, indent=2)
            
            # Rename temp file to target file (atomic operation)
            os.replace(temp_path, profile_path)
            
            # Update cache
            with self.cache_lock:
                self.profile_cache[student_id] = profile
                self.modified_profiles.add(student_id)
            
            self.logger.debug(f"Saved profile for student {student_id}")
            
        except (IOError, OSError) as e:
            error_msg = f"Failed to save profile for student {student_id}: {str(e)}"
            self.logger.error(error_msg)
            
            # Clean up temp file if it exists
            if os.path.exists(f"{profile_path}.tmp"):
                try:
                    os.remove(f"{profile_path}.tmp")
                except:
                    pass
                
            raise StudentProfileError(error_msg)
    
    def delete_student_profile(self, student_id: str) -> bool:
        """
        Delete a student profile
        
        Args:
            student_id: Student identifier
            
        Returns:
            True if profile was deleted, False if not found
            
        Raises:
            StudentProfileError: If deletion fails
        """
        # Validate student ID format
        self._validate_student_id(student_id)
        
        # Check if profile exists
        profile_path = os.path.join(self.storage_dir, f"{student_id}.json")
        
        if not os.path.exists(profile_path):
            self.logger.warning(f"Profile not found for student {student_id}")
            return False
        
        try:
            # Create backup before deletion
            self._backup_profile(student_id)
            
            # Delete profile file
            os.remove(profile_path)
            
            # Remove from cache
            with self.cache_lock:
                if student_id in self.profile_cache:
                    del self.profile_cache[student_id]
                self.modified_profiles.discard(student_id)
            
            self.logger.info(f"Deleted profile for student {student_id}")
            return True
            
        except (IOError, OSError) as e:
            error_msg = f"Failed to delete profile for student {student_id}: {str(e)}"
            self.logger.error(error_msg)
            raise StudentProfileError(error_msg)
    
    def _validate_student_id(self, student_id: str) -> None:
        """
        Validate student ID format
        
        Args:
            student_id: Student identifier
            
        Raises:
            StudentProfileError: If ID format is invalid
        """
        # Check if ID is a string
        if not isinstance(student_id, str):
            raise StudentProfileError(f"Student ID must be a string, got {type(student_id)}")
        
        # Check ID format with regex pattern
        if not re.match(r"^[a-zA-Z0-9_-]{4,64}$", student_id):
            raise StudentProfileError(
                f"Invalid student ID format: {student_id}. " + 
                "ID must be 4-64 characters and contain only letters, numbers, underscores, and hyphens."
            )
    
    def _validate_profile(self, profile: Dict[str, Any]) -> None:
        """
        Validate a student profile against the schema
        
        Args:
            profile: Student profile to validate
            
        Raises:
            ValidationError: If profile is invalid
        """
        validate(instance=profile, schema=ProfileValidationSchema.STUDENT_PROFILE_SCHEMA)
    
    def _is_empty_or_corrupt(self, profile_path: str) -> bool:
        """
        Check if a profile file is empty or corrupt
        
        Args:
            profile_path: Path to profile file
            
        Returns:
            True if file is empty or corrupt, False otherwise
        """
        try:
            # Check file size
            if os.path.getsize(profile_path) == 0:
                return True
            
            # Try to load JSON
            with open(profile_path, "r") as f:
                profile = json.load(f)
            
            # Check if it has required fields
            if not isinstance(profile, dict) or "student_id" not in profile:
                return True
            
            return False
            
        except (IOError, json.JSONDecodeError):
            return True
    
    def _backup_profile(self, student_id: str) -> bool:
        """
        Create a backup of a student profile
        
        Args:
            student_id: Student identifier
            
        Returns:
            True if backup was created, False otherwise
        """
        profile_path = os.path.join(self.storage_dir, f"{student_id}.json")
        
        if not os.path.exists(profile_path):
            return False
            
        try:
            # Create timestamp for backup filename
            timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
            backup_path = os.path.join(self.backup_dir, f"{student_id}_{timestamp}.json")
            
            # Copy file to backup directory
            with open(profile_path, "r") as src:
                with open(backup_path, "w") as dst:
                    dst.write(src.read())
            
            self.logger.debug(f"Created backup of profile for student {student_id}")
            return True
            
        except (IOError, OSError) as e:
            self.logger.error(f"Failed to backup profile for student {student_id}: {str(e)}")
            return False
            
    def _load_from_backup(self, student_id: str) -> Optional[Dict[str, Any]]:
        """
        Load a student profile from the latest backup
        
        Args:
            student_id: Student identifier
        
        Returns:
            Student profile from backup, or None if no backup found
        """
        # Find all backups for this student
        backup_files = []
        for filename in os.listdir(self.backup_dir):
            if filename.startswith(f"{student_id}_") and filename.endswith(".json"):
                backup_files.append(filename)
        
        if not backup_files:
            return None
        
        # Sort by timestamp (descending)
        backup_files.sort(reverse=True)
        
        # Try to load the latest backup
        latest_backup = os.path.join(self.backup_dir, backup_files[0])
        
        try:
            with open(latest_backup, "r") as f:
                profile = json.load(f)
            
            # Validate the profile
            self._validate_profile(profile)
            
            return profile
            
        except (IOError, json.JSONDecodeError, ValidationError) as e:
            self.logger.error(f"Failed to load backup for student {student_id}: {str(e)}")
            return None
    
    def _needs_version_migration(self, profile: Dict[str, Any]) -> bool:
        """
        Check if a profile needs version migration
        
        Args:
            profile: Student profile to check
            
        Returns:
            True if migration is needed, False otherwise
        """
        # Check if profile has a version
        if "profile_version" not in profile:
            return True
        
        # Compare versions (simple string comparison for now)
        return profile["profile_version"] != self.profile_version
    
    def _migrate_profile(self, profile: Dict[str, Any]) -> Dict[str, Any]:
        """
        Migrate a profile to the current version
        
        Args:
            profile: Student profile to migrate
            
        Returns:
            Migrated profile
        """
        # Get current version
        current_version = profile.get("profile_version", "1.0.0")
        
        # Clone the profile
        migrated = dict(profile)
        
        # Update version
        migrated["profile_version"] = self.profile_version
        
        # Perform migration based on current version
        if current_version == "1.0.0":
            # Migrate from 1.0.0 to 1.1.0
            if "knowledge_state" not in migrated:
                migrated["knowledge_state"] = {"concepts": {}, "domains": {}}
            if "goals" not in migrated:
                migrated["goals"] = []
        
        if current_version <= "1.1.0":
            # Migrate from 1.1.0 to 1.2.0
            if "adaptive_parameters" not in migrated:
                migrated["adaptive_parameters"] = {
                    "difficulty_factor": 1.0,
                    "content_diversity": 0.5,
                    "repetition_interval": 7,
                    "mastery_threshold": 0.8
                }
        
        if current_version <= "1.2.0":
            # Migrate from 1.2.0 to 1.3.0
            if "learning_history" not in migrated:
                migrated["learning_history"] = []
        
        self.logger.info(f"Migrated profile from version {current_version} to {self.profile_version}")
        
        return migrated
    
    def _backup_thread(self) -> None:
        """Background thread for automatic profile backups"""
        while self.backup_thread_running:
            try:
                # Sleep for the backup interval
                time.sleep(self.backup_interval)
                
                # Get modified profiles
                modified = set()
                with self.cache_lock:
                    modified = self.modified_profiles.copy()
                    self.modified_profiles.clear()
                
                # Backup each modified profile
                for student_id in modified:
                    self._backup_profile(student_id)
                
                if modified:
                    self.logger.info(f"Auto-backup completed for {len(modified)} profiles")
                
            except Exception as e:
                self.logger.error(f"Error in backup thread: {str(e)}")
    
    def shutdown(self) -> None:
        """Shutdown the profile manager and save all cached profiles"""
        self.logger.info("Shutting down profile manager")
        
        # Stop backup thread
        if hasattr(self, 'backup_thread_running'):
            self.backup_thread_running = False
            if hasattr(self, 'backup_thread') and self.backup_thread.is_alive():
                self.backup_thread.join(timeout=5)
        
        # Save all cached profiles
        with self.cache_lock:
            for student_id, profile in self.profile_cache.items():
                try:
                    # Save each profile
                    profile_path = os.path.join(self.storage_dir, f"{student_id}.json")
                    with open(profile_path, "w") as f:
                        json.dump(profile, f, indent=2)
                    
                    # Create backup
                    self._backup_profile(student_id)
                    
                except Exception as e:
                    self.logger.error(f"Error saving profile {student_id} during shutdown: {str(e)}")
        
        self.logger.info("Profile manager shutdown complete") 

---


## ComponentConfig.py

In [ ]:
"""
Component Configuration Classes
------------------------------
Defines typed, validated configuration classes for all workflow components.
Ensures proper configuration validation at initialization time.
"""

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Union, Any
import os
from enum import Enum


class OCRQuality(Enum):
    """OCR quality levels"""
    LOW = "low"
    STANDARD = "standard" 
    HIGH = "high"
    HIGHEST = "highest"


class VideoQuality(Enum):
    """Video quality levels"""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    ULTRA = "ultra"


@dataclass
class PDFExtractorConfig:
    """Configuration for PDF extraction"""
    # Feature toggles
    extract_images: bool = True
    extract_tables: bool = True
    extract_equations: bool = True
    
    # OCR configuration
    ocr_quality: OCRQuality = OCRQuality.HIGH
    ocr_language: str = "eng"
    multi_language_detection: bool = False
    tesseract_path: Optional[str] = None
    
    # Scanned document options
    process_scanned: bool = True
    deskew_pages: bool = True
    denoise_images: bool = True
    contrast_enhancement: bool = False
    
    # Storage
    output_dir: Optional[str] = None
    
    def validate(self) -> None:
        """Validate the configuration"""
        if self.ocr_language and not isinstance(self.ocr_language, str):
            raise ValueError("OCR language must be a string")
        
        if self.output_dir and not os.path.exists(self.output_dir):
            raise ValueError(f"Output directory does not exist: {self.output_dir}")


@dataclass
class ContentAnalysisConfig:
    """Configuration for content analysis"""
    domain: str = "general"
    depth_level: str = "standard"
    use_domain_specific: bool = True
    domain_confidence_threshold: float = 0.7
    apply_iterative_refinement: bool = True
    context_preservation: bool = True
    semantic_enrichment: bool = True
    knowledge_graph_integration: bool = True
    
    def validate(self) -> None:
        """Validate the configuration"""
        valid_domains = ["general", "auto", "mathematics", "physics", 
                        "chemistry", "biology", "computer_science", 
                        "history", "literature"]
        
        if self.domain not in valid_domains and not self.domain.startswith("custom_"):
            raise ValueError(f"Invalid domain: {self.domain}")
        
        valid_depth = ["basic", "standard", "deep", "comprehensive"]
        if self.depth_level not in valid_depth:
            raise ValueError(f"Invalid depth level: {self.depth_level}")
        
        if not 0.0 <= self.domain_confidence_threshold <= 1.0:
            raise ValueError("Domain confidence threshold must be between 0 and 1")


@dataclass
class ScriptStyleConfig:
    """Configuration for script generation style"""
    tone: str = "neutral"
    verbosity: str = "moderate"
    target_audience: str = "general"
    language: str = "en-US"
    
    def validate(self) -> None:
        """Validate the configuration"""
        valid_tones = ["formal", "neutral", "friendly", "enthusiastic", "professional"]
        if self.tone not in valid_tones:
            raise ValueError(f"Invalid tone: {self.tone}")
        
        valid_verbosity = ["minimal", "moderate", "detailed", "comprehensive"]
        if self.verbosity not in valid_verbosity:
            raise ValueError(f"Invalid verbosity: {self.verbosity}")


@dataclass
class AnimationStyleConfig:
    """Configuration for animation style"""
    style: str = "modern"
    color_scheme: str = "educational"
    animation_level: str = "standard"
    scheduler_type: str = "ddim"
    
    def validate(self) -> None:
        """Validate the configuration"""
        valid_styles = ["modern", "classic", "minimalist", "vibrant", "technical"]
        if self.style not in valid_styles:
            raise ValueError(f"Invalid style: {self.style}")
        
        valid_schemes = ["educational", "vibrant", "monochrome", "pastel", "high-contrast"]
        if self.color_scheme not in valid_schemes:
            raise ValueError(f"Invalid color scheme: {self.color_scheme}")
        
        valid_animation = ["simple", "standard", "complex"]
        if self.animation_level not in valid_animation:
            raise ValueError(f"Invalid animation level: {self.animation_level}")


@dataclass
class VoiceOptionsConfig:
    """Configuration for voice generation"""
    voice_id: str = "neutral"
    speed: float = 1.0
    pitch: int = 0
    language_code: str = "en-US"
    
    def validate(self) -> None:
        """Validate the configuration"""
        if not 0.5 <= self.speed <= 2.0:
            raise ValueError("Speed must be between 0.5 and 2.0")
        
        if not -10 <= self.pitch <= 10:
            raise ValueError("Pitch must be between -10 and 10")


@dataclass
class MultimediaConfig:
    """Configuration for multimedia generation"""
    script_style: ScriptStyleConfig = field(default_factory=ScriptStyleConfig)
    animation_style: AnimationStyleConfig = field(default_factory=AnimationStyleConfig)
    voice_options: VoiceOptionsConfig = field(default_factory=VoiceOptionsConfig)
    video_format: str = "mp4"
    video_quality: VideoQuality = VideoQuality.HIGH
    api_token: Optional[str] = None
    
    def validate(self) -> None:
        """Validate the configuration"""
        valid_formats = ["mp4", "webm", "mov"]
        if self.video_format not in valid_formats:
            raise ValueError(f"Invalid video format: {self.video_format}")
        
        self.script_style.validate()
        self.animation_style.validate()
        self.voice_options.validate()


@dataclass
class AdaptiveLearningConfig:
    """Configuration for adaptive learning"""
    difficulty_adjustment: bool = True
    personalization_level: str = "medium"
    focus_areas: List[str] = field(default_factory=list)
    
    def validate(self) -> None:
        """Validate the configuration"""
        valid_levels = ["low", "medium", "high"]
        if self.personalization_level not in valid_levels:
            raise ValueError(f"Invalid personalization level: {self.personalization_level}")


@dataclass
class WorkflowConfig:
    """Master configuration for the complete workflow"""
    extraction_options: PDFExtractorConfig = field(default_factory=PDFExtractorConfig)
    analysis_options: ContentAnalysisConfig = field(default_factory=ContentAnalysisConfig)
    multimedia_options: MultimediaConfig = field(default_factory=MultimediaConfig)
    adaptive_options: AdaptiveLearningConfig = field(default_factory=AdaptiveLearningConfig)
    
    # System-level configuration
    output_dir: str = "output"
    async_processing: bool = False
    use_distributed: bool = False
    
    # Auto-scaling options
    auto_scaling: Dict[str, Any] = field(default_factory=lambda: {
        "enabled": False,
        "cpu_threshold": 0.8,
        "memory_threshold": 0.8,
        "time_threshold": 300
    })
    
    def validate(self) -> None:
        """Validate the entire configuration"""
        # Validate component configs
        self.extraction_options.validate()
        self.analysis_options.validate()
        self.multimedia_options.validate()
        self.adaptive_options.validate()
        
        # Validate system config
        if not os.path.exists(self.output_dir):
            try:
                os.makedirs(self.output_dir, exist_ok=True)
            except Exception as e:
                raise ValueError(f"Could not create output directory: {e}")
    
    @classmethod
    def from_dict(cls, config_dict: Dict[str, Any]) -> 'WorkflowConfig':
        """Create a typed configuration from a dictionary"""
        # Extract component configs
        extraction_dict = config_dict.get("extraction_options", {})
        analysis_dict = config_dict.get("analysis_options", {})
        multimedia_dict = config_dict.get("multimedia_options", {})
        adaptive_dict = config_dict.get("adaptive_options", {})
        
        # Create component configs
        extraction_config = PDFExtractorConfig(**extraction_dict)
        
        analysis_config = ContentAnalysisConfig(**analysis_dict)
        
        # Handle nested multimedia configs
        script_style = ScriptStyleConfig(**multimedia_dict.get("script_style", {}))
        animation_style = AnimationStyleConfig(**multimedia_dict.get("animation_style", {}))
        voice_options = VoiceOptionsConfig(**multimedia_dict.get("voice_options", {}))
        
        multimedia_config = MultimediaConfig(
            script_style=script_style,
            animation_style=animation_style,
            voice_options=voice_options,
            video_format=multimedia_dict.get("video_format", "mp4"),
            video_quality=VideoQuality(multimedia_dict.get("video_quality", "high")),
            api_token=multimedia_dict.get("api_token")
        )
        
        adaptive_config = AdaptiveLearningConfig(**adaptive_dict)
        
        # Create the main config
        return cls(
            extraction_options=extraction_config,
            analysis_options=analysis_config,
            multimedia_options=multimedia_config,
            adaptive_options=adaptive_config,
            output_dir=config_dict.get("output_dir", "output"),
            async_processing=config_dict.get("async_processing", False),
            use_distributed=config_dict.get("use_distributed", False),
            auto_scaling=config_dict.get("auto_scaling", {"enabled": False})
        ) 

---


## config.py

In [ ]:
"""
Configuration management for the VisualizationElementExtractor module.

This module provides centralized configuration for the visualization element extraction
pipeline, making it easier to adjust parameters for different environments 
(development, testing, production).
"""

import os
import logging
from typing import Dict, Any

class Config:
    """Configuration class for the VisualizationElementExtractor module."""
    
    # Environment settings
    ENV = os.getenv("EXTRACTION_ENV", "development")  # development, testing, production
    
    # Logging configuration
    LOG_LEVEL = os.getenv("LOG_LEVEL", "INFO")
    LOG_FORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    LOG_FILE = os.getenv("LOG_FILE", "extraction_pipeline.log")
    
    # Processing parameters
    BATCH_SIZE = int(os.getenv("BATCH_SIZE", "10"))
    MAX_WORKERS = int(os.getenv("MAX_WORKERS", "4"))
    
    # Image preprocessing parameters
    GAUSSIAN_BLUR_KERNEL = (5, 5)
    CANNY_THRESHOLD1 = 50
    CANNY_THRESHOLD2 = 150
    BINARY_THRESHOLD = 127
    
    # Extraction parameters
    MIN_CONTOUR_AREA = 500
    MIN_LINE_LENGTH = 50
    LINE_GAP = 10
    HOUGH_THRESHOLD = 50
    
    # OCR parameters
    TESSERACT_CONFIG = "--oem 3 --psm 6"
    TESSERACT_LANG = "eng"
    
    # Caching
    ENABLE_CACHE = os.getenv("ENABLE_CACHE", "False").lower() == "true"
    CACHE_DIR = os.getenv("CACHE_DIR", "/tmp/extraction_cache")
    CACHE_TTL = int(os.getenv("CACHE_TTL", "3600"))  # seconds
    
    # Output settings
    OUTPUT_FORMAT = os.getenv("OUTPUT_FORMAT", "json")  # json, csv, xml
    
    @classmethod
    def get_env_config(cls) -> Dict[str, Any]:
        """Get environment-specific configuration."""
        if cls.ENV == "production":
            return {
                "LOG_LEVEL": "WARNING",
                "ENABLE_CACHE": True,
                "BATCH_SIZE": 50,
                "MAX_WORKERS": 8,
            }
        elif cls.ENV == "testing":
            return {
                "LOG_LEVEL": "DEBUG",
                "ENABLE_CACHE": False,
                "BATCH_SIZE": 2,
                "MAX_WORKERS": 2,
            }
        else:  # development
            return {
                "LOG_LEVEL": "DEBUG",
                "ENABLE_CACHE": True,
                "BATCH_SIZE": 5,
                "MAX_WORKERS": 2,
            }
    
    @classmethod
    def setup_logging(cls):
        """Configure logging based on current settings."""
        log_level = getattr(logging, cls.LOG_LEVEL)
        logging.basicConfig(
            level=log_level,
            format=cls.LOG_FORMAT,
            handlers=[
                logging.FileHandler(cls.LOG_FILE),
                logging.StreamHandler()
            ]
        )
        
    @classmethod
    def update_from_env(cls):
        """Update configuration from environment variables."""
        env_config = cls.get_env_config()
        for key, value in env_config.items():
            if hasattr(cls, key):
                setattr(cls, key, value)


# Initialize configuration
Config.update_from_env() 

---


## ContentAnalysisPipeline.py

In [ ]:
import json
from collections.abc import Mapping
from jsonschema import validate, ValidationError
import logging
from typing import Dict, List, Any, Optional, Union, Tuple
import os
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Importing required classes from other files in the same directory
from pipeline.pdf_extraction.EnhancedPDFExtractor import EnhancedPDFExtractor
from pipeline.content_analysis.ContentAnalyzer import ContentAnalyzer
from pipeline.adaptive_learning.DomainSpecificProcessor import DomainSpecificProcessor
from pipeline.content_analysis.ContentRelationshipAnalyzer import ContentRelationshipAnalyzer
from pipeline.content_analysis.ContentEnrichment import ContentEnrichmentProcessor
from pipeline.content_analysis.VisualizationElementExtractor import VisualizationElementExtractor
from pipeline.multimedia.mistralModel import OpenRouterMistralClient
from pipeline.errors.WorkflowErrors import (
    ContentAnalysisError, DomainDetectionError, ContentStructuringError, SummarizationError
)

# Try to download NLTK resources, handle if already exists or network unavailable
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
except Exception as e:
    pass  # Silently continue if downloads fail (might already be available)

def print_key_structure(d: dict) -> None:
    """
    Given a dictionary `d`, prints its keys and any nested subkeys
    as a JSON-formatted structure where leaf values are replaced by {}.
    """
    def extract_keys(obj):
        if isinstance(obj, Mapping):
            return {k: extract_keys(v) for k, v in obj.items()}
        else:
            # nonâ€dict value â†’ leaf key
            return type(obj).__name__
    
    key_tree = extract_keys(d)
    print(json.dumps(key_tree, indent=4))
    
class ContentAnalysisSchema:
    """Defines the schema for content analysis output validation"""
    
    # Schema for the overall analysis result
    ANALYSIS_RESULT_SCHEMA = {
        "type": "object",
        "required": ["topic", "summary", "concepts", "structure", "metadata"],
        "properties": {
            "topic": {"type": "string", "minLength": 1},
            "summary": {
                "type": "object",
                "required": ["brief", "key_points"],
                "properties": {
                    "brief": {"type": "string", "minLength": 10},
                    "key_points": {
                        "type": "array",
                        "items": {"type": "string", "minLength": 5},
                        "minItems": 1
                    },
                    "difficulty_level": {"type": "string"},
                    "estimated_reading_time": {"type": "number", "minimum": 0}
                }
            },
            "concepts": {
                "type": "array",
                "items": {
                    "type": "object",
                    "required": ["name", "description"],
                    "properties": {
                        "name": {"type": "string", "minLength": 1},
                        "description": {"type": "string", "minLength": 10},
                        "importance": {"type": "number", "minimum": 0, "maximum": 1},
                        "related_concepts": {
                            "type": "array",
                            "items": {"type": "string"}
                        }
                    }
                },
                "minItems": 1
            },
            "structure": {
                "type": "object",
                "required": ["sections"],
                "properties": {
                    "sections": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "required": ["title", "content"],
                            "properties": {
                                "title": {"type": "string"},
                                "content": {"type": "string", "minLength": 10},
                                "level": {"type": "integer", "minimum": 1},
                                "subsections": {
                                    "type": "array",
                                    "items": {"$ref": "#/properties/structure/properties/sections/items"}
                                }
                            }
                        },
                        "minItems": 1
                    },
                    "narrative_flow": {
                        "type": "array",
                        "items": {"type": "string"}
                    }
                }
            },
            "metadata": {
                "type": "object",
                "properties": {
                    "domain": {"type": "string"},
                    "domain_confidence": {"type": "number", "minimum": 0, "maximum": 1},
                    "language": {"type": "string"},
                    "source_type": {"type": "string"},
                    "analysis_version": {"type": "string"}
                }
            },
            "quiz": {
                "type": "array",
                "items": {
                    "type": "object",
                    "required": ["question", "options", "correct_answer"],
                    "properties": {
                        "question": {"type": "string", "minLength": 10},
                        "options": {
                            "type": "array",
                            "items": {"type": "string", "minLength": 1},
                            "minItems": 2
                        },
                        "correct_answer": {"type": ["string", "integer"]},
                        "explanation": {"type": "string"},
                        "difficulty": {"type": "string", "enum": ["easy", "medium", "hard"]}
                    }
                }
            }
        },
        "additionalProperties": true
    }
    
    # Schema for domain-specific analysis results
    DOMAIN_SPECIFIC_SCHEMAS = {
        "mathematics": {
            "type": "object",
            "required": ["equations", "variables", "complexity_level"],
            "properties": {
                "equations": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": ["latex", "description"],
                        "properties": {
                            "latex": {"type": "string"},
                            "description": {"type": "string"},
                            "variables": {
                                "type": "array",
                                "items": {"type": "string"}
                            }
                        }
                    }
                },
                "variables": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": ["name", "description"],
                        "properties": {
                            "name": {"type": "string"},
                            "description": {"type": "string"},
                            "units": {"type": "string"}
                        }
                    }
                },
                "complexity_level": {"type": "string"}
            }
        },
        "physics": {
            "type": "object",
            "required": ["principles", "experiments"],
            "properties": {
                "principles": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "description": {"type": "string"},
                            "equations": {
                                "type": "array",
                                "items": {"type": "string"}
                            }
                        }
                    }
                },
                "experiments": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "description": {"type": "string"},
                            "outcomes": {
                                "type": "array",
                                "items": {"type": "string"}
                            }
                        }
                    }
                }
            }
        },
        "computer_science": {
            "type": "object",
            "required": ["algorithms", "code_examples"],
            "properties": {
                "algorithms": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "description": {"type": "string"},
                            "complexity": {"type": "string"}
                        }
                    }
                },
                "code_examples": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "language": {"type": "string"},
                            "code": {"type": "string"},
                            "explanation": {"type": "string"}
                        }
                    }
                }
            }
        }
    }
    
class ContentAnalysisPipeline:
    """
    Pipeline for analyzing extracted PDF content,
    with robust schema validation and domain-specific processing
    """
    
    def __init__(self, domain: str = "general", model_path: Optional[str] = None):
        """
        Initialize the content analysis pipeline
        
        Args:
            domain: Default domain for content analysis
            model_path: Path to domain classification model
        """
        self.domain = domain
        self.model_path = model_path
        self.logger = logging.getLogger(self.__class__.__name__)
        
        # Configure processing pipeline
        self._configure_pipeline()
    
    def _configure_pipeline(self):
        """Configure the analysis pipeline components"""
        # Initialize content analyzer for text processing
        self.content_analyzer = ContentAnalyzer()
        
        # Initialize content relationship analyzer
        self.relationship_analyzer = ContentRelationshipAnalyzer()
        
        # Initialize content enrichment processor
        self.enrichment_processor = ContentEnrichmentProcessor()
        
        # Initialize visualization element extractor
        self.visualization_extractor = VisualizationElementExtractor()
        
        # Initialize LLM client for advanced analysis if available
        try:
            # The model_path parameter is used to provide the OpenRouter API key
            if self.model_path:
                # Initialize with API key if provided
                self.llm_client = OpenRouterMistralClient(api_key=self.model_path)
                self.logger.info("LLM client initialized with provided API key")
            else:
                # Initialize without API key (will need to be updated later)
                self.llm_client = OpenRouterMistralClient()
                self.logger.warning("LLM client initialized without API key. Some features may be unavailable.")
            
            self.llm_available = True
            self.logger.info("LLM client initialized successfully")
        except Exception as e:
            self.llm_available = False
            self.logger.warning(f"LLM client initialization failed: {str(e)}. Using fallback methods.")
        
        # Initialize domain processors with lazy loading
        self.domain_processors = {}
        
        # Initialize text processing tools
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # TF-IDF vectorizer for keyword extraction
        self.tfidf = TfidfVectorizer(max_features=100, stop_words='english')
        
        # Domain keywords for classification
        self.domain_keywords = {
            "mathematics": ["equation", "theorem", "formula", "calculation", "algebra", "geometry", 
                           "calculus", "function", "variable", "constant", "proof", "matrix"],
            "physics": ["force", "energy", "motion", "gravity", "particle", "wave", "quantum", 
                       "relativity", "momentum", "acceleration", "velocity", "mass", "charge"],
            "computer_science": ["algorithm", "programming", "code", "function", "data", "variable", 
                               "class", "object", "database", "network", "system", "software", "hardware"],
            "biology": ["cell", "organism", "dna", "evolution", "species", "protein", "gene", 
                       "tissue", "enzyme", "chromosome", "ecology", "metabolism"],
            "chemistry": ["element", "compound", "reaction", "molecule", "acid", "base", "bond", 
                         "solution", "atom", "electron", "periodic", "organic"]
        }
        
        self.logger.info("Content analysis pipeline configured successfully")
    
    def analyze_content(self, extracted_content: Dict[str, Any], output_dir: str,
                       domain_focus: Optional[str] = None, grade_level: str = "general",
                       detail_level: str = "standard", include_quiz: bool = False,
                       language: str = "en") -> Dict[str, Any]:
        """
        Analyze the extracted content
        
        Args:
            extracted_content: The extracted PDF content
            output_dir: Directory to store analysis output
            domain_focus: Domain to focus analysis on (e.g., "mathematics", "physics")
            grade_level: Educational grade level (e.g., "elementary", "university")
            detail_level: Level of detail for analysis
            include_quiz: Whether to generate quiz questions
            language: Content language
            
        Returns:
            Dictionary containing analyzed content
            
        Raises:
            ContentAnalysisError: If analysis fails
        """
        try:
            # Validate input content
            self._validate_input_content(extracted_content)
            
            # Detect content domain if not specified
            detected_domain, domain_confidence = self._detect_domain(extracted_content, domain_focus)
            
            # Analyze content structure
            structure = self._analyze_structure(extracted_content, grade_level, detail_level)
            
            # Extract key concepts
            concepts = self._extract_concepts(extracted_content, detected_domain, grade_level)
            
            # Generate summary
            summary = self._generate_summary(extracted_content, detected_domain, 
                                           grade_level, detail_level)
            
            # Generate domain-specific analysis
            domain_specific = self._generate_domain_specific_analysis(
                extracted_content, detected_domain, grade_level, detail_level
            )
            
            # Generate quiz if requested
            quiz = self._generate_quiz(extracted_content, concepts, grade_level) if include_quiz else []
            
            # Compile results
            result = {
                "topic": self._generate_topic(extracted_content, detected_domain),
                "summary": summary,
                "concepts": concepts,
                "structure": structure,
                "metadata": {
                    "domain": detected_domain,
                    "domain_confidence": domain_confidence,
                    "grade_level": grade_level,
                    "detail_level": detail_level,
                    "language": language,
                    "source_type": "pdf",
                    "analysis_version": "1.0"
                }
            }
            
            # Add domain-specific analysis if available
            if domain_specific:
                result["domain_specific"] = domain_specific
            
            # Add quiz if generated
            if quiz:
                result["quiz"] = quiz
            
            # Validate the output against schema
            self._validate_analysis_result(result, detected_domain)
            
            # Save results to output directory
            os.makedirs(output_dir, exist_ok=True)
            with open(os.path.join(output_dir, "analysis_result.json"), "w") as f:
                json.dump(result, f, indent=2)
            
            return result
            
        except ValidationError as e:
            # Handle schema validation errors
            schema_error = f"Schema validation error: {e.message}"
            self.logger.error(schema_error)
            raise ContentStructuringError(schema_error, details={"schema_path": e.schema_path})
            
        except Exception as e:
            # Handle other errors
            error_message = f"Content analysis failed: {str(e)}"
            self.logger.error(error_message)
            
            if isinstance(e, ContentAnalysisError):
                raise
            else:
                raise ContentAnalysisError(error_message)
    
    def _validate_input_content(self, content: Dict[str, Any]) -> None:
        """
        Validate input content structure
        
        Args:
            content: Extracted content dictionary
            
        Raises:
            ContentAnalysisError: If content is invalid
        """
        if not isinstance(content, dict):
            raise ContentAnalysisError("Extracted content must be a dictionary")
        
        # Check for required fields
        required_fields = ["content", "pages"]
        missing_fields = [field for field in required_fields if field not in content]
        
        if missing_fields:
            raise ContentAnalysisError(
                f"Extracted content missing required fields: {', '.join(missing_fields)}"
            )
        
        # Check content type and minimum length
        if not isinstance(content.get("content"), str) or len(content.get("content", "")) < 50:
            raise ContentAnalysisError(
                "Extracted content must contain text content of sufficient length"
            )
        
        # Check pages structure
        if not isinstance(content.get("pages"), list) or not content.get("pages"):
            raise ContentAnalysisError("Extracted content must contain pages data")
    
    def _validate_analysis_result(self, result: Dict[str, Any], domain: str) -> None:
        """
        Validate analysis result against schema
        
        Args:
            result: Analysis result dictionary
            domain: Detected content domain
            
        Raises:
            ValidationError: If result does not match schema
        """
        # Validate against base schema
        validate(instance=result, schema=ContentAnalysisSchema.ANALYSIS_RESULT_SCHEMA)
        
        # Validate domain-specific parts if present
        if domain in ContentAnalysisSchema.DOMAIN_SPECIFIC_SCHEMAS and "domain_specific" in result:
            domain_schema = ContentAnalysisSchema.DOMAIN_SPECIFIC_SCHEMAS[domain]
            validate(instance=result["domain_specific"], schema=domain_schema)
    
    def _detect_domain(self, content: Dict[str, Any], domain_focus: Optional[str] = None) \
            -> Tuple[str, float]:
        """
        Detect the content domain
        
        Args:
            content: Extracted content dictionary
            domain_focus: Optional domain focus
            
        Returns:
            Tuple of (detected_domain, confidence)
        """
        # If domain focus is specified, use it with high confidence
        if domain_focus and domain_focus != "auto":
            return domain_focus, 1.0
        
        try:
            # Extract text content
            text = content.get("content", "")
            
            # Tokenize and preprocess text
            tokens = word_tokenize(text.lower())
            
            # Remove stopwords and lemmatize
            filtered_tokens = [
                self.lemmatizer.lemmatize(token) 
                for token in tokens 
                if token.isalpha() and token not in self.stop_words
            ]
            
            # Count domain keywords in text
            domain_scores = {}
            for domain, keywords in self.domain_keywords.items():
                # Calculate the presence of domain keywords in filtered tokens
                keyword_count = sum(1 for token in filtered_tokens if token in keywords)
                # Weight by document length to avoid bias towards longer documents
                domain_scores[domain] = keyword_count / max(1, len(filtered_tokens)) * 100
            
            # If we have LLM access, enhance with more sophisticated analysis
            if self.llm_available and len(text) > 200:
                try:
                    # Check if API key is available
                    if hasattr(self.llm_client, 'api_key') and self.llm_client.api_key:
                        # Prepare a prompt for domain classification
                        prompt = f"Classify the following text into one of these domains: {', '.join(self.domain_keywords.keys())}.\n\nText: {text[:2000]}\n\nDomain:"
                        llm_result = self.llm_client.generate(prompt, max_tokens=20)
                        llm_domain = llm_result.lower().strip()
                        
                        # If LLM returns a recognized domain, boost its score
                        for domain in self.domain_keywords:
                            if domain in llm_domain:
                                domain_scores[domain] = domain_scores.get(domain, 0) + 30
                                break
                else:
                        self.logger.warning("LLM domain classification skipped: API key not set")
                except Exception as e:
                    self.logger.warning(f"LLM domain classification failed: {str(e)}")
            
            # Determine the most likely domain
            if domain_scores:
                best_domain = max(domain_scores.items(), key=lambda x: x[1])
                domain_name, score = best_domain
                
                # Calculate confidence (normalize score to 0.5-0.95 range)
                confidence = min(0.95, 0.5 + (score / 200))
                
                # Only return a specific domain if confidence is above threshold
                if confidence > 0.6:
                    return domain_name, confidence
            
            # Check for tables and equations as indicators of technical domains
            has_tables = len(content.get("tables", [])) > 0
            has_equations = "equations" in content or "formulas" in text.lower()
            
            if has_equations and has_tables:
                return "mathematics", 0.7
            elif has_tables and "data" in text.lower():
                return "computer_science", 0.65
            
            # Default to general with moderate confidence
            return "general", 0.6
            
        except Exception as e:
            self.logger.error(f"Domain detection failed: {str(e)}")
            # Fallback to general domain with low confidence
            return "general", 0.5
    
    def _analyze_structure(self, content: Dict[str, Any], grade_level: str, detail_level: str) \
            -> Dict[str, Any]:
        """
        Analyze content structure
        
        Args:
            content: Extracted content dictionary
            grade_level: Educational grade level
            detail_level: Level of detail
            
        Returns:
            Content structure dictionary
        """
        try:
            # Extract the pages data
            pages = content.get("pages", [])
            full_text = content.get("content", "")
            
            # Initialize structure components
            sections = []
            narrative_flow = ["introduction"]
            
            # If no pages, create a basic structure
            if not pages:
                if full_text:
                    # Try to identify sections from the full text
                    paragraphs = full_text.split('\n\n')
                    
                    if len(paragraphs) > 3:
                        intro = paragraphs[0]
                        main_content = ' '.join(paragraphs[1:-1])
                        conclusion = paragraphs[-1]
                        
                        sections = [
                            {"title": "Introduction", "content": intro, "level": 1},
                            {"title": "Main Content", "content": main_content, "level": 1},
                            {"title": "Conclusion", "content": conclusion, "level": 1}
                        ]
                        narrative_flow = ["introduction", "main_content", "conclusion"]
                                else:
                        sections = [{"title": "Content", "content": full_text, "level": 1}]
                        narrative_flow = ["content"]
                        else:
                    # Return minimal structure if no content
                    return {
                        "sections": [{"title": "Empty Document", "content": "No content available", "level": 1}],
                        "narrative_flow": ["empty"]
                    }
                else:
                # Process page data to identify sections
                section_candidates = []
                current_level = 1
                
                # Analyze text for headings
                heading_pattern = re.compile(r'^(#+|\d+\.|\b(?:CHAPTER|SECTION)\b)', re.IGNORECASE)
                
                # Extract potential sections from page data
                for page in pages:
                    page_text = page.get("text", "")
                    lines = page_text.split('\n')
                    
                    for line in lines:
                        line = line.strip()
                        if not line:
                            continue
                            
                        # Check if this line looks like a heading
                        if heading_pattern.match(line) or (len(line) < 100 and line.isupper()):
                            # Determine heading level
                            level = 1
                            if line.startswith('##'):
                                level = 3 if line.startswith('###') else 2
                            elif line.startswith('#'):
                                level = 1
                            elif '.' in line and line[0].isdigit():
                                # Check for nested numbering like "1.2.3"
                                dots = line.split('.', 1)[0].count('.')
                                level = dots + 1
                            
                            # Save as section candidate
                            section_candidates.append({
                                "title": line.lstrip('#. '),
                                "content": "",
                                "level": level,
                                "index": len(section_candidates)
                            })
                        elif section_candidates:
                            # Append content to the latest section
                            section_candidates[-1]["content"] += line + " "
                
                # If we found sections, organize them into a hierarchy
                if section_candidates:
                    # Normalize section levels to ensure they start at 1
                    min_level = min(section["level"] for section in section_candidates)
                    for section in section_candidates:
                        section["level"] = max(1, section["level"] - min_level + 1)
                    
                    # Organize sections into a nested structure
                    sections = self._organize_sections(section_candidates)
                    
                    # Define the narrative flow based on sections
                    narrative_flow = ["introduction"]
                    if len(sections) > 2:
                        narrative_flow.extend(["development"] * (len(sections) - 2))
                    if len(sections) > 1:
                        narrative_flow.append("conclusion")
                    else:
                    # If no section candidates, create generic sections based on pages
                    if len(pages) > 3:
                        # First page as intro, last as conclusion
                        sections = [
                            {"title": "Introduction", "content": pages[0].get("text", ""), "level": 1},
                            {"title": "Main Content", "content": " ".join(p.get("text", "") for p in pages[1:-1]), "level": 1},
                            {"title": "Conclusion", "content": pages[-1].get("text", ""), "level": 1}
                        ]
                        narrative_flow = ["introduction", "main_content", "conclusion"]
                    else:
                        # For short documents, just use all content
                        sections = [{"title": "Content", "content": " ".join(p.get("text", "") for p in pages), "level": 1}]
                        narrative_flow = ["content"]
            
            # Adjust level of detail based on parameter
            if detail_level == "minimal":
                # Keep only top-level sections
                sections = [s for s in sections if s["level"] == 1]
            elif detail_level == "detailed":
                # Keep all sections, already handled above
                pass
            else:  # "standard" detail level
                # Keep up to level 3
                sections = self._filter_sections_by_max_level(sections, 3)
            
            # Adjust for grade level
            if grade_level == "elementary":
                # Simplify content for younger students
                for section in sections:
                    if len(section["content"]) > 300:
                        section["content"] = section["content"][:300] + "..."
            
            return {
                "sections": sections,
                "narrative_flow": narrative_flow
            }
            
        except Exception as e:
            self.logger.error(f"Structure analysis failed: {str(e)}")
            # Return a basic structure as fallback
            return {
                "sections": [{"title": "Content", "content": content.get("content", "")[:1000], "level": 1}],
                "narrative_flow": ["content"]
            }
    
    def _organize_sections(self, section_candidates: List[Dict]) -> List[Dict]:
        """Helper to organize sections into a properly nested structure"""
        if not section_candidates:
            return []
            
        # Sort by original index to maintain order
        sorted_sections = sorted(section_candidates, key=lambda x: x["index"])
        
        # Create a new list without the index field
        clean_sections = []
        for section in sorted_sections:
            new_section = {
                "title": section["title"],
                "content": section["content"].strip(),
                "level": section["level"]
            }
            
            # Add empty subsections list if it might have children
            if any(s["level"] > section["level"] for s in sorted_sections):
                new_section["subsections"] = []
                
            clean_sections.append(new_section)
        
        # Build the hierarchy (only for top-level sections)
        result = []
        i = 0
        while i < len(clean_sections):
            if clean_sections[i]["level"] == 1:
                # Start with a top-level section
                current_section = clean_sections[i]
                i += 1
                
                # Process subsections
                self._add_subsections(current_section, clean_sections, i)
                
                # Add to result
                result.append(current_section)
                else:
                # Skip non-top-level sections as they'll be handled by _add_subsections
                i += 1
                
        return result
    
    def _add_subsections(self, parent: Dict, all_sections: List[Dict], start_idx: int) -> int:
        """Recursively add subsections to a parent section"""
        parent_level = parent["level"]
        i = start_idx
        
        # Make sure parent has subsections field
        if "subsections" not in parent:
            parent["subsections"] = []
            
        while i < len(all_sections):
            current = all_sections[i]
            
            if current["level"] <= parent_level:
                # If same or higher level, we're done with this parent
                return i
            
            if current["level"] == parent_level + 1:
                # Direct child of parent
                parent["subsections"].append(current)
                i += 1
                
                # Process children of this child
                i = self._add_subsections(current, all_sections, i)
                    else:
                # Skip sections that are too deeply nested
                i += 1
                
        return i
    
    def _filter_sections_by_max_level(self, sections: List[Dict], max_level: int) -> List[Dict]:
        """Filter sections to keep only up to a certain level"""
        result = []
        
        for section in sections:
            new_section = {
                "title": section["title"],
                "content": section["content"],
                "level": section["level"]
            }
            
            # Handle subsections
            if "subsections" in section and section["level"] < max_level:
                new_section["subsections"] = self._filter_sections_by_max_level(
                    section["subsections"], max_level)
                
            result.append(new_section)
            
        return result
    
    def _extract_concepts(self, content: Dict[str, Any], domain: str, grade_level: str) \
            -> List[Dict[str, Any]]:
        """
        Extract key concepts from content
        
        Args:
            content: Extracted content dictionary
            domain: Content domain
            grade_level: Educational grade level
            
        Returns:
            List of concept dictionaries
        """
        try:
            # Get text content
            text = content.get("content", "")
            if not text:
                return [{"name": "Unknown", "description": "No content available for concept extraction.", 
                        "importance": 0.5, "related_concepts": []}]
            
            # Extract domain-specific concepts first if available
            domain_concepts = []
            
            # Check if we need to load a domain processor
            if domain != "general" and domain not in self.domain_processors:
                try:
                    # Lazy-load domain processor
                    processor_class = DomainSpecificProcessor.get_processor_for_domain(domain)
                    if processor_class:
                        self.domain_processors[domain] = processor_class()
                except Exception as e:
                    self.logger.warning(f"Failed to load domain processor for {domain}: {str(e)}")
            
            # Use domain processor if available
            if domain in self.domain_processors:
                domain_concepts = self.domain_processors[domain].extract_concepts(content, grade_level)
            
            # If domain-specific concepts were found, return them
            if domain_concepts:
                return domain_concepts
                
            # Otherwise, use general NLP approach
            # Tokenize content and remove stopwords
            tokens = word_tokenize(text.lower())
            filtered_tokens = [self.lemmatizer.lemmatize(token) for token in tokens 
                              if token.isalpha() and token not in self.stop_words]
            
            # Create corpus for TF-IDF
            sentences = sent_tokenize(text)
            
            if not sentences:
                return [{"name": "Unknown", "description": "Content could not be processed for concepts.", 
                        "importance": 0.5, "related_concepts": []}]
            
            # Process sentences for TF-IDF
            try:
                # Fit TF-IDF vectorizer
                self.tfidf.fit(sentences)
                
                # Get feature names
                feature_names = self.tfidf.get_feature_names_out()
                
                # Transform the corpus
                tfidf_matrix = self.tfidf.transform(sentences)
                
                # Get top terms by TF-IDF score
                max_val = tfidf_matrix.max(axis=0).toarray().ravel()
                sorted_indices = max_val.argsort()[::-1][:15]  # Get top 15 terms
                
                # Get the actual terms
                top_terms = [feature_names[i] for i in sorted_indices if max_val[i] > 0]
                
                # If we have too few terms, add more based on frequency
                if len(top_terms) < 5:
                    # Count word frequencies
                    word_freq = {}
                    for token in filtered_tokens:
                        word_freq[token] = word_freq.get(token, 0) + 1
                        
                    # Add most frequent words not already in top_terms
                    freq_terms = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
                    for term, freq in freq_terms:
                        if term not in top_terms and len(term) > 3:  # Skip very short words
                            top_terms.append(term)
                            if len(top_terms) >= 10:
                                break
            except Exception as e:
                self.logger.warning(f"TF-IDF processing failed: {str(e)}. Using fallback method.")
                # Fallback: Use simple frequency
                word_freq = {}
                for token in filtered_tokens:
                    word_freq[token] = word_freq.get(token, 0) + 1
                
                # Get top terms by frequency
                top_terms = [term for term, freq in sorted(word_freq.items(), 
                                                         key=lambda x: x[1], 
                                                         reverse=True)[:15]
                            if len(term) > 3]  # Skip very short words
            
            # Use LLM to enhance concept descriptions if available
            concepts = []
            for term in top_terms[:10]:  # Limit to top 10
                # Find relevant sentences
                relevant_sentences = [s for s in sentences if term.lower() in s.lower()]
                context = " ".join(relevant_sentences[:3])  # Use up to 3 sentences
                
                if self.llm_available and hasattr(self.llm_client, 'api_key') and self.llm_client.api_key:
                    try:
                        # Ask LLM to generate concept description
                        prompt = f"Based on the following context, provide a concise description (max 50 words) of the concept '{term}'.\n\nContext: {context}\n\nDescription:"
                        description = self.llm_client.generate(prompt, max_tokens=100)
                        
                        # Also get related concepts
                        related_prompt = f"List 3 concepts related to '{term}' based on this context: {context}\n\nRelated concepts (comma-separated):"
                        related_concepts_text = self.llm_client.generate(related_prompt, max_tokens=50)
                        related_concepts = [c.strip() for c in related_concepts_text.split(',')]
                    except Exception as e:
                        self.logger.warning(f"LLM concept enhancement failed: {str(e)}. Using fallback.")
                        description = f"A key concept related to {term}."
                        related_concepts = []
            else:
                    # Fallback without LLM or when API key not set
                    description = context[:200] + "..." if len(context) > 200 else context
                    related_concepts = []
                
                # Calculate importance based on position in top_terms
                importance = 1.0 - (top_terms.index(term) / len(top_terms))
                
                concepts.append({
                    "name": term.capitalize(),
                    "description": description,
                    "importance": round(importance, 2),
                    "related_concepts": related_concepts[:3]  # Limit to 3 related concepts
                })
            
            return concepts
            
        except Exception as e:
            self.logger.error(f"Concept extraction failed: {str(e)}")
            # Return a minimal placeholder in case of errors
            return [
                {
                    "name": "Main Topic",
                    "description": "The central topic of the document.",
                    "importance": 1.0,
                    "related_concepts": []
                }
            ]
    
    def _generate_summary(self, content: Dict[str, Any], domain: str, 
                        grade_level: str, detail_level: str) -> Dict[str, Any]:
        """
        Generate content summary
        
        Args:
            content: Extracted content dictionary
            domain: Content domain
            grade_level: Educational grade level
            detail_level: Level of detail
            
        Returns:
            Summary dictionary
        """
        try:
            # Get the text content
            text = content.get("content", "")
            if not text:
                return {
                    "brief": "No content available to summarize.",
                    "key_points": ["Content unavailable"],
                    "difficulty_level": grade_level,
                    "estimated_reading_time": 0
                }
            
            # Calculate estimated reading time (avg adult reads ~250 words per minute)
            word_count = len(text.split())
            reading_time = word_count / 250  # In minutes
            
            # Adjust reading time based on grade level
            if grade_level == "elementary":
                # Younger readers take longer
                reading_time *= 1.5
            elif grade_level == "university":
                # Advanced readers may be faster
                reading_time *= 0.9
            
            # Determine difficulty level based on content
            difficulty = self._estimate_difficulty(text, grade_level)
            
            # Generate summary based on detail level
            summary_length = {
                "minimal": 100,    # Very short summary
                "standard": 200,   # Medium length summary
                "detailed": 400    # Longer summary
            }.get(detail_level, 200)
            
            # Use LLM for abstractive summarization if available
            brief_summary = ""
            if self.llm_available and hasattr(self.llm_client, 'api_key') and self.llm_client.api_key:
                try:
                    # Truncate text if too long for prompt
                    truncated_text = text[:4000] if len(text) > 4000 else text
                    
                    # Prompt for general summary
                    prompt = f"Summarize the following text in about {summary_length} words. Focus on the main ideas and key concepts. Make the summary coherent and readable.\n\nText: {truncated_text}\n\nSummary:"
                    brief_summary = self.llm_client.generate(prompt, max_tokens=summary_length * 2)
                    
                    # Prompt for key points
                    points_prompt = f"Extract the 5 most important points from this text as a numbered list:\n\nText: {truncated_text}\n\nKey Points:"
                    key_points_text = self.llm_client.generate(points_prompt, max_tokens=300)
                    
                    # Parse key points into a list
                    key_points = []
                    for line in key_points_text.strip().split('\n'):
                        # Remove numbering and strip whitespace
                        clean_line = re.sub(r'^\d+\.?\s*', '', line).strip()
                        if clean_line:
                            key_points.append(clean_line)
                    
                    # If we didn't get enough key points, add generic ones
                    while len(key_points) < 3:
                        key_points.append(f"Additional important information from the {domain} content")
                    
                except Exception as e:
                    self.logger.warning(f"LLM summarization failed: {str(e)}. Using fallback method.")
                    brief_summary = ""  # Reset to use fallback
            
            # Fallback to extractive summarization if LLM failed or not available
            if not brief_summary:
                # Split text into sentences
                sentences = sent_tokenize(text)
                
                if not sentences:
                    return {
                        "brief": "Unable to process content for summarization.",
                        "key_points": ["Content could not be processed"],
                        "difficulty_level": difficulty,
                        "estimated_reading_time": reading_time
                    }
                
                # Calculate sentence scores
                scores = {}
                for sentence in sentences:
                    # Skip very short sentences
                    if len(sentence.split()) < 4:
                        continue

                    # Simple scoring based on sentence position and keywords
                    score = 0
                    
                    # First and last sentences often contain important information
                    if sentence == sentences[0]:
                        score += 0.3
                    elif sentence == sentences[-1]:
                        score += 0.2
                    
                    # Check for domain keywords
                    if domain in self.domain_keywords:
                        for keyword in self.domain_keywords[domain]:
                            if keyword in sentence.lower():
                                score += 0.1
                                break
                    
                    # Check for informative phrases
                    info_phrases = ["important", "significant", "key", "main", "essential", "fundamental"]
                    for phrase in info_phrases:
                        if phrase in sentence.lower():
                            score += 0.05
                            break
                    
                    scores[sentence] = score
                
                # Sort sentences by score
                sorted_sentences = sorted(scores.items(), key=lambda x: x[1], reverse=True)
                
                # Select top sentences
                top_sentence_count = max(3, int(len(sentences) * 0.2))  # At least 3 sentences or 20%
                if detail_level == "minimal":
                    top_sentence_count = min(3, top_sentence_count)
                elif detail_level == "detailed":
                    top_sentence_count = max(5, top_sentence_count)
                
                # Get top sentences and sort them by original order
                top_sentences = [s[0] for s in sorted_sentences[:top_sentence_count]]
                ordered_sentences = [s for s in sentences if s in top_sentences]
                
                # Create brief summary
                brief_summary = " ".join(ordered_sentences)
                
                # Truncate if too long
                if len(brief_summary) > summary_length * 5:  # Average word ~5 chars
                    brief_summary = brief_summary[:summary_length * 5] + "..."
                
                # Extract key points
                key_points = []
                for sentence in ordered_sentences[:5]:
                    # Simplify long sentences
                    if len(sentence.split()) > 15:
                        key_points.append(" ".join(sentence.split()[:15]) + "...")
            else:
                    key_points.append(sentence)
                
                # Ensure we have at least 3 key points
                while len(key_points) < 3:
                    key_points.append(f"Additional information from the {domain} content")
            
            return {
                "brief": brief_summary,
                "key_points": key_points[:5],  # Limit to 5 key points
                "difficulty_level": difficulty,
                "estimated_reading_time": round(reading_time, 1)
            }
            
        except Exception as e:
            self.logger.error(f"Summary generation failed: {str(e)}")
            # Return a default summary in case of errors
            return {
                "brief": "Summary unavailable due to processing error.",
                "key_points": ["Content available in full document"],
                "difficulty_level": grade_level,
                "estimated_reading_time": 5.0
            }
    
    def _estimate_difficulty(self, text: str, grade_level: str) -> str:
        """Estimate the difficulty level of the text"""
        # Simple implementation to estimate reading difficulty
        sentences = sent_tokenize(text)
        if not sentences:
            return grade_level
        
        # Get average sentence length
        avg_sentence_length = sum(len(s.split()) for s in sentences) / len(sentences)
        
        # Get percentage of complex words (rough approximation)
        words = [w.lower() for s in sentences for w in word_tokenize(s)]
        complex_word_count = sum(1 for w in words if len(w) > 8)
        complex_word_pct = complex_word_count / max(1, len(words))
        
        # Calculate a complexity score
        complexity_score = (avg_sentence_length * 0.6) + (complex_word_pct * 100 * 0.4)
        
        # Adjust based on input grade level
        if grade_level == "elementary":
            complexity_score -= 5
        elif grade_level == "university":
            complexity_score += 5
        
        # Map score to difficulty level
        if complexity_score < 10:
            return "beginner"
        elif complexity_score < 15:
            return "intermediate"
        elif complexity_score < 20:
            return "advanced"
                    else:
            return "expert"
    
    def _generate_domain_specific_analysis(self, content: Dict[str, Any], domain: str,
                                         grade_level: str, detail_level: str) \
            -> Optional[Dict[str, Any]]:
        """
        Generate domain-specific analysis
        
        Args:
            content: Extracted content dictionary
            domain: Content domain
            grade_level: Educational grade level
            detail_level: Level of detail
            
        Returns:
            Domain-specific analysis or None
        """
        try:
            # Only generate for supported domains
            if domain not in ContentAnalysisSchema.DOMAIN_SPECIFIC_SCHEMAS:
                return None
            
            # Extract text content
            text = content.get("content", "")
            if not text:
                return None
            
            # Use LLM for domain-specific analysis if available
            if self.llm_available and hasattr(self.llm_client, 'api_key') and self.llm_client.api_key:
                try:
                    # Truncate text if too long
                    truncated_text = text[:3000] if len(text) > 3000 else text
                    
                    # Create domain-specific prompts
                    if domain == "mathematics":
                        prompt = f"""
                        Extract mathematical equations and variables from this text:
                        {truncated_text}
                        
                        Format as JSON:
                        {{
                            "equations": [
                                {{"latex": "equation1", "description": "what it represents"}}
                            ],
                            "variables": [
                                {{"name": "variable name", "description": "what it represents", "units": "units of measurement"}}
                            ],
                            "complexity_level": "beginner/intermediate/advanced"
                        }}
                        """
                    elif domain == "physics":
                        prompt = f"""
                        Extract physics principles and experiments from this text:
                        {truncated_text}
                        
                        Format as JSON:
                        {{
                            "principles": [
                                {{"name": "principle name", "description": "what it states"}}
                            ],
                            "experiments": [
                                {{"name": "experiment name", "description": "what it demonstrates"}}
                            ]
                        }}
                        """
                    elif domain == "computer_science":
                        prompt = f"""
                        Extract algorithms and code examples from this text:
                        {truncated_text}
                        
                        Format as JSON:
                        {{
                            "algorithms": [
                                {{"name": "algorithm name", "description": "what it does", "complexity": "time complexity"}}
                            ],
                            "code_examples": [
                                {{"language": "programming language", "code": "code snippet", "explanation": "what it does"}}
                            ]
                        }}
                        """
        else:
                        return None
                    
                    # Generate LLM response
                    llm_response = self.llm_client.generate(prompt, max_tokens=500)
                    
                    # Try to parse JSON response
                    try:
                        # Extract JSON object from response
                        json_str = re.search(r'({.*})', llm_response, re.DOTALL)
                        if json_str:
                            domain_analysis = json.loads(json_str.group(1))
                            return domain_analysis
                    except Exception as e:
                        self.logger.warning(f"Failed to parse LLM domain analysis: {str(e)}")
                
                except Exception as e:
                    self.logger.warning(f"LLM domain analysis failed: {str(e)}")
            
            # Fallback to basic templates if LLM unavailable or failed
            if domain == "mathematics":
                return {
                    "equations": [{"latex": "E = mc^2", "description": "Energy-mass equivalence"}],
                    "variables": [{"name": "E", "description": "Energy", "units": "joules"}],
                    "complexity_level": "intermediate"
                }
            elif domain == "physics":
                return {
                    "principles": [{"name": "Conservation of Energy", "description": "Energy cannot be created or destroyed"}],
                    "experiments": [{"name": "Double Slit", "description": "Demonstrates wave-particle duality"}]
                }
            elif domain == "computer_science":
                return {
                    "algorithms": [{"name": "Quicksort", "description": "Sorting algorithm", "complexity": "O(n log n)"}],
                    "code_examples": [{"language": "Python", "code": "print('Hello world')", "explanation": "Basic output"}]
                }
            
            return None
            
        except Exception as e:
            self.logger.error(f"Domain-specific analysis failed: {str(e)}")
            return None
    
    def _generate_quiz(self, content: Dict[str, Any], concepts: List[Dict[str, Any]],
                     grade_level: str) -> List[Dict[str, Any]]:
        """
        Generate quiz questions based on content
        
        Args:
            content: Extracted content dictionary
            concepts: Extracted concepts
            grade_level: Educational grade level
            
        Returns:
            List of quiz questions
        """
        try:
            # If no concepts available, return a simple default quiz
            if not concepts:
                return [{
                    "question": "What is the main topic discussed in this content?",
                    "options": ["Topic A", "Topic B", "Topic C", "Topic D"],
                    "correct_answer": 0,
                    "explanation": "The content primarily discusses Topic A.",
                    "difficulty": "medium"
                }]
            
            # Extract text content
            text = content.get("content", "")
            if not text:
                return []
                
            quiz_questions = []
            
            # Generate questions based on top concepts
            if self.llm_available and hasattr(self.llm_client, 'api_key') and self.llm_client.api_key:
                try:
                    # Process top concepts (maximum 5)
                    for i, concept in enumerate(concepts[:5]):
                        concept_name = concept["name"]
                        concept_desc = concept["description"]
                        
                        # Find relevant sentences for context
                        sentences = sent_tokenize(text)
                        relevant_sentences = [s for s in sentences if concept_name.lower() in s.lower()]
                        context = " ".join(relevant_sentences[:3]) if relevant_sentences else text[:500]
                        
                        # Generate a question about this concept
                        question_prompt = f"""
                        Create a multiple-choice question about the concept '{concept_name}' based on this context:
                        {context}
                        
                        Format:
                        Question: (the question)
                        Option A: (first option)
                        Option B: (second option)
                        Option C: (third option)
                        Option D: (fourth option)
                        Correct: (A, B, C, or D)
                        Explanation: (why this answer is correct)
                        Difficulty: (easy, medium, or hard)
                        """
                        
                        question_response = self.llm_client.generate(question_prompt, max_tokens=300)
                        
                        # Parse the response
                        try:
                            lines = question_response.strip().split('\n')
                            question_text = next((line.split(':', 1)[1].strip() 
                                              for line in lines if line.startswith('Question:')), 
                                             f"What best describes {concept_name}?")
                            
                            options = []
                            for letter in ['A', 'B', 'C', 'D']:
                                option = next((line.split(':', 1)[1].strip() 
                                           for line in lines if line.startswith(f'Option {letter}:')), 
                                          f"Option about {concept_name}")
                                options.append(option)
                            
                            correct_letter = next((line.split(':', 1)[1].strip() 
                                              for line in lines if line.startswith('Correct:')), 'A')
                            correct_index = ord(correct_letter[0]) - ord('A')  # Convert letter to index
                            
                            explanation = next((line.split(':', 1)[1].strip() 
                                            for line in lines if line.startswith('Explanation:')), 
                                           f"Related to the concept of {concept_name}")
                            
                            difficulty = next((line.split(':', 1)[1].strip() 
                                           for line in lines if line.startswith('Difficulty:')), 'medium')
                            
                            # Validate difficulty
                            if difficulty not in ['easy', 'medium', 'hard']:
                                difficulty = 'medium'
                                
                            # Validate correct index
                            if correct_index < 0 or correct_index >= len(options):
                                correct_index = 0
                            
                            quiz_questions.append({
                                "question": question_text,
                                "options": options,
                                "correct_answer": correct_index,
                                "explanation": explanation,
                                "difficulty": difficulty
                            })
                            
                        except Exception as e:
                            self.logger.warning(f"Failed to parse LLM quiz question: {str(e)}")
                            # Add a fallback question
                            quiz_questions.append({
                                "question": f"Which statement best describes {concept_name}?",
                                "options": [
                                    concept_desc[:100],
                                    f"The opposite of {concept_name}",
                                    f"A concept unrelated to {concept_name}",
                                    f"None of the above"
                                ],
                                "correct_answer": 0,
                                "explanation": f"The first option correctly describes {concept_name}.",
                                "difficulty": "medium"
                            })
                            
                except Exception as e:
                    self.logger.warning(f"LLM quiz generation failed: {str(e)}. Using fallback method.")
                    # Will use fallback below
            
            # If we didn't generate any questions with LLM (or it failed), use fallback
            if not quiz_questions:
                # Generate simple questions based on concepts
                for i, concept in enumerate(concepts[:5]):
                    concept_name = concept["name"]
                    concept_desc = concept["description"]
                    
                    # Create basic question
                    quiz_questions.append({
                        "question": f"Which of the following best describes {concept_name}?",
                        "options": [
                            concept_desc[:100] + "..." if len(concept_desc) > 100 else concept_desc,
                            f"The opposite of {concept_name}",
                            f"A process unrelated to {concept_name}",
                            f"None of the above"
                        ],
                        "correct_answer": 0,
                        "explanation": f"The description matches the concept of {concept_name}.",
                        "difficulty": "medium"
                    })
            
            # Adjust quiz difficulty based on grade level
            for question in quiz_questions:
                if grade_level == "elementary":
                    question["difficulty"] = "easy"
                elif grade_level == "university":
                    if question["difficulty"] == "easy":
                        question["difficulty"] = "medium"
            
            return quiz_questions
            
        except Exception as e:
            self.logger.error(f"Quiz generation failed: {str(e)}")
            # Return a simple fallback quiz
            return [{
                "question": "What is the main topic of this content?",
                "options": ["Topic A", "Topic B", "Topic C", "Topic D"],
                "correct_answer": 0,
                "explanation": "The content primarily discusses Topic A.",
                "difficulty": "medium"
            }]
    
    def _generate_topic(self, content: Dict[str, Any], domain: str) -> str:
        """
        Generate a topic title for the content
        
        Args:
            content: Extracted content dictionary
            domain: Content domain
            
        Returns:
            Topic title string
        """
        try:
            # Extract text content
            text = content.get("content", "")
            
            if not text:
                return "Untitled Document"
                
            # Try to find title in metadata if available
            if "title" in content:
                return content["title"]
                
            # Try to extract from first page
            if "pages" in content and content["pages"]:
                first_page = content["pages"][0]
                if "title" in first_page:
                    return first_page["title"]
                    
                # Try to find a title-like line in the first page text
                if "text" in first_page:
                    lines = first_page["text"].split("\n")
                    # Look for short lines at the beginning that might be titles
                    for line in lines[:5]:
                        if 3 <= len(line.split()) <= 10 and any(c.isupper() for c in line):
                            return line.strip()
            
            # Use LLM to generate title if available
            if self.llm_available and hasattr(self.llm_client, 'api_key') and self.llm_client.api_key:
                try:
                    # Truncate text if too long
                    truncated_text = text[:3000] if len(text) > 3000 else text
                    
                    # Prompt for title generation
                    prompt = f"Based on the following text, generate a concise, descriptive title (maximum 10 words):\n\n{truncated_text}\n\nTitle:"
                    title = self.llm_client.generate(prompt, max_tokens=50).strip()
                    
                    # Check if we got a reasonable title
                    if title and len(title.split()) <= 12:
                        return title
                except Exception as e:
                    self.logger.warning(f"LLM title generation failed: {str(e)}. Using fallback method.")
            
            # Fallback: Extract key phrases for title
            sentences = sent_tokenize(text)
            if sentences:
                # Use first sentence for keywords
                first_sentence = sentences[0]
                
                # Remove stop words and extract nouns/important words
                words = word_tokenize(first_sentence.lower())
                filtered_words = [w for w in words if w.isalpha() and w not in self.stop_words]
                
                # Take the first few important words
                if filtered_words:
                    title_words = filtered_words[:4]
                    title = " ".join(w.capitalize() for w in title_words)
                    return f"{title} ({domain.capitalize()})"
            
            # Default title with domain
            return f"{domain.capitalize()} Content"
            
        except Exception as e:
            self.logger.error(f"Topic generation failed: {str(e)}")
            # Return generic title
            return f"Educational Content ({domain.capitalize()})"
    
    def set_api_key(self, api_key: str) -> bool:
        """
        Set or update the API key for the LLM client
        
        Args:
            api_key: OpenRouter API key (starts with sk-...)
            
        Returns:
            bool: True if successful, False otherwise
        """
        try:
            if self.llm_available and hasattr(self.llm_client, 'update_api_key'):
                success = self.llm_client.update_api_key(api_key)
                self.model_path = api_key  # Update the stored model_path as well
                self.logger.info("API key updated successfully")
                return success
            else:
                self.logger.warning("Cannot update API key: LLM client not available or doesn't support key updates")
                return False
        except Exception as e:
            self.logger.error(f"Failed to update API key: {str(e)}")
            return False
    

---


## ContentAnalyzer.py

In [ ]:
from contentHelper import *
from pipeline.adaptive_learning.DomainRulesManager import DomainRulesManager

class ContentAnalyzer:
    def __init__(self, language_model=None, domain=None, extractor=None):
        """
        Initialize with optional connection to an extractor
        
        Args:
            language_model: Optional pre-trained language model
            domain: Optional domain specification
            extractor: Optional PDFContentExtractor to use for segmentation
        """
        self.language_model = language_model
        self.domain = domain
        self.domain_specific_rules = self._load_domain_rules(domain)
        self.extractor = extractor
        
        # Create helper objects
        self.text_processor = TextProcessor()
        self.semantic_analyzer = SemanticAnalyzer(self.domain_specific_rules)
        self.keyword_extractor = KeywordExtractor(self.text_processor.stop_words, self.domain_specific_rules)
        self.sentiment_analyzer = SentimentAnalyzer()
        self.text_summarizer = TextSummarizer(self.keyword_extractor)
    
    def _load_domain_rules(self, domain):
        """
        Load domain-specific rules and patterns for content analysis.

        Args:
            domain: The domain to load rules for

        Returns:
            Dictionary of rules and patterns specific to the domain
        """
        return DomainRulesManager.get_domain_rules(domain)
    
    def analyze_content(self, extraction_result):
        """
        Main method to analyze extracted content from a PDF.
        
        Args:
            extraction_result: Dictionary containing the extracted content from PDFContentExtractor
            
        Returns:
            Dictionary containing the original content enriched with semantic annotations
        """
        analyzed_result = extraction_result.copy()
        if self.language_model is not None:
            return self._analyze_with_language_model(extraction_result)
        
        # Add semantic analysis to the content
        if 'content_blocks' in analyzed_result:
            analyzed_result['content_blocks'] = self.tag_content_blocks(analyzed_result['content_blocks'])
        
        # Extract keywords and concepts
        if 'text_content' in analyzed_result:
            analyzed_result['keywords_and_concepts'] = self.keyword_extractor.extract_keywords_and_concepts(
                analyzed_result['text_content'], self.domain
            )
        
        # Generate summaries for sections
        if 'sections' in analyzed_result:
            analyzed_result['section_summaries'] = {
                section_id: self.text_summarizer.generate_summaries(section_content)
                for section_id, section_content in analyzed_result['sections'].items()
            }
        
        # Segment content into logical units
        analyzed_result['content_segments'] = self.segment_content(extraction_result)
        
        # Add sentiment analysis if text is available
        if 'text_content' in analyzed_result:
            analyzed_result['sentiment_analysis'] = self.sentiment_analyzer.perform_sentiment_analysis(
                analyzed_result['text_content']
            )
        
        if self.domain == 'literature':
            try:
                # Extract full text content
                text_content = extraction_result.get('text_content', '')
                work_subtype = self._determine_literary_work_subtype(text_content)
    
                if work_subtype:
                    # Add literary analysis (without causing recursion)
                    analyzed_result['literary_analysis'] = self.analyze_literary_work(
                        extraction_result, work_subtype
                    )
            except Exception as e:
                # Fail gracefully if the literary analysis can't be performed
                analyzed_result['literary_analysis_error'] = str(e)
    
        return analyzed_result
    
    def segment_content(self, extraction_result):
        """
        Divides content into logical segments using extractor if available,
        otherwise uses built-in segmentation
        
        Args:
            extraction_result: Dictionary with extracted content
            
        Returns:
            List of content segments with semantic annotations
        """
        segments = []
        
        # Use extractor for segmentation if available
        if self.extractor and hasattr(self.extractor, 'segment_text'):
            # If we have structured content already
            if 'pages' in extraction_result:
                for page in extraction_result['pages']:
                    for block in page.get('blocks', []):
                        # Get base segmentation from extractor
                        if 'text' in block:
                            text_segments = self.extractor.segment_text(block['text'])
                            for text in text_segments:
                                # Add semantic annotation 
                                segment_type = self.semantic_analyzer.identify_segment_type(text)
                                segments.append({
                                    'type': segment_type,
                                    'content': text,
                                    'semantic_tags': self.semantic_analyzer.generate_semantic_tags(text)
                                })
            # If we only have raw text
            elif 'text_content' in extraction_result:
                text_segments = self.extractor.segment_text(extraction_result['text_content'])
                for text in text_segments:
                    segment_type = self.semantic_analyzer.identify_segment_type(text)
                    segments.append({
                        'type': segment_type,
                        'content': text,
                        'semantic_tags': self.semantic_analyzer.generate_semantic_tags(text)
                    })
        else:
            # Fall back to built-in segmentation
            if 'document_structure' in extraction_result:
                structure = extraction_result['document_structure']
                for section in structure:
                    # Create a segment for the heading
                    if 'heading' in section:
                        segments.append({
                            'type': 'heading',
                            'level': section.get('level', 1),
                            'content': section['heading'],
                            'semantic_tags': self.semantic_analyzer.generate_semantic_tags(section['heading'])
                        })
                    
                    # Process the content under this heading
                    if 'content' in section:
                        content_segments = self.text_processor.segment_text_block(section['content'])
                        segments.extend(content_segments)
            
            # If no structure is available, segment the raw text
            elif 'text_content' in extraction_result:
                segments = self.text_processor.segment_text_block(extraction_result['text_content'])
        
        return segments
    
    def tag_content_blocks(self, blocks):
        """
        Classify content blocks into specific types.
        
        Args:
            blocks: List of content blocks from the extraction result
            
        Returns:
            List of content blocks with type classification added
        """
        tagged_blocks = []
        
        for block in blocks:
            block_copy = block.copy()
            
            # Determine the block type based on content and structure
            if 'text' in block:
                block_copy['block_type'] = self.semantic_analyzer.identify_segment_type(block['text'])
                
                # Add additional semantic tags if possible
                block_copy['semantic_tags'] = self.semantic_analyzer.generate_semantic_tags(block['text'])
            
            tagged_blocks.append(block_copy)
            
        return tagged_blocks
    
    def analyze_literary_work(self, extraction_result, work_type='novel'):
        """
        Special method for analyzing literary works like Shakespeare plays

        Args:
            extraction_result: Dictionary containing the extracted content
            work_type: Type of literary work ('play', 'novel', 'poem')

        Returns:
            Dictionary with literary analysis results
        """
        # Create literary work analyzer
        literary_analyzer = LiteraryWorkAnalyzer(work_type)

        # Extract full text content
        text_content = extraction_result.get('text_content', '')

        # Initialize result dictionary
        literary_analysis = {
            'characters': literary_analyzer.analyze_characters(text_content),
            'structure': literary_analyzer.identify_scenes(text_content),
            'literary_devices': literary_analyzer.detect_literary_devices(text_content)
        }

        # For plays and poetry, add verse analysis
        if work_type in ['play', 'poem']:
            literary_analysis['verse_patterns'] = literary_analyzer.analyze_verse_patterns(text_content)

        # For Shakespeare works, add theme analysis
        if work_type == 'play:shakespeare':
            literary_analysis['themes'] = literary_analyzer.detect_shakespearean_themes(text_content)

        return literary_analysis

    # Delegate methods to appropriate classes for interface compatibility
    def extract_keywords_and_concepts(self, text, domain=None, max_terms=20, min_word_length=3):
        return self.keyword_extractor.extract_keywords_and_concepts(text, domain, max_terms, min_word_length)
    
    def extract_keywords_with_positions(self, text, domain=None, max_terms=20, min_word_length=3):
        return self.keyword_extractor.extract_keywords_with_positions(text, domain, max_terms, min_word_length)
    
    def extract_key_terms_with_importance(self, text, max_terms=20, min_word_length=3):
        return self.keyword_extractor.extract_key_terms_with_importance(text, self.domain_specific_rules, max_terms, min_word_length)
    
    def generate_summaries(self, sections, max_length=200):
        return self.text_summarizer.generate_summaries(sections, max_length)
    
    def perform_sentiment_analysis(self, text):
        return self.sentiment_analyzer.perform_sentiment_analysis(text)
    
    def _determine_literary_work_subtype(self, text_content):
        """
        Determine the subtype of literary work based on content analysis.

        Args:
            text_content: The raw text content

        Returns:
            String indicating work subtype or None
        """
        # Check for play-like structure (character names followed by dialogue)
        dialogue_patterns = re.findall(r'([A-Z]{2,}[A-Z\s]*)[\.:]\s', text_content)

        if len(dialogue_patterns) > 10:
            # This looks like a play
            work_subtype = 'play'

            # Check for Shakespeare-specific markers
            shakespeare_markers = ['thee', 'thou', 'thy', 'hath', 'doth', 'prithee', 'forsooth', 'anon']
            shakespeare_count = sum(text_content.lower().count(' ' + marker + ' ') for marker in shakespeare_markers)

            if shakespeare_count > 5:
                work_subtype += ':shakespeare'
        elif re.search(r'Chapter [IVXLCDM]+|Chapter \d+', text_content):
            work_subtype = 'novel'
        elif self.text_processor.detect_poetic_structure(text_content):
            work_subtype = 'poem'
        else:
            work_subtype = 'general'

        return work_subtype
    
    def _find_term_locations(self, text, term):
        return self.text_processor.find_term_locations(text, term)

    def _analyze_with_language_model(self, extraction_result):
        """
        Use the provided language model to perform content analysis.

        Args:
            extraction_result: Dictionary containing the extracted content

        Returns:
            Dictionary with enhanced analysis from the language model
        """
        analyzed_result = extraction_result.copy()
        text_content = extraction_result.get('text_content', '')

        if not text_content:
            # Fallback to original method if no text content is available
            return self.analyze_content(extraction_result)

        # Get comprehensive analysis from language model
        model_analysis = {
            'keywords_and_concepts': self.language_model.extract_keywords(text_content, self.domain),
            'sentiment_analysis': self.language_model.analyze_sentiment(text_content),
            'content_segments': self.language_model.segment_content(text_content),
            'semantic_tags': self.language_model.generate_semantic_tags(text_content, self.domain)
        }

        # Add section summaries if sections exist
        if 'sections' in extraction_result:
            model_analysis['section_summaries'] = {}
            for section_id, section_content in extraction_result['sections'].items():
                model_analysis['section_summaries'][section_id] = self.language_model.summarize_text(
                    section_content, max_length=200
                )

        # Add domain-specific analysis
        if self.domain == 'literature':
            try:
                work_subtype = self._determine_literary_work_subtype(text_content)
                if work_subtype:
                    model_analysis['literary_analysis'] = self.language_model.analyze_literary_work(
                        text_content, work_subtype
                    )
            except Exception as e:
                model_analysis['literary_analysis_error'] = str(e)

        # Merge the language model analysis with the extraction result
        analyzed_result.update(model_analysis)

        return analyzed_result

---


## ContentEnrichment.py

In [ ]:
from pipeline.content_analysis.ContentAnalyzer import ContentAnalyzer
import re
from collections import defaultdict

class ContentEnrichmentProcessor:
    def __init__(self, knowledge_base=None, content_analyzer=None):
        # Initialize with optional knowledge base
        self.knowledge_base = knowledge_base
        # Use composition to leverage ContentAnalyzer
        self.content_analyzer = content_analyzer if content_analyzer else ContentAnalyzer()
        
    def enrich_content(self, extraction_result):
        # Add contextual information to extraction result
        # Handle both dict and object types
        if hasattr(extraction_result, 'copy'):
            enriched_result = extraction_result.copy()
        elif isinstance(extraction_result, dict):
            enriched_result = extraction_result.copy()
        else:
            enriched_result = extraction_result
        
        # Extract concepts and terms for enrichment
        concepts = self.extract_concepts(enriched_result)
        
        # Add explanations for complex terms
        enriched_result = self.generate_explanations(enriched_result, concepts)
        
        # Add cross-references between related sections
        enriched_result = self.add_cross_references(enriched_result)
        
        # Suggest additional learning resources
        enriched_result = self.suggest_learning_resources(enriched_result, concepts)
        
        return enriched_result
        
    def extract_concepts(self, extraction_result):
        """Extract key concepts and terms from the extraction result with knowledge base enrichment"""
        concepts = []
        
        # Check if there's content to analyze
        if not extraction_result:
            return concepts
            
        # Get content text from extraction result
        text = self._extract_text_from_result(extraction_result)
        if not text:
            return concepts
            
        # Use ContentAnalyzer to extract keywords - leverage the core class!
        analysis_result = self.content_analyzer.extract_keywords_and_concepts(text)
        
        # Get keywords from the analysis result
        if isinstance(analysis_result, dict) and 'keywords' in analysis_result:
            # Handle case where keywords are returned as tuples of (word, frequency)
            if analysis_result['keywords'] and isinstance(analysis_result['keywords'][0], tuple):
                concepts = [keyword[0] for keyword in analysis_result['keywords']]
            else:
                concepts = analysis_result['keywords']
        
        # Add domain-specific enrichment
        if self.knowledge_base:
            # Filter concepts based on domain knowledge
            domain_concepts = []
            for concept in concepts:
                # Check if concept exists in our knowledge base
                if concept in self.knowledge_base:
                    # Prioritize concepts that are in our knowledge base
                    domain_concepts.append(concept)
            
            # Add specialized domain concepts from knowledge base that might be relevant
            for key in self.knowledge_base:
                # If term is in the text but wasn't caught by frequency analysis
                if key.lower() in text.lower() and key not in concepts:
                    domain_concepts.append(key)
            
            # If we found domain-specific concepts, prioritize them
            if domain_concepts:
                # Combine domain concepts with general concepts, prioritizing domain ones
                remaining_slots = max(20 - len(domain_concepts), 0)
                general_concepts = [c for c in concepts if c not in domain_concepts]
                concepts = domain_concepts + general_concepts[:remaining_slots]
        
        # Process multi-word concepts (which may not be caught by frequency analysis)
        section_titles = self._extract_section_titles(extraction_result)
        if section_titles:
            for title in section_titles:
                # Extract potential multi-word concepts
                matches = re.findall(r'[A-Z][a-z]+(?:\s+[a-z]+){1,3}', title)
                for match in matches:
                    if match.lower() in text.lower() and match not in concepts:
                        concepts.append(match)
        
        return concepts
    
    def _extract_text_from_result(self, extraction_result):
        """Helper method to extract text content from different extraction result formats"""
        if hasattr(extraction_result, 'content'):
            content = extraction_result.content
            return ' '.join(content) if isinstance(content, list) else content
        elif isinstance(extraction_result, dict):
            if 'content' in extraction_result:
                content = extraction_result['content']
                return ' '.join(content) if isinstance(content, list) else content
            elif 'text_content' in extraction_result:
                return extraction_result['text_content']
            elif 'sections' in extraction_result:
                # Extract text from sections
                sections_text = []
                for section in extraction_result['sections']:
                    if isinstance(section, dict):
                        if 'content' in section:
                            sections_text.append(section['content'])
                        elif 'text' in section:
                            sections_text.append(section['text'])
                return ' '.join(sections_text)
        return ""
    
    def _extract_section_titles(self, extraction_result):
        """Helper method to extract section titles from different extraction result formats"""
        section_titles = []
        
        if hasattr(extraction_result, 'sections'):
            for section in extraction_result.sections:
                if hasattr(section, 'title'):
                    section_titles.append(section.title)
                elif isinstance(section, dict) and 'title' in section:
                    section_titles.append(section['title'])
        elif isinstance(extraction_result, dict) and 'sections' in extraction_result:
            for section in extraction_result['sections']:
                if isinstance(section, dict) and 'title' in section:
                    section_titles.append(section['title'])
        
        return section_titles
        
    def generate_explanations(self, extraction_result, complex_terms):
        """Generate explanations for complex terms or concepts"""
        # Handle both object and dictionary access patterns
        if hasattr(extraction_result, 'explanations'):
            explanations = extraction_result.explanations
        elif isinstance(extraction_result, dict):
            if 'explanations' not in extraction_result:
                extraction_result['explanations'] = {}
            explanations = extraction_result['explanations']
        else:
            # If it's not a dict or doesn't have explanations attribute, attach one
            extraction_result.explanations = {}
            explanations = extraction_result.explanations
            
        for term in complex_terms:
            if self.knowledge_base and term in self.knowledge_base:
                explanations[term] = self.knowledge_base[term]
            else:
                # Generate explanation based on surrounding context
                context = self._find_term_context(self._extract_text_from_result(extraction_result), term)
                if context:
                    explanations[term] = f"Based on context: {context}"
                else:
                    explanations[term] = f"Term related to the document's subject matter"
                
        return extraction_result
    
    def _find_term_context(self, text, term, window_size=100):
        """Find a term in text and return surrounding context"""
        if not text or not term:
            return ""
            
        pattern = re.compile(r'\b' + re.escape(term) + r'\b', re.IGNORECASE)
        match = pattern.search(text)
        
        if not match:
            return ""
            
        start = max(0, match.start() - window_size)
        end = min(len(text), match.end() + window_size)
        
        # Get context and format it
        context = text[start:end].strip()
        if start > 0:
            context = "..." + context
        if end < len(text):
            context = context + "..."
            
        return context
        
    def add_cross_references(self, extraction_result):
        """Add references to related sections within the document"""
        # Handle both object and dictionary access patterns
        if hasattr(extraction_result, 'cross_references'):
            cross_references = extraction_result.cross_references
        elif isinstance(extraction_result, dict):
            if 'cross_references' not in extraction_result:
                extraction_result['cross_references'] = {}
            cross_references = extraction_result['cross_references']
        else:
            # If it's not a dict or doesn't have cross_references attribute, attach one
            extraction_result.cross_references = {}
            cross_references = extraction_result.cross_references
            
        # Build section index
        sections = {}
        if hasattr(extraction_result, 'sections'):
            for i, section in enumerate(extraction_result.sections):
                # Extract key terms from section
                if hasattr(section, 'title'):
                    section_title = section.title
                    section_content = section.content if hasattr(section, 'content') else ""
                elif isinstance(section, dict):
                    section_title = section.get('title', f"Section {i}")
                    section_content = section.get('content', '')
                else:
                    continue
                    
                sections[section_title] = {
                    'index': i,
                    'content': section_content,
                    'key_terms': self._extract_key_terms_from_text(section_content)
                }
        elif isinstance(extraction_result, dict) and 'sections' in extraction_result:
            for i, section in enumerate(extraction_result['sections']):
                if isinstance(section, dict):
                    section_title = section.get('title', f"Section {i}")
                    section_content = section.get('content', '')
                    sections[section_title] = {
                        'index': i,
                        'content': section_content,
                        'key_terms': self._extract_key_terms_from_text(section_content)
                    }
        
        # Find cross-references between sections based on shared key terms
        for title, section_data in sections.items():
            related_sections = []
            for other_title, other_data in sections.items():
                if title != other_title:
                    # Check for shared key terms
                    shared_terms = set(section_data['key_terms']) & set(other_data['key_terms'])
                    if shared_terms and len(shared_terms) >= 2:  # Require at least 2 shared terms
                        related_sections.append({
                            'title': other_title,
                            'index': other_data['index'],
                            'shared_terms': list(shared_terms)
                        })
            
            if related_sections:
                cross_references[title] = sorted(
                    related_sections, 
                    key=lambda x: len(x['shared_terms']), 
                    reverse=True
                )
                
        return extraction_result
    
    def _extract_key_terms_from_text(self, text):
        """Extract key terms from a section's text"""
        if not text:
            return []
            
        # Use ContentAnalyzer to extract terms
        analysis_result = self.content_analyzer.extract_keywords_and_concepts(text, max_terms=10)
        
        # Get keywords from the analysis result
        if isinstance(analysis_result, dict) and 'keywords' in analysis_result:
            # Handle case where keywords are returned as tuples of (word, frequency)
            if analysis_result['keywords'] and isinstance(analysis_result['keywords'][0], tuple):
                return [keyword[0] for keyword in analysis_result['keywords']]
            else:
                return analysis_result['keywords']
        
        return []
        
    def suggest_learning_resources(self, extraction_result, concepts):
        """Suggest additional resources for understanding concepts"""
        # Handle both object and dictionary access patterns
        if hasattr(extraction_result, 'learning_resources'):
            learning_resources = extraction_result.learning_resources
        elif isinstance(extraction_result, dict):
            if 'learning_resources' not in extraction_result:
                extraction_result['learning_resources'] = {}
            learning_resources = extraction_result['learning_resources']
        else:
            # If it's not a dict or doesn't have learning_resources attribute, attach one
            extraction_result.learning_resources = {}
            learning_resources = extraction_result.learning_resources
        
        # Map concepts to resource types based on their characteristics
        resource_types = {
            'article': ['introduction', 'overview', 'guide'],
            'video': ['tutorial', 'demonstration', 'explanation'],
            'interactive': ['exercise', 'practice', 'quiz']
        }
        
        # For a real implementation, would connect to a resource database
        # Generate more realistic resource suggestions based on concept
        for concept in concepts[:5]:  # Limit to top 5 concepts
            resources = []
            
            # Generate different types of resources
            for resource_type, prefixes in resource_types.items():
                # Choose a prefix randomly for variety
                import random
                prefix = random.choice(prefixes)
                
                # Generate a resource with realistic URL patterns
                concept_slug = concept.lower().replace(' ', '-')
                
                if resource_type == 'article':
                    resources.append({
                        'title': f"{prefix.capitalize()} to {concept.capitalize()}",
                        'type': resource_type,
                        'url': f"https://learn.example.com/articles/{concept_slug}",
                        'description': f"A comprehensive {prefix} covering the fundamentals of {concept}"
                    })
                elif resource_type == 'video':
                    resources.append({
                        'title': f"{concept.capitalize()} {prefix} video",
                        'type': resource_type,
                        'url': f"https://example.com/videos/{concept_slug}",
                        'duration': f"{random.randint(5, 20)} min",
                        'description': f"Visual {prefix} of {concept} with examples"
                    })
                elif resource_type == 'interactive':
                    resources.append({
                        'title': f"{concept.capitalize()} {prefix}",
                        'type': resource_type,
                        'url': f"https://practice.example.com/{prefix}/{concept_slug}",
                        'estimated_time': f"{random.randint(10, 30)} min",
                        'description': f"Hands-on {prefix} to test your understanding of {concept}"
                    })
            
            learning_resources[concept] = resources
            
        return extraction_result

---


## contentHelper.py

In [ ]:
import re

class TextProcessor:
    """Provides text manipulation and preprocessing utilities"""
    
    def __init__(self):
        self.stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 
                     'by', 'this', 'with', 'i', 'you', 'it', 'not', 'or', 'be', 'are', 
                     'from', 'at', 'as', 'your', 'have', 'has', 'had', 'was', 'were', 
                     'they', 'their', 'them', 'we', 'our', 'us', 'he', 'she', 'his', 
                     'her', 'him', 'an', 'can', 'will', 'just', 'do', 'does', 'did',
                     'but', 'if', 'so', 'what', 'when', 'where', 'how', 'all', 'any',
                     'both', 'each', 'few', 'more', 'most', 'some', 'such', 'no', 'nor',
                     'too', 'very', 'one', 'every', 'least', 'less', 'many', 'now',
                     'ever', 'never', 'say', 'says', 'said', 'also', 'like', 'time'}
    
    def segment_text_block(self, text):
        """Break a block of text into logical segments."""
        segments = []
        paragraphs = text.split('\n\n')
        
        for paragraph in paragraphs:
            if not paragraph.strip():
                continue
                
            segments.append({
                'type': 'paragraph',  # Default type
                'content': paragraph.strip()
            })
            
        return segments
    
    def find_term_positions(self, text, term):
        """Find all positions of a term in text."""
        positions = []
        pattern = r'\b' + re.escape(term) + r'\b'

        for match in re.finditer(pattern, text, re.IGNORECASE):
            positions.append(match.start())

        return positions
    
    def detect_poetic_structure(self, text):
        """Detect if text has poetic structure based on line length patterns"""
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        
        if not lines:
            return False
        
        # Count lines with similar length (potential verse structure)
        line_lengths = [len(line) for line in lines]
        
        # Get average line length
        avg_length = sum(line_lengths) / len(line_lengths)
        
        # Count lines within 50% of average length (consistent structure)
        consistent_lines = sum(1 for length in line_lengths if 0.5*avg_length <= length <= 1.5*avg_length)
        
        # If at least 60% of lines have consistent length, likely poetry
        return (consistent_lines / len(line_lengths)) > 0.6

    def find_term_locations(self, text, term):
        """
        Find locations of a term in the text.

        Args:
            text (str): The text to search in
            term (str): The term to locate

        Returns:
            List of positions (character indices) where the term appears
        """
        import re

        locations = []
        pattern = r'\b' + re.escape(term) + r'\b'

        for match in re.finditer(pattern, text, re.IGNORECASE):
            locations.append(match.start())

        return locations

    def count_term_occurrences(self, text, term):
        """
        Count occurrences of a term in text.

        Args:
            text (str): The text to search in
            term (str): The term to count

        Returns:
            Number of occurrences
        """
        import re
        # Use word boundaries to ensure we match whole words
        pattern = r'\b' + re.escape(term) + r'\b'
        return len(re.findall(pattern, text, re.IGNORECASE))

    def get_context_around_term(self, text, term, context_size=100):
        """
        Get text context around a term.

        Args:
            text (str): The text to search in
            term (str): The term to find context for
            context_size (int): Number of characters around the term

        Returns:
            String with context
        """
        import re

        # Find all occurrences of the term
        matches = list(re.finditer(r'\b' + re.escape(term) + r'\b', text, re.IGNORECASE))

        if not matches:
            return ""

        # Use the first occurrence
        match = matches[0]
        start_pos = max(0, match.start() - context_size)
        end_pos = min(len(text), match.end() + context_size)

        # Extract context
        context = text[start_pos:end_pos]

        # Add ellipsis if needed
        if start_pos > 0:
            context = "..." + context
        if end_pos < len(text):
            context = context + "..."

        return context

    def extract_noun_phrases(self, text):
        """
        Extract noun phrases from text.

        Args:
            text (str): Text to extract noun phrases from

        Returns:
            List of noun phrases
        """
        # Simple implementation using basic patterns
        import re

        # Pattern for simple noun phrases
        noun_phrase_pattern = re.compile(r'(?:[A-Z][a-z]+\s)?(?:[A-Z][a-z]+\s)?[A-Z]?[a-z]+')
        matches = noun_phrase_pattern.findall(text)

        # Filter short phrases and common words
        stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 'by', 'this', 'with'}
        phrases = [match for match in matches if len(match) > 4 and match.lower() not in stop_words]

        return phrases


class SemanticAnalyzer:
    """Handles semantic analysis of text content"""
    
    def __init__(self, domain_rules=None):
        self.domain_rules = domain_rules or {}
    
    def identify_segment_type(self, text):
        """Identify the type of a text segment."""
        text = text.strip().lower()
        
        # Check if it's a definition
        if text.startswith('definition') or ': is defined as' in text or 'refers to' in text:
            return 'definition'
            
        # Check if it's an example
        if text.startswith('example') or text.startswith('for example') or 'for instance' in text:
            return 'example'
            
        # Check if it's a theorem/proposition
        if text.startswith('theorem') or text.startswith('proposition') or text.startswith('lemma'):
            return 'theorem'
            
        # Check if it's a proof
        if text.startswith('proof') or text.endswith('q.e.d.') or text.endswith('â–¡'):
            return 'proof'
            
        # Check domain-specific section types
        if 'section_types' in self.domain_rules:
            for section_type in self.domain_rules['section_types']:
                if section_type.lower() in text[:20]:
                    return section_type
        
        # Default to paragraph
        return 'paragraph'
    
    def generate_semantic_tags(self, text):
        """Generate semantic tags for a piece of text."""
        tags = []
        
        # Check for domain-specific terminology
        if 'terminologies' in self.domain_rules:
            for term in self.domain_rules['terminologies']:
                if term.lower() in text.lower():
                    tags.append(term)
        
        # Check if text contains numerical data
        if any(char.isdigit() for char in text):
            tags.append('numerical_data')
            
        # Check if text contains questions
        if '?' in text:
            tags.append('question')
            
        # Check if text contains lists
        if any(line.strip().startswith(('-', 'â€¢', '*')) for line in text.split('\n')):
            tags.append('list')
            
        return tags


class KeywordExtractor:
    """Extracts keywords and concepts from text"""
    
    def __init__(self, stop_words=None, domain_rules=None):
        self.stop_words = stop_words or set()
        self.domain_rules = domain_rules or {}
    
    def extract_keywords_and_concepts(self, text, domain=None, max_terms=20, min_word_length=3):
        """Extract key terms and concepts from text, optionally customized by domain."""
        # Split text into words and clean them
        if not text:
            return {'keywords': [], 'concepts': []}

        words = text.lower().split()
        words = [word.strip('.,;:()[]{}"\'-') for word in words if word.strip('.,;:()[]{}"\'-')]

        # Remove stop words and short words
        filtered_words = [word for word in words if word not in self.stop_words and len(word) > min_word_length]

        # Count word frequencies
        word_freq = {}
        for word in filtered_words:
            word_freq[word] = word_freq.get(word, 0) + 1

        # Get top keywords by frequency
        keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:max_terms]

        # Extract multi-word concepts (bigrams)
        bigrams = []
        for i in range(len(filtered_words) - 1):
            if filtered_words[i] and filtered_words[i+1]:
                bigram = filtered_words[i] + ' ' + filtered_words[i+1]
                bigrams.append(bigram)

        bigram_counts = {}
        for bigram in bigrams:
            bigram_counts[bigram] = bigram_counts.get(bigram, 0) + 1

        # Get top bigrams
        top_bigrams = sorted(bigram_counts.items(), key=lambda x: x[1], reverse=True)[:max_terms//2]

        # Extract domain-specific concepts if available
        domain_concepts = []
        if domain and self.domain_rules:
            if 'entity_patterns' in self.domain_rules:
                for pattern in self.domain_rules['entity_patterns']:
                    matches = re.findall(pattern, text)
                    domain_concepts.extend(matches)

        return {
            'keywords': [keyword for keyword, _ in keywords],
            'bigrams': [bigram for bigram, _ in top_bigrams],
            'domain_concepts': domain_concepts,
            'word_frequencies': word_freq,
            'raw_keywords': keywords,  # Include frequency information
            'raw_bigrams': top_bigrams  # Include frequency information
        }
    
    def extract_keywords_with_positions(self, text, domain=None, max_terms=20, min_word_length=3):
        """Extract keywords with their positions in the text."""
        # Get base extraction results
        base_results = self.extract_keywords_and_concepts(text, domain, max_terms, min_word_length)

        # Add position information
        terms_with_positions = {}
        text_processor = TextProcessor()

        # Find positions for keywords
        for keyword in base_results['keywords']:
            positions = text_processor.find_term_positions(text, keyword)
            if positions:
                terms_with_positions[keyword] = positions

        # Find positions for bigrams
        for bigram in base_results['bigrams']:
            positions = text_processor.find_term_positions(text, bigram)
            if positions:
                terms_with_positions[bigram] = positions

        # Find positions for domain concepts
        for concept in base_results['domain_concepts']:
            positions = text_processor.find_term_positions(text, concept)
            if positions:
                terms_with_positions[concept] = positions

        # Add position information to results
        base_results['term_positions'] = terms_with_positions

        return base_results
    
    def extract_key_terms_with_importance(self, text, domain_rules=None, max_terms=20, min_word_length=3):
        """Extract key terms with importance scores."""
        result = self.extract_keywords_and_concepts(text, domain_rules, max_terms*2, min_word_length)
        term_importance = {}
        
        # Add single words with their importance
        for term, count in result['raw_keywords']:
            # Calculate importance using frequency
            term_importance[term] = count / max(1, len(text.split())) * 0.8  # Single words get slightly lower weight
        
        # Add bigrams with higher weight
        for bigram, count in result['raw_bigrams']:
            if count > 1:  # Only include bigrams that occur more than once
                term_importance[bigram] = count / max(1, len(result.get('bigrams', []))) * 1.2  # Bigrams get higher weight
        
        # Add domain-specific concepts with highest weight
        for concept in result.get('domain_concepts', []):
            # Domain concepts get highest weight
            term_importance[concept] = 1.5
        
        # Sort by importance and return top terms
        sorted_terms = sorted(term_importance.items(), key=lambda x: x[1], reverse=True)
        
        return sorted_terms[:max_terms]

    def count_bigram_frequency(self, text, bigram):
        """Count frequency of a bigram in text"""
        words = text.lower().split()
        count = 0
        bigram_parts = bigram.lower().split()
        
        if len(bigram_parts) != 2:
            return 0
            
        for i in range(len(words) - 1):
            if words[i] == bigram_parts[0] and words[i+1] == bigram_parts[1]:
                count += 1
        
        return count
    
    def get_ngrams(self, text, n=3):
        """Generate n-grams from text"""
        return [text[i:i+n] for i in range(len(text)-n+1)]
    
    def calculate_term_similarity(self, term_a, term_b):
        """
        Calculate similarity between two terms using n-grams.
        
        Args:
            term_a (str): First term
            term_b (str): Second term
            
        Returns:
            Similarity score between 0 and 1
        """
        # Convert to lower case
        term_a = term_a.lower()
        term_b = term_b.lower()
        
        # Generate character 3-grams
        ngrams_a = set(self.get_ngrams(term_a))
        ngrams_b = set(self.get_ngrams(term_b))
        
        # Calculate Jaccard similarity
        if not ngrams_a or not ngrams_b:
            return 0
        
        intersection = len(ngrams_a.intersection(ngrams_b))
        union = len(ngrams_a.union(ngrams_b))
        
        return intersection / union if union > 0 else 0

class SentimentAnalyzer:
    """Analyzes sentiment and tone in text"""
    
    def perform_sentiment_analysis(self, text):
        """Analyze tone and sentiment of content sections using industry-standard NLP libraries."""
        try:
            # Using NLTK's VADER (Valence Aware Dictionary and sEntiment Reasoner)
            # VADER is specifically attuned to sentiments expressed in social media and works well on texts
            from nltk.sentiment.vader import SentimentIntensityAnalyzer
            import nltk
            from textblob import TextBlob
            
            # Ensure NLTK resources are downloaded
            try:
                nltk.data.find('vader_lexicon')
            except LookupError:
                nltk.download('vader_lexicon', quiet=True)
            
            # Initialize VADER
            sid = SentimentIntensityAnalyzer()
            
            # Get sentiment scores from VADER
            vader_scores = sid.polarity_scores(text)
            
            # Use TextBlob as a second opinion (combines pattern analyzer and NaiveBayesAnalyzer)
            blob = TextBlob(text)
            textblob_polarity = blob.sentiment.polarity  # Range: -1.0 to 1.0
            
            # Combine scores (weight VADER more as it's specifically designed for this)
            compound_score = vader_scores['compound']  # Range: -1.0 to 1.0
            
            # Weighted average (70% VADER, 30% TextBlob)
            weighted_score = (0.7 * compound_score) + (0.3 * textblob_polarity)
            
            # Determine overall sentiment
            if weighted_score >= 0.05:
                sentiment = 'positive'
            elif weighted_score <= -0.05:
                sentiment = 'negative'
            else:
                sentiment = 'neutral'
            
            # Check for academic/formal language
            academic_markers = ['therefore', 'thus', 'consequently', 'furthermore', 'moreover', 
                               'subsequently', 'hence', 'accordingly', 'evidently', 'demonstrably']
            academic_count = sum(text.lower().count(' ' + marker + ' ') for marker in academic_markers)
            
            # More sophisticated approach to detect academic tone
            # Higher ratio of long words often indicates academic writing
            words = text.split()
            long_words = [word for word in words if len(word) > 8]
            long_word_ratio = len(long_words) / max(len(words), 1)
            
            # Use both markers and vocabulary complexity as indicators
            is_academic = (academic_count > len(words) / 200) or (long_word_ratio > 0.15)
            
            return {
                'overall_sentiment': sentiment,
                'compound_score': round(weighted_score, 3),
                'vader_scores': {
                    'negative': round(vader_scores['neg'], 3),
                    'neutral': round(vader_scores['neu'], 3),
                    'positive': round(vader_scores['pos'], 3),
                    'compound': round(vader_scores['compound'], 3)
                },
                'textblob_polarity': round(textblob_polarity, 3),
                'is_academic_tone': is_academic,
                'long_word_ratio': round(long_word_ratio, 3)
            }
        
        except Exception as e:
            # Fallback to a simpler method if libraries aren't available
            print(f"Sentiment analysis error: {str(e)}. Using fallback method.")
            
            # Basic fallback implementation
            positive_words = ['good', 'excellent', 'positive', 'beneficial', 'advantage', 
                             'effective', 'success', 'improve', 'best', 'important']
            negative_words = ['bad', 'poor', 'negative', 'problem', 'disadvantage', 
                             'ineffective', 'failure', 'worse', 'worst', 'difficult']
            
            # Count occurrences of positive and negative words
            text_lower = text.lower()
            positive_count = sum(text_lower.count(' ' + word + ' ') for word in positive_words)
            negative_count = sum(text_lower.count(' ' + word + ' ') for word in negative_words)
            
            # Determine overall sentiment
            if positive_count > negative_count * 1.5:
                sentiment = 'positive'
            elif negative_count > positive_count * 1.5:
                sentiment = 'negative'
            else:
                sentiment = 'neutral'
            
            # Check for academic/formal language
            academic_markers = ['therefore', 'thus', 'consequently', 'furthermore', 'moreover']
            academic_count = sum(text_lower.count(' ' + marker + ' ') for marker in academic_markers)
            is_academic = academic_count > len(text.split()) / 200
            
            return {
                'overall_sentiment': sentiment,
                'positive_count': positive_count,
                'negative_count': negative_count,
                'is_academic_tone': is_academic,
                'fallback_method': True
            }


class TextSummarizer:
    """Generates summaries of text content"""
    
    def __init__(self, keyword_extractor=None):
        self.keyword_extractor = keyword_extractor or KeywordExtractor()
    
    def generate_summaries(self, sections, max_length=200):
        """Create concise summaries of complex sections."""
        # Simple extractive summarization implementation
        
        if isinstance(sections, str):
            text = sections
        else:
            # If sections is a dictionary or list, concatenate all text
            text = ' '.join([section if isinstance(section, str) else section.get('text', '')
                             for section in sections])
        
        # Split into sentences
        sentences = text.replace('!', '.').replace('?', '.').split('.')
        sentences = [s.strip() for s in sentences if s.strip()]
        
        if not sentences:
            return ""
        
        # Simple scoring: prefer sentences with keywords and at the beginning/end
        keywords = self.keyword_extractor.extract_keywords_and_concepts(text)['keywords'][:10]
        
        scored_sentences = []
        for i, sentence in enumerate(sentences):
            # Score based on position (beginning and end sentences are important)
            position_score = 1.0
            if i < len(sentences) / 10:  # First 10%
                position_score = 2.0
            elif i > len(sentences) * 0.9:  # Last 10%
                position_score = 1.5
                
            # Score based on keyword presence
            keyword_score = sum(1.0 for keyword in keywords if keyword in sentence.lower())
            
            # Score based on sentence length (not too short, not too long)
            length = len(sentence.split())
            length_score = 1.0
            if 5 <= length <= 20:
                length_score = 1.5
                
            # Combine scores
            total_score = position_score + keyword_score + length_score
            scored_sentences.append((sentence, total_score))
        
        # Sort by score and select top sentences
        scored_sentences.sort(key=lambda x: x[1], reverse=True)
        
        # Select top sentences up to max_length
        summary = ""
        for sentence, _ in scored_sentences:
            if len(summary) + len(sentence) <= max_length:
                summary += sentence + ". "
            else:
                break
                
        return summary.strip()


---


## ContentRelationshipAnalyzer.py

In [ ]:
from ContentAnalyzer import ContentAnalyzer

class ContentRelationshipAnalyzer:
    def __init__(self, content_analyzer=None):
        """
        Initialize the ContentRelationshipAnalyzer for understanding relationships between content elements.
        """
        # Store the content extraction result for analysis
        self.extraction_result = None
        
        # Internal data structures for analysis
        self.concept_graph = None
        self.dependency_tree = None
        self.concept_index = {}
        
        # Use composition to leverage ContentAnalyzer
        self.content_analyzer = content_analyzer if content_analyzer else ContentAnalyzer()
        
    def build_concept_graph(self, extraction_result):
        """
        Create a graph of related concepts in the document.
        
        Args:
            extraction_result (dict): The extraction result containing document content
            
        Returns:
            Dictionary representing a graph of concepts and their relationships
        """
        self.extraction_result = extraction_result
        
        # Initialize concept graph structure
        concept_graph = {
            'nodes': [],
            'edges': []
        }
        
        # Extract concepts from the document
        concepts = self._extract_concepts(extraction_result)
        
        # Add concepts as nodes to the graph
        for i, concept in enumerate(concepts):
            concept_node = {
                'id': f"concept_{i}",
                'label': concept['term'],
                'type': concept['type'],
                'frequency': concept['frequency'],
                'locations': concept['locations']
            }
            concept_graph['nodes'].append(concept_node)
            
            # Build index for quick lookup
            self.concept_index[concept['term']] = i
        
        # Build edges between related concepts
        for i, concept_a in enumerate(concepts):
            for j, concept_b in enumerate(concepts):
                if i == j:
                    continue
                
                # Calculate relationship strength between concepts
                relationship = self._calculate_concept_relationship(concept_a, concept_b)
                
                if relationship['strength'] > 0.3:  # Threshold for meaningful relationships
                    edge = {
                        'source': f"concept_{i}",
                        'target': f"concept_{j}",
                        'weight': relationship['strength'],
                        'type': relationship['type']
                    }
                    concept_graph['edges'].append(edge)
        
        self.concept_graph = concept_graph
        return concept_graph
    
    def _extract_concepts(self, extraction_result):
        """
        Extract concepts from document content with positions and types.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            List of concept dictionaries with term, type, frequency, and locations
        """
        concepts = []
        text_content = extraction_result.get('text_content', '')
        
        # First check if domain analysis is available - class-specific logic
        if 'domain_analysis' in extraction_result:
            domain_analysis = extraction_result['domain_analysis']
            domain_entities = domain_analysis.get('domain_entities', {})
            
            # Add domain-specific entities as concepts
            for entity_type, entities in domain_entities.items():
                if isinstance(entities, list):
                    for entity in entities:
                        if isinstance(entity, dict) and 'name' in entity:
                            entity_name = entity['name']
                            concepts.append({
                                'term': entity_name,
                                'type': entity_type,
                                'frequency': self._count_term_occurrences(text_content, entity_name),
                                'locations': self._find_term_locations(text_content, entity_name)
                            })
                        elif isinstance(entity, str):
                            concepts.append({
                                'term': entity,
                                'type': entity_type,
                                'frequency': self._count_term_occurrences(text_content, entity),
                                'locations': self._find_term_locations(text_content, entity)
                            })
        
        # Extract key terms from headings - class-specific logic
        structure = extraction_result.get('document_structure', [])
        heading_terms = set()
        for section in structure:
            if 'heading' in section:
                heading = section['heading']
                # Extract noun phrases from heading
                terms = self._extract_noun_phrases(heading)
                heading_terms.update(terms)
        
        # Add heading terms as concepts
        for term in heading_terms:
            if not any(concept['term'] == term for concept in concepts):
                concepts.append({
                    'term': term,
                    'type': 'heading_term',
                    'frequency': self._count_term_occurrences(text_content, term),
                    'locations': self._find_term_locations(text_content, term)
                })
        
        # Use ContentAnalyzer for core extraction - leverage the core class!
        domain = getattr(self.content_analyzer, 'domain', None)
        analysis_result = self.content_analyzer.extract_keywords_with_positions(text_content, domain)
        
        # Add keywords as concepts
        for term in analysis_result['keywords']:
            if not any(concept['term'] == term for concept in concepts):
                concepts.append({
                    'term': term,
                    'type': 'frequent_term',
                    'frequency': analysis_result['word_frequencies'].get(term, 0),
                    'locations': analysis_result['term_positions'].get(term, [])
                })
                
        # Add bigrams as concepts
        for term in analysis_result.get('bigrams', []):
            if not any(concept['term'] == term for concept in concepts):
                concepts.append({
                    'term': term,
                    'type': 'bigram_term',
                    'frequency': self._count_bigram_frequency(text_content, term),
                    'locations': analysis_result['term_positions'].get(term, [])
                })
        
        return concepts 

    def _count_bigram_frequency(self, text, bigram):
        """Count frequency of a bigram in text"""
        words = text.lower().split()
        count = 0
        bigram_parts = bigram.lower().split()
        
        if len(bigram_parts) != 2:
            return 0
            
        for i in range(len(words) - 1):
            if words[i] == bigram_parts[0] and words[i+1] == bigram_parts[1]:
                count += 1
        
        return count
    
    def _extract_noun_phrases(self, text):
        """
        Extract noun phrases from text.
        
        Args:
            text (str): Text to extract noun phrases from
            
        Returns:
            List of noun phrases
        """
        # Simple implementation using basic patterns
        # In a production environment, you would use a more sophisticated NLP library
        import re
        
        # Pattern for simple noun phrases
        noun_phrase_pattern = re.compile(r'(?:[A-Z][a-z]+\s)?(?:[A-Z][a-z]+\s)?[A-Z]?[a-z]+')
        matches = noun_phrase_pattern.findall(text)
        
        # Filter short phrases and common words
        stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 'by', 'this', 'with'}
        phrases = [match for match in matches if len(match) > 4 and match.lower() not in stop_words]
        
        return phrases
    
    def _extract_frequent_terms(self, text_content, max_terms=30):
        """
        Extract frequent terms from text content.
        
        Args:
            text_content (str): Text to analyze
            max_terms (int): Maximum number of terms to extract
            
        Returns:
            List of (term, frequency) tuples sorted by frequency
        """
        # Delegate to the content analyzer for core extraction logic
        return self.content_analyzer.extract_frequent_terms(text_content, max_terms)
    
    def _count_term_occurrences(self, text, term):
        return self.content_analyzer.text_processor.count_term_occurrences(text, term)
    
    def _find_term_locations(self, text, term):
        return self.content_analyzer.text_processor.find_term_locations(text, term)
    
    def _calculate_concept_relationship(self, concept_a, concept_b):
        """
        Calculate relationship between two concepts.
        
        Args:
            concept_a (dict): First concept
            concept_b (dict): Second concept
            
        Returns:
            Dictionary with relationship type and strength
        """
        # Calculate co-occurrence proximity
        proximity_score = self._calculate_proximity(concept_a['locations'], concept_b['locations'])
        
        # Check for term similarity
        similarity_score = self._calculate_term_similarity(concept_a['term'], concept_b['term'])
        
        # Check if one is a subterm of the other
        subterm_score = 0
        if concept_a['term'].lower() in concept_b['term'].lower():
            subterm_score = 0.8
        elif concept_b['term'].lower() in concept_a['term'].lower():
            subterm_score = 0.8
        
        # Calculate overall relationship strength
        strength = max(proximity_score, similarity_score, subterm_score)
        
        # Determine relationship type
        relationship_type = 'related'
        if subterm_score > 0:
            relationship_type = 'is_a'
        elif proximity_score > 0.6:
            relationship_type = 'co_occurs'
        elif similarity_score > 0.7:
            relationship_type = 'similar_to'
        
        return {
            'strength': strength,
            'type': relationship_type
        }
    
    def _calculate_proximity(self, locations_a, locations_b, max_distance=500):
        """
        Calculate proximity score based on locations.
        
        Args:
            locations_a (list): List of locations for first term
            locations_b (list): List of locations for second term
            max_distance (int): Maximum distance to consider for proximity
            
        Returns:
            Proximity score between 0 and 1
        """
        if not locations_a or not locations_b:
            return 0
        
        # Find minimum distance between any two locations
        min_distance = float('inf')
        for loc_a in locations_a:
            for loc_b in locations_b:
                distance = abs(loc_a - loc_b)
                min_distance = min(min_distance, distance)
        
        # Convert distance to proximity score
        if min_distance == float('inf') or min_distance > max_distance:
            return 0
        
        return 1 - (min_distance / max_distance)
    
    def _calculate_term_similarity(self, term_a, term_b):
        return self.content_analyzer.keyword_extractor.calculate_term_similarity(term_a, term_b)
    
    def identify_prerequisites(self, sections):
        """
        Determine which sections depend on understanding other sections.
        
        Args:
            sections (list): List of section dictionaries with content
            
        Returns:
            Dictionary mapping section IDs to their prerequisites
        """
        if not sections:
            return {}
        
        prerequisites = {}
        
        # Build terminology dictionary for each section
        section_terms = {}
        for i, section in enumerate(sections):
            section_id = section.get('id', f"section_{i}")
            content = section.get('content', '')
            
            # Extract key terms from this section
            if isinstance(content, str):
                terms = self._extract_key_terms(content)
            else:
                # If content is not a string (e.g., a list or dict), convert it
                content_str = str(content)
                terms = self._extract_key_terms(content_str)
            
            section_terms[section_id] = terms
        
        # For each section, identify which earlier sections introduce terms it uses
        for i, section in enumerate(sections):
            section_id = section.get('id', f"section_{i}")
            content = section.get('content', '')
            
            if not content:
                continue
            
            # Convert content to string if it's not already
            if not isinstance(content, str):
                content = str(content)
            
            # Find terms from previous sections that appear in this section
            section_prerequisites = {}
            
            for j, prior_section in enumerate(sections[:i]):
                prior_id = prior_section.get('id', f"section_{j}")
                prior_terms = section_terms.get(prior_id, [])
                
                # Count how many terms from the prior section appear in this section
                referenced_terms = []
                for term, _ in prior_terms:
                    if term in content and len(term) > 4:  # Minimum length to avoid common words
                        referenced_terms.append(term)
                
                if referenced_terms:
                    section_prerequisites[prior_id] = {
                        'strength': len(referenced_terms) / max(1, len(prior_terms)),
                        'referenced_terms': referenced_terms
                    }
            
            # Store prerequisites with strength above threshold
            filtered_prerequisites = {
                section_id: data
                for section_id, data in section_prerequisites.items()
                if data['strength'] > 0.2  # Threshold for prerequisite relationship
            }
            
            if filtered_prerequisites:
                prerequisites[section_id] = filtered_prerequisites
        
        return prerequisites
    
    def _extract_key_terms(self, text, max_terms=20):
        """
        Extract key terms from text using the ContentAnalyzer.
        
        Args:
            text (str): Text to analyze
            max_terms (int): Maximum number of terms to extract
            
        Returns:
            List of (term, importance) tuples sorted by importance
        """
        # Use the content analyzer for core extraction logic
        return self.content_analyzer.extract_key_terms_with_importance(text, max_terms)
    
    def find_related_content(self, concept, extraction_result):
        """
        Find all content related to a specific concept.
        
        Args:
            concept (str): The concept to find related content for
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary of related content sections
        """
        if not extraction_result:
            return {}
        
        related_content = {
            'sections': [],
            'mentions': [],
            'related_concepts': []
        }
        
        # First, ensure we have built the concept graph
        if not self.concept_graph:
            self.build_concept_graph(extraction_result)
        
        # Find the concept in our index
        concept_id = None
        for i, node in enumerate(self.concept_graph.get('nodes', [])):
            if node['label'].lower() == concept.lower():
                concept_id = node['id']
                break
        
        # If concept not found, try substring matching
        if not concept_id:
            for i, node in enumerate(self.concept_graph.get('nodes', [])):
                if concept.lower() in node['label'].lower() or node['label'].lower() in concept.lower():
                    concept_id = node['id']
                    break
        
        # If still not found, return empty result
        if not concept_id:
            return related_content
        
        # Find all edges connected to this concept
        related_concept_ids = []
        for edge in self.concept_graph.get('edges', []):
            if edge['source'] == concept_id:
                related_concept_ids.append(edge['target'])
            elif edge['target'] == concept_id:
                related_concept_ids.append(edge['source'])
        
        # Get the related concepts
        for node in self.concept_graph.get('nodes', []):
            if node['id'] in related_concept_ids:
                related_concepts = related_content['related_concepts']
                related_concepts.append({
                    'term': node['label'],
                    'type': node['type'],
                    'frequency': node['frequency']
                })
        
        # Get content sections that mention the concept
        text_content = extraction_result.get('text_content', '')
        document_structure = extraction_result.get('document_structure', [])
        
        # Find the concept node
        concept_node = None
        for node in self.concept_graph.get('nodes', []):
            if node['id'] == concept_id:
                concept_node = node
                break
        
        if concept_node:
            concept_term = concept_node['label']
            
            # Find all sections that mention the concept
            for section in document_structure:
                section_content = section.get('content', '')
                if not isinstance(section_content, str):
                    section_content = str(section_content)
                
                if concept_term.lower() in section_content.lower():
                    related_content['sections'].append({
                        'heading': section.get('heading', 'Untitled Section'),
                        'level': section.get('level', 1),
                        'content_preview': self._get_context_around_term(section_content, concept_term)
                    })
            
            # Find specific mentions with context
            related_content['mentions'] = self._find_mentions_with_context(text_content, concept_term)
        
        return related_content
    
    def _get_context_around_term(self, text, term, context_size=100):
        """
        Get text context around a term.
        
        Args:
            text (str): The text to search in
            term (str): The term to find context for
            context_size (int): Number of characters around the term
            
        Returns:
            String with context
        """
        import re
        
        # Find all occurrences of the term
        matches = list(re.finditer(r'\b' + re.escape(term) + r'\b', text, re.IGNORECASE))
        
        if not matches:
            return ""
        
        # Use the first occurrence
        match = matches[0]
        start_pos = max(0, match.start() - context_size)
        end_pos = min(len(text), match.end() + context_size)
        
        # Extract context
        context = text[start_pos:end_pos]
        
        # Add ellipsis if needed
        if start_pos > 0:
            context = "..." + context
        if end_pos < len(text):
            context = context + "..."
        
        return context
    
    def _find_mentions_with_context(self, text, term, max_mentions=5, context_size=100):
        """
        Find mentions of a term with surrounding context.
        
        Args:
            text (str): The text to search in
            term (str): The term to find mentions of
            max_mentions (int): Maximum number of mentions to return
            context_size (int): Number of characters around each mention
            
        Returns:
            List of mention dictionaries with context
        """
        import re
        
        # Find all occurrences of the term
        pattern = r'\b' + re.escape(term) + r'\b'
        matches = list(re.finditer(pattern, text, re.IGNORECASE))
        
        mentions = []
        for match in matches[:max_mentions]:
            start_pos = max(0, match.start() - context_size)
            end_pos = min(len(text), match.end() + context_size)
            
            # Extract context
            context = text[start_pos:end_pos]
            
            # Add ellipsis if needed
            if start_pos > 0:
                context = "..." + context
            if end_pos < len(text):
                context = context + "..."
            
            mentions.append({
                'position': match.start(),
                'context': context
            })
        
        return mentions
    
    def create_dependency_tree(self, extraction_result):
        """
        Build a tree showing how concepts build upon each other.
        
        Args:
            extraction_result (dict): The extraction result
            
        Returns:
            Dictionary representing a dependency tree of concepts
        """
        # Ensure we have built the concept graph
        if not self.concept_graph:
            self.build_concept_graph(extraction_result)
        
        # Create dependency tree structure
        dependency_tree = {
            'nodes': [],
            'links': []
        }
        
        # Use document structure to identify concept ordering
        document_structure = extraction_result.get('document_structure', [])
        
        # Extract concepts in order of appearance
        ordered_concepts = []
        for section in document_structure:
            section_content = section.get('content', '')
            if not isinstance(section_content, str):
                section_content = str(section_content)
            
            # Find concepts mentioned in this section
            section_concepts = []
            for node in self.concept_graph.get('nodes', []):
                concept_term = node['label']
                if concept_term.lower() in section_content.lower():
                    section_concepts.append({
                        'id': node['id'],
                        'term': concept_term,
                        'first_position': section_content.lower().find(concept_term.lower())
                    })
            
            # Sort concepts by order of appearance
            section_concepts.sort(key=lambda x: x['first_position'])
            
            # Add to ordered concepts
            ordered_concepts.extend(section_concepts)
        
        # Remove duplicates while preserving order
        seen_concepts = set()
        ordered_concepts_unique = []
        for concept in ordered_concepts:
            if concept['id'] not in seen_concepts:
                seen_concepts.add(concept['id'])
                ordered_concepts_unique.append(concept)
        
        # Build the dependency tree
        # The tree is built based on concept ordering and relationships
        for i, concept in enumerate(ordered_concepts_unique):
            # Add node to tree
            dependency_tree['nodes'].append({
                'id': concept['id'],
                'label': concept['term'],
                'level': self._determine_concept_level(concept, ordered_concepts_unique[:i])
            })
            
            # Link to prerequisite concepts
            prerequisites = self._identify_concept_prerequisites(concept, ordered_concepts_unique[:i])
            for prereq in prerequisites:
                dependency_tree['links'].append({
                    'source': prereq['id'],
                    'target': concept['id'],
                    'strength': prereq['strength']
                })
        
        self.dependency_tree = dependency_tree
        return dependency_tree
    
    def _determine_concept_level(self, concept, prior_concepts):
        """
        Determine the hierarchy level of a concept.
        
        Args:
            concept (dict): The concept to determine level for
            prior_concepts (list): List of concepts that appear earlier
            
        Returns:
            Integer level (1 is highest level, higher numbers are more specific/dependent)
        """
        # If no prior concepts, this is a root concept (level 1)
        if not prior_concepts:
            return 1
        
        # Check if this concept is a subterm of any prior concept
        for prior in prior_concepts:
            # If this concept is a more specific version of a prior concept, increase level
            if concept['term'].lower() != prior['term'].lower() and concept['term'].lower() in prior['term'].lower():
                return 2
            
            # If this concept contains a prior concept, it might be a higher level concept
            if prior['term'].lower() in concept['term'].lower():
                return 1
        
        # Default level based on position
        return min(3, 1 + len(prior_concepts) // 5)
    
    def _identify_concept_prerequisites(self, concept, prior_concepts, max_prereqs=3):
        """
        Identify prerequisite concepts for a given concept.
        
        Args:
            concept (dict): The concept to find prerequisites for
            prior_concepts (list): List of concepts that appear earlier
            max_prereqs (int): Maximum number of prerequisites to identify
            
        Returns:
            List of prerequisite concepts with strength scores
        """
        if not prior_concepts:
            return []
        
        prerequisites = []
        
        # Calculate relationship strengths with prior concepts
        for prior in prior_concepts:
            # Skip if same concept
            if concept['id'] == prior['id']:
                continue
            
            # Calculate relationship strength
            term_similarity = self._calculate_term_similarity(concept['term'], prior['term'])
            
            # Check if prior concept is contained in this concept
            subterm_score = 0
            if prior['term'].lower() in concept['term'].lower():
                subterm_score = 0.7
            
            # Check if concepts are connected in the concept graph
            graph_connection = 0
            for edge in self.concept_graph.get('edges', []):
                if (edge['source'] == concept['id'] and edge['target'] == prior['id']) or \
                   (edge['target'] == concept['id'] and edge['source'] == prior['id']):
                    graph_connection = edge['weight']
                    break
            
            # Calculate overall prerequisite strength
            strength = max(term_similarity * 0.5, subterm_score, graph_connection)
            
            if strength > 0.3:  # Threshold for prerequisite relationship
                prerequisites.append({
                    'id': prior['id'],
                    'term': prior['term'],
                    'strength': strength
                })
        
        # Sort by strength and return top prerequisites
        prerequisites.sort(key=lambda x: x['strength'], reverse=True)
        return prerequisites[:max_prereqs]

---


## LiteraryWorkAnalyzer.py

In [ ]:
class LiteraryWorkAnalyzer:
    """Support class for analyzing literary works including Shakespeare plays"""
    def __init__(self, work_subtype=None):
        """
        Initialize with optional work subtype specification
        
        Args:
            work_subtype: Subtype of literary work (e.g., 'play:shakespeare', 'novel', 'poem')
        """
        self.work_type = work_subtype.split(':')[0] if work_subtype and ':' in work_subtype else work_subtype
        self.work_subtype = work_subtype.split(':')[1] if work_subtype and ':' in work_subtype else None
        self.character_map = {}
        self.scene_breakdown = {}
        self.literary_devices = {
            'metaphor': [],
            'simile': [],
            'alliteration': [],
            'personification': [],
            'irony': [],
            'foreshadowing': []
        }
        
    def analyze_characters(self, text):
        """
        Identify and analyze characters in the text
        
        Args:
            text: Full text content to analyze
            
        Returns:
            Dictionary with character information
        """
        import re
        
        # First pass: identify potential character names from speech patterns
        # Look for patterns like "HAMLET:" or "ROMEO."
        character_pattern = r'([A-Z]{2,}[A-Z\s]*)[\.:]\s'
        potential_characters = set(re.findall(character_pattern, text))
        
        # Filter out stage directions often in all caps
        filtered_characters = [char for char in potential_characters 
                              if char not in ['EXIT', 'ENTER', 'EXEUNT', 'ALL']]
        
        character_data = {}
        for character in filtered_characters:
            # Count lines spoken
            line_count = len(re.findall(f"{character}[\.:]", text))
            
            # Find all speech by this character
            speech_pattern = f"{character}[\.:]\\s*(.*?)(?=([A-Z]{{2,}}[A-Z\\s]*[\\.:]|$))"
            speeches = re.findall(speech_pattern, text, re.DOTALL)
            speech_text = ' '.join([s[0] for s in speeches if s[0]])
            
            # Calculate word count and average speech length
            words = speech_text.split()
            word_count = len(words)
            avg_speech_length = word_count / max(1, line_count)
            
            character_data[character] = {
                'line_count': line_count,
                'word_count': word_count,
                'avg_speech_length': avg_speech_length,
                'sample_speech': speech_text[:200] + '...' if len(speech_text) > 200 else speech_text
            }
        
        self.character_map = character_data
        return character_data
        
    def identify_scenes(self, text):
        """
        Break down text into acts and scenes for plays
        
        Args:
            text: Full text content
            
        Returns:
            Dictionary with act/scene structure
        """
        import re
        
        acts_scenes = {}
        
        # For Shakespeare and similar structured plays
        if self.work_type == 'play':
            # Find act divisions
            act_pattern = r'ACT\s+([IVXLCDM]+|[0-9]+)(.+?)(?=ACT\s+[IVXLCDM]+|[0-9]+|$)'
            acts = re.findall(act_pattern, text, re.DOTALL)
            
            for act_num, act_content in acts:
                # Find scene divisions within this act
                scene_pattern = r'SCENE\s+([IVXLCDM]+|[0-9]+)(.+?)(?=SCENE\s+[IVXLCDM]+|[0-9]+|$)'
                scenes = re.findall(scene_pattern, act_content, re.DOTALL)
                
                act_key = f"ACT {act_num}"
                acts_scenes[act_key] = {}
                
                for scene_num, scene_content in scenes:
                    scene_key = f"SCENE {scene_num}"
                    
                    # Try to extract setting information
                    setting_match = re.search(r'\[(.*?)\]', scene_content)
                    setting = setting_match.group(1) if setting_match else "Unknown setting"
                    
                    # Get characters in scene
                    character_pattern = r'([A-Z]{2,}[A-Z\s]*)[\.:]\s'
                    scene_characters = set(re.findall(character_pattern, scene_content))
                    
                    acts_scenes[act_key][scene_key] = {
                        'setting': setting,
                        'characters': list(scene_characters),
                        'length': len(scene_content),
                        'word_count': len(scene_content.split())
                    }
        
        # For novels and other chapter-based works
        elif self.work_type == 'novel':
            # Find chapter divisions
            chapter_pattern = r'CHAPTER\s+([IVXLCDM]+|[0-9]+)(.+?)(?=CHAPTER\s+[IVXLCDM]+|[0-9]+|$)'
            chapters = re.findall(chapter_pattern, text, re.DOTALL)
            
            for chapter_num, chapter_content in chapters:
                chapter_key = f"CHAPTER {chapter_num}"
                
                # Try to extract title if available
                title_match = re.search(r'CHAPTER\s+[IVXLCDM0-9]+\.\s+(.*?)[\n\r]', chapter_content)
                title = title_match.group(1) if title_match else "Untitled"
                
                acts_scenes[chapter_key] = {
                    'title': title,
                    'length': len(chapter_content),
                    'word_count': len(chapter_content.split())
                }
        
        self.scene_breakdown = acts_scenes
        return acts_scenes
    
    def detect_literary_devices(self, text):
        """
        Identify common literary devices in the text
        
        Args:
            text: Text to analyze
            
        Returns:
            Dictionary with literary devices found
        """
        import re
        
        devices = {
            'metaphor': [],
            'simile': [],
            'alliteration': [],
            'personification': []
        }
        
        # Break text into sentences for analysis
        sentences = re.split(r'[.!?]', text)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        # Find similes (like/as comparisons)
        simile_patterns = [
            r'\b\w+\s+like\s+\w+',  # "white like snow"
            r'\b\w+\s+as\s+\w+\s+as\s+\w+'  # "cold as winter night"
        ]
        
        for sentence in sentences:
            # Check for similes
            for pattern in simile_patterns:
                matches = re.findall(pattern, sentence, re.IGNORECASE)
                if matches:
                    devices['simile'].extend(matches)
            
            # Check for alliteration (3+ words starting with same consonant)
            words = sentence.split()
            if len(words) >= 3:
                for i in range(len(words) - 2):
                    if (words[i] and words[i+1] and words[i+2] and
                        len(words[i]) > 0 and len(words[i+1]) > 0 and len(words[i+2]) > 0):
                        first_chars = [w[0].lower() for w in [words[i], words[i+1], words[i+2]]]
                        if (first_chars[0] == first_chars[1] == first_chars[2] and 
                            first_chars[0] not in 'aeiou'):
                            devices['alliteration'].append(' '.join([words[i], words[i+1], words[i+2]]))
            
            # Basic personification detection (non-human things doing human actions)
            personification_verbs = ['spoke', 'whispered', 'thought', 'laughed', 'cried', 'smiled', 'frowned']
            personification_subjects = ['sky', 'moon', 'sun', 'wind', 'tree', 'forest', 'sea', 'ocean', 'mountain']
            
            for subject in personification_subjects:
                for verb in personification_verbs:
                    if f"{subject} {verb}" in sentence.lower():
                        devices['personification'].append(f"{subject} {verb}")
        
        self.literary_devices = devices
        return devices
        
    def analyze_verse_patterns(self, text):
        """
        Analyze verse patterns in poetic works including Shakespeare
        
        Args:
            text: Text to analyze
            
        Returns:
            Dictionary with verse analysis
        """
        # Break text into lines
        lines = text.split('\n')
        lines = [line.strip() for line in lines if line.strip()]
        
        verse_analysis = {
            'iambic_pentameter_count': 0,
            'rhyming_couplets': 0,
            'blank_verse_sections': 0,
            'sonnets': 0,
            'average_syllables_per_line': 0
        }
        
        total_syllables = 0
        
        # Count syllables for each line
        for i, line in enumerate(lines):
            # Simple syllable counting - can be improved
            syllable_count = self._count_syllables(line)
            total_syllables += syllable_count
            
            # Check for potential iambic pentameter (10 syllables)
            if 9 <= syllable_count <= 11:
                verse_analysis['iambic_pentameter_count'] += 1
                
            # Check for potential rhyming couplets
            if i > 0 and len(lines[i-1]) > 2 and len(line) > 2:
                if lines[i-1][-2:].lower() == line[-2:].lower():
                    verse_analysis['rhyming_couplets'] += 1
        
        # Calculate average syllables per line
        if lines:
            verse_analysis['average_syllables_per_line'] = total_syllables / len(lines)
            
        return verse_analysis
    
    def _count_syllables(self, word):
        """
        Simple syllable counter for English text
        
        Args:
            word: Word to count syllables for
            
        Returns:
            Estimated syllable count
        """
        # Remove punctuation
        word = ''.join(c for c in word if c.isalpha() or c.isspace())
        
        # Split into individual words
        words = word.split()
        count = 0
        
        for w in words:
            w = w.lower()
            
            # Count vowel groups
            vowels = "aeiouy"
            on_vowel = False
            in_word = False
            syllables = 0
            
            # Count syllables in the word
            for char in w:
                if char in vowels:
                    if not on_vowel:
                        syllables += 1
                        on_vowel = True
                else:
                    on_vowel = False
                    
            # Adjust for endings
            if w.endswith('e') and not w.endswith('le'):
                syllables -= 1
                
            if syllables == 0:
                syllables = 1
                
            count += syllables
            
        return count
    
    def detect_shakespearean_themes(self, text):
        """
        Identify common Shakespearean themes in the text
        
        Args:
            text: Text to analyze
            
        Returns:
            Dictionary with theme analysis
        """
        # Define common Shakespearean themes and related keywords
        themes = {
            'power_corruption': ['crown', 'throne', 'king', 'queen', 'rule', 'power', 'tyrant', 'ambition'],
            'love_romance': ['love', 'heart', 'passion', 'desire', 'kiss', 'marry', 'marriage', 'lovers'],
            'betrayal': ['betray', 'trust', 'loyal', 'faithful', 'deceive', 'traitor', 'backstab'],
            'appearance_reality': ['seem', 'appear', 'mask', 'disguise', 'truth', 'deception', 'reality'],
            'supernatural': ['ghost', 'witch', 'fairy', 'spirit', 'magic', 'dream', 'fate', 'omen'],
            'order_chaos': ['order', 'chaos', 'balance', 'harmony', 'discord', 'peace', 'war', 'tumult'],
            'revenge': ['revenge', 'vengeance', 'avenge', 'wrong', 'justice', 'blood', 'death']
        }
        
        text_lower = text.lower()
        theme_scores = {}
        
        # Score each theme based on keyword presence
        for theme, keywords in themes.items():
            # Count occurrences of theme keywords
            score = sum(text_lower.count(' ' + keyword + ' ') for keyword in keywords)
            theme_scores[theme] = score
        
        # Normalize scores by text length (per 10,000 words)
        word_count = len(text_lower.split())
        for theme in theme_scores:
            theme_scores[theme] = (theme_scores[theme] * 10000) / max(1, word_count)
        
        # Find the dominant themes (scores above average)
        avg_score = sum(theme_scores.values()) / len(theme_scores)
        dominant_themes = {theme: score for theme, score in theme_scores.items() if score > avg_score}
        
        return {
            'all_themes': theme_scores,
            'dominant_themes': dominant_themes
        }
    

---


## test_visualization_extractor.py

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import logging
import traceback
import argparse
from pathlib import Path
from VisualizationElementExtractor import VisualizationElementExtractor, Config

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("visualization_test.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("VisualizationTest")

# Test configuration
class TestConfig:
    """Configuration for visualization tests"""
    # Output directory for test results
    OUTPUT_DIR = "output"
    
    # Test image dimensions
    IMAGE_WIDTH = 640
    IMAGE_HEIGHT = 480
    
    # Chart parameters
    BAR_WIDTH = 40
    BAR_SPACE = 30
    BAR_VALUES = [150, 250, 180, 300, 220]
    BAR_COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255)]
    
    # Pie chart parameters
    PIE_SEGMENTS = [30, 60, 75, 120, 75]  # Angles in degrees
    
    # Number of scatter points to generate
    SCATTER_POINTS = 30
    
    # Random seed for reproducibility
    RANDOM_SEED = 42


def test_visualization_extractor(chart_types=None, output_dir=None):
    """
    Test the VisualizationElementExtractor with sample visualizations.
    
    Args:
        chart_types: List of chart types to test, or None for all
        output_dir: Directory to save output files
    """
    try:
        # Initialize the extractor
        logger.info("Initializing VisualizationElementExtractor")
        extractor = VisualizationElementExtractor()
        
        # Set output directory
        if output_dir:
            output_path = Path(output_dir)
        else:
            output_path = Path(TestConfig.OUTPUT_DIR)
            
        # Create output directory if it doesn't exist
        output_path.mkdir(exist_ok=True, parents=True)
        logger.info(f"Test results will be saved to {output_path.absolute()}")
        
        # Define chart types to test
        if chart_types is None:
            chart_types = ["bar_chart", "line_chart", "scatter_plot", "pie_chart"]
        
        # Create a sample extraction result with different visualization types
        test_results = {}
        
        # Process each chart type
        for chart_type in chart_types:
            try:
                logger.info(f"Creating and processing {chart_type}")
                
                # Create the chart image
                if chart_type == "bar_chart":
                    image = create_sample_bar_chart()
                elif chart_type == "line_chart":
                    image = create_sample_line_chart()
                elif chart_type == "scatter_plot":
                    image = create_sample_scatter_plot()
                elif chart_type == "pie_chart":
                    image = create_sample_pie_chart()
                else:
                    logger.warning(f"Unknown chart type: {chart_type}, skipping...")
                    continue
                
                # Process the image
                result = process_image(extractor, image, chart_type)
                test_results[chart_type] = result
                
                # Save the original image for reference
                original_path = output_path / f"{chart_type}_original.png"
                cv2.imwrite(str(original_path), image)
                logger.debug(f"Saved original {chart_type} to {original_path}")
                
            except Exception as e:
                logger.error(f"Error processing {chart_type}: {str(e)}")
                logger.debug(traceback.format_exc())
        
        # Visualize and save results
        success_count = 0
        for chart_type, result in test_results.items():
            try:
                output_file = output_path / f"{chart_type}_result.png"
                visualization = extractor.visualize_extraction_result(result, str(output_file))
                
                if 'figures' in result and result['figures']:
                    data_points_count = len(result.get('figures', [])[0].get('data_points', []))
                    logger.info(f"Processed {chart_type}, extracted {data_points_count} data points")
                    success_count += 1
                else:
                    logger.warning(f"No figures detected in {chart_type}")
            except Exception as e:
                logger.error(f"Error visualizing {chart_type} result: {str(e)}")
                logger.debug(traceback.format_exc())
                
        logger.info(f"Testing complete! Successfully processed {success_count}/{len(chart_types)} chart types")
        return test_results
        
    except Exception as e:
        logger.error(f"Test failed with error: {str(e)}")
        logger.debug(traceback.format_exc())
        return {}
    
def process_image(extractor, image, chart_type):
    """
    Process an image with the VisualizationElementExtractor.
    
    Args:
        extractor: VisualizationElementExtractor instance
        image: Image data as numpy array
        chart_type: Type of chart for labeling
        
    Returns:
        Dictionary with extraction results
    """
    try:
        logger.debug(f"Processing {chart_type} image of shape {image.shape}")
        
        # Create a mock extraction result with the image
        extraction_result = {
            'images': [
                {
                    'image_data': image,
                    'page_num': 1,
                    'position': {'x': 0, 'y': 0, 'width': image.shape[1], 'height': image.shape[0]}
                }
            ],
            'text_blocks': []
        }
        
        # Extract figures from the image
        extraction_result = extractor.analyze_figures(extraction_result)
        
        return extraction_result
    except Exception as e:
        logger.error(f"Error in process_image for {chart_type}: {str(e)}")
        logger.debug(traceback.format_exc())
        return {'error': str(e), 'images': [{'image_data': image}]}

def create_sample_bar_chart(width=None, height=None):
    """
    Create a sample bar chart for testing.
    
    Args:
        width: Image width (defaults to TestConfig.IMAGE_WIDTH)
        height: Image height (defaults to TestConfig.IMAGE_HEIGHT)
        
    Returns:
        Numpy array containing the bar chart image
    """
    width = width or TestConfig.IMAGE_WIDTH
    height = height or TestConfig.IMAGE_HEIGHT
    
    # Create blank image
    image = np.ones((height, width, 3), dtype=np.uint8) * 255
    
    # Draw x and y axes
    cv2.line(image, (50, height-50), (width-50, height-50), (0, 0, 0), 2)  # x-axis
    cv2.line(image, (50, 50), (50, height-50), (0, 0, 0), 2)  # y-axis
    
    # Draw bars
    bar_width = TestConfig.BAR_WIDTH
    space = TestConfig.BAR_SPACE
    x_positions = [100, 200, 300, 400, 500]
    values = TestConfig.BAR_VALUES
    colors = TestConfig.BAR_COLORS
    
    for i, (x, val, color) in enumerate(zip(x_positions, values, colors)):
        bar_height = val
        cv2.rectangle(image, (x, height - 50 - bar_height), (x + bar_width, height - 50), color, -1)
        
        # Add value text
        cv2.putText(image, str(val), (x + 10, height - 60 - bar_height), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
        # Add x-label
        cv2.putText(image, f"Item {i+1}", (x, height - 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    # Add chart title
    cv2.putText(image, "Sample Bar Chart", (width // 3, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    return image

def create_sample_line_chart(width=None, height=None):
    """
    Create a sample line chart for testing.
    
    Args:
        width: Image width (defaults to TestConfig.IMAGE_WIDTH)
        height: Image height (defaults to TestConfig.IMAGE_HEIGHT)
        
    Returns:
        Numpy array containing the line chart image
    """
    width = width or TestConfig.IMAGE_WIDTH
    height = height or TestConfig.IMAGE_HEIGHT
    
    # Create blank image
    image = np.ones((height, width, 3), dtype=np.uint8) * 255
    
    # Draw x and y axes
    cv2.line(image, (50, height-50), (width-50, height-50), (0, 0, 0), 2)  # x-axis
    cv2.line(image, (50, 50), (50, height-50), (0, 0, 0), 2)  # y-axis
    
    # Draw grid lines
    for y in range(100, height-50, 50):
        cv2.line(image, (50, y), (width-50, y), (200, 200, 200), 1)
    
    # Draw two lines
    x_points = np.linspace(50, width-50, 10).astype(int)
    y_points1 = np.array([250, 200, 300, 270, 350, 330, 280, 320, 270, 390])
    y_points2 = np.array([150, 180, 100, 200, 250, 220, 180, 150, 200, 170])
    
    # Line 1
    for i in range(len(x_points)-1):
        cv2.line(image, (x_points[i], y_points1[i]), (x_points[i+1], y_points1[i+1]), (255, 0, 0), 2)
        if i % 2 == 0:  # Add sample points
            cv2.circle(image, (x_points[i], y_points1[i]), 5, (255, 0, 0), -1)
    
    # Line 2
    for i in range(len(x_points)-1):
        cv2.line(image, (x_points[i], y_points2[i]), (x_points[i+1], y_points2[i+1]), (0, 0, 255), 2)
        if i % 2 == 0:  # Add sample points
            cv2.circle(image, (x_points[i], y_points2[i]), 5, (0, 0, 255), -1)
    
    # Add legend
    cv2.rectangle(image, (width-150, 70), (width-50, 120), (255, 255, 255), -1)
    cv2.rectangle(image, (width-150, 70), (width-50, 120), (0, 0, 0), 1)
    cv2.line(image, (width-140, 85), (width-110, 85), (255, 0, 0), 2)
    cv2.line(image, (width-140, 105), (width-110, 105), (0, 0, 255), 2)
    cv2.putText(image, "Series 1", (width-105, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(image, "Series 2", (width-105, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    # Add chart title
    cv2.putText(image, "Sample Line Chart", (width // 3, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    return image

def create_sample_scatter_plot(width=None, height=None):
    """
    Create a sample scatter plot for testing.
    
    Args:
        width: Image width (defaults to TestConfig.IMAGE_WIDTH)
        height: Image height (defaults to TestConfig.IMAGE_HEIGHT)
        
    Returns:
        Numpy array containing the scatter plot image
    """
    width = width or TestConfig.IMAGE_WIDTH
    height = height or TestConfig.IMAGE_HEIGHT
    
    # Create blank image
    image = np.ones((height, width, 3), dtype=np.uint8) * 255
    
    # Draw x and y axes
    cv2.line(image, (50, height-50), (width-50, height-50), (0, 0, 0), 2)  # x-axis
    cv2.line(image, (50, 50), (50, height-50), (0, 0, 0), 2)  # y-axis
    
    # Generate random points
    np.random.seed(TestConfig.RANDOM_SEED)  # For reproducibility
    x_coords = np.random.randint(70, width-70, TestConfig.SCATTER_POINTS)
    y_coords = np.random.randint(70, height-70, TestConfig.SCATTER_POINTS)
    
    # Draw scatter points
    for x, y in zip(x_coords, y_coords):
        cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
    
    # Add chart title
    cv2.putText(image, "Sample Scatter Plot", (width // 3, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    return image

def create_sample_pie_chart(width=None, height=None):
    """
    Create a sample pie chart for testing.
    
    Args:
        width: Image width (defaults to TestConfig.IMAGE_WIDTH)
        height: Image height (defaults to TestConfig.IMAGE_HEIGHT)
        
    Returns:
        Numpy array containing the pie chart image
    """
    width = width or TestConfig.IMAGE_WIDTH
    height = height or TestConfig.IMAGE_HEIGHT
    
    # Create blank image
    image = np.ones((height, width, 3), dtype=np.uint8) * 255
    
    # Define center and radius
    center = (width // 2, height // 2)
    radius = min(width, height) // 3
    
    # Define segments and colors
    segments = TestConfig.PIE_SEGMENTS
    colors = TestConfig.BAR_COLORS
    
    # Draw pie segments
    start_angle = 0
    for angle, color in zip(segments, colors):
        end_angle = start_angle + angle
        
        # Convert angles to radians for cv2.ellipse
        start_rad = start_angle * np.pi / 180
        end_rad = end_angle * np.pi / 180
        
        # Draw the pie segment
        cv2.ellipse(image, center, (radius, radius), 0, start_angle, end_angle, color, -1)
        
        # Add a line from center to the segment edge
        end_x = int(center[0] + radius * np.cos(start_rad))
        end_y = int(center[1] + radius * np.sin(start_rad))
        cv2.line(image, center, (end_x, end_y), (0, 0, 0), 1)
        
        # Calculate label position
        mid_angle = (start_angle + end_angle) / 2
        mid_rad = mid_angle * np.pi / 180
        label_x = int(center[0] + (radius + 30) * np.cos(mid_rad))
        label_y = int(center[1] + (radius + 30) * np.sin(mid_rad))
        
        # Add percentage text
        percentage = angle / 360 * 100
        cv2.putText(image, f"{percentage:.1f}%", (label_x-20, label_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
        start_angle = end_angle
    
    # Add chart title
    cv2.putText(image, "Sample Pie Chart", (width // 3, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    return image

def parse_arguments():
    """Parse command line arguments"""
    parser = argparse.ArgumentParser(description='Test VisualizationElementExtractor')
    parser.add_argument('--charts', type=str, nargs='+', 
                        choices=['bar_chart', 'line_chart', 'scatter_plot', 'pie_chart'],
                        help='Chart types to test')
    parser.add_argument('--output', type=str, help='Output directory')
    parser.add_argument('--log-level', type=str, default='INFO',
                        choices=['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'],
                        help='Logging level')
    return parser.parse_args()

if __name__ == "__main__":
    args = parse_arguments()
    
    # Set log level
    logging.getLogger().setLevel(getattr(logging, args.log_level))
    
    # Run tests
    test_visualization_extractor(args.charts, args.output) 

---


## utils.py

In [ ]:
"""
Utility functions for the visualization element extraction pipeline.

This module contains helper functions, error handling, and logging utilities
to support the main extraction functionality.
"""

import os
import time
import json
import logging
import functools
import traceback
from typing import Any, Callable, Dict, List, Optional, Tuple, TypeVar, Union, cast

import cv2
import numpy as np

from .config import Config

# Set up logger
logger = logging.getLogger(__name__)
Config.setup_logging()

# Type variables for function decorators
F = TypeVar('F', bound=Callable[..., Any])


class ExtractionError(Exception):
    """Base exception class for extraction errors."""
    pass


class PreprocessingError(ExtractionError):
    """Exception raised when image preprocessing fails."""
    pass


class ExtractionFailedError(ExtractionError):
    """Exception raised when data extraction fails."""
    pass


class ValidationError(ExtractionError):
    """Exception raised when data validation fails."""
    pass


def log_execution_time(func: F) -> F:
    """
    Decorator to log the execution time of a function.
    
    Args:
        func: The function to be decorated.
        
    Returns:
        The wrapped function.
    """
    @functools.wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        start_time = time.time()
        try:
            result = func(*args, **kwargs)
            execution_time = time.time() - start_time
            logger.debug(f"Function {func.__name__} executed in {execution_time:.4f} seconds")
            return result
        except Exception as e:
            execution_time = time.time() - start_time
            logger.error(f"Function {func.__name__} failed after {execution_time:.4f} seconds: {str(e)}")
            raise
    
    return cast(F, wrapper)


def retry(max_attempts: int = 3, delay: float = 1.0, backoff: float = 2.0, 
           exceptions: Tuple[Exception, ...] = (Exception,)) -> Callable[[F], F]:
    """
    Retry decorator with exponential backoff.
    
    Args:
        max_attempts: Maximum number of attempts to retry the function.
        delay: Initial delay between retries in seconds.
        backoff: Factor by which the delay increases after each retry.
        exceptions: Exceptions to catch and retry on.
        
    Returns:
        Decorator function.
    """
    def decorator(func: F) -> F:
        @functools.wraps(func)
        def wrapper(*args: Any, **kwargs: Any) -> Any:
            mtries, mdelay = max_attempts, delay
            while mtries > 1:
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    logger.warning(f"Retrying {func.__name__} after error: {str(e)}. "
                                 f"Attempts remaining: {mtries-1}")
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return func(*args, **kwargs)
        return cast(F, wrapper)
    return decorator


def safe_execute(default_return: Any = None) -> Callable[[F], F]:
    """
    Decorator to safely execute a function and return a default value on exception.
    
    Args:
        default_return: The default value to return in case of exception.
        
    Returns:
        Decorator function.
    """
    def decorator(func: F) -> F:
        @functools.wraps(func)
        def wrapper(*args: Any, **kwargs: Any) -> Any:
            try:
                return func(*args, **kwargs)
            except Exception as e:
                logger.error(f"Error in {func.__name__}: {str(e)}")
                logger.debug(f"Traceback: {traceback.format_exc()}")
                return default_return
        return cast(F, wrapper)
    return decorator


def validate_image(image: np.ndarray) -> bool:
    """
    Validate that an image is properly loaded and usable.
    
    Args:
        image: The image to validate.
        
    Returns:
        True if the image is valid, False otherwise.
    
    Raises:
        ValidationError: If the image is invalid.
    """
    if image is None or image.size == 0:
        raise ValidationError("Image is empty or None")
    
    if len(image.shape) < 2:
        raise ValidationError(f"Invalid image dimensions: {image.shape}")
    
    if not isinstance(image, np.ndarray):
        raise ValidationError(f"Image is not a numpy array, got {type(image)}")
    
    return True


def ensure_directory_exists(directory_path: str) -> None:
    """
    Ensure that the specified directory exists, creating it if necessary.
    
    Args:
        directory_path: Path to the directory.
    """
    if not os.path.exists(directory_path):
        try:
            os.makedirs(directory_path, exist_ok=True)
            logger.debug(f"Created directory: {directory_path}")
        except OSError as e:
            logger.error(f"Failed to create directory {directory_path}: {str(e)}")
            raise


def save_extraction_results(results: Dict[str, Any], output_path: str, 
                            format: str = Config.OUTPUT_FORMAT) -> str:
    """
    Save extraction results to a file.
    
    Args:
        results: Dictionary containing extraction results.
        output_path: Path where the results should be saved.
        format: Output format (json, csv, xml).
        
    Returns:
        Path to the saved file.
    """
    ensure_directory_exists(os.path.dirname(output_path))
    
    try:
        if format.lower() == 'json':
            with open(output_path, 'w') as f:
                json.dump(results, f, indent=2)
        elif format.lower() == 'csv':
            # Implement CSV export logic if needed
            pass
        elif format.lower() == 'xml':
            # Implement XML export logic if needed
            pass
        else:
            logger.warning(f"Unsupported format: {format}, defaulting to JSON")
            with open(output_path, 'w') as f:
                json.dump(results, f, indent=2)
                
        logger.info(f"Saved extraction results to {output_path}")
        return output_path
    
    except Exception as e:
        logger.error(f"Failed to save results to {output_path}: {str(e)}")
        raise


def load_image(image_path: str) -> np.ndarray:
    """
    Safely load an image from a file path.
    
    Args:
        image_path: Path to the image file.
        
    Returns:
        Loaded image as numpy array.
        
    Raises:
        FileNotFoundError: If the image file doesn't exist.
        PreprocessingError: If the image couldn't be loaded.
    """
    if not os.path.exists(image_path):
        logger.error(f"Image file not found: {image_path}")
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise PreprocessingError(f"Failed to load image: {image_path}")
        
        validate_image(image)
        logger.debug(f"Successfully loaded image: {image_path}")
        return image
    
    except Exception as e:
        logger.error(f"Error loading image {image_path}: {str(e)}")
        raise PreprocessingError(f"Error loading image {image_path}: {str(e)}") 

---


## VisualizationElementExtractor.py

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from PIL import Image
import pytesseract
from typing import Dict, List, Tuple, Any, Optional
import logging
import os
import traceback

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("visualization_extractor.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("VisualizationElementExtractor")

# Configuration class for settings
class Config:
    """Configuration settings for visualization extraction"""
    # Threshold values
    MIN_CONTOUR_AREA_RATIO = 0.001
    MIN_CIRCULAR_AREA = 10
    MAX_CIRCULAR_AREA_RATIO = 0.05
    CIRCULARITY_MIN = 0.6
    CIRCULARITY_MAX = 1.4
    
    # Detector parameters
    CANNY_THRESHOLD1 = 50
    CANNY_THRESHOLD2 = 150
    HOUGH_THRESHOLD = 30
    HOUGH_MIN_LINE_LENGTH_RATIO = 0.05
    HOUGH_MAX_LINE_GAP = 20
    
    # Preprocessing parameters
    GAUSSIAN_KERNEL_SIZE = (5, 5)
    GAUSSIAN_SIGMA = 0
    
    # Visualization settings
    MAX_VISUALIZATION_DIM = 1200
    
    # OCR settings
    OCR_CONFIG = "--psm 11"


class VisualizationElementExtractor:
    """
    A class focused on identifying and processing visual elements beyond just images in PDFs.
    This includes diagrams, charts, graphs, and understanding their relationships with text.
    """
    
    def __init__(self, ocr_engine=None, diagram_classifier=None):
        """
        Initialize visualization analysis tools.
        
        Args:
            ocr_engine: Optional OCR engine to use (defaults to pytesseract if None)
            diagram_classifier: Optional pre-trained classifier for diagram types
        """
        self.ocr_engine = ocr_engine or pytesseract
        self.diagram_classifier = diagram_classifier
        logger.info("VisualizationElementExtractor initialized")
        
    def extract_diagrams(self, extraction_result: Dict) -> Dict:
        """
        Identify and classify diagrams (flow charts, process diagrams, etc.)
        
        Args:
            extraction_result: Dictionary containing PDF extraction results
            
        Returns:
            Dictionary with identified diagrams and their classification
        """
        try:
            logger.info("Starting diagram extraction")
        diagrams = []
        
        for idx, image_info in enumerate(extraction_result.get('images', [])):
            image = image_info.get('image_data')
            if image is None:
                    logger.warning(f"Image data is None for image index {idx}")
                continue
                
            # Determine if the image is a diagram
            if self._is_diagram(image):
                    try:
                diagram_type = self._classify_diagram(image)
                diagram_text = self._extract_diagram_text(image)
                diagram_components = self._identify_diagram_components(image)
                
                diagrams.append({
                    'id': f"diagram_{idx}",
                    'page_num': image_info.get('page_num'),
                    'position': image_info.get('position'),
                    'type': diagram_type,
                    'text': diagram_text,
                    'components': diagram_components,
                    'connections': self._extract_component_connections(image, diagram_components)
                })
                        logger.info(f"Extracted diagram of type {diagram_type} from image {idx}")
                    except Exception as e:
                        logger.error(f"Error processing diagram {idx}: {str(e)}")
                        logger.debug(traceback.format_exc())
        
        # Add diagrams to the extraction result
        extraction_result['diagrams'] = diagrams
            logger.info(f"Completed diagram extraction. Found {len(diagrams)} diagrams")
            return extraction_result
        except Exception as e:
            logger.error(f"Error in extract_diagrams: {str(e)}")
            logger.debug(traceback.format_exc())
            # In production, we should return the original result with an error flag
            # rather than raising the exception
            extraction_result['error'] = str(e)
            extraction_result['diagrams'] = []
        return extraction_result
    
    def analyze_figures(self, extraction_result: Dict) -> Dict:
        """
        Analyze figures, charts, and graphs to extract data and meaning.
        
        Args:
            extraction_result: Dictionary containing PDF extraction results
            
        Returns:
            Dictionary with analyzed figures and their data
        """
        figures = []
        
        for idx, image_info in enumerate(extraction_result.get('images', [])):
            image = image_info.get('image_data')
            if image is None:
                continue
                
            # Check if image is a chart or graph
            visualization_type = self.classify_visualization_types(image)
            
            if visualization_type in ['chart', 'graph', 'plot']:
                figure_data = {
                    'id': f"figure_{idx}",
                    'page_num': image_info.get('page_num'),
                    'position': image_info.get('position'),
                    'type': visualization_type,
                    'subtype': self._get_chart_subtype(image, visualization_type),
                    'axes_labels': self._extract_axes_labels(image),
                    'legend_items': self._extract_legend(image),
                    'data_points': self._extract_data_points(image, visualization_type),
                    'chart_title': self._extract_chart_title(image)
                }
                figures.append(figure_data)
        
        # Add figures to the extraction result
        extraction_result['figures'] = figures
        return extraction_result
    
    def extract_visual_relationships(self, extraction_result: Dict) -> Dict:
        """
        Understand relationships between visuals and text in the document.
        
        Args:
            extraction_result: Dictionary containing PDF extraction results
            
        Returns:
            Dictionary with visual-text relationships identified
        """
        visual_relationships = []
        
        # Get all visual elements (images, diagrams, figures)
        visuals = (
            extraction_result.get('images', []) + 
            extraction_result.get('diagrams', []) + 
            extraction_result.get('figures', [])
        )
        
        text_blocks = extraction_result.get('text_blocks', [])
        
        for visual in visuals:
            page_num = visual.get('page_num')
            visual_position = visual.get('position')
            
            # Skip if missing critical information
            if not page_num or not visual_position:
                continue
            
            # Find related text based on proximity and references
            related_text = self._find_related_text(visual, text_blocks, page_num)
            
            # Determine relationship type (caption, explanation, reference)
            relationship_type = self._determine_relationship_type(visual, related_text)
            
            visual_relationships.append({
                'visual_id': visual.get('id'),
                'visual_type': visual.get('type', 'image'),
                'page_num': page_num,
                'related_text_blocks': related_text,
                'relationship_type': relationship_type,
                'visual_context': self._extract_visual_context(visual, related_text)
            })
        
        # Add visual relationships to the extraction result
        extraction_result['visual_relationships'] = visual_relationships
        return extraction_result
    
    def classify_visualization_types(self, image: np.ndarray) -> str:
        """
        Determine the type of visualization (chart, diagram, graph, etc.)
        
        Args:
            image: Image data as numpy array
            
        Returns:
            String identifying the visualization type
        """
        # Convert to grayscale if it's not already
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        # Extract features that help identify visualization type
        edge_density = self._calculate_edge_density(gray)
        text_density = self._calculate_text_density(image)
        has_grid_lines = self._detect_grid_lines(gray)
        color_variety = self._calculate_color_variety(image)
        
        # Use features to classify
        if edge_density > 0.15 and edge_density < 0.3 and text_density < 0.1:
            if has_grid_lines:
                return "chart"
            else:
                return "diagram"
        elif has_grid_lines and text_density > 0.1:
            return "graph"
        elif color_variety > 20 and text_density < 0.05:
            return "illustration"
        elif text_density > 0.2:
            return "text_with_visuals"
        else:
            return "image"
    
    def _is_diagram(self, image: np.ndarray) -> bool:
        """
        Determine if an image is likely a diagram rather than a photo or other image type.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Boolean indicating if the image is likely a diagram
        """
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Calculate features indicative of diagrams
        edge_density = self._calculate_edge_density(gray)
        color_count = self._count_distinct_colors(image)
        text_regions = self._count_text_regions(image)
        
        # Diagrams typically have:
        # 1. High edge density
        # 2. Limited color palette
        # 3. Multiple text regions
        return edge_density > 0.1 and color_count < 20 and text_regions > 3
    
    def _classify_diagram(self, image: np.ndarray) -> str:
        """
        Classify the type of diagram.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            String identifying the diagram type
        """
        # If a pre-trained classifier was provided, use it
        if self.diagram_classifier:
            return self.diagram_classifier.predict(image)
        
        # Otherwise use heuristics to classify
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Detect lines in the image
        edges = cv2.Canny(gray, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)
        
        if lines is None:
            return "unknown_diagram"
            
        # Count horizontal, vertical, and diagonal lines
        h_lines, v_lines, d_lines = 0, 0, 0
        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
            
            if angle < 10 or angle > 170:
                h_lines += 1
            elif 80 < angle < 100:
                v_lines += 1
            else:
                d_lines += 1
        
        # Extract text to look for keywords
        text = self._extract_diagram_text(image).lower()
        
        # Classify based on features
        if "flow" in text or "process" in text or (h_lines > 5 and v_lines > 5 and d_lines > 5):
            return "flowchart"
        elif "class" in text or "uml" in text or (h_lines > 10 and v_lines > 10 and d_lines < 5):
            return "uml_diagram"
        elif "entity" in text or "relationship" in text or (h_lines > 10 and v_lines > 10):
            return "er_diagram"
        elif "network" in text or "topology" in text or (d_lines > h_lines + v_lines):
            return "network_diagram"
        elif "sequence" in text or (h_lines > v_lines and d_lines < 5):
            return "sequence_diagram"
        else:
            return "general_diagram"
    
    def _extract_diagram_text(self, image: np.ndarray) -> str:
        """
        Extract text from a diagram using OCR.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            String containing extracted text
        """
        # Use the OCR engine to extract text
        if isinstance(self.ocr_engine, type(pytesseract)):
            text = pytesseract.image_to_string(Image.fromarray(image))
        else:
            # Assuming the OCR engine has a compatible interface
            text = self.ocr_engine.extract_text(image)
            
        return text
    
    def _identify_diagram_components(self, image: np.ndarray) -> List[Dict]:
        """
        Identify components within a diagram (boxes, circles, arrows, etc.)
        
        Args:
            image: Image data as numpy array
            
        Returns:
            List of dictionaries containing component information
        """
        components = []
        
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Threshold the image
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for i, contour in enumerate(contours):
            # Skip small contours
            if cv2.contourArea(contour) < 100:
                continue
                
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Determine shape type
            shape_type = self._identify_shape(contour)
            
            # Extract text within the component
            component_image = image[y:y+h, x:x+w]
            component_text = self._extract_diagram_text(component_image)
            
            components.append({
                'id': f"component_{i}",
                'type': shape_type,
                'position': {'x': int(x), 'y': int(y), 'width': int(w), 'height': int(h)},
                'text': component_text
            })
            
        return components
    
    def _extract_component_connections(self, image: np.ndarray, components: List[Dict]) -> List[Dict]:
        """
        Extract connections between diagram components (arrows, lines).
        
        Args:
            image: Image data as numpy array
            components: List of identified components
            
        Returns:
            List of dictionaries describing connections between components
        """
        connections = []
        
        # Convert to grayscale if needed
        if len(image.shape) > 2 and image.shape[2] > 1:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Create a mask excluding components to isolate lines/arrows
        mask = np.ones_like(gray) * 255
        
        for component in components:
            pos = component['position']
            x, y, w, h = pos['x'], pos['y'], pos['width'], pos['height']
            mask[y:y+h, x:x+w] = 0
            
        # Apply mask to the grayscale image
        masked_gray = cv2.bitwise_and(gray, mask)
        
        # Detect edges and lines
        edges = cv2.Canny(masked_gray, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=30, minLineLength=30, maxLineGap=10)
        
        if lines is None:
            return connections
        
        # Group line segments that likely form a single connection
        grouped_lines = self._group_line_segments(lines)
        
        for i, line_group in enumerate(grouped_lines):
            # Determine start and end points
            start_point, end_point = self._get_line_endpoints(line_group)
            
            # Find components connected by this line
            start_component = self._find_closest_component(start_point, components)
            end_component = self._find_closest_component(end_point, components)
            
            if start_component and end_component and start_component != end_component:
                # Determine if it's an arrow and its direction
                is_arrow, direction = self._check_arrow_properties(line_group, gray)
                
                connections.append({
                    'id': f"connection_{i}",
                    'from_component': start_component,
                    'to_component': end_component,
                    'type': 'arrow' if is_arrow else 'line',
                    'direction': direction if is_arrow else None,
                    'path': [{'x': int(x1), 'y': int(y1), 'x2': int(x2), 'y2': int(y2)} 
                            for x1, y1, x2, y2 in line_group]
                })
        
        return connections
    
    def _get_chart_subtype(self, image: np.ndarray, visualization_type: str) -> str:
        """
        Determine the specific subtype of a chart based on visualization type.
        
        Args:
            image: Image data as numpy array
            visualization_type: Base visualization type
            
        Returns:
            String indicating chart subtype
        """
        if 'bar' in visualization_type:
            # Check for horizontal orientation
            if self._detect_horizontal_orientation(image):
                return 'horizontal_bar'
                
            # Check for stacked bars
            if self._detect_stacked_bars(image):
                return 'stacked_bar'
                
            # Check for grouped bars
            if self._detect_grouped_bars(image):
                return 'grouped_bar'
                
            return 'bar_chart'  # Default
            
        elif 'line' in visualization_type:
            # Check for multiple lines
            color_variety = self._calculate_color_variety(image)
            
            # Check for area chart
            if self._detect_filled_area(image):
                return 'area_chart'
                
            if color_variety > 3:
                return 'multi_line'
                
            return 'line_chart'  # Default
            
        elif 'pie' in visualization_type:
            # Check for donut chart
            if self._detect_donut(image):
                return 'donut_chart'
                
            return 'pie_chart'  # Default
            
        elif 'scatter' in visualization_type:
            # Check for bubble chart
            if self._detect_bubble_sizes(image):
                return 'bubble_chart'
                
            # Check for connected scatter
            if self._detect_lines(image):
                return 'connected_scatter'
                
            return 'scatter_plot'  # Default
        
        # Return base type for other visualizations
        return visualization_type
        
    def _detect_horizontal_orientation(self, image: np.ndarray) -> bool:
        """
        Detect if a bar chart has horizontal orientation.
        
        Args:
            image: Image data
            
        Returns:
            True if horizontal orientation detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply edge detection
        edges = cv2.Canny(gray, 50, 150)
        
        # Calculate horizontal and vertical projections
        h_projection = np.sum(edges, axis=1)
        v_projection = np.sum(edges, axis=0)
        
        # Analyze signal characteristics
        h_signal = np.var(h_projection)
        v_signal = np.var(v_projection)
        
        # Horizontal bars create stronger vertical edges
        return v_signal > h_signal * 1.2
        
    def _calculate_color_variety(self, image: np.ndarray) -> int:
        """
        Calculate the number of distinct colors in an image.
        
        Args:
            image: Image data
            
        Returns:
            Estimated number of distinct colors
        """
        # If grayscale, return 1
        if len(image.shape) == 2:
            return 1
            
        # Downsample image for efficiency
        small = cv2.resize(image, (100, 100))
        
        # Reshape for k-means
        pixels = small.reshape(-1, 3).astype(np.float32)
        
        # Use k-means to find clusters of colors
        k = 8  # Max number of colors to detect
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        
        # Count significant color clusters (>5% of pixels)
        unique, counts = np.unique(labels, return_counts=True)
        significant = sum(1 for count in counts if count > pixels.shape[0] * 0.05)
        
        return significant
        
    def _detect_nodes(self, image: np.ndarray) -> bool:
        """
        Detect if image contains nodes (e.g., for scatter plots, networks).
        
        Args:
            image: Image data
            
        Returns:
            True if nodes detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Apply binary threshold
        _, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)
        
        # Set up blob detector parameters
        params = cv2.SimpleBlobDetector_Params()
        params.filterByArea = True
        params.minArea = 10
        params.maxArea = 500
        params.filterByCircularity = True
        params.minCircularity = 0.7
        
        # Create detector
        detector = cv2.SimpleBlobDetector_create(params)
        
        # Detect blobs
        keypoints = detector.detect(thresh)
        
        # Return True if significant number of circular blobs found
        return len(keypoints) > 5
        
    def _detect_lines(self, image: np.ndarray) -> bool:
        """
        Detect if image contains significant line segments.
        
        Args:
            image: Image data
            
        Returns:
            True if lines detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply edge detection
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        
        # Use Hough Line Transform
        lines = cv2.HoughLinesP(
            edges, 1, np.pi/180, threshold=30, 
            minLineLength=max(gray.shape) * 0.1, 
            maxLineGap=20
        )
        
        # Return True if enough line segments found
        return lines is not None and len(lines) > 5
        
    def _detect_scatter_points(self, image: np.ndarray) -> bool:
        """
        Detect if image contains scatter plot points.
        
        Args:
            image: Image data
            
        Returns:
            True if scatter points detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                else:
            gray = image.copy()
            
        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Apply binary threshold
        _, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Count small, roughly circular contours
        scatter_points = 0
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < 200 and area > 10:  # Small but not too small
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
                if 0.6 < circularity < 1.4:  # Fairly circular
                    scatter_points += 1
        
        # Return True if significant number of scatter points found
        return scatter_points > 10
        
    def _detect_bars(self, image: np.ndarray) -> bool:
        """
        Detect if image contains bars (for bar charts).
        
        Args:
            image: Image data
            
        Returns:
            True if bars detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply binary threshold
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Count rectangular shapes
        bar_count = 0
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < thresh.size * 0.5 and area > thresh.size * 0.001:  # Reasonable size
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h if h > 0 else 0
                # Bars typically have aspect ratio far from 1
                if aspect_ratio > 3 or aspect_ratio < 0.33:
                    # Verify the shape is rectangular
                    rect_area = w * h
                    if area / rect_area > 0.8:  # Fills at least 80% of bounding rectangle
                        bar_count += 1
        
        # Return True if significant number of bars found
        return bar_count > 3
        
    def _detect_stacked_bars(self, image: np.ndarray) -> bool:
        """
        Detect if image contains stacked bars.
        
        Args:
            image: Image data
            
        Returns:
            True if stacked bars detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply edge detection
        edges = cv2.Canny(gray, 50, 150)
        
        # Find contours
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Check hierarchy to identify stacked regions
        if hierarchy is None:
            return False
            
        # Count potential stacked bars
        is_horizontal = self._detect_horizontal_orientation(image)
        
        # Get image dimensions
        height, width = gray.shape
        
        # Group contours by x position for vertical bars or by y position for horizontal bars
        groups = {}
        
        for i, contour in enumerate(contours):
            x, y, w, h = cv2.boundingRect(contour)
            
            # Skip very small contours
            if cv2.contourArea(contour) < width * height * 0.001:
                continue
                
            if is_horizontal:
                # Group by y position (similar y = same bar)
                key = y // (height // 10)  # Discretize y-coordinate
                if key not in groups:
                    groups[key] = []
                groups[key].append((x, y, w, h))
            else:
                # Group by x position (similar x = same bar)
                key = x // (width // 10)  # Discretize x-coordinate
                if key not in groups:
                    groups[key] = []
                groups[key].append((x, y, w, h))
        
        # Count groups with multiple rectangles
        stacked_groups = 0
        for rects in groups.values():
            if len(rects) > 1:
                stacked_groups += 1
        
        # Return True if significant number of stacked groups found
        return stacked_groups > 2
        
    def _detect_grouped_bars(self, image: np.ndarray) -> bool:
        """
        Detect if image contains grouped bars.
        
        Args:
            image: Image data
            
        Returns:
            True if grouped bars detected, False otherwise
        """
        # Calculate color variety
        color_variety = self._calculate_color_variety(image)
        
        # Check for bars
        has_bars = self._detect_bars(image)
        
        # If we have multiple colors and bars, likely grouped
        if color_variety > 2 and has_bars:
            # Convert to grayscale if needed
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                gray = image.copy()
                
            # Find contours
            _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Get image dimensions
            height, width = gray.shape
            
            # Extract rectangles for bars
            rects = []
            for contour in contours:
                if cv2.contourArea(contour) > width * height * 0.001:
                    x, y, w, h = cv2.boundingRect(contour)
                    rects.append((x, y, w, h))
            
            # Sort rectangles by x-position
            rects.sort(key=lambda r: r[0])
            
            # Check for consistent grouping pattern
            if len(rects) >= 6:  # At least 2 groups of 3 bars
                # Calculate distances between consecutive bars
                distances = [rects[i+1][0] - (rects[i][0] + rects[i][2]) for i in range(len(rects)-1)]
                
                # Check if distances show a pattern (small gap between grouped bars, larger gap between groups)
                if len(distances) >= 3:
                    distances = np.array(distances)
                    mean_dist = np.mean(distances)
                    large_gaps = np.where(distances > mean_dist * 1.5)[0]
                    
                    # If we find large gaps at regular intervals, likely grouped bars
                    if len(large_gaps) >= 1:
                        return True
        
        return False
        
    def _detect_filled_area(self, image: np.ndarray) -> bool:
        """
        Detect if image contains a filled area under a line (area chart).
        
        Args:
            image: Image data
            
        Returns:
            True if filled area detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply binary threshold
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Get image dimensions
        height, width = gray.shape
        
        # Look for large filled areas
        for contour in contours:
            # Skip small contours
            if cv2.contourArea(contour) < width * height * 0.05:
                    continue
                    
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(contour)
            
            # Typical area chart has a filled region that extends to bottom of chart
            if h > height * 0.3 and y + h > height * 0.8:
                # Check if top edge is more like a line (not flat)
                contour_image = np.zeros_like(thresh)
                cv2.drawContours(contour_image, [contour], 0, 255, -1)
                
                # Extract top edge profile
                top_profile = []
                for i in range(x, x+w):
                    for j in range(y, y+h):
                        if contour_image[j, i] > 0:
                            top_profile.append(j)
                            break
                
                # Calculate variance of top profile
                if top_profile and len(top_profile) > w * 0.5:
                    variance = np.var(top_profile)
                    # If top edge has significant variation, likely an area chart
                    if variance > 100:
                        return True
        
        return False
        
    def _detect_donut(self, image: np.ndarray) -> bool:
        """
        Detect if image contains a donut chart.
        
        Args:
            image: Image data
            
        Returns:
            True if donut chart detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply binary threshold
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Look for circular hole in the middle
        potential_donut = False
        
        # Find the largest contour (likely the whole chart)
        if not contours:
            return False
            
        largest_contour = max(contours, key=cv2.contourArea)
        largest_area = cv2.contourArea(largest_contour)
        
        # Find the center of the pie chart
        M = cv2.moments(largest_contour)
        if M["m00"] == 0:
            return False
            
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
        
        # Look for a hole near the center
        for contour in contours:
            # Skip the largest contour
            if np.array_equal(contour, largest_contour):
                continue
                
            # Get contour center
            M = cv2.moments(contour)
            if M["m00"] == 0:
                continue
                
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            
            # Check if this contour is near the center
            distance = np.sqrt((cx - center_x)**2 + (cy - center_y)**2)
            
            # Check if roughly circular
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
            
            # If there's a circular hole near the center, it's likely a donut chart
            if distance < largest_area**0.5 * 0.2 and circularity > 0.7:
                potential_donut = True
                break
        
        return potential_donut
        
    def _detect_bubble_sizes(self, image: np.ndarray) -> bool:
        """
        Detect if image contains bubble chart (variable-sized circles).
        
        Args:
            image: Image data
            
        Returns:
            True if bubble chart detected, False otherwise
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
        # Apply binary threshold
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter contours to find circular shapes
        areas = []
        
        for contour in contours:
            area = cv2.contourArea(contour)
            # Skip very small contours
            if area < 20:
                continue
                
            # Check if roughly circular
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
            
            if 0.7 < circularity < 1.3:  # Fairly circular
                areas.append(area)
        
        # Must have enough circular shapes
        if len(areas) < 5:
            return False
            
        # Calculate size variation - bubble charts should have variable sizes
        areas = np.array(areas)
        max_area = np.max(areas)
        min_area = np.min(areas)
        
        # If there's significant size variation, likely a bubble chart
        return max_area > min_area * 3

    def _group_line_segments(self, lines):
        """
        Group line segments that might belong to the same line.
        
        Args:
            lines: Detected line segments from Hough transform
            
        Returns:
            List of line groups
        """
        if lines is None:
            return []
            
        # Extract line segments
        segments = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            segments.append(((x1, y1), (x2, y2)))
        
        # Initialize groups
        groups = []
        
        # Helper function to check if a segment is close to a group
        def is_close_to_group(segment, group, threshold=10):
            for g_segment in group:
                # Calculate minimum distance between segment endpoints and group segments
                s1, s2 = segment
                g1, g2 = g_segment
                
                # Check all endpoint combinations
                d1 = np.sqrt((s1[0] - g1[0])**2 + (s1[1] - g1[1])**2)
                d2 = np.sqrt((s1[0] - g2[0])**2 + (s1[1] - g2[1])**2)
                d3 = np.sqrt((s2[0] - g1[0])**2 + (s2[1] - g1[1])**2)
                d4 = np.sqrt((s2[0] - g2[0])**2 + (s2[1] - g2[1])**2)
                
                min_dist = min(d1, d2, d3, d4)
                
                if min_dist < threshold:
                    return True
            
            return False
        
        # Group segments
        for segment in segments:
            added_to_group = False
            
            for group in groups:
                if is_close_to_group(segment, group):
                    group.append(segment)
                    added_to_group = True
                    break
            
            if not added_to_group:
                groups.append([segment])
        
        # Convert groups back to line format for consistency
        result = []
        for group in groups:
            group_lines = []
            for segment in group:
                (x1, y1), (x2, y2) = segment
                group_lines.append(np.array([[x1, y1, x2, y2]]))
            result.append(group_lines)
        
        return result
            
    def _extract_generic_chart_data(self, binary_image: np.ndarray) -> List[Dict]:
        """
        Generic data extraction for unspecified chart types.
        
        Args:
            binary_image: Preprocessed binary image
            
        Returns:
            List of extracted data points
        """
        # Find contours
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Get image dimensions
        height, width = binary_image.shape
        
        data_points = []
        
        for contour in contours:
            # Skip very small contours
            if cv2.contourArea(contour) < width * height * 0.001:
                continue
                
            # Get contour center
            M = cv2.moments(contour)
            if M["m00"] == 0:
                continue
                
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            
            # Normalize coordinates
            norm_x = cx / width
            norm_y = 1.0 - (cy / height)  # Invert y-axis
            
            # Add to data points
            data_points.append({
                'x': norm_x,
                'y': norm_y,
                'contour_index': len(data_points)
            })
        
        return data_points
    
    def _extract_axes_labels(self, image: np.ndarray) -> Dict[str, str]:
        """
        Extract labels for the x and y axes from charts and graphs.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            Dictionary with x and y axis labels
        """
        # Get image dimensions
        height, width = image.shape[:2]
        
        # Define regions where axis labels are typically found
        x_axis_region = image[int(height * 0.85):height, int(width * 0.1):int(width * 0.9)]
        y_axis_region = image[int(height * 0.1):int(height * 0.9), 0:int(width * 0.15)]
        
        # Extract text from these regions
        x_label = self._extract_diagram_text(x_axis_region).strip()
        y_label = self._extract_diagram_text(y_axis_region).strip()
        
        return {'x_axis': x_label, 'y_axis': y_label}
    
    def _extract_legend(self, image: np.ndarray) -> List[Dict]:
        """
        Extract legend items from charts and graphs.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            List of legend items with color and label
        """
        # Common places for legends are top right, bottom right, or bottom of image
        height, width = image.shape[:2]
        
        # Check top right corner
        top_right = image[0:int(height * 0.3), int(width * 0.7):width]
        # Check bottom right corner
        bottom_right = image[int(height * 0.7):height, int(width * 0.7):width]
        # Check bottom center
        bottom_center = image[int(height * 0.85):height, int(width * 0.2):int(width * 0.8)]
        
        # Try to extract legend from these regions
        legend_regions = [top_right, bottom_right, bottom_center]
        
        for region in legend_regions:
            legend_items = self._process_legend_region(region)
            if legend_items:
                return legend_items
        
        return []
    
    def _process_legend_region(self, region: np.ndarray) -> List[Dict]:
        """
        Process a potential legend region to extract legend items.
        
        Args:
            region: Image region to process
            
        Returns:
            List of legend items
        """
        legend_items = []
        
        # Convert to grayscale
        if len(region.shape) > 2 and region.shape[2] > 1:
            gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
        else:
            gray = region.copy()
            
        # Threshold to find boxes/markers
        _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            # Skip small contours
            if cv2.contourArea(contour) < 20:
                continue
                
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Check if there's text next to this symbol/color
            text_region = region[max(0, y-5):min(region.shape[0], y+h+5), 
                               min(region.shape[1]-1, x+w+5):min(region.shape[1], x+w+150)]
            
            legend_text = self._extract_diagram_text(text_region).strip()
            
            if legend_text:
                # Get the color of the legend marker
                marker_region = region[y:y+h, x:x+w]
                color = self._get_dominant_color(marker_region)
                
                legend_items.append({
                    'label': legend_text,
                    'color': color,
                    'marker_type': self._identify_marker_type(marker_region)
                })
        
        return legend_items
    
    def _extract_data_points(self, image: np.ndarray, visualization_type: str) -> List[Dict]:
        """
        Extract data points from a visualization.
        
        Args:
            image: Image data as numpy array
            visualization_type: Type of visualization (e.g., 'bar_chart', 'line_chart', etc.)
            
        Returns:
            List of extracted data points
        """
        # Preprocess the image
        preprocessed = self._preprocess_image(image)
        
        # Determine the specific chart subtype
        chart_subtype = self._get_chart_subtype(image, visualization_type)
        
        # Extract data based on visualization type
        if 'bar' in visualization_type or 'bar' in chart_subtype:
            is_horizontal = 'horizontal' in chart_subtype
            is_stacked = 'stacked' in chart_subtype
            is_grouped = 'grouped' in chart_subtype
            data_points = self._extract_bar_chart_data(
                preprocessed, 
                horizontal=is_horizontal,
                stacked=is_stacked,
                grouped=is_grouped
            )
        elif 'line' in visualization_type or 'area' in chart_subtype:
            multi_line = 'multi_line' in chart_subtype
            is_area = 'area' in chart_subtype
            data_points = self._extract_line_chart_data(
                preprocessed,
                multi_line=multi_line,
                area_chart=is_area
            )
        elif 'pie' in visualization_type or 'donut' in chart_subtype:
            is_donut = 'donut' in chart_subtype
            data_points = self._extract_pie_chart_data(
                preprocessed,
                donut=is_donut
            )
        elif 'scatter' in visualization_type or 'bubble' in chart_subtype:
            is_bubble = 'bubble' in chart_subtype
            is_connected = 'connected' in chart_subtype
            data_points = self._extract_scatter_plot_data(
                preprocessed,
                bubble_chart=is_bubble,
                connected=is_connected
            )
        elif 'histogram' in visualization_type:
            data_points = self._extract_bar_chart_data(
                preprocessed,
                histogram=True
            )
        elif 'heatmap' in visualization_type:
            data_points = self._extract_heatmap_data(preprocessed)
        else:
            # Fallback to generic extraction
            data_points = self._extract_generic_chart_data(preprocessed)
            
        return data_points
    
    def _preprocess_image(self, image: np.ndarray) -> np.ndarray:
        """
        Preprocess image for data extraction.
        
        Args:
            image: Original image data
            
        Returns:
            Preprocessed binary image
        """
        try:
        # Convert to grayscale if needed
            if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
            
            # Apply Gaussian blur to reduce noise
            blurred = cv2.GaussianBlur(
                gray, 
                Config.GAUSSIAN_KERNEL_SIZE, 
                Config.GAUSSIAN_SIGMA
            )
            
            # Apply Otsu's thresholding
            _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            
            return binary
        except Exception as e:
            logger.error(f"Error in _preprocess_image: {str(e)}")
            logger.debug(traceback.format_exc())
            # Return original image if preprocessing fails
            if len(image.shape) == 3:
                return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            return image
        
    def _extract_bar_chart_data(self, binary_image: np.ndarray, horizontal: bool = False, 
                              stacked: bool = False, grouped: bool = False, 
                              histogram: bool = False) -> List[Dict]:
        """
        Extract data from a bar chart.
        
        Args:
            binary_image: Preprocessed binary image
            horizontal: Whether bars are horizontal
            stacked: Whether bars are stacked
            grouped: Whether bars are grouped
            histogram: Whether this is a histogram (specialized bar chart)
            
        Returns:
            List of data points
        """
        # Find contours
        contours, hierarchy = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Get image dimensions for normalization
        height, width = binary_image.shape
        
        # Filter and process bar contours
        data_points = []
        
        for contour in contours:
            # Skip small contours
            if cv2.contourArea(contour) < (width * height * 0.001):
                continue
                
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(contour)
            
            # Check if shape matches a bar
            if horizontal and w > h:
                # Horizontal bar
                value = w / width
                position = 1.0 - ((y + h/2) / height)  # Normalized position
                
                data_points.append({
                    'x': position,
                    'y': value,
                    'width': h / height,
                    'height': w / width
                })
            elif not horizontal and h > w:
                # Vertical bar
                value = h / height
                position = (x + w/2) / width  # Normalized position
                
                data_points.append({
                    'x': position,
                    'y': value,
                    'width': w / width,
                    'height': h / height
                })
                
        # If no bars found, try fallback method
        if not data_points:
            # For histograms, use column-wise projection
            if histogram:
                # Sum binary image columns (for vertical histograms)
                if not horizontal:
                    projection = np.sum(binary_image, axis=0)
                    for i, value in enumerate(projection):
                        if value > 0:  # Only include non-zero columns
                            norm_value = value / height
                            norm_position = i / width
                            
                            data_points.append({
                                'x': norm_position,
                                'y': norm_value,
                                'width': 1 / width,
                                'height': norm_value
                            })
                else:
                    # For horizontal histograms
                    projection = np.sum(binary_image, axis=1)
                    for i, value in enumerate(projection):
                        if value > 0:  # Only include non-zero rows
                            norm_value = value / width
                            norm_position = 1.0 - (i / height)
                            
                            data_points.append({
                                'x': norm_position,
                                'y': norm_value,
                                'width': norm_value,
                                'height': 1 / height
                            })
        else:
                # Fallback to column projection for regular charts
                projection = np.sum(binary_image, axis=0) if not horizontal else np.sum(binary_image, axis=1)
                
                # Find peaks in the projection
                from scipy.signal import find_peaks
                peaks, _ = find_peaks(projection, height=np.max(projection) * 0.1, distance=width/20)
                
                for peak in peaks:
                    if not horizontal:
                        norm_value = projection[peak] / height
                        norm_position = peak / width
                        
                        data_points.append({
                            'x': norm_position,
                            'y': norm_value,
                            'width': width/len(peaks) / width,
                            'height': norm_value
                        })
                    else:
                        norm_value = projection[peak] / width
                        norm_position = 1.0 - (peak / height)
                        
                        data_points.append({
                            'x': norm_position,
                            'y': norm_value,
                            'width': norm_value,
                            'height': height/len(peaks) / height
                        })
                        
        return data_points
        
    def _extract_line_chart_data(self, binary_image: np.ndarray, multi_line: bool = False, 
                                area_chart: bool = False) -> List[Dict]:
        """
        Extract data from a line chart.
        
        Args:
            binary_image: Preprocessed binary image
            multi_line: Whether chart contains multiple lines
            area_chart: Whether this is an area chart
            
        Returns:
            List of data points
        """
        try:
            # Get image dimensions for normalization
            height, width = binary_image.shape
            
            # Use edge detection for finding lines
            edges = cv2.Canny(
                binary_image, 
                Config.CANNY_THRESHOLD1, 
                Config.CANNY_THRESHOLD2, 
                apertureSize=3
            )
            
            # Extract line segments using Hough Line Transform
            lines = cv2.HoughLinesP(
                edges, 1, np.pi/180, 
                threshold=Config.HOUGH_THRESHOLD, 
                minLineLength=width * Config.HOUGH_MIN_LINE_LENGTH_RATIO,
                maxLineGap=Config.HOUGH_MAX_LINE_GAP
            )
            
            if lines is None:
                logger.info("No lines detected in image using Hough transform")
                return []
                
            # Group line segments that might belong to the same line
            line_groups = self._group_line_segments(lines)
            
            data_points = []
            line_index = 0
            
            for group in line_groups:
                # Skip groups with too few segments (noise)
                if len(group) < 2:
                    continue
                    
                # Sort segments by x-coordinate to connect them properly
                sorted_segments = []
                for segment in group:
                    for line in segment:
                        x1, y1, x2, y2 = line[0]
                        # Ensure points are ordered by x-coordinate
                        if x1 > x2:
                            x1, y1, x2, y2 = x2, y2, x1, y1
                        sorted_segments.append((x1, y1, x2, y2))
                        
                sorted_segments.sort(key=lambda s: s[0])  # Sort by starting x-coordinate
                
                # Sample points along the line
                points = []
                for x1, y1, x2, y2 in sorted_segments:
                    # Add starting point
                    points.append((x1, y1))
                    # Add ending point of the last segment
                    if (x1, y1, x2, y2) == sorted_segments[-1]:
                        points.append((x2, y2))
                
                # For multi-line charts, add a line identifier
                line_id = line_index if multi_line else None
                line_index += 1
                
                # Convert points to normalized data points
                for x, y in points:
                    norm_x = x / width
                    norm_y = 1.0 - (y / height)  # Invert y-axis
                    
                    data_points.append({
                        'x': norm_x,
                        'y': norm_y,
                        'line_id': line_id
                    })
                    
            # If no lines were found with Hough transform or it's an area chart, try contour method
            if not data_points or area_chart:
                logger.info("Using contour method for area chart or as fallback")
                contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            # Skip small contours
                    if cv2.contourArea(contour) < width * height * Config.MIN_CONTOUR_AREA_RATIO:
                continue
                
                    # For area charts, find the top edge of the filled area
                    if area_chart:
                        # Get the top-most points for each x-coordinate
                        contour_points = contour.reshape(-1, 2)
                        x_coords = {}
                        
                        for x, y in contour_points:
                            if x not in x_coords or y < x_coords[x]:
                                x_coords[x] = y
                                
                        # Convert to normalized data points
                        for x, y in sorted(x_coords.items()):
                            norm_x = x / width
                            norm_y = 1.0 - (y / height)  # Invert y-axis
                            
                            data_points.append({
                                'x': norm_x,
                                'y': norm_y,
                                'line_id': 0  # Single line for area charts
                            })
                    else:
                        # For regular lines, simplify the contour to get key points
                        epsilon = 0.01 * cv2.arcLength(contour, True)
                        approx = cv2.approxPolyDP(contour, epsilon, True)
                        
                        # Convert to normalized data points
                        for point in approx:
                            x, y = point[0]
                            norm_x = x / width
                            norm_y = 1.0 - (y / height)  # Invert y-axis
                            
                            data_points.append({
                                'x': norm_x,
                                'y': norm_y,
                                'line_id': line_index
                            })
                        line_index += 1
            
            # Sort data points by x-coordinate within each line
            if multi_line:
                # Group by line_id
                lines_dict = {}
                for point in data_points:
                    line_id = point['line_id']
                    if line_id not in lines_dict:
                        lines_dict[line_id] = []
                    lines_dict[line_id].append(point)
                    
                # Sort each line by x-coordinate
                data_points = []
                for line_id, points in lines_dict.items():
                    sorted_points = sorted(points, key=lambda p: p['x'])
                    data_points.extend(sorted_points)
            else:
                data_points = sorted(data_points, key=lambda p: p['x'])
                
            logger.info(f"Extracted {len(data_points)} data points from line chart")
            return data_points
        except Exception as e:
            logger.error(f"Error in _extract_line_chart_data: {str(e)}")
            logger.debug(traceback.format_exc())
            return []
        
    def _extract_scatter_plot_data(self, binary_image: np.ndarray, bubble_chart: bool = False,
                                 connected: bool = False) -> List[Dict]:
        """
        Extract data from a scatter plot.
        
        Args:
            binary_image: Preprocessed binary image
            bubble_chart: Whether this is a bubble chart (variable-sized points)
            connected: Whether points are connected
            
        Returns:
            List of data points
        """
        try:
            # Get image dimensions for normalization
            height, width = binary_image.shape
            
            # Find contours for the points
            contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            data_points = []
        
        for contour in contours:
            area = cv2.contourArea(contour)
                
                # Skip very small or very large contours
                if area < Config.MIN_CIRCULAR_AREA or area > width * height * Config.MAX_CIRCULAR_AREA_RATIO:
                    continue
                    
                # Check if shape is roughly circular
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
                
                if Config.CIRCULARITY_MIN <= circularity <= Config.CIRCULARITY_MAX:  # Fairly circular
                    # Get center and size
                    M = cv2.moments(contour)
                    if M["m00"] == 0:
                        continue
                        
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    
                    # Normalize coordinates
                    norm_x = cx / width
                    norm_y = 1.0 - (cy / height)  # Invert y-axis
                    
                    # For bubble charts, include size information
                    if bubble_chart:
                        # Calculate radius from area
                        radius = np.sqrt(area / np.pi)
                        norm_radius = radius / (min(width, height) / 2)
                        
                        data_points.append({
                            'x': norm_x,
                            'y': norm_y,
                            'size': norm_radius,
                            'area': area
                        })
        else:
                        data_points.append({
                            'x': norm_x,
                            'y': norm_y
                        })
            
            # For connected scatter plots, add connection information
            if connected and len(data_points) > 1:
                # Sort points by x-coordinate
                data_points.sort(key=lambda p: p['x'])
                
                # Add connection information
                for i in range(len(data_points) - 1):
                    data_points[i]['next_point'] = i + 1
                    
            logger.info(f"Extracted {len(data_points)} data points from scatter plot")
            return data_points
        except Exception as e:
            logger.error(f"Error in _extract_scatter_plot_data: {str(e)}")
            logger.debug(traceback.format_exc())
            return []
        
    def _extract_pie_chart_data(self, binary_image: np.ndarray, donut: bool = False) -> List[Dict]:
        """
        Extract data from a pie chart.
        
        Args:
            binary_image: Preprocessed binary image
            donut: Whether this is a donut chart
            
        Returns:
            List of data points (pie segments)
        """
        # Get image dimensions
        height, width = binary_image.shape
        
        # Find contours
        contours, hierarchy = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        if not contours:
            return []
            
        # Find the largest contour (likely the outer circle of the pie chart)
        largest_contour = max(contours, key=cv2.contourArea)
        largest_area = cv2.contourArea(largest_contour)
        
        # Get the center of the pie chart
        M = cv2.moments(largest_contour)
        if M["m00"] == 0:
            return []
            
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
        
        # For donut charts, find the inner circle
        inner_circle = None
        inner_radius = 0
        
        if donut:
            for contour in contours:
                if np.array_equal(contour, largest_contour):
                    continue
                    
                # Calculate distance from center
                M = cv2.moments(contour)
                if M["m00"] == 0:
                    continue
                    
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                
                distance = np.sqrt((cx - center_x)**2 + (cy - center_y)**2)
                
                # Check if roughly circular
                area = cv2.contourArea(contour)
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
                
                # If circular and near center, it's likely the inner circle
                if distance < largest_area**0.5 * 0.2 and circularity > 0.7:
                    if inner_circle is None or area > cv2.contourArea(inner_circle):
                        inner_circle = contour
                        # Approximate radius from area
                        inner_radius = np.sqrt(area / np.pi)
        
        # Create a mask for the pie chart
        mask = np.zeros_like(binary_image)
        cv2.drawContours(mask, [largest_contour], 0, 255, -1)
        
        # If it's a donut chart, remove the inner circle
        if donut and inner_circle is not None:
            cv2.drawContours(mask, [inner_circle], 0, 0, -1)
        
        # Use watershed or other segmentation techniques to separate pie segments
        # For simplicity, we'll use color-based segmentation on the original image
        
        # Find contours in the pie mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        data_points = []
        total_area = cv2.contourArea(largest_contour)
        if donut and inner_circle is not None:
            total_area -= cv2.contourArea(inner_circle)
        
        # Process each segment
        for i, contour in enumerate(contours):
            # Skip small contours
            if cv2.contourArea(contour) < total_area * 0.01:
                continue
                
            # Calculate segment properties
            area = cv2.contourArea(contour)
            
            # Calculate angle range for the segment
            points = contour.reshape(-1, 2)
            angles = []
            
            for x, y in points:
                # Calculate angle relative to center
                angle = np.arctan2(y - center_y, x - center_x)
                angles.append(angle)
            
            # Find start and end angles
            min_angle = min(angles)
            max_angle = max(angles)
            
            # Convert to degrees and ensure positive values
            start_angle = (min_angle * 180 / np.pi) % 360
            end_angle = (max_angle * 180 / np.pi) % 360
            
            # Ensure proper ordering
            if end_angle < start_angle:
                end_angle += 360
                
            # Calculate percentage
            percentage = area / total_area
            
            data_points.append({
                'segment_id': i,
                'start_angle': start_angle,
                'end_angle': end_angle,
                'percentage': percentage,
                'area': area
            })
        
        return data_points
        
    def _extract_heatmap_data(self, binary_image: np.ndarray) -> List[Dict]:
        """
        Extract data from a heatmap.
        
        Args:
            binary_image: Preprocessed binary image
            
        Returns:
            List of data points (cells with intensity values)
        """
        # Get image dimensions
        height, width = binary_image.shape
        
        # For heatmaps, we need to analyze the intensity distribution
        # Convert back to grayscale if binary
        gray_image = 255 - binary_image  # Invert because binary image has dark areas as foreground
        
        # Try to detect grid structure
        # Use edge detection
        edges = cv2.Canny(gray_image, 50, 150)
        
        # Detect horizontal and vertical lines
        h_lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=width*0.3, maxLineGap=20)
        v_lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=height*0.3, maxLineGap=20)
        
        # If grid lines detected, use them to define cells
        if h_lines is not None and v_lines is not None:
            # Extract x and y coordinates of lines
            h_positions = []
            for line in h_lines:
                x1, y1, x2, y2 = line[0]
                h_positions.append(y1)
                
            v_positions = []
            for line in v_lines:
                x1, y1, x2, y2 = line[0]
                v_positions.append(x1)
                
            # Sort and remove duplicates
            h_positions = sorted(list(set([int(y) for y in h_positions])))
            v_positions = sorted(list(set([int(x) for x in v_positions])))
            
            # Create cells from grid intersections
            data_points = []
            
            for i in range(len(h_positions)-1):
                for j in range(len(v_positions)-1):
                    # Define cell boundaries
                    x1, y1 = v_positions[j], h_positions[i]
                    x2, y2 = v_positions[j+1], h_positions[i+1]
                    
                    # Skip if cell is too small
                    if x2 - x1 < 5 or y2 - y1 < 5:
                        continue
                        
                    # Calculate average intensity in the cell
                    cell_region = gray_image[y1:y2, x1:x2]
                    avg_intensity = np.mean(cell_region)
                    
                    # Normalize coordinates and intensity
                    norm_x = (x1 + (x2 - x1) / 2) / width
                    norm_y = 1.0 - ((y1 + (y2 - y1) / 2) / height)  # Invert y-axis
                    norm_intensity = avg_intensity / 255.0
                    
                    data_points.append({
                        'x': norm_x,
                        'y': norm_y,
                        'value': norm_intensity,
                        'row': i,
                        'col': j
                    })
        else:
            # If no grid structure detected, divide the image into cells
            cell_size = min(width, height) // 20  # Approximate cell size
            
            data_points = []
            
            for i in range(0, height, cell_size):
                for j in range(0, width, cell_size):
                    # Define cell boundaries
                    x1, y1 = j, i
                    x2, y2 = min(j + cell_size, width), min(i + cell_size, height)
                    
                    # Calculate average intensity in the cell
                    cell_region = gray_image[y1:y2, x1:x2]
                    if cell_region.size == 0:
                        continue
                        
                    avg_intensity = np.mean(cell_region)
                    
                    # Only include cells with significant intensity
                    if avg_intensity > 10:
                        # Normalize coordinates and intensity
                        norm_x = (x1 + (x2 - x1) / 2) / width
                        norm_y = 1.0 - ((y1 + (y2 - y1) / 2) / height)  # Invert y-axis
                        norm_intensity = avg_intensity / 255.0
                        
                        data_points.append({
                            'x': norm_x,
                            'y': norm_y,
                            'value': norm_intensity,
                            'row': i // cell_size,
                            'col': j // cell_size
                        })
        
        return data_points
        
    def _extract_chart_title(self, image: np.ndarray) -> str:
        """
        Extract the title of a chart.
        
        Args:
            image: Image data as numpy array
            
        Returns:
            String containing the extracted chart title
        """
        # Chart titles are typically at the top of the image
        height, width = image.shape[:2]
        
        # Define region for title (top 15% of the image)
        title_region = image[0:int(height * 0.15), 0:width]
        
        # Extract text from this region
        title_text = self._extract_diagram_text(title_region).strip()
        
        return title_text
        
    def visualize_extraction_result(self, extraction_result: Dict, output_path: str = None) -> np.ndarray:
        """
        Create a visualization of the extraction results.
        
        Args:
            extraction_result: Dictionary with extraction results
            output_path: Optional path to save the visualization
            
        Returns:
            Numpy array containing visualization image
        """
        try:
        # Create a copy of the first page image as background
        images = extraction_result.get('images', [])
        if not images or 'image_data' not in images[0]:
                logger.warning("No image data found for visualization, creating blank canvas")
            # Create a blank canvas if no image is available
            visualization = np.ones((800, 600, 3), dtype=np.uint8) * 255
        else:
            # Use the first image as background
            visualization = images[0]['image_data'].copy()
            
        # Resize if too large
            max_dim = Config.MAX_VISUALIZATION_DIM
        h, w = visualization.shape[:2]
        if h > max_dim or w > max_dim:
            if h > w:
                visualization = cv2.resize(visualization, (int(w * max_dim / h), max_dim))
            else:
                visualization = cv2.resize(visualization, (max_dim, int(h * max_dim / w)))
                logger.debug(f"Resized visualization to {visualization.shape}")
                
        # Draw bounding boxes around identified diagrams
        for diagram in extraction_result.get('diagrams', []):
            pos = diagram.get('position', {})
            if pos:
                x, y = pos.get('x', 0), pos.get('y', 0)
                w, h = pos.get('width', 0), pos.get('height', 0)
                
                # Draw rectangle
                cv2.rectangle(visualization, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                # Add label
                label = f"Diagram: {diagram.get('type', 'unknown')}"
                cv2.putText(visualization, label, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                        
        # Draw bounding boxes around identified figures
        for figure in extraction_result.get('figures', []):
            pos = figure.get('position', {})
            if pos:
                x, y = pos.get('x', 0), pos.get('y', 0)
                w, h = pos.get('width', 0), pos.get('height', 0)
                
                # Draw rectangle
                cv2.rectangle(visualization, (x, y), (x+w, y+h), (255, 0, 0), 2)
                
                # Add label
                label = f"Figure: {figure.get('subtype', figure.get('type', 'unknown'))}"
                cv2.putText(visualization, label, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                        
        # Draw lines for visual relationships
        for rel in extraction_result.get('visual_relationships', []):
            # Find the visual element
            visual_id = rel.get('visual_id')
            visual_element = None
            
            for element_list in [
                extraction_result.get('images', []),
                extraction_result.get('diagrams', []),
                extraction_result.get('figures', [])
            ]:
                for element in element_list:
                    if element.get('id') == visual_id:
                        visual_element = element
                        break
                if visual_element:
                    break
                    
            if not visual_element:
                continue
                
            # Get visual position
            vis_pos = visual_element.get('position', {})
            if not vis_pos:
                continue
                
            vis_x = vis_pos.get('x', 0) + vis_pos.get('width', 0) // 2
            vis_y = vis_pos.get('y', 0) + vis_pos.get('height', 0) // 2
            
            # Draw connections to related text blocks
            for block in rel.get('related_text_blocks', []):
                block_pos = block.get('position', {})
                if not block_pos:
                    continue
                    
                block_x = block_pos.get('x', 0) + block_pos.get('width', 0) // 2
                block_y = block_pos.get('y', 0) + block_pos.get('height', 0) // 2
                
                # Draw line between visual and text
                cv2.line(visualization, (vis_x, vis_y), (block_x, block_y), (0, 0, 255), 1)
                
                # Add relationship type
                mid_x = (vis_x + block_x) // 2
                mid_y = (vis_y + block_y) // 2
                rel_type = rel.get('relationship_type', 'unknown')
                cv2.putText(visualization, rel_type, (mid_x, mid_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            
        # Save if output path is provided
        if output_path:
                os.makedirs(os.path.dirname(output_path) or '.', exist_ok=True)
            cv2.imwrite(output_path, visualization)
                logger.info(f"Visualization saved to {output_path}")
            
        return visualization
        except Exception as e:
            logger.error(f"Error in visualize_extraction_result: {str(e)}")
            logger.debug(traceback.format_exc())
            # Return a blank image if visualization fails
            return np.ones((800, 600, 3), dtype=np.uint8) * 255

---


## DependencyContainer.py

In [ ]:
"""
Dependency Injection Container
-----------------------------
Manages component creation, configuration, and lifecycle.
Provides a central point for component dependencies and service location.
"""

import logging
from typing import Dict, Any, Type, Optional, Callable, TypeVar

# Import component configurations
from pipeline.config.ComponentConfig import WorkflowConfig

# Type variable for generic component types
T = TypeVar('T')

class DependencyContainer:
    """
    A simple dependency injection container that manages component creation and lifecycle.
    """
    
    def __init__(self, config: WorkflowConfig):
        """
        Initialize the container with a configuration.
        
        Args:
            config: The typed configuration for all components
        """
        self.config = config
        self.logger = logging.getLogger(__name__)
        
        # Validate the configuration
        try:
            self.config.validate()
        except ValueError as e:
            self.logger.error(f"Configuration validation failed: {e}")
            raise
        
        # Component registry - stores factories for creating components
        self._component_factories: Dict[str, Callable[[], Any]] = {}
        
        # Component instances - lazily instantiated components
        self._component_instances: Dict[str, Any] = {}
        
        # Initialize the built-in components
        self._initialize_default_components()
    
    def _initialize_default_components(self) -> None:
        """Initialize the default component factories"""
        # These will be initialized on first use
        from pipeline.pdf_extraction.EnhancedPDFExtractor import EnhancedPDFExtractor
        from pipeline.content_analysis.ContentAnalysisPipeline import ContentAnalysisPipeline
        from pipeline.multimedia.MultimediaPipeline import MultimediaGenerationPipeline
        from pipeline.adaptive_learning.AdaptiveLearningManager import AdaptiveLearningManager
        from pipeline.adaptive_learning.StudentProfileManager import StudentProfileManager
        
        # Register PDF extractor factory
        self.register_component(
            "pdf_extractor",
            lambda: EnhancedPDFExtractor(
                tesseract_path=self.config.extraction_options.tesseract_path,
                math_recognition=self.config.extraction_options.extract_equations,
                chem_recognition=True
            )
        )
        
        # Register content analysis pipeline factory
        self.register_component(
            "content_pipeline", 
            lambda: ContentAnalysisPipeline(
                domain=self.config.analysis_options.domain
            )
        )
        
        # Register multimedia pipeline factory
        self.register_component(
            "multimedia_pipeline",
            lambda: MultimediaGenerationPipeline(
                api_token=self.config.multimedia_options.api_token,
                output_dir=f"{self.config.output_dir}/multimedia",
                video_format=self.config.multimedia_options.video_format,
                video_quality=self.config.multimedia_options.video_quality.value,
                scheduler_type=self.config.multimedia_options.animation_style.scheduler_type
            )
        )
        
        # Register adaptive learning components
        self.register_component(
            "learning_manager",
            lambda: AdaptiveLearningManager()
        )
        
        self.register_component(
            "student_manager",
            lambda: StudentProfileManager()
        )
    
    def register_component(self, name: str, factory: Callable[[], Any]) -> None:
        """
        Register a component factory.
        
        Args:
            name: Component name
            factory: Factory function that creates the component
        """
        self._component_factories[name] = factory
        # Remove any cached instance if re-registering
        if name in self._component_instances:
            del self._component_instances[name]
            
    def get_component(self, name: str) -> Any:
        """
        Get a component by name, creating it if necessary.
        
        Args:
            name: The component name
            
        Returns:
            The component instance
            
        Raises:
            KeyError: If the component is not registered
        """
        # Return cached instance if available
        if name in self._component_instances:
            return self._component_instances[name]
        
        # Check if factory exists
        if name not in self._component_factories:
            raise KeyError(f"Component not registered: {name}")
        
        # Create and cache the component
        try:
            instance = self._component_factories[name]()
            self._component_instances[name] = instance
            return instance
        except Exception as e:
            self.logger.error(f"Error creating component '{name}': {e}")
            raise
    
    def get_component_typed(self, name: str, component_type: Type[T]) -> T:
        """
        Get a component by name and validate its type.
        
        Args:
            name: The component name
            component_type: Expected component type
            
        Returns:
            The component instance cast to the expected type
            
        Raises:
            KeyError: If the component is not registered
            TypeError: If the component is not of the expected type
        """
        component = self.get_component(name)
        
        if not isinstance(component, component_type):
            raise TypeError(
                f"Component '{name}' is of type {type(component).__name__}, "
                f"expected {component_type.__name__}"
            )
        
        return component
    
    def has_component(self, name: str) -> bool:
        """
        Check if a component is registered.
        
        Args:
            name: The component name
            
        Returns:
            True if the component is registered, False otherwise
        """
        return name in self._component_factories
    
    def remove_component(self, name: str) -> None:
        """
        Remove a component from the registry.
        
        Args:
            name: The component name
        """
        if name in self._component_factories:
            del self._component_factories[name]
        
        if name in self._component_instances:
            del self._component_instances[name]
    
    def reset(self) -> None:
        """Reset all component instances but keep factories"""
        self._component_instances = {}
        
    def shutdown(self) -> None:
        """Shutdown the container and all components"""
        for name, instance in self._component_instances.items():
            try:
                # Call shutdown method if available
                if hasattr(instance, "shutdown") and callable(getattr(instance, "shutdown")):
                    instance.shutdown()
                # Close method is also common
                elif hasattr(instance, "close") and callable(getattr(instance, "close")):
                    instance.close()
            except Exception as e:
                self.logger.error(f"Error shutting down component '{name}': {e}")
        
        # Clear all instances and factories
        self._component_instances = {}
        self._component_factories = {} 

---


## WorkflowErrors.py

In [ ]:
"""
Workflow Error Classes
--------------------
Defines a structured error hierarchy for different types of workflow failures.
Enables more precise error handling and better diagnostics.
"""

from typing import Optional, Dict, Any, List


class WorkflowError(Exception):
    """Base class for all workflow-related errors"""
    
    def __init__(self, message: str, workflow_id: Optional[str] = None, details: Optional[Dict[str, Any]] = None):
        """
        Initialize workflow error.
        
        Args:
            message: Error message
            workflow_id: ID of the workflow that encountered the error
            details: Additional error details
        """
        super().__init__(message)
        self.workflow_id = workflow_id
        self.details = details or {}
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert error to dictionary for serialization"""
        return {
            "error_type": self.__class__.__name__,
            "message": str(self),
            "workflow_id": self.workflow_id,
            "details": self.details
        }


# --- Input/Configuration Errors ---

class ConfigurationError(WorkflowError):
    """Error raised when there's an issue with the system or workflow configuration"""
    pass


class InvalidInputError(WorkflowError):
    """Error raised when workflow input is invalid or missing"""
    pass


class ResourceNotFoundError(WorkflowError):
    """Error raised when a required resource is not found"""
    
    def __init__(self, resource_type: str, resource_id: str, *args, **kwargs):
        """
        Initialize resource not found error.
        
        Args:
            resource_type: Type of resource (file, model, etc.)
            resource_id: Identifier for the resource
        """
        super().__init__(f"{resource_type} not found: {resource_id}", *args, **kwargs)
        self.resource_type = resource_type
        self.resource_id = resource_id
        self.details["resource_type"] = resource_type
        self.details["resource_id"] = resource_id


# --- PDF Extraction Errors ---

class PDFExtractionError(WorkflowError):
    """Base class for PDF extraction errors"""
    pass


class PDFParsingError(PDFExtractionError):
    """Error raised when PDF parsing fails"""
    pass


class OCRError(PDFExtractionError):
    """Error raised when OCR processing fails"""
    pass


class ContentExtractionError(PDFExtractionError):
    """Error raised when content extraction fails"""
    pass


# --- Content Analysis Errors ---

class ContentAnalysisError(WorkflowError):
    """Base class for content analysis errors"""
    pass


class DomainDetectionError(ContentAnalysisError):
    """Error raised when domain detection fails"""
    pass


class ContentStructuringError(ContentAnalysisError):
    """Error raised when content structuring fails"""
    pass


class SummarizationError(ContentAnalysisError):
    """Error raised when content summarization fails"""
    pass


# --- Multimedia Generation Errors ---

class MultimediaGenerationError(WorkflowError):
    """Base class for multimedia generation errors"""
    pass


class ScriptGenerationError(MultimediaGenerationError):
    """Error raised when script generation fails"""
    pass


class AnimationError(MultimediaGenerationError):
    """Error raised when animation generation fails"""
    pass


class AudioGenerationError(MultimediaGenerationError):
    """Error raised when audio generation fails"""
    pass


class VideoRenderingError(MultimediaGenerationError):
    """Error raised when video rendering fails"""
    pass


# --- Adaptive Learning Errors ---

class AdaptiveLearningError(WorkflowError):
    """Base class for adaptive learning errors"""
    pass


class StudentProfileError(AdaptiveLearningError):
    """Error raised when student profile operations fail"""
    pass


class LearningPathwayError(AdaptiveLearningError):
    """Error raised when learning pathway creation fails"""
    pass


# --- System/Resource Errors ---

class ResourceLimitError(WorkflowError):
    """Error raised when a resource limit is exceeded"""
    
    def __init__(self, resource_type: str, limit: Any, actual: Any, *args, **kwargs):
        """
        Initialize resource limit error.
        
        Args:
            resource_type: Type of resource (memory, CPU, etc.)
            limit: Resource limit
            actual: Actual resource usage
        """
        super().__init__(
            f"{resource_type} limit exceeded: {actual} > {limit}", 
            *args, **kwargs
        )
        self.resource_type = resource_type
        self.limit = limit
        self.actual = actual
        self.details.update({
            "resource_type": resource_type,
            "limit": str(limit),
            "actual": str(actual)
        })


class TimeoutError(WorkflowError):
    """Error raised when a workflow or step times out"""
    
    def __init__(self, operation: str, timeout: float, *args, **kwargs):
        """
        Initialize timeout error.
        
        Args:
            operation: The operation that timed out
            timeout: The timeout duration in seconds
        """
        super().__init__(f"{operation} timed out after {timeout} seconds", *args, **kwargs)
        self.operation = operation
        self.timeout = timeout
        self.details.update({
            "operation": operation,
            "timeout_seconds": timeout
        })


class ConcurrencyError(WorkflowError):
    """Error raised for concurrency-related issues"""
    pass


# --- Utility functions ---

def error_from_dict(error_dict: Dict[str, Any]) -> WorkflowError:
    """
    Create an error instance from a dictionary.
    
    Args:
        error_dict: Dictionary containing error information
        
    Returns:
        A WorkflowError instance
        
    Raises:
        ValueError: If the error type is not recognized
    """
    error_type = error_dict.get("error_type", "WorkflowError")
    message = error_dict.get("message", "Unknown error")
    workflow_id = error_dict.get("workflow_id")
    details = error_dict.get("details", {})
    
    # Get the error class by name
    error_classes = {cls.__name__: cls for cls in WorkflowError.__subclasses__()}
    
    # Add subclasses of subclasses
    for subcls in list(error_classes.values()):
        for subsub in subcls.__subclasses__():
            error_classes[subsub.__name__] = subsub
    
    # Create the error instance
    if error_type in error_classes:
        return error_classes[error_type](message, workflow_id=workflow_id, details=details)
    else:
        return WorkflowError(message, workflow_id=workflow_id, details=details) 

---


## EventSystem.py

In [ ]:
"""
Event System
-----------
Implements a simple event system for communication between workflow components.
Enables loose coupling between components and better error handling.
"""

import logging
import uuid
from typing import Dict, Any, List, Callable, Optional, Set
from dataclasses import dataclass, field
from enum import Enum, auto
import time
import threading


class EventType(Enum):
    """Defines the standard workflow event types"""
    # Workflow lifecycle events
    WORKFLOW_STARTED = auto()
    WORKFLOW_COMPLETED = auto()
    WORKFLOW_FAILED = auto()
    
    # Step lifecycle events
    EXTRACTION_STARTED = auto()
    EXTRACTION_COMPLETED = auto()
    EXTRACTION_FAILED = auto()
    
    ANALYSIS_STARTED = auto()
    ANALYSIS_COMPLETED = auto()
    ANALYSIS_FAILED = auto()
    
    VIDEO_GENERATION_STARTED = auto()
    VIDEO_GENERATION_COMPLETED = auto()
    VIDEO_GENERATION_FAILED = auto()
    
    ADAPTIVE_LEARNING_STARTED = auto()
    ADAPTIVE_LEARNING_COMPLETED = auto()
    ADAPTIVE_LEARNING_FAILED = auto()
    
    # Performance and monitoring events
    PERFORMANCE_METRICS_UPDATED = auto()
    AUTO_SCALING_TRIGGERED = auto()
    RESOURCE_WARNING = auto()


@dataclass
class Event:
    """Represents a workflow event with payload data"""
    event_type: EventType
    workflow_id: str
    timestamp: float = field(default_factory=time.time)
    event_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    payload: Dict[str, Any] = field(default_factory=dict)


# Type definition for event handlers
EventHandler = Callable[[Event], None]


class EventBus:
    """
    Central event bus that manages publishing and subscribing to events.
    Implements the observer pattern for workflow components.
    """
    
    def __init__(self):
        """Initialize the event bus"""
        self.logger = logging.getLogger(__name__)
        self._handlers: Dict[EventType, Set[EventHandler]] = {}
        self._global_handlers: Set[EventHandler] = set()
        self._lock = threading.RLock()  # For thread safety
    
    def subscribe(self, event_type: EventType, handler: EventHandler) -> None:
        """
        Subscribe to a specific event type.
        
        Args:
            event_type: The event type to subscribe to
            handler: Function that will be called when the event occurs
        """
        with self._lock:
            if event_type not in self._handlers:
                self._handlers[event_type] = set()
            
            self._handlers[event_type].add(handler)
            self.logger.debug(f"Subscribed handler to {event_type.name}")
    
    def subscribe_to_all(self, handler: EventHandler) -> None:
        """
        Subscribe to all events.
        
        Args:
            handler: Function that will be called for all events
        """
        with self._lock:
            self._global_handlers.add(handler)
            self.logger.debug("Subscribed handler to all events")
    
    def unsubscribe(self, event_type: EventType, handler: EventHandler) -> None:
        """
        Unsubscribe from a specific event type.
        
        Args:
            event_type: The event type to unsubscribe from
            handler: The handler to remove
        """
        with self._lock:
            if event_type in self._handlers:
                if handler in self._handlers[event_type]:
                    self._handlers[event_type].remove(handler)
                    self.logger.debug(f"Unsubscribed handler from {event_type.name}")
    
    def unsubscribe_from_all(self, handler: EventHandler) -> None:
        """
        Unsubscribe from all events.
        
        Args:
            handler: The handler to remove
        """
        with self._lock:
            # Remove from global handlers
            if handler in self._global_handlers:
                self._global_handlers.remove(handler)
            
            # Remove from specific event handlers
            for event_type in self._handlers:
                if handler in self._handlers[event_type]:
                    self._handlers[event_type].remove(handler)
            
            self.logger.debug("Unsubscribed handler from all events")
    
    def publish(self, event: Event) -> None:
        """
        Publish an event to all subscribers.
        
        Args:
            event: The event to publish
        """
        with self._lock:
            # Get relevant handlers
            handlers = set()
            
            # Add specific handlers for this event type
            if event.event_type in self._handlers:
                handlers.update(self._handlers[event.event_type])
            
            # Add global handlers
            handlers.update(self._global_handlers)
        
        # Release lock before calling handlers to avoid deadlocks
        for handler in handlers:
            try:
                handler(event)
            except Exception as e:
                self.logger.error(f"Error in event handler: {e}")
    
    def publish_for_workflow(self, 
                           workflow_id: str, 
                           event_type: EventType, 
                           payload: Optional[Dict[str, Any]] = None) -> Event:
        """
        Convenience method to create and publish an event for a specific workflow.
        
        Args:
            workflow_id: The workflow ID
            event_type: The event type
            payload: Optional event payload
            
        Returns:
            The published event
        """
        event = Event(
            event_type=event_type,
            workflow_id=workflow_id,
            payload=payload or {}
        )
        
        self.publish(event)
        return event


# Singleton instance for application-wide event bus
_event_bus: Optional[EventBus] = None


def get_event_bus() -> EventBus:
    """
    Get the global event bus instance.
    
    Returns:
        The global event bus
    """
    global _event_bus
    if _event_bus is None:
        _event_bus = EventBus()
    return _event_bus


class EventMonitor:
    """
    Monitors and records events for debugging and analysis.
    """
    
    def __init__(self, max_events: int = 1000):
        """
        Initialize the event monitor.
        
        Args:
            max_events: Maximum number of events to keep in memory
        """
        self.max_events = max_events
        self.events: List[Event] = []
        self.logger = logging.getLogger(__name__)
        
        # Subscribe to all events
        get_event_bus().subscribe_to_all(self._on_event)
    
    def _on_event(self, event: Event) -> None:
        """Event handler that records events"""
        self.events.append(event)
        
        # Trim events if exceeding max limit
        if len(self.events) > self.max_events:
            self.events = self.events[-self.max_events:]
    
    def get_events_for_workflow(self, workflow_id: str) -> List[Event]:
        """
        Get all events for a specific workflow.
        
        Args:
            workflow_id: The workflow ID
            
        Returns:
            List of events for the workflow
        """
        return [event for event in self.events if event.workflow_id == workflow_id]
    
    def get_events_by_type(self, event_type: EventType) -> List[Event]:
        """
        Get all events of a specific type.
        
        Args:
            event_type: The event type
            
        Returns:
            List of events of the specified type
        """
        return [event for event in self.events if event.event_type == event_type]
    
    def clear(self) -> None:
        """Clear all recorded events"""
        self.events = []
    
    def shutdown(self) -> None:
        """Unsubscribe from the event bus"""
        get_event_bus().unsubscribe_from_all(self._on_event) 

---


## AnimationGenerator.py

In [ ]:
"""
Animation Generator Module
-----------------------
Converts script scenes into detailed animation and visualization instructions.
Generates specifications for animations, diagrams, and visual elements.
Integrates Stable Diffusion for AI-powered visual generation.
"""

import re
import json
import random
import os
import io
import logging
import numpy as np
from typing import Dict, List, Any, Optional, Union, Tuple
from PIL import Image
import requests
import base64
from pathlib import Path

# Stable Diffusion requirements
try:
    from diffusers import StableDiffusionPipeline, DiffusionPipeline
    from diffusers.schedulers import DDIMScheduler, LMSDiscreteScheduler
    import torch
    from torch import autocast
    STABLE_DIFFUSION_AVAILABLE = True
except ImportError:
    STABLE_DIFFUSION_AVAILABLE = False

class AnimationGenerator:
    """Generates animation instructions and visuals from script data using AI"""
    
    def __init__(self, style="modern", color_scheme="educational", 
                animation_level="standard", include_transitions=True,
                use_ai=True, model_id="stabilityai/stable-diffusion-2-1-base",
                scheduler_type="ddim", output_dir="output/animations"):
        """
        Initialize the animation generator with style preferences and AI model
        
        Args:
            style: Visual style of animations (modern, minimalist, playful)
            color_scheme: Color palette (educational, vibrant, monochrome)
            animation_level: Level of animation complexity (simple, standard, complex)
            include_transitions: Whether to include transitions between scenes
            use_ai: Whether to use AI for generating visuals (requires GPU)
            model_id: HuggingFace model ID for Stable Diffusion
            scheduler_type: Type of scheduler to use (ddim, lms)
            output_dir: Directory to save generated animations
        """
        self.style = style
        self.color_scheme = color_scheme
        self.animation_level = animation_level
        self.include_transitions = include_transitions
        self.use_ai = use_ai and STABLE_DIFFUSION_AVAILABLE
        self.model_id = model_id
        self.scheduler_type = scheduler_type.lower()
        self.output_dir = output_dir
        self.device = "cuda" if torch.cuda.is_available() and self.use_ai else "cpu"
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Configure logging
        logging.basicConfig(level=logging.INFO, 
                           format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger("AnimationGenerator")
        
        # Initialize AI model if available
        if self.use_ai and STABLE_DIFFUSION_AVAILABLE:
            self._initialize_ai_model()
        elif self.use_ai and not STABLE_DIFFUSION_AVAILABLE:
            self.logger.warning("Stable Diffusion not available. Install with: pip install diffusers transformers torch")
            self.use_ai = False
        
        # Define color palettes for different schemes
        self.color_palettes = {
            "educational": {
                "primary": "#4285F4",  # Google Blue
                "secondary": "#34A853",  # Google Green
                "tertiary": "#FBBC05",  # Google Yellow
                "accent": "#EA4335",  # Google Red
                "background": "#FFFFFF",
                "text": "#202124"
            },
            "vibrant": {
                "primary": "#FF5252",  # Bright Red
                "secondary": "#42A5F5",  # Bright Blue
                "tertiary": "#FFCA28",  # Bright Yellow
                "accent": "#66BB6A",  # Bright Green
                "background": "#FAFAFA",
                "text": "#212121"
            },
            "monochrome": {
                "primary": "#455A64",  # Blue Grey
                "secondary": "#607D8B",  # Lighter Blue Grey
                "tertiary": "#90A4AE",  # Even Lighter Blue Grey
                "accent": "#263238",  # Dark Blue Grey
                "background": "#ECEFF1",
                "text": "#263238"
            }
        }
        
        # Animation templates for different visual types
        self.animation_templates = {
            "title_slide": {
                "simple": {
                    "animation_type": "fade_in",
                    "duration": 3,
                    "elements": ["title", "subtitle"]
                },
                "standard": {
                    "animation_type": "sequential_fade",
                    "duration": 5,
                    "elements": ["title", "subtitle", "background_element"]
                },
                "complex": {
                    "animation_type": "parallax_zoom",
                    "duration": 7,
                    "elements": ["background", "title_animated", "subtitle_animated", "decorative_elements"]
                }
            },
            "text_slide": {
                "simple": {
                    "animation_type": "simple_text_appear",
                    "duration": 2,
                    "elements": ["title", "content"]
                },
                "standard": {
                    "animation_type": "text_build",
                    "duration": 4,
                    "elements": ["title", "content_sequential"]
                },
                "complex": {
                    "animation_type": "dynamic_text_display",
                    "duration": 6,
                    "elements": ["animated_title", "animated_content", "highlight_effects"]
                }
            },
            "bullet_points": {
                "simple": {
                    "animation_type": "sequential_bullets",
                    "duration": 2,
                    "elements": ["title", "bullets"]
                },
                "standard": {
                    "animation_type": "bullet_build_with_icons",
                    "duration": 4,
                    "elements": ["title", "bullets_with_icons", "highlight"]
                },
                "complex": {
                    "animation_type": "dynamic_bullet_reveal",
                    "duration": 6,
                    "elements": ["animated_title", "interactive_bullets", "bullet_connections"]
                }
            },
            "diagram": {
                "simple": {
                    "animation_type": "diagram_display",
                    "duration": 5,
                    "elements": ["title", "diagram"]
                },
                "standard": {
                    "animation_type": "diagram_build",
                    "duration": 8,
                    "elements": ["title", "diagram_sequential", "labels"]
                },
                "complex": {
                    "animation_type": "interactive_diagram",
                    "duration": 12,
                    "elements": ["title", "diagram_parts", "animated_connections", "highlight_sequence"]
                }
            },
            "figure": {
                "simple": {
                    "animation_type": "figure_display",
                    "duration": 4,
                    "elements": ["figure", "caption"]
                },
                "standard": {
                    "animation_type": "figure_zoom",
                    "duration": 6,
                    "elements": ["figure", "caption", "highlight_areas"]
                },
                "complex": {
                    "animation_type": "figure_analysis",
                    "duration": 10,
                    "elements": ["figure", "callouts", "animated_explanations", "zoom_sequence"]
                }
            },
            "whiteboard": {
                "simple": {
                    "animation_type": "basic_whiteboard",
                    "duration": 8,
                    "elements": ["hand_drawn_elements", "simple_text"]
                },
                "standard": {
                    "animation_type": "sequential_whiteboard",
                    "duration": 12,
                    "elements": ["hand_drawn_sequential", "appearing_text", "basic_highlights"]
                },
                "complex": {
                    "animation_type": "dynamic_whiteboard",
                    "duration": 18,
                    "elements": ["complex_drawings", "animated_text", "color_highlights", "transitions"]
                }
            },
            "concept_graph": {
                "simple": {
                    "animation_type": "graph_display",
                    "duration": 5,
                    "elements": ["nodes", "edges"]
                },
                "standard": {
                    "animation_type": "graph_build",
                    "duration": 10,
                    "elements": ["nodes_sequential", "edges_sequential", "labels"]
                },
                "complex": {
                    "animation_type": "interactive_graph",
                    "duration": 15,
                    "elements": ["nodes_animated", "edges_animated", "highlights", "zoom_areas"]
                }
            },
            "summary_slide": {
                "simple": {
                    "animation_type": "summary_fade",
                    "duration": 5,
                    "elements": ["title", "summary_points"]
                },
                "standard": {
                    "animation_type": "summary_build",
                    "duration": 8,
                    "elements": ["title_animated", "summary_points_sequential", "conclusion_element"]
                },
                "complex": {
                    "animation_type": "dynamic_summary",
                    "duration": 12,
                    "elements": ["title_animated", "summary_points_interactive", "recap_visuals", "call_to_action"]
                }
            }
        }
        
        # Transition templates
        self.transition_templates = {
            "simple": [
                {"type": "fade", "duration": 0.5},
                {"type": "cut", "duration": 0},
                {"type": "dissolve", "duration": 1}
            ],
            "standard": [
                {"type": "wipe_right", "duration": 0.8},
                {"type": "fade_through_black", "duration": 1.2},
                {"type": "zoom_out_in", "duration": 1.5},
                {"type": "push_left", "duration": 1}
            ],
            "complex": [
                {"type": "page_turn", "duration": 1.5},
                {"type": "morph", "duration": 2},
                {"type": "3d_rotation", "duration": 2.2},
                {"type": "particle_transition", "duration": 2.5},
                {"type": "parallax_transition", "duration": 2}
            ]
        }
    
    def _initialize_ai_model(self):
        """Initialize the Stable Diffusion model"""
        try:
            # Use CUDA if available for faster generation
            self.logger.info(f"Initializing Stable Diffusion model {self.model_id}")
            self.logger.info(f"Using device: {self.device}")
            
            # For local model usage
            if self.device == "cuda":
                # Choose scheduler based on scheduler_type
                if self.scheduler_type == "lms":
                    self.logger.info("Using LMSDiscreteScheduler")
                    scheduler = LMSDiscreteScheduler.from_pretrained(self.model_id, subfolder="scheduler")
                else:
                    # Default to DDIM
                    self.logger.info("Using DDIMScheduler")
                scheduler = DDIMScheduler.from_pretrained(self.model_id, subfolder="scheduler")
                
                # Initialize Stable Diffusion pipeline
                self.pipe = StableDiffusionPipeline.from_pretrained(
                    self.model_id,
                    scheduler=scheduler,
                    torch_dtype=torch.float16
                ).to(self.device)
                self.pipe.enable_attention_slicing()  # Reduce memory usage
                
                # Also initialize a DiffusionPipeline for text-to-image for special cases
                if self.model_id in ["stabilityai/stable-diffusion-2-1-base", "stabilityai/stable-diffusion-2"]:
                    self.logger.info("Also initializing DiffusionPipeline for special cases")
                    try:
                        self.diffusion_pipe = DiffusionPipeline.from_pretrained(
                            self.model_id,
                            torch_dtype=torch.float16,
                        ).to(self.device)
                    except Exception as e:
                        self.logger.warning(f"Could not initialize DiffusionPipeline: {str(e)}")
                        self.diffusion_pipe = None
                else:
                    self.diffusion_pipe = None
            else:
                # For CPU fallback, use API-based generation
                self.pipe = None
                self.diffusion_pipe = None
                self.logger.warning("GPU not available. Using API-based generation which may be slower.")
        except Exception as e:
            self.logger.error(f"Error initializing Stable Diffusion: {str(e)}")
            self.use_ai = False
            self.pipe = None
            self.diffusion_pipe = None
    
    def generate_animations(self, script_data: Dict) -> Dict[str, Any]:
        """
        Generate animation instructions and visuals from script data
        
        Args:
            script_data: The script data generated by ScriptGenerator
            
        Returns:
            Dictionary with animation specifications and paths to generated visuals
        """
        self.logger.info("Generating animation instructions")
        
        scenes = script_data.get("scenes", [])
        if not scenes:
            self.logger.warning("No scenes found in script data")
            return {"scenes": [], "metadata": {}}
        
        # Generate animation instructions for each scene
        animated_scenes = []
        
        for i, scene in enumerate(scenes):
            # Generate animation spec for this scene
            animation_spec = self._generate_scene_animation(scene)
            
            # Generate actual visuals if AI is enabled
            if self.use_ai:
                self.logger.info(f"Generating visuals for scene {i+1}/{len(scenes)}")
                visuals = self._generate_scene_visuals(scene, animation_spec)
                animation_spec["visuals"] = visuals
            
            # Add transition to next scene (except for the last scene)
            if i < len(scenes) - 1 and self.include_transitions:
                animation_spec["transition_to_next"] = self._generate_transition()
            
            animated_scenes.append({
                "scene_id": scene.get("scene_id", f"scene_{i+1}"),
                "title": scene.get("title", f"Scene {i+1}"),
                "narration": scene.get("narration", ""),
                "duration_estimate": scene.get("duration_estimate", 10),
                "animation_spec": animation_spec
            })
        
        # Calculate total animation time
        total_animation_time = sum(scene["animation_spec"]["total_duration"] 
                                 for scene in animated_scenes)
        
        # Add metadata
        result = {
            "scenes": animated_scenes,
            "metadata": {
                "style": self.style,
                "color_scheme": self.color_scheme,
                "color_palette": self.color_palettes[self.color_scheme],
                "animation_level": self.animation_level,
                "total_frames": int(total_animation_time * 30),  # Assuming 30fps
                "total_duration": total_animation_time,
                "ai_generated": self.use_ai,
                "scheduler_type": self.scheduler_type
            }
        }
        
        self.logger.info(f"Animation generation complete: {len(animated_scenes)} scenes, {total_animation_time:.1f} seconds")
        return result
    
    def _generate_scene_visuals(self, scene: Dict, animation_spec: Dict) -> Dict[str, Any]:
        """Generate visual elements for a scene using Stable Diffusion"""
        visuals = {}
        scene_title = scene.get("title", "Scene")
        scene_id = scene.get("scene_id", "scene_1")
        
        # Create a folder for this scene
        scene_folder = os.path.join(self.output_dir, f"{scene_id}")
        os.makedirs(scene_folder, exist_ok=True)
        
        # Generate visuals for each animation element
        for i, element in enumerate(animation_spec.get("elements", [])):
            element_type = element.get("type", "text_slide")
            element_content = element.get("content", {})
            
            # Generate a prompt based on the element type and content
            prompt = self._create_visual_prompt(element_type, element_content, scene)
            
            # Generate the image
            try:
                if self.device == "cuda" and self.pipe is not None:
                    # Local generation on GPU
                    image = self._generate_image_local(prompt, element_type)
                else:
                    # API-based generation (fallback)
                    image = self._generate_image_api(prompt, element_type)
                
                if image:
                    # Save the image
                    image_path = os.path.join(scene_folder, f"{element_type}_{i+1}.png")
                    image.save(image_path)
                    
                    # Add to visuals dictionary
                    if element_type not in visuals:
                        visuals[element_type] = []
                    
                    visuals[element_type].append({
                        "path": image_path,
                        "prompt": prompt,
                        "element_index": i
                    })
                    
                    self.logger.info(f"Generated {element_type} visual for {scene_title}")
                
            except Exception as e:
                self.logger.error(f"Error generating visual: {str(e)}")
        
        return visuals
    
    def _create_visual_prompt(self, element_type: str, content: Dict, scene: Dict) -> str:
        """Create a prompt for image generation based on element type and content"""
        style_words = {
            "modern": "clean, modern, minimal design, professional, sharp lines",
            "minimalist": "minimalist, simple, clean, uncluttered, essential elements only",
            "playful": "playful, vibrant, fun, colorful, engaging, dynamic"
        }
        
        style_prompt = style_words.get(self.style, "modern")
        
        # Base prompts by element type
        if element_type == "title_slide":
            title = content.get("title", scene.get("title", "Educational Content"))
            subtitle = content.get("subtitle", "")
            prompt = f"A {self.style} educational title slide with title '{title}'"
            if subtitle:
                prompt += f" and subtitle '{subtitle}'"
            prompt += f". {style_prompt}, educational presentation slide"
            
        elif element_type == "text_slide":
            title = content.get("title", "")
            text = content.get("text", "")
            prompt = f"A {self.style} educational slide with title '{title}' and text content. {style_prompt}, educational presentation"
            
        elif element_type == "bullet_points":
            title = content.get("title", "Key Points")
            points = content.get("points", [])
            points_text = ", ".join(points[:3])  # Include first few points
            prompt = f"A {self.style} slide with bullet points titled '{title}' including points about: {points_text}. {style_prompt}, educational presentation"
            
        elif element_type == "diagram":
            diagram_type = content.get("diagram_type", "generic")
            title = content.get("title", "Diagram")
            prompt = f"A {diagram_type} diagram titled '{title}' in {self.style} style. {style_prompt}, educational, clear diagram"
            
        elif element_type == "figure":
            caption = content.get("caption", "")
            prompt = f"An educational figure with caption '{caption}'. {style_prompt}, clear illustration, educational"
            
        elif element_type == "whiteboard":
            title = content.get("title", scene.get("title", ""))
            content_text = content.get("content", "")
            prompt = f"A whiteboard drawing explaining '{title}'. {style_prompt}, hand-drawn, educational whiteboard"
            
        elif element_type == "concept_graph":
            prompt = f"A concept graph or mind map in {self.style} style. {style_prompt}, nodes and connections, educational diagram"
            
        elif element_type == "summary_slide":
            title = content.get("title", "Summary")
            prompt = f"A summary slide titled '{title}' in {self.style} style. {style_prompt}, educational, conclusion slide"
            
        else:
            # Generic fallback
            prompt = f"Educational content slide in {self.style} style. {style_prompt}, educational presentation"
        
        # Add color scheme information
        prompt += f". Color scheme: {self.color_scheme}"
        
        return prompt
    
    def _generate_image_local(self, prompt: str, element_type: str) -> Optional[Image.Image]:
        """Generate an image using local Stable Diffusion model"""
        if not self.pipe:
            return None
            
        try:
            # Use specialized diffusion pipeline for diagrams and concept graphs if available
            if element_type in ["diagram", "concept_graph"] and self.diffusion_pipe is not None:
                try:
                    self.logger.info(f"Using DiffusionPipeline for {element_type}")
                    with autocast("cuda"):
                        image = self.diffusion_pipe(
                            prompt=prompt,
                            num_inference_steps=30,
                            guidance_scale=7.5,
                        ).images[0]
                    return image
                except Exception as e:
                    self.logger.warning(f"Error with DiffusionPipeline, falling back to standard: {str(e)}")
                    # Fall back to standard pipeline
            
            # Use standard StableDiffusionPipeline
            with autocast("cuda"):
                image = self.pipe(
                    prompt=prompt,
                    num_inference_steps=30,
                    guidance_scale=7.5,
                ).images[0]
                
            return image
            
        except Exception as e:
            self.logger.error(f"Error generating image locally: {str(e)}")
            return None
    
    def _generate_image_api(self, prompt: str, element_type: str) -> Optional[Image.Image]:
        """Generate an image using HuggingFace Inference API (fallback method)"""
        try:
            # Use HuggingFace Inference API
            API_URL = f"https://api-inference.huggingface.co/models/{self.model_id}"
            
            # Get API token from environment variable
            api_token = os.environ.get("HUGGINGFACE_API_TOKEN", "")
            if not api_token:
                self.logger.warning("No Hugging Face API token found. Set HUGGINGFACE_API_TOKEN environment variable.")
                # Generate a placeholder image
                return self._generate_placeholder_image(prompt, element_type)
            
            headers = {
                "Authorization": f"Bearer {api_token}",
                "Content-Type": "application/json"
            }
            
            payload = {
                "inputs": prompt,
                "parameters": {
                    "num_inference_steps": 30,
                    "guidance_scale": 7.5,
                }
            }
            
            response = requests.post(API_URL, headers=headers, json=payload)
            if response.status_code == 200:
                # Convert to PIL Image
                image = Image.open(io.BytesIO(response.content))
                return image
            else:
                self.logger.error(f"API Error: {response.status_code}, {response.text}")
                # Generate a placeholder image as fallback
                return self._generate_placeholder_image(prompt, element_type)
        
        except Exception as e:
            self.logger.error(f"Error generating image via API: {str(e)}")
            # Generate a placeholder image as fallback
            return self._generate_placeholder_image(prompt, element_type)
    
    def _generate_placeholder_image(self, prompt: str, element_type: str) -> Image.Image:
        """Generate a placeholder image with text when AI generation fails"""
        # Create a simple colored background with text
        width, height = 512, 512
        background_color = self.color_palettes[self.color_scheme]["background"]
        text_color = self.color_palettes[self.color_scheme]["text"]
        
        # Convert hex colors to RGB
        def hex_to_rgb(hex_color):
            hex_color = hex_color.lstrip('#')
            return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        
        bg_color = hex_to_rgb(background_color)
        
        # Create image
        image = Image.new('RGB', (width, height), bg_color)
        
        # Return the placeholder image
        return image

    def _generate_scene_animation(self, scene: Dict) -> Dict:
        """Generate animation specification for a single scene"""
        visual_elements = scene.get("visual_elements", [])
        narration_duration = scene.get("duration_estimate", 10)
        
        # Initialize animation elements for this scene
        animation_elements = []
        total_duration = 0
        
        # Process each visual element
        for visual in visual_elements:
            visual_type = visual.get("type", "text_slide")
            
            # Get animation template for this type of visual
            templates = self.animation_templates.get(visual_type, self.animation_templates["text_slide"])
            template = templates.get(self.animation_level, templates["standard"])
            
            # Generate animation element based on the template
            animation_element = {
                "type": visual_type,
                "animation_type": template["animation_type"],
                "duration": template["duration"],
                "start_time": total_duration,
                "elements": []
            }
            
            # Add visual content
            if visual_type == "title_slide":
                animation_element["content"] = {
                    "title": visual.get("title", scene["title"]),
                    "subtitle": visual.get("subtitle", ""),
                    "background_color": self.color_palettes[self.color_scheme]["background"],
                    "text_color": self.color_palettes[self.color_scheme]["text"],
                    "accent_color": self.color_palettes[self.color_scheme]["accent"]
                }
            
            elif visual_type == "text_slide":
                animation_element["content"] = {
                    "title": visual.get("title", ""),
                    "text": visual.get("content", ""),
                    "background_color": self.color_palettes[self.color_scheme]["background"],
                    "text_color": self.color_palettes[self.color_scheme]["text"],
                    "highlight_color": self.color_palettes[self.color_scheme]["accent"]
                }
            
            elif visual_type == "bullet_points":
                animation_element["content"] = {
                    "title": visual.get("title", "Key Points"),
                    "points": visual.get("points", []),
                    "background_color": self.color_palettes[self.color_scheme]["background"],
                    "text_color": self.color_palettes[self.color_scheme]["text"],
                    "bullet_color": self.color_palettes[self.color_scheme]["primary"]
                }
            
            elif visual_type == "diagram":
                # Get diagram content from the visual element
                diagram_content = visual.get("content", {})
                animation_element["content"] = {
                    "diagram_type": diagram_content.get("type", "generic"),
                    "title": diagram_content.get("title", "Diagram"),
                    "svg_content": diagram_content.get("svg", ""),
                    "components": diagram_content.get("components", []),
                    "background_color": self.color_palettes[self.color_scheme]["background"],
                    "primary_color": self.color_palettes[self.color_scheme]["primary"],
                    "secondary_color": self.color_palettes[self.color_scheme]["secondary"]
                }
            
            elif visual_type == "figure":
                # Get figure content from the visual element
                figure_content = visual.get("content", {})
                animation_element["content"] = {
                    "image_path": figure_content.get("path", ""),
                    "caption": figure_content.get("caption", ""),
                    "alt_text": figure_content.get("alt_text", ""),
                    "highlight_areas": figure_content.get("highlights", [])
                }
            
            elif visual_type == "whiteboard":
                animation_element["content"] = {
                    "title": visual.get("title", scene["title"]),
                    "content": visual.get("content", ""),
                    "drawing_style": self.style,
                    "background_color": "#FFFFFF",  # Always white for whiteboard
                    "ink_color": "#333333"  # Dark gray ink
                }
            
            elif visual_type == "concept_graph":
                # Get graph content from the visual element
                graph_content = visual.get("content", {})
                animation_element["content"] = {
                    "nodes": graph_content.get("nodes", []),
                    "edges": graph_content.get("edges", []),
                    "node_color": self.color_palettes[self.color_scheme]["primary"],
                    "edge_color": self.color_palettes[self.color_scheme]["tertiary"],
                    "highlight_color": self.color_palettes[self.color_scheme]["accent"],
                    "background_color": self.color_palettes[self.color_scheme]["background"]
                }
            
            elif visual_type == "summary_slide":
                animation_element["content"] = {
                    "title": visual.get("title", "Summary"),
                    "points": visual.get("points", []),
                    "background_color": self.color_palettes[self.color_scheme]["background"],
                    "text_color": self.color_palettes[self.color_scheme]["text"],
                    "accent_color": self.color_palettes[self.color_scheme]["accent"]
                }
            
            # Add animation-specific properties
            animation_element["timing"] = {
                "start_time": total_duration,
                "duration": template["duration"],
                "end_time": total_duration + template["duration"]
            }
            
            # Apply style customizations
            self._apply_style_customizations(animation_element)
            
            # Add to scene elements and update total duration
            animation_elements.append(animation_element)
            total_duration += template["duration"]
        
        # If narration is longer than animations, extend the last animation
        if total_duration < narration_duration and animation_elements:
            last_element = animation_elements[-1]
            duration_extension = narration_duration - total_duration
            last_element["timing"]["duration"] += duration_extension
            last_element["timing"]["end_time"] += duration_extension
            total_duration = narration_duration
        
        # Create the final animation specification
        animation_spec = {
            "elements": animation_elements,
            "total_duration": total_duration,
            "style": self.style,
            "color_scheme": self.color_scheme,
            "animation_level": self.animation_level
        }
        
        return animation_spec
    
    def _apply_style_customizations(self, animation_element: Dict):
        """Apply style-specific customizations to animation elements"""
        element_type = animation_element["type"]
        
        # Apply modern style customizations
        if self.style == "modern":
            # Add drop shadows to text elements
            if element_type in ["title_slide", "text_slide", "bullet_points", "summary_slide"]:
                animation_element["style_effects"] = {
                    "text_shadow": {"enabled": True, "blur": 4, "color": "rgba(0,0,0,0.2)"},
                    "use_sans_serif": True,
                    "rounded_corners": True,
                    "border_radius": 8
                }
            
            # Clean, minimal diagrams with thin lines
            elif element_type in ["diagram", "concept_graph"]:
                animation_element["style_effects"] = {
                    "line_thickness": "thin",
                    "use_gradients": True,
                    "shadow": {"enabled": True, "blur": 10, "opacity": 0.2},
                    "rounded_corners": True
                }
        
        # Apply minimalist style customizations
        elif self.style == "minimalist":
            # Simple, clean text without decorations
            if element_type in ["title_slide", "text_slide", "bullet_points", "summary_slide"]:
                animation_element["style_effects"] = {
                    "text_shadow": {"enabled": False},
                    "use_sans_serif": True,
                    "use_thin_font": True,
                    "uppercase_headings": True
                }
            
            # Simple diagrams with monochrome colors
            elif element_type in ["diagram", "concept_graph"]:
                animation_element["style_effects"] = {
                    "line_thickness": "thin",
                    "use_gradients": False,
                    "monochromatic": True,
                    "shadow": {"enabled": False}
                }
        
        # Apply playful style customizations
        elif self.style == "playful":
            # Playful text with rounded fonts and bright colors
            if element_type in ["title_slide", "text_slide", "bullet_points", "summary_slide"]:
                animation_element["style_effects"] = {
                    "text_shadow": {"enabled": True, "blur": 0, "offset_y": 2, "color": "rgba(0,0,0,0.3)"},
                    "use_rounded_font": True,
                    "use_bright_colors": True,
                    "add_decorative_elements": True
                }
            
            # Colorful diagrams with fun elements
            elif element_type in ["diagram", "concept_graph"]:
                animation_element["style_effects"] = {
                    "line_thickness": "medium",
                    "use_gradients": True,
                    "use_bright_colors": True,
                    "add_decorative_elements": True,
                    "rounded_corners": True,
                    "playful_icons": True
                }
        
        # Additional customizations based on animation level
        if self.animation_level == "complex":
            animation_element["advanced_effects"] = {
                "use_parallax": True,
                "use_3d_elements": element_type in ["diagram", "concept_graph"],
                "use_particle_effects": True,
                "dynamic_backgrounds": True
            }
    
    def _generate_transition(self) -> Dict:
        """Generate a transition specification between scenes"""
        # Select transition type based on animation level
        transitions = self.transition_templates.get(self.animation_level, self.transition_templates["standard"])
        transition = random.choice(transitions)
        
        # Apply style-specific customizations
        if self.style == "minimalist":
            # Prefer simple transitions for minimalist style
            transition = random.choice(self.transition_templates["simple"])
        
        return transition
    
    def customize_animation_style(self, animation_data: Dict, style=None, 
                               color_scheme=None, animation_level=None) -> Dict:
        """
        Customize the style of existing animation data
        
        Args:
            animation_data: Existing animation data
            style: New style to apply
            color_scheme: New color scheme
            animation_level: New animation complexity level
            
        Returns:
            Updated animation data
        """
        # Update style parameters
        if style:
            self.style = style
        if color_scheme:
            self.color_scheme = color_scheme
        if animation_level:
            self.animation_level = animation_level
        
        # Update metadata
        animation_data["metadata"].update({
            "style": self.style,
            "color_scheme": self.color_scheme,
            "color_palette": self.color_palettes[self.color_scheme],
            "animation_level": self.animation_level
        })
        
        # Mark for regeneration
        animation_data["needs_regeneration"] = True
        
        return animation_data 

---


## mistralModel.py

In [ ]:
import requests
import json

class OpenRouterMistralClient:
    CHAR_LIMIT = 30000  # approximate character limit to avoid context overflow

    def __init__(self, api_key: str = None,
                 model: str = "mistralai/mistral-7b-instruct-v0.2"):
        """
        LM client for Mistralâ€‘7Bâ€‘Instruct via OpenRouter's free API.

        Args:
            api_key: Your OpenRouter API key (starts with sk-â€¦)
            model:   The model identifier on OpenRouter
        """
        self.url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = api_key
        self.headers = {
            "Authorization": f"Bearer {api_key}" if api_key else "",
            "Content-Type": "application/json"
        }
        self.model = model

    def _chunk_text(self, text: str) -> list[str]:
        """
        Split text into chunks under CHAR_LIMIT characters.
        """
        chunks = []
        start = 0
        while start < len(text):
            end = min(start + self.CHAR_LIMIT, len(text))
            # try to split at last newline within limit for cleaner chunks
            if end < len(text):
                nl = text.rfind("\n", start, end)
                if nl > start:
                    end = nl
            chunks.append(text[start:end])
            start = end
        return chunks

    def update_api_key(self, api_key: str):
        """
        Update the API key after initialization
        
        Args:
            api_key: OpenRouter API key (starts with sk-...)
        """
        self.api_key = api_key
        self.headers["Authorization"] = f"Bearer {api_key}"
        return True

    def generate(self, prompt: str, max_tokens: int = 256) -> str:
        """
        Generate text completion for a prompt.
        
        Args:
            prompt: The full text prompt to complete
            max_tokens: Maximum number of tokens to generate
            
        Returns:
            Generated text completion
        """
        if not self.api_key:
            raise ValueError("API key is required. Set it in constructor or call update_api_key()")
        
        return self._generate(prompt_template="{text}", text=prompt, max_tokens=max_tokens)

    def _generate(self, prompt_template: str, text: str, max_tokens: int = 256) -> str:
        """
        Core chatâ€‘completion call to OpenRouter, with input chunking.

        Args:
            prompt_template: a format string expecting placeholders subbed
            text:            the raw text to insert into the prompt
            max_tokens:      model's max new tokens

        Returns:
            Combined responses across all text chunks.
        """
        if not self.api_key:
            raise ValueError("API key is required. Set it in constructor or call update_api_key()")
            
        responses = []
        for chunk in self._chunk_text(text):
            prompt = prompt_template.replace("{text}", chunk)
            payload = {
                "model": self.model,
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": max_tokens
            }
            resp = requests.post(self.url, headers=self.headers, json=payload)
            if not resp.ok:
                raise RuntimeError(
                    f"OpenRouter API error {resp.status_code}: {resp.text}"
                )
            data = resp.json()
            if "choices" not in data:
                raise KeyError(f"OpenRouter response missing 'choices': {json.dumps(data)}")
            content = data["choices"][0]["message"]["content"]
            responses.append(content)
        # Join chunk responses naturally
        return "\n".join(responses)

    def extract_keywords(self, text: str, domain: str = None):
        """Extract key terms and concepts from text."""
        template = (
            "Extract the key terms and concepts from the following {domain} text. "
            "Return a JSON with fields 'keywords' and 'concepts':\n\n{text}"
        )
        return self._generate(template.replace("{domain}", domain or 'general'), text)

    def analyze_sentiment(self, text: str):
        """Analyze sentiment of text."""
        template = (
            "Provide a sentiment analysis of the following text. "
            "Return a JSON with fields 'sentiment' and 'score':\n\n{text}"
        )
        return self._generate(template, text)

    def segment_content(self, text: str):
        """Split text into logical segments."""
        template = (
            "Split the following into logical content segments. "
            "Return a JSON array of segments:\n\n{text}"
        )
        result = self._generate(template, text, max_tokens=512)
        try:
            # Try to parse as JSON
            segments = json.loads(result)
            return segments if isinstance(segments, list) else [result]
        except json.JSONDecodeError:
            # Fall back to line splitting if not valid JSON
            return [seg.strip() for seg in result.split("\n") if seg.strip()]

    def generate_semantic_tags(self, text: str, domain: str = None):
        """Generate semantic tags for text."""
        template = (
            "Generate a list of semantic tags relevant to {domain} domain for this text. "
            "Return a JSON array of tags:\n\n{text}"
        )
        result = self._generate(template.replace("{domain}", domain or 'general'), text)
        try:
            # Try to parse as JSON
            tags = json.loads(result)
            return tags if isinstance(tags, list) else result.split(",")
        except json.JSONDecodeError:
            # Fall back to comma splitting if not valid JSON
            return [tag.strip() for tag in result.split(",")]

    def summarize_text(self, text: str, max_length: int = 200):
        """Summarize text within max length."""
        template = (
            "Summarize the following text in no more than {max} words:\n\n{text}"
        )
        return self._generate(template.replace("{max}", str(max_length)), text, max_tokens=max_length)

    def analyze_literary_work(self, text: str, subtype: str):
        """Analyze literary work by subtype."""
        template = (
            "As a literary critic analyzing a {subtype}, provide a JSON with fields "
            "'characters', 'structure', 'literary_devices', and 'themes':\n\n{text}"
        )
        return self._generate(template.replace("{subtype}", subtype), text, max_tokens=512)
    
    # New domain-specific analysis methods
    
    def analyze_scientific_content(self, text: str):
        """Analyze scientific content."""
        template = (
            "Analyze this scientific text and provide a JSON with fields "
            "'terminology', 'methods', 'findings', and 'conclusions':\n\n{text}"
        )
        return self._generate(template, text, max_tokens=512)
    
    def analyze_mathematical_content(self, text: str):
        """Analyze mathematical content."""
        template = (
            "Analyze this mathematical text and provide a JSON with fields "
            "'theorems', 'proofs', 'equations', 'definitions', and 'problem_types':\n\n{text}"
        )
        return self._generate(template, text, max_tokens=512)
    
    def analyze_chemical_content(self, text: str):
        """Analyze chemical content."""
        template = (
            "Analyze this chemistry text and provide a JSON with fields "
            "'compounds', 'reactions', and 'reaction_conditions':\n\n{text}"
        )
        return self._generate(template, text, max_tokens=512)
    
    def analyze_biological_content(self, text: str):
        """Analyze biological content."""
        template = (
            "Analyze this biology text and provide a JSON with fields "
            "'species', 'genes', and 'biological_pathways':\n\n{text}"
        )
        return self._generate(template, text, max_tokens=512)
    
    def analyze_physics_content(self, text: str):
        """Analyze physics content."""
        template = (
            "Analyze this physics text and provide a JSON with fields "
            "'physical_laws', 'physical_quantities', and 'physics_equations':\n\n{text}"
        )
        return self._generate(template, text, max_tokens=512)
    
    def analyze_section(self, text: str, domain: str = None):
        """Analyze a specific section with domain context."""
        template = (
            "Analyze this {domain} section and provide a JSON with fields "
            "'summary', 'key_points', and 'relevant_entities':\n\n{text}"
        )
        return self._generate(template.replace("{domain}", domain or 'general'), text, max_tokens=384)

class MistralSmallClient:
    """
    Client for interacting with Mistral Small model for script generation and other tasks.
    
    This class is optimized for educational script generation using Mistral Small 3.1,
    which is well-suited for knowledge-intensive tasks like educational content creation.
    """
    
    CHAR_LIMIT = 32000  # Higher context limit for Mistral Small
    
    def __init__(self, api_key: str = None, 
                 model: str = "mistralai/mistral-small-latest"):
        """
        Initialize the Mistral Small client.
        
        Args:
            api_key: Your Mistral API key
            model: The model identifier (defaults to Mistral Small)
        """
        self.api_key = api_key
        self.model = model
        self.url = "https://api.mistral.ai/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}" if api_key else "",
            "Content-Type": "application/json"
        }
    
    def _chunk_text(self, text: str) -> list[str]:
        """
        Split text into chunks under CHAR_LIMIT characters.
        """
        chunks = []
        start = 0
        while start < len(text):
            end = min(start + self.CHAR_LIMIT, len(text))
            # try to split at last newline within limit for cleaner chunks
            if end < len(text):
                nl = text.rfind("\n", start, end)
                if nl > start:
                    end = nl
            chunks.append(text[start:end])
            start = end
        return chunks
    
    def update_api_key(self, api_key: str):
        """
        Update the API key after initialization
        
        Args:
            api_key: Mistral API key
        """
        self.api_key = api_key
        self.headers["Authorization"] = f"Bearer {api_key}"
        return True
    
    def generate(self, prompt: str, max_tokens: int = 1024, 
                temperature: float = 0.7, system_prompt: str = None) -> str:
        """
        Generate text using Mistral Small model.
        
        Args:
            prompt: The text prompt to complete
            max_tokens: Maximum number of tokens to generate
            temperature: Controls randomness (0.0-1.0)
            system_prompt: Optional system prompt for context
            
        Returns:
            Generated text
        """
        if not self.api_key:
            raise ValueError("API key is required. Set it in constructor or call update_api_key()")
        
        messages = []
        
        # Add system prompt if provided
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
            
        # Add user prompt
        messages.append({"role": "user", "content": prompt})
        
        payload = {
            "model": self.model,
            "messages": messages,
            "max_tokens": max_tokens,
            "temperature": temperature
        }
        
        resp = requests.post(self.url, headers=self.headers, json=payload)
        if not resp.ok:
            raise RuntimeError(
                f"Mistral API error {resp.status_code}: {resp.text}"
            )
        
        data = resp.json()
        if "choices" not in data:
            raise KeyError(f"Mistral response missing 'choices': {json.dumps(data)}")
        
        return data["choices"][0]["message"]["content"]
    
    def _generate_with_messages(self, messages: list, max_tokens: int = 1024,
                               temperature: float = 0.7) -> str:
        """
        Generate response using a list of message objects.
        
        Args:
            messages: List of message objects (role, content)
            max_tokens: Maximum tokens to generate
            temperature: Controls randomness
            
        Returns:
            Generated text
        """
        if not self.api_key:
            raise ValueError("API key is required. Set it in constructor or call update_api_key()")
        
        payload = {
            "model": self.model,
            "messages": messages,
            "max_tokens": max_tokens,
            "temperature": temperature
        }
        
        resp = requests.post(self.url, headers=self.headers, json=payload)
        if not resp.ok:
            raise RuntimeError(
                f"Mistral API error {resp.status_code}: {resp.text}"
            )
        
        data = resp.json()
        if "choices" not in data:
            raise KeyError(f"Mistral response missing 'choices': {json.dumps(data)}")
        
        return data["choices"][0]["message"]["content"]
    
    def generate_educational_script(self, topic: str, key_concepts: list, 
                                  audience: str = "general", tone: str = "neutral",
                                  max_length: int = 2000) -> str:
        """
        Generate an educational script focused on explaining a topic.
        
        Args:
            topic: The main topic of the script
            key_concepts: List of key concepts to cover
            audience: Target audience (elementary, secondary, university, general)
            tone: Desired tone (neutral, conversational, formal)
            max_length: Maximum length of the script in tokens
            
        Returns:
            Generated educational script
        """
        system_prompt = (
            "You are an expert educational script writer. Your task is to create "
            "clear, engaging, and accurate educational scripts for videos. "
            "Incorporate proper transitions between topics and use language "
            "appropriate for the target audience."
        )
        
        # Format key concepts as a comma-separated list
        concepts_text = ", ".join(key_concepts) if isinstance(key_concepts, list) else key_concepts
        
        user_prompt = f"""
        Create an educational script about {topic} for a {audience} audience.
        
        Key concepts to cover:
        {concepts_text}
        
        The script should:
        - Use a {tone} tone
        - Include a clear introduction and conclusion
        - Explain each concept in a way appropriate for the {audience} audience
        - Include appropriate transition phrases between sections
        - Mark places where a visual should be displayed with [VISUAL: description]
        
        Format the script with clear section headers and concise content.
        """
        
        return self.generate(user_prompt, max_tokens=max_length, 
                           temperature=0.7, system_prompt=system_prompt)
    
    def generate_section_content(self, section_title: str, description: str, 
                               key_concepts: list, audience: str = "general",
                               tone: str = "neutral", max_length: int = 500) -> str:
        """
        Generate content for a specific section of an educational script.
        
        Args:
            section_title: Title of the section
            description: Brief description of the section
            key_concepts: Key concepts to cover in this section
            audience: Target audience
            tone: Desired tone
            max_length: Maximum length of the section content
            
        Returns:
            Generated section content
        """
        system_prompt = (
            "You are writing a section for an educational script. Your writing should be "
            "clear, accurate, and engaging. Structure your response to flow naturally "
            "in a spoken format, with appropriate transitions and language for the target audience."
        )
        
        # Format key concepts
        concepts_text = ", ".join(key_concepts) if isinstance(key_concepts, list) else key_concepts
        
        user_prompt = f"""
        Write the content for the "{section_title}" section of an educational script.
        
        Section description: {description}
        
        Key concepts to address: {concepts_text}
        
        Requirements:
        - Use a {tone} tone appropriate for a {audience} audience
        - Keep the content focused and concise
        - Include appropriate transitions for a spoken script
        - Mark visual cues with [VISUAL: description]
        - Limit to approximately {max_length//5} words
        
        The content should be ready to be performed as part of a video script.
        """
        
        return self.generate(user_prompt, max_tokens=max_length, 
                           temperature=0.7, system_prompt=system_prompt)
    
    def generate_introduction(self, topic: str, key_concepts: list, 
                            audience: str = "general", tone: str = "neutral") -> str:
        """
        Generate an engaging introduction for an educational script.
        
        Args:
            topic: The main topic
            key_concepts: Key concepts to mention
            audience: Target audience 
            tone: Desired tone
            
        Returns:
            Generated introduction
        """
        # Format key concepts
        concepts_text = ", ".join(key_concepts) if isinstance(key_concepts, list) else key_concepts
        
        system_prompt = "You are writing an engaging introduction for an educational video script."
        
        user_prompt = f"""
        Create an engaging introduction for an educational video about {topic}.
        
        The introduction should:
        - Use a {tone} tone appropriate for a {audience} audience
        - Briefly mention that we'll cover these key concepts: {concepts_text}
        - Hook the viewer's interest with a compelling opening
        - Be approximately 100-150 words
        - Include a visual cue if appropriate with [VISUAL: description]
        
        The introduction should naturally lead into the main content of the video.
        """
        
        return self.generate(user_prompt, max_tokens=350, 
                           temperature=0.7, system_prompt=system_prompt)
    
    def generate_conclusion(self, topic: str, key_concepts: list, 
                          audience: str = "general", tone: str = "neutral") -> str:
        """
        Generate a conclusion for an educational script.
        
        Args:
            topic: The main topic
            key_concepts: Key concepts to summarize
            audience: Target audience 
            tone: Desired tone
            
        Returns:
            Generated conclusion
        """
        # Format key concepts
        concepts_text = ", ".join(key_concepts) if isinstance(key_concepts, list) else key_concepts
        
        system_prompt = "You are writing a concise, effective conclusion for an educational video script."
        
        user_prompt = f"""
        Create a strong conclusion for an educational video about {topic}.
        
        The conclusion should:
        - Use a {tone} tone appropriate for a {audience} audience
        - Briefly summarize the key concepts we covered: {concepts_text}
        - Emphasize why this topic matters or how it connects to broader contexts
        - Include a call to action or thought-provoking final statement
        - Be approximately 80-120 words
        
        The conclusion should provide a sense of closure while reinforcing the main takeaways.
        """
        
        return self.generate(user_prompt, max_tokens=300, 
                           temperature=0.7, system_prompt=system_prompt)
    
    def analyze_content_for_script(self, content: str, domain: str = None) -> dict:
        """
        Analyze content to extract key elements for script generation.
        
        Args:
            content: The content to analyze
            domain: Optional domain context
            
        Returns:
            Dictionary with extracted script elements
        """
        system_prompt = (
            "You are an expert content analyst for educational video production. "
            "Your task is to extract key elements from content that will be useful "
            "for creating an educational script."
        )
        
        domain_context = f"in the {domain} domain" if domain else ""
        
        user_prompt = f"""
        Analyze the following content {domain_context} and extract elements for an educational script.
        
        Content:
        {content[:4000]}
        
        Extract and return a JSON object with the following structure:
        {{
            "topic": "The main topic of the content",
            "key_concepts": ["List of 5-8 key concepts"],
            "suggested_structure": [
                {{"section": "Introduction", "description": "Brief intro to topic"}},
                {{"section": "Concept 1", "description": "Explanation of first key concept"}},
                // Additional sections
                {{"section": "Conclusion", "description": "Summary and key takeaways"}}
            ],
            "visual_elements": [
                {{"type": "diagram/chart/image", "description": "What should be shown"}}
            ],
            "target_audience": "Suggested target audience based on content complexity",
            "estimated_duration": "Estimated video length in minutes"
        }}
        
        Ensure the suggested structure follows a logical flow and covers all key concepts.
        """
        
        response = self.generate(user_prompt, max_tokens=1500, 
                               temperature=0.3, system_prompt=system_prompt)
        
        # Try to extract JSON from the response
        try:
            # Find JSON object in response
            json_start = response.find('{')
            json_end = response.rfind('}') + 1
            if json_start >= 0 and json_end > json_start:
                json_str = response[json_start:json_end]
                return json.loads(json_str)
            else:
                return {"error": "Could not extract JSON from response", "raw_response": response}
        except json.JSONDecodeError:
            return {"error": "Failed to parse JSON", "raw_response": response}

---


## MultimediaPipeline.py

In [ ]:
"""
Multimedia Generation Pipeline
----------------------------
Main pipeline for generating educational videos from structured content.
Integrates script generation, animation, text-to-speech, and video compilation.
"""

import os
import json
import logging
from typing import Dict, List, Any, Optional, Union
import time
import threading
import psutil
import uuid
from concurrent.futures import ThreadPoolExecutor

from pipeline.multimedia.ScriptGenerator import ScriptGenerator
from pipeline.multimedia.AnimationGenerator import AnimationGenerator
from pipeline.multimedia.TextToSpeech import TextToSpeechGenerator
from pipeline.multimedia.VideoCompiler import VideoCompiler
from pipeline.errors.WorkflowErrors import (
    MultimediaGenerationError, ScriptGenerationError, 
    AnimationError, AudioGenerationError, VideoRenderingError,
    ResourceLimitError, TimeoutError
)

class RenderQuality:
    """Video rendering quality levels with resource requirements"""
    LOW = {
        "name": "low",
        "resolution": "720p",
        "fps": 24,
        "bitrate": "1M",
        "cpu_usage": 0.3,
        "memory_usage": 1024,  # MB
        "disk_usage": 50,  # MB per minute
        "rendering_factor": 0.5,  # Relative time factor
    }
    MEDIUM = {
        "name": "medium",
        "resolution": "1080p",
        "fps": 30,
        "bitrate": "3M",
        "cpu_usage": 0.5,
        "memory_usage": 2048,  # MB
        "disk_usage": 150,  # MB per minute
        "rendering_factor": 1.0,  # Relative time factor
    }
    HIGH = {
        "name": "high",
        "resolution": "1440p",
        "fps": 30,
        "bitrate": "8M",
        "cpu_usage": 0.7,
        "memory_usage": 4096,  # MB
        "disk_usage": 300,  # MB per minute
        "rendering_factor": 2.0,  # Relative time factor
    }
    ULTRA = {
        "name": "ultra",
        "resolution": "2160p",
        "fps": 60,
        "bitrate": "15M",
        "cpu_usage": 0.9,
        "memory_usage": 8192,  # MB
        "disk_usage": 600,  # MB per minute
        "rendering_factor": 4.0,  # Relative time factor
    }
    
    @classmethod
    def from_string(cls, quality_name: str) -> 'RenderQuality':
        """Get quality enum from string name"""
        for quality in cls:
            if quality.value["name"] == quality_name.lower():
                return quality
        return cls.MEDIUM  # Default

class RenderPriority:
    """Priority levels for rendering jobs"""
    LOW = 3
    NORMAL = 2
    HIGH = 1
    CRITICAL = 0

class RenderJob:
    """Represents a video rendering job in the queue"""
    def __init__(self, job_id: str, workflow_id: str, script_path: str, analysis_result: Dict[str, Any], output_path: str, quality: RenderQuality, priority: RenderPriority, submitted_time: float, parameters: Dict[str, Any]):
        self.job_id = job_id
        self.workflow_id = workflow_id
        self.script_path = script_path
        self.analysis_result = analysis_result
        self.output_path = output_path
        self.quality = quality
        self.priority = priority
        self.submitted_time = submitted_time
        self.parameters = parameters
        self.status = "queued"
        self.progress = 0.0
        self.result = None
        self.error = None

    def __lt__(self, other: 'RenderJob') -> bool:
        """Compare jobs for priority queue ordering"""
        # First compare by priority level
        if self.priority.value != other.priority.value:
            return self.priority.value < other.priority.value
        
        # Then by submission time (FIFO)
        return self.submitted_time < other.submitted_time

class ResourceMonitor:
    """Monitors system resources and adapts rendering settings"""
    
    def __init__(self, poll_interval: int = 5, 
                cpu_threshold: float = 0.8, 
                memory_threshold: float = 0.8):
        """
        Initialize resource monitor
        
        Args:
            poll_interval: Seconds between resource checks
            cpu_threshold: CPU usage threshold (0.0-1.0)
            memory_threshold: Memory usage threshold (0.0-1.0)
        """
        self.poll_interval = poll_interval
        self.cpu_threshold = cpu_threshold
        self.memory_threshold = memory_threshold
        self.logger = logging.getLogger(self.__class__.__name__)
        
        # Resource metrics
        self.metrics = {
            "cpu_usage": 0.0,
            "memory_usage": 0.0,
            "memory_available": 0,
            "disk_usage": 0.0,
            "disk_available": 0,
            "last_updated": 0
        }
        
        # Start monitoring thread
        self.running = True
        self.monitor_thread = threading.Thread(target=self._monitor_loop, daemon=True)
        self.monitor_thread.start()
    
    def _monitor_loop(self) -> None:
        """Background thread that monitors system resources"""
        while self.running:
            try:
                # Update resource metrics
                self.metrics["cpu_usage"] = psutil.cpu_percent(interval=1) / 100.0
                
                memory = psutil.virtual_memory()
                self.metrics["memory_usage"] = memory.percent / 100.0
                self.metrics["memory_available"] = memory.available / (1024 * 1024)  # MB
                
                disk = psutil.disk_usage(os.path.abspath(os.sep))
                self.metrics["disk_usage"] = disk.percent / 100.0
                self.metrics["disk_available"] = disk.free / (1024 * 1024)  # MB
                
                self.metrics["last_updated"] = time.time()
                
                # Log if resources are constrained
                if self.metrics["cpu_usage"] > self.cpu_threshold:
                    self.logger.warning(f"CPU usage high: {self.metrics['cpu_usage']:.2f}")
                
                if self.metrics["memory_usage"] > self.memory_threshold:
                    self.logger.warning(f"Memory usage high: {self.metrics['memory_usage']:.2f}")
                
            except Exception as e:
                self.logger.error(f"Error monitoring resources: {str(e)}")
            
            time.sleep(self.poll_interval)
    
    def get_metrics(self) -> Dict[str, Any]:
        """Get current resource metrics"""
        return self.metrics.copy()
    
    def can_handle_quality(self, quality: RenderQuality) -> bool:
        """
        Check if system can handle a specific render quality
        
        Args:
            quality: The desired render quality
            
        Returns:
            True if system has sufficient resources
        """
        # Check if metrics are recent
        if time.time() - self.metrics["last_updated"] > self.poll_interval * 2:
            # Force an update
            self._update_metrics()
        
        # Check CPU availability (consider current usage + required usage)
        if self.metrics["cpu_usage"] + quality.value["cpu_usage"] > 0.95:
            return False
        
        # Check memory availability
        if self.metrics["memory_available"] < quality.value["memory_usage"] * 1.2:
            return False
        
        # Check disk availability (ensure at least 10GB or 10x expected usage)
        min_disk_required = max(10 * 1024, quality.value["disk_usage"] * 10)
        if self.metrics["disk_available"] < min_disk_required:
            return False
        
        return True
    
    def recommend_quality(self, requested_quality: RenderQuality) -> RenderQuality:
        """
        Recommend a quality level based on available resources
        
        Args:
            requested_quality: The desired quality
            
        Returns:
            Recommended quality level
        """
        # If we can handle the requested quality, return it
        if self.can_handle_quality(requested_quality):
            return requested_quality
        
        # Otherwise, try to find a lower quality that will work
        qualities = list(RenderQuality)
        requested_index = qualities.index(requested_quality)
        
        for i in range(requested_index - 1, -1, -1):
            if self.can_handle_quality(qualities[i]):
                return qualities[i]
        
        # If even the lowest quality won't work, return it anyway
        # but log a warning
        self.logger.warning("System resources may be insufficient even for low quality rendering")
        return RenderQuality.LOW
    
    def _update_metrics(self) -> None:
        """Force an immediate update of resource metrics"""
        # Update CPU usage
        self.metrics["cpu_usage"] = psutil.cpu_percent(interval=0.5) / 100.0
        
        # Update memory usage
        memory = psutil.virtual_memory()
        self.metrics["memory_usage"] = memory.percent / 100.0
        self.metrics["memory_available"] = memory.available / (1024 * 1024)  # MB
        
        # Update disk usage
        disk = psutil.disk_usage(os.path.abspath(os.sep))
        self.metrics["disk_usage"] = disk.percent / 100.0
        self.metrics["disk_available"] = disk.free / (1024 * 1024)  # MB
        
        self.metrics["last_updated"] = time.time()
    
    def shutdown(self) -> None:
        """Stop the resource monitoring thread"""
        self.running = False
        if self.monitor_thread.is_alive():
            self.monitor_thread.join(timeout=2)

class MultimediaGenerationPipeline:
    """
    Pipeline for generating multimedia content from analysis results
    with resource-aware rendering and job prioritization
    """
    
    def __init__(self, api_token: Optional[str] = None, output_dir: str = "output",
                video_format: str = "mp4", video_quality: str = "high",
                max_concurrent_renders: int = 2, scheduler_type: str = "ddim"):
        """
        Initialize the multimedia generation pipeline
        
        Args:
            api_token: API token for external services
            output_dir: Output directory for generated content
            video_format: Output video format
            video_quality: Default video quality
            max_concurrent_renders: Maximum concurrent rendering jobs
            scheduler_type: Diffusion scheduler type
        """
        self.api_token = api_token
        self.output_dir = output_dir
        self.video_format = video_format
        self.default_quality = RenderQuality.from_string(video_quality)
        self.max_concurrent_renders = max_concurrent_renders
        self.scheduler_type = scheduler_type
        
        self.logger = logging.getLogger(self.__class__.__name__)
        
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Initialize resource monitor
        self.resource_monitor = ResourceMonitor()
        
        # Job queue and tracking
        self.job_queue = PriorityQueue()
        self.active_jobs: Dict[str, RenderJob] = {}
        self.completed_jobs: Dict[str, RenderJob] = {}
        self.job_lock = threading.RLock()
        
        # Start worker thread pool
        self.workers = ThreadPoolExecutor(max_workers=max_concurrent_renders)
        
        # Start scheduler thread
        self.running = True
        self.scheduler_thread = threading.Thread(target=self._scheduler_loop, daemon=True)
        self.scheduler_thread.start()
        
        self.logger.info(f"Multimedia pipeline initialized with {max_concurrent_renders} "
                      f"render workers and {video_quality} default quality")
    
    def generate_video(self, analysis_result: Dict[str, Any], extracted_content: Dict[str, Any],
                     output_dir: Optional[str] = None, **kwargs) -> Dict[str, Any]:
        """
        Generate a video from analysis results
        
        Args:
            analysis_result: Content analysis result
            extracted_content: Extracted PDF content
            output_dir: Output directory (overrides default)
            **kwargs: Additional parameters
            
        Returns:
            Dictionary with video generation results
            
        Raises:
            MultimediaGenerationError: If video generation fails
        """
        # Validate inputs
        if not analysis_result or not isinstance(analysis_result, dict):
            raise MultimediaGenerationError("Invalid analysis result")
        
        if not extracted_content or not isinstance(extracted_content, dict):
            raise MultimediaGenerationError("Invalid extracted content")
        
        try:
            # Generate script
            script_result = self._generate_script(analysis_result, extracted_content, **kwargs)
            
            # Get target directory
            target_dir = output_dir or self.output_dir
            os.makedirs(target_dir, exist_ok=True)
        
            # Determine quality based on resource availability
            requested_quality = kwargs.get("video_quality", self.default_quality)
            if isinstance(requested_quality, str):
                requested_quality = RenderQuality.from_string(requested_quality)
            
            recommended_quality = self.resource_monitor.recommend_quality(requested_quality)
            
            if recommended_quality != requested_quality:
                self.logger.warning(f"Adjusted render quality from {requested_quality.value['name']} "
                                 f"to {recommended_quality.value['name']} due to resource constraints")
        
            # Check if we need to render synchronously or asynchronously
            if kwargs.get("async_render", False):
                # Submit as job and return job ID
                job_id = self._submit_render_job(
                    workflow_id=kwargs.get("workflow_id", "unknown"),
                    script_path=script_result["script_file"],
                    analysis_result=analysis_result,
                    output_path=target_dir,
                    quality=recommended_quality,
                    priority=RenderPriority.NORMAL,
                    parameters=kwargs
                )
                
                return {
                    "job_id": job_id,
                    "script_file": script_result["script_file"],
                    "status": "rendering_queued",
                    "estimated_completion": self._estimate_completion_time(job_id),
                    "quality": recommended_quality.value["name"]
                }
            else:
                # Render synchronously
                render_result = self._render_video(
                    script_result["script_file"],
                    analysis_result,
                    target_dir,
                    recommended_quality,
                    kwargs
                )
                
                # Combine script and render results
        result = {
                    **script_result,
                    **render_result,
                    "quality": recommended_quality.value["name"]
                }
                
        return result
    
        except Exception as e:
            error_msg = f"Video generation failed: {str(e)}"
            self.logger.error(error_msg, exc_info=True)
            
            if isinstance(e, MultimediaGenerationError):
                raise
            else:
                raise MultimediaGenerationError(error_msg)
    
    def _generate_script(self, analysis_result: Dict[str, Any], 
                       extracted_content: Dict[str, Any], **kwargs) -> Dict[str, Any]:
        """
        Generate a video script from analysis results
        
        Args:
            analysis_result: Content analysis result
            extracted_content: Extracted PDF content
            **kwargs: Additional parameters
            
        Returns:
            Dictionary with script generation results
            
        Raises:
            ScriptGenerationError: If script generation fails
        """
        try:
            # Implementation of script generation...
            
            # For now, return placeholder
            script_file = os.path.join(kwargs.get("output_dir", self.output_dir), "script.txt")
            
            # Ensure directory exists
            os.makedirs(os.path.dirname(script_file), exist_ok=True)
            
            # Write placeholder script
            with open(script_file, "w") as f:
                f.write(f"Title: {analysis_result.get('topic', 'Untitled')}\n\n")
                f.write("Introduction:\n")
                f.write(f"{analysis_result.get('summary', {}).get('brief', 'Introduction content')}\n\n")
                
                # Write sections
                for section in analysis_result.get("structure", {}).get("sections", []):
                    f.write(f"Section: {section.get('title', 'Untitled Section')}\n")
                    f.write(f"{section.get('content', 'Section content')}\n\n")
                
                f.write("Conclusion:\n")
                f.write("This concludes the presentation.\n")
            
            return {
                "script_file": script_file,
                "scenes": 3,
                "duration_estimate": 120,
                "tone": kwargs.get("script_style", {}).get("tone", "neutral")
            }
            
        except Exception as e:
            error_msg = f"Script generation failed: {str(e)}"
            self.logger.error(error_msg)
            raise ScriptGenerationError(error_msg)
    
    def _render_video(self, script_path: str, analysis_result: Dict[str, Any],
                    output_path: str, quality: RenderQuality, 
                    parameters: Dict[str, Any]) -> Dict[str, Any]:
        """
        Render a video from a script
        
        Args:
            script_path: Path to script file
            analysis_result: Content analysis result
            output_path: Output directory
            quality: Render quality
            parameters: Additional parameters
            
        Returns:
            Dictionary with render results
            
        Raises:
            VideoRenderingError: If rendering fails
        """
        try:
            # Implementation of video rendering...
            
            # Check if we have sufficient resources for this quality
            if not self.resource_monitor.can_handle_quality(quality):
                # Log warning and proceed (we've already adjusted quality if needed)
                self.logger.warning(f"System resources may be insufficient for {quality.value['name']} "
                                 f"quality rendering")
            
            # For now, return placeholder
            video_file = os.path.join(output_path, "video.mp4")
            
            # Simulate rendering time based on quality
            rendering_time = 2.0 * quality.value["rendering_factor"]
            time.sleep(rendering_time)
            
            # Create empty video file
            with open(video_file, "wb") as f:
                f.write(b"PLACEHOLDER")
            
            return {
                "video_file": video_file,
                "video_length": 120,
                "resolution": quality.value["resolution"],
                "fps": quality.value["fps"],
                "format": self.video_format,
                "render_time": rendering_time
            }
            
        except Exception as e:
            error_msg = f"Video rendering failed: {str(e)}"
            self.logger.error(error_msg)
            raise VideoRenderingError(error_msg)
    
    def _submit_render_job(self, workflow_id: str, script_path: str, 
                         analysis_result: Dict[str, Any], output_path: str,
                         quality: RenderQuality, priority: RenderPriority,
                         parameters: Dict[str, Any]) -> str:
        """
        Submit a rendering job to the queue
        
        Args:
            workflow_id: ID of the associated workflow
            script_path: Path to script file
            analysis_result: Content analysis result
            output_path: Output directory
            quality: Render quality
            priority: Job priority
            parameters: Additional parameters
            
        Returns:
            Job ID
        """
        job_id = str(uuid.uuid4())
        
        job = RenderJob(
            job_id=job_id,
            workflow_id=workflow_id,
            script_path=script_path,
            analysis_result=analysis_result,
            output_path=output_path,
            quality=quality,
            priority=priority,
            submitted_time=time.time(),
            parameters=parameters
        )
        
        with self.job_lock:
            # Add to queue
            self.job_queue.put(job)
            
            # Store in active jobs
            self.active_jobs[job_id] = job
        
        self.logger.info(f"Submitted render job {job_id} with {priority.name} priority "
                      f"and {quality.value['name']} quality")
        
        return job_id
    
    def _scheduler_loop(self) -> None:
        """Background thread that schedules render jobs"""
        while self.running:
            try:
                # Check how many active renders we have
                active_renders = sum(1 for job in self.active_jobs.values() 
                                  if job.status == "rendering")
                
                # Process jobs if we have capacity
                if active_renders < self.max_concurrent_renders and not self.job_queue.empty():
                    # Get next job
                    job = self.job_queue.get(block=False)
                    
                    # Update job status
                    with self.job_lock:
                        job.status = "rendering"
                        self.active_jobs[job.job_id] = job
                    
                    # Submit to worker pool
                    self.workers.submit(self._process_render_job, job)
                    
                    self.logger.info(f"Started rendering job {job.job_id} with "
                                  f"{job.quality.value['name']} quality")
            
            except Exception as e:
                if not isinstance(e, queue.Empty):
                    self.logger.error(f"Error in scheduler loop: {str(e)}")
            
            # Sleep briefly
            time.sleep(0.5)
    
    def _process_render_job(self, job: RenderJob) -> None:
        """
        Process a render job
        
        Args:
            job: Render job to process
        """
        try:
            # Render the video
            render_result = self._render_video(
                job.script_path,
                job.analysis_result,
                job.output_path,
                job.quality,
                job.parameters
            )
            
            # Update job with result
            with self.job_lock:
                job.status = "completed"
                job.progress = 1.0
                job.result = render_result
                
                # Move from active to completed
                if job.job_id in self.active_jobs:
                    del self.active_jobs[job.job_id]
                self.completed_jobs[job.job_id] = job
            
            self.logger.info(f"Completed render job {job.job_id}")
            
        except Exception as e:
            # Update job with error
            with self.job_lock:
                job.status = "failed"
                job.error = str(e)
                
                # Move from active to completed
                if job.job_id in self.active_jobs:
                    del self.active_jobs[job.job_id]
                self.completed_jobs[job.job_id] = job
            
            self.logger.error(f"Render job {job.job_id} failed: {str(e)}")
    
    def get_job_status(self, job_id: str) -> Dict[str, Any]:
        """
        Get the status of a render job
        
        Args:
            job_id: Job ID
            
        Returns:
            Job status dictionary
            
        Raises:
            KeyError: If job not found
        """
        with self.job_lock:
            if job_id in self.active_jobs:
                job = self.active_jobs[job_id]
            elif job_id in self.completed_jobs:
                job = self.completed_jobs[job_id]
            else:
                raise KeyError(f"Job {job_id} not found")
            
            status = {
                "job_id": job.job_id,
                "status": job.status,
                "progress": job.progress,
                "quality": job.quality.value["name"],
                "submitted_time": job.submitted_time,
                "elapsed_time": time.time() - job.submitted_time
            }
            
            if job.status == "completed" and job.result:
                status["result"] = job.result
            elif job.status == "failed" and job.error:
                status["error"] = job.error
            elif job.status == "queued" or job.status == "rendering":
                status["estimated_completion"] = self._estimate_completion_time(job_id)
        
            return status
    
    def cancel_job(self, job_id: str) -> bool:
        """
        Cancel a render job
        
        Args:
            job_id: Job ID
            
        Returns:
            True if job was cancelled, False otherwise
        """
        with self.job_lock:
            if job_id in self.active_jobs:
                job = self.active_jobs[job_id]
                
                # Can only cancel queued jobs
                if job.status == "queued":
                    # Mark as cancelled
                    job.status = "cancelled"
                    
                    # Move from active to completed
                    del self.active_jobs[job_id]
                    self.completed_jobs[job_id] = job
                    
                    self.logger.info(f"Cancelled render job {job_id}")
                    return True
            
            return False
    
    def _estimate_completion_time(self, job_id: str) -> float:
        """
        Estimate when a job will complete
        
        Args:
            job_id: Job ID
            
        Returns:
            Estimated completion time timestamp
        """
        with self.job_lock:
            if job_id not in self.active_jobs:
                return 0
            
            job = self.active_jobs[job_id]
            
            # If job is rendering, estimate based on progress
            if job.status == "rendering":
                if job.progress > 0:
                    # Estimate from current progress
                    elapsed = time.time() - job.submitted_time
                    total_estimated = elapsed / job.progress
                    remaining = total_estimated - elapsed
                    return time.time() + remaining
                else:
                    # Fall back to quality-based estimate
                    return time.time() + (120 * job.quality.value["rendering_factor"])
            
            # If job is queued, estimate based on queue position and active jobs
            elif job.status == "queued":
                # Count jobs ahead in queue
                queue_position = 0
                for q_job in list(self.active_jobs.values()):
                    if q_job.status == "queued" and q_job.priority.value <= job.priority.value:
                        if q_job.submitted_time < job.submitted_time:
                            queue_position += 1
                
                # Estimate wait time based on queue position and active jobs
                wait_time = queue_position * (120 * job.quality.value["rendering_factor"])
                return time.time() + wait_time
            
            return 0
    
    def shutdown(self) -> None:
        """Shutdown the pipeline and release resources"""
        self.logger.info("Shutting down multimedia pipeline")
        
        # Stop scheduler
        self.running = False
        if self.scheduler_thread.is_alive():
            self.scheduler_thread.join(timeout=2)
        
        # Shutdown workers
        self.workers.shutdown(wait=True)
        
        # Shutdown resource monitor
        self.resource_monitor.shutdown()
        
        self.logger.info("Multimedia pipeline shutdown complete") 

---


## ScriptGenerator.py

In [ ]:
"""
Script Generator Module
----------------------
This module provides functionality for generating educational scripts from content analysis data.
It works with the MultimediaGenerationPipeline to create structured scripts for educational videos.
"""

import os
import json
import time
import logging
import requests
from typing import Dict, Any, List, Optional, Tuple
from datetime import datetime

from pipeline.content_analysis.AnalysisResult import AnalysisResult
from pipeline.multimedia.mistralModel import OpenRouterMistralClient, MistralSmallClient

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class ScriptGenerator:
    """
    Generates educational scripts from content analysis data.
    
    This class takes content analysis data and transforms it into a structured script
    that can be used for educational video creation. It supports various customization
    options for tone, verbosity, and target audience.
    """
    
    def __init__(self, mistral_client: Optional[OpenRouterMistralClient] = None,
                 mistral_small_client: Optional[MistralSmallClient] = None):
        """
        Initialize the ScriptGenerator with the specified Mistral clients.
        
        Args:
            mistral_client: Optional Mistral client for text generation via OpenRouter
            mistral_small_client: Optional Mistral Small client for improved script generation
        """
        self.mistral_client = mistral_client
        self.mistral_small_client = mistral_small_client
        self.script_structure = {
            "introduction": "Introduction to the topic",
            "main_concepts": "Explanation of main concepts",
            "examples": "Illustrative examples",
            "practical_applications": "Real-world applications",
            "conclusion": "Summary and takeaways"
        }
        
        logger.info("Initialized ScriptGenerator with Mistral clients")
    
    def generate_script(self, analysis_result: AnalysisResult, 
                      structure: Optional[Dict[str, Dict]] = None) -> Dict[str, Any]:
        """
        Generate an educational script from analysis results.
        
        Args:
            analysis_result: Content analysis results object
            structure: Optional custom script structure
            
        Returns:
            Dictionary containing the complete script
        """
        # Extract key elements from analysis result
        topic = self._extract_topic(analysis_result)
        subject = self._extract_subject(analysis_result)
        key_concepts = self._extract_key_concepts(analysis_result)
        
        # Use provided structure or create default
        if not structure:
            structure = self._create_default_structure(topic, subject, key_concepts)
        
        # Determine audience and tone
        audience = self._determine_audience(analysis_result)
        tone = self._determine_tone(analysis_result)
        
        # Initialize script output
        script = {
            "title": topic,
            "subject": subject,
            "audience": audience,
            "tone": tone,
            "generated_date": datetime.now().isoformat(),
            "sections": {}
        }
        
        # Check if we have a Mistral client
        if not self.mistral_client:
            # Generate simple placeholder content for each section
            for section_id, section_info in structure.items():
                script["sections"][section_id] = {
                    "title": section_info["title"],
                    "content": f"Content for {section_info['title']} section about {section_info['description']}."
                }
        else:
            # Use Mistral client to generate content for each section
            for section_id, section_info in structure.items():
                prompt = f"Write content for a {section_id} section of an educational script about {topic}. "
                prompt += f"This section should cover {section_info['description']}. "
                prompt += f"Use a {tone} tone appropriate for a {audience} audience."
                
                # Generate content with Mistral
                content = self.mistral_client.generate(prompt, max_tokens=400)
                
                # Add to script
                script["sections"][section_id] = {
                    "title": section_info["title"],
                    "content": content
                }
        
        return script
    
    def generate_script_with_mistral_small(self, analysis_result: AnalysisResult, 
                                         structure: Optional[Dict[str, Dict]] = None) -> Dict[str, Any]:
        """
        Generate an enhanced educational script using Mistral Small.
        
        This method uses the more advanced Mistral Small model for higher quality script generation,
        with specialized prompts for educational content creation.
        
        Args:
            analysis_result: Content analysis results object
            structure: Optional custom script structure
            
        Returns:
            Dictionary containing the complete script with sections
        """
        # Check if we have a Mistral Small client
        if not self.mistral_small_client:
            # Fall back to standard generation
            return self.generate_script(analysis_result, structure)
        
        # Extract key elements from analysis result
        topic = self._extract_topic(analysis_result)
        subject = self._extract_subject(analysis_result)
        key_concepts = self._extract_key_concepts(analysis_result)
        
        # Determine audience and tone
        audience = self._determine_audience(analysis_result)
        tone = self._determine_tone(analysis_result)
        
        # Initialize script output
        script = {
            "title": topic,
                "subject": subject,
            "audience": audience,
            "tone": tone,
            "generated_date": datetime.now().isoformat(),
            "sections": {}
        }
        
        # Option 1: Generate entire script at once
        if not structure:
            # Generate full script in one call
            full_script = self.mistral_small_client.generate_educational_script(
                topic=topic,
                key_concepts=key_concepts,
                audience=audience,
                tone=tone,
                max_length=2500
            )
            
            # Parse the generated script into sections
            sections = self._parse_generated_script(full_script)
            script["full_text"] = full_script
            script["sections"] = sections
            
            return script
        
        # Option 2: Generate section by section with custom structure
        # Generate introduction
        intro_content = self.mistral_small_client.generate_introduction(
            topic=topic,
            key_concepts=key_concepts,
            audience=audience,
            tone=tone
        )
        
        script["sections"]["introduction"] = {
            "title": "Introduction",
            "content": intro_content
        }
        
        # Generate body sections
        for section_id, section_info in structure.items():
            # Skip introduction and conclusion which are handled separately
            if section_id in ["introduction", "conclusion"]:
                continue
                
            section_content = self.mistral_small_client.generate_section_content(
                section_title=section_info["title"],
                description=section_info["description"],
                key_concepts=section_info.get("concepts", key_concepts),
                audience=audience,
                tone=tone,
                max_length=600
            )
            
            script["sections"][section_id] = {
                "title": section_info["title"],
                "content": section_content
            }
        
        # Generate conclusion
        conclusion_content = self.mistral_small_client.generate_conclusion(
            topic=topic,
            key_concepts=key_concepts,
            audience=audience,
            tone=tone
        )
        
        script["sections"]["conclusion"] = {
            "title": "Conclusion",
            "content": conclusion_content
        }
        
        return script
    
    def _extract_topic(self, analysis_result: AnalysisResult) -> str:
        """
        Extract the primary topic from analysis results.
        
        Args:
            analysis_result: Content analysis results object
            
        Returns:
            Topic string
        """
        # Try to get topic from metadata
        if analysis_result.metadata and 'title' in analysis_result.metadata:
            return analysis_result.metadata['title']
        
        # Fallback to first key concept
        if analysis_result.key_concepts and len(analysis_result.key_concepts) > 0:
            return analysis_result.key_concepts[0]
        
        # Last resort
        return "Educational Content"
    
    def _extract_subject(self, analysis_result: AnalysisResult) -> str:
        """
        Extract the subject area from analysis results.
        
        Args:
            analysis_result: Content analysis results object
            
        Returns:
            Subject string
        """
        # Get domain from analysis result
        if analysis_result.domain:
            return analysis_result.domain
        
        # Fallback to general
        return "general"
    
    def _extract_key_concepts(self, analysis_result: AnalysisResult) -> List[str]:
        """
        Extract key concepts from analysis results.
        
        Args:
            analysis_result: Content analysis results object
            
        Returns:
            List of key concept strings
        """
        concepts = []
        
        # Add explicit key concepts
        if analysis_result.key_concepts:
            concepts.extend(analysis_result.key_concepts)
        
        # Add important entities
        if analysis_result.entities:
            for entity in analysis_result.entities:
                if entity.get('importance', 0) > 0.7 and entity.get('name'):
                    concepts.append(entity['name'])
        
        # Deduplicate
        return list(set(concepts))
    
    def _create_default_structure(self, topic: str, subject: str, key_concepts: List[str]) -> Dict[str, Dict]:
        """
        Create a default script structure.
        
        Args:
            topic: The main topic
            subject: The subject area
            key_concepts: List of key concepts
            
        Returns:
            Dictionary of script sections
        """
        structure = {}
        
        # Introduction section
        structure["introduction"] = {
            "title": "Introduction",
            "description": f"Introduction to {topic}",
            "concepts": key_concepts[:2] if len(key_concepts) > 2 else key_concepts
        }
        
        # Main concepts
        if len(key_concepts) > 0:
            for i, concept in enumerate(key_concepts[:3]):  # Limit to 3 main concepts
                section_id = f"concept_{i+1}"
                structure[section_id] = {
                    "title": concept,
                    "description": f"Explanation of {concept}",
                    "concepts": [concept]
                }
        
        # Examples section
        structure["examples"] = {
            "title": "Examples",
            "description": f"Examples relating to {topic}",
            "concepts": key_concepts
        }
        
        # Conclusion
        structure["conclusion"] = {
            "title": "Conclusion",
            "description": f"Summary and key takeaways about {topic}",
            "concepts": key_concepts
        }
        
        return structure
    
    def _determine_audience(self, analysis_result: AnalysisResult) -> str:
        """
        Determine the appropriate audience based on content complexity.
        
        Args:
            analysis_result: Content analysis results
            
        Returns:
            Audience string (elementary, secondary, university, general)
        """
        # If explicitly provided in metadata
        if analysis_result.metadata and 'audience' in analysis_result.metadata:
            return analysis_result.metadata['audience']
        
        # Default to general
        return "general"
    
    def _determine_tone(self, analysis_result: AnalysisResult) -> str:
        """
        Determine the appropriate tone based on content and domain.
        
        Args:
            analysis_result: Content analysis results
            
        Returns:
            Tone string (conversational, formal, neutral)
        """
        # If explicitly provided in metadata
        if analysis_result.metadata and 'tone' in analysis_result.metadata:
            return analysis_result.metadata['tone']
        
        # Based on domain
        domain = analysis_result.domain
        if domain == "elementary_education":
            return "conversational"
        elif domain in ["scientific", "academic", "medical"]:
            return "formal"
        
        # Default to neutral
        return "neutral"
    
    def _parse_generated_script(self, script_text: str) -> Dict[str, Dict[str, str]]:
        """
        Parse a full generated script into structured sections.
        
        Args:
            script_text: The full text of the generated script
            
        Returns:
            Dictionary of sections with title and content
        """
        sections = {}
        current_section = None
        current_content = []
        
        # Split the script by lines
        lines = script_text.split('\n')
        
        for line in lines:
            # Check if this line is a section header
            # Section headers are typically capitalized or have # or == markers
            if (line.strip().isupper() or 
                line.strip().startswith('#') or 
                line.strip().startswith('==') or
                (line.strip() and line.strip() == line.strip().title() and len(line.strip()) < 50)):
                
                # Save previous section if there was one
                if current_section:
                    sections[current_section.lower().replace(' ', '_')] = {
                        "title": current_section,
                        "content": '\n'.join(current_content)
                    }
                
                # Start new section
                current_section = line.strip().replace('#', '').replace('=', '').strip()
                current_content = []
            else:
                # Add line to current section content
                if current_section:
                    current_content.append(line)
        
        # Add the last section
        if current_section:
            sections[current_section.lower().replace(' ', '_')] = {
                "title": current_section,
                "content": '\n'.join(current_content)
            }
        
        # If no sections were found, create a single section
        if not sections:
            sections["main_content"] = {
                "title": "Main Content",
                "content": script_text
            }
        
        return sections
    
    def save_script(self, script: Dict[str, Any], output_dir: str, filename: Optional[str] = None) -> str:
        """
        Save the generated script to a file.
        
        Args:
            script: The script dictionary
            output_dir: Directory to save the script
            filename: Optional filename (default: based on title)
            
        Returns:
            Path to the saved script file
        """
        # Create directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Generate filename if not provided
        if not filename:
            # Create filename from title
            title_slug = script["title"].lower().replace(' ', '_')
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{title_slug}_{timestamp}.json"
        
        # Ensure filename has .json extension
        if not filename.endswith('.json'):
            filename += '.json'
        
        # Full path
        file_path = os.path.join(output_dir, filename)
        
        # Save script to file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(script, f, indent=2, ensure_ascii=False)
        
        logger.info(f"Saved script to {file_path}")
        return file_path 

---


## TextToSpeech.py

In [ ]:
"""
Text-to-Speech Module
-------------------
Converts script text into natural-sounding narration audio.
Handles voice selection, pronunciation, pacing, and audio file generation.
Uses gTTS (Google Text-to-Speech) for free speech synthesis.
"""

import re
import os
import json
import logging
import time
from typing import Dict, List, Any, Optional, Union
import uuid
from pathlib import Path

# Import TTS libraries
try:
    from gtts import gTTS
    import pygame
    from pydub import AudioSegment
    TTS_AVAILABLE = True
except ImportError:
    TTS_AVAILABLE = False

class TextToSpeechGenerator:
    """Generates speech audio from script text using Google Text-to-Speech"""
    
    def __init__(self, voice_id="neutral", language_code="en-US", 
                voice_speed=1.0, pitch=0, use_ssml=False):
        """
        Initialize the TTS generator with voice preferences
        
        Args:
            voice_id: Voice identifier or style
            language_code: Language code (e.g., en-US, fr-FR)
            voice_speed: Speech rate multiplier (0.5-2.0)
            pitch: Voice pitch adjustment (-10 to 10)
            use_ssml: Whether to use SSML for enhanced speech control
        """
        self.voice_id = voice_id
        self.language_code = language_code
        self.voice_speed = max(0.5, min(2.0, voice_speed))  # Clamp to valid range
        self.pitch = max(-10, min(10, pitch))  # Clamp to valid range
        self.use_ssml = use_ssml
        
        # Output directory for audio files
        self.output_dir = os.path.join("output", "audio")
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Configure logging
        logging.basicConfig(level=logging.INFO, 
                           format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger("TextToSpeechGenerator")
        
        # Define voice profiles mapped to language codes
        self.voice_profiles = {
            "neutral": {
                "gender": "neutral",
                "age": "adult",
                "style": "neutral",
                "pitch_adjustment": 0,
                "rate_adjustment": 0,
                "lang_map": {"en-US": "en", "fr-FR": "fr", "es-ES": "es", "de-DE": "de"}
            },
            "professional": {
                "gender": "male",
                "age": "adult",
                "style": "formal",
                "pitch_adjustment": -1,
                "rate_adjustment": -0.1,
                "lang_map": {"en-US": "en", "fr-FR": "fr", "es-ES": "es", "de-DE": "de"}
            },
            "friendly": {
                "gender": "female",
                "age": "adult",
                "style": "conversational",
                "pitch_adjustment": 1,
                "rate_adjustment": 0.1,
                "lang_map": {"en-US": "en", "fr-FR": "fr", "es-ES": "es", "de-DE": "de"}
            },
            "energetic": {
                "gender": "male",
                "age": "young-adult",
                "style": "enthusiastic",
                "pitch_adjustment": 2,
                "rate_adjustment": 0.2,
                "lang_map": {"en-US": "en", "fr-FR": "fr", "es-ES": "es", "de-DE": "de"}
            },
            "calm": {
                "gender": "female",
                "age": "adult",
                "style": "gentle",
                "pitch_adjustment": -1,
                "rate_adjustment": -0.2,
                "lang_map": {"en-US": "en", "fr-FR": "fr", "es-ES": "es", "de-DE": "de"}
            },
            "authoritative": {
                "gender": "male",
                "age": "middle-aged",
                "style": "authoritative",
                "pitch_adjustment": -2,
                "rate_adjustment": -0.1,
                "lang_map": {"en-US": "en", "fr-FR": "fr", "es-ES": "es", "de-DE": "de"}
            }
        }
        
        # Initialize the TTS engine
        self._initialize_tts_engine()
    
    def generate_speech(self, script_data: Dict) -> Dict[str, Any]:
        """
        Generate speech audio from script data
        
        Args:
            script_data: The script data generated by ScriptGenerator
            
        Returns:
            Dictionary with audio specifications for each scene
        """
        self.logger.info("Generating speech audio")
        
        scenes = script_data.get("scenes", [])
        if not scenes:
            self.logger.warning("No scenes found in script data")
            return {"scenes": [], "metadata": {}}
        
        # Ensure TTS engine is initialized
        if not TTS_AVAILABLE:
            self.logger.warning("TTS engine not available. Install required packages: pip install gtts pygame pydub")
            return {"scenes": [], "metadata": {"error": "TTS engine not available"}}
        
        # Generate audio for each scene
        audio_scenes = []
        total_audio_duration = 0
        
        for scene in scenes:
            # Generate audio for this scene
            narration_text = scene.get("narration", "")
            scene_title = scene.get("title", "Untitled Scene")
            scene_id = scene.get("scene_id", f"scene_{len(audio_scenes)+1}")
            
            if not narration_text:
                self.logger.warning(f"No narration text found for scene: {scene_title}")
                audio_spec = {
                    "audio_file": None,
                    "duration": 0,
                    "word_timings": []
                }
            else:
                # Process the narration text
                processed_text = self._process_narration_text(narration_text)
                
                # Generate audio spec
                audio_spec = self._generate_scene_audio(processed_text, scene_id, scene_title)
                total_audio_duration += audio_spec["duration"]
            
            # Add to audio scenes
            audio_scenes.append({
                "scene_id": scene_id,
                "title": scene_title,
                "audio_spec": audio_spec
            })
        
        # Create metadata
        result = {
            "scenes": audio_scenes,
            "metadata": {
                "voice_id": self.voice_id,
                "language_code": self.language_code,
                "voice_speed": self.voice_speed,
                "pitch": self.pitch,
                "total_duration": total_audio_duration,
                "word_count": sum(len(scene.get("narration", "").split()) for scene in scenes)
            }
        }
        
        self.logger.info(f"Speech generation complete: {len(audio_scenes)} scenes, {total_audio_duration:.1f} seconds")
        return result
    
    def _initialize_tts_engine(self):
        """Initialize the TTS engine"""
        if not TTS_AVAILABLE:
            self.logger.warning("TTS libraries not available. Install with: pip install gtts pygame pydub")
            return
            
        self.logger.info("Initializing TTS engine (Google Text-to-Speech)")
        
        # Initialize pygame for audio playback (needed for duration calculation)
        try:
            pygame.mixer.init()
        except Exception as e:
            self.logger.warning(f"Could not initialize audio playback: {str(e)}")
    
    def _process_narration_text(self, text: str) -> str:
        """
        Process narration text for better speech synthesis
        
        Args:
            text: The raw narration text
            
        Returns:
            Processed text ready for TTS
        """
        # Clean up whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Handle abbreviations and acronyms
        abbreviations = {
            r'\bDr\.': 'Doctor',
            r'\bMr\.': 'Mister',
            r'\bMrs\.': 'Misses',
            r'\bMs\.': 'Miss',
            r'\bPh\.D\.': 'P H D',
            r'\be\.g\.': 'for example',
            r'\bi\.e\.': 'that is',
            r'\betc\.': 'etcetera',
        }
        
        for abbr, expansion in abbreviations.items():
            text = re.sub(abbr, expansion, text)
        
        # Convert numbers to spoken form for better pronunciation
        number_patterns = [
            (r'\b(\d+)\%', r'\1 percent'),
            (r'\$(\d+)', r'\1 dollars'),
            (r'\b(\d{4})\b', self._process_year)  # Special handling for years
        ]
        
        for pattern, replacement in number_patterns:
            if callable(replacement):
                # Find all matches and process each one
                for match in re.finditer(pattern, text):
                    full_match = match.group(0)
                    # Extract the captured group
                    captured = match.group(1)
                    # Replace with the function's output
                    text = text.replace(full_match, replacement(captured))
            else:
                text = re.sub(pattern, replacement, text)
        
        # Note: gTTS doesn't support SSML, so we don't use it
        return text
    
    def _process_year(self, year_str: str) -> str:
        """Process 4-digit years into spoken form"""
        year = int(year_str)
        
        # Handle special cases for years
        if 1000 <= year <= 1999:
            # Years like "nineteen ninety-nine" for 1999
            century = year // 100
            remainder = year % 100
            
            century_words = {
                10: "ten", 11: "eleven", 12: "twelve", 13: "thirteen",
                14: "fourteen", 15: "fifteen", 16: "sixteen",
                17: "seventeen", 18: "eighteen", 19: "nineteen"
            }
            
            if remainder == 0:
                return f"{century_words[century]} hundred"
            elif remainder < 10:
                return f"{century_words[century]} oh {remainder}"
            else:
                return f"{century_words[century]} {remainder}"
        else:
            # Just return the digits for other years (TTS will usually handle these well)  
            return year_str
    
    def _convert_to_ssml(self, text: str) -> str:
        """
        Convert plain text to SSML for better speech control.
        Note: gTTS doesn't support SSML, so this is kept for future implementation.
        
        Args:
            text: Plain text to convert
            
        Returns:
            SSML-formatted text
        """
        # Basic SSML wrapper
        ssml = f'<speak><prosody rate="{self.voice_speed}" pitch="{self.pitch:+d}st">{text}</prosody></speak>'
        
        # Add pauses for punctuation
        ssml = ssml.replace('. ', '. <break time="300ms"/>')
        ssml = ssml.replace('! ', '! <break time="300ms"/>')
        ssml = ssml.replace('? ', '? <break time="300ms"/>')
        ssml = ssml.replace(', ', ', <break time="200ms"/>')
        ssml = ssml.replace('; ', '; <break time="250ms"/>')
        ssml = ssml.replace(':', ': <break time="200ms"/>')
        ssml = ssml.replace('... ', '... <break time="500ms"/>')
        ssml = ssml.replace('â€” ', 'â€” <break time="250ms"/>')
        
        # Process text formatting patterns for different types of emphasis
        # Strong emphasis (bold)
        ssml = re.sub(r'\*\*([^*]+)\*\*', r'<emphasis level="strong">\1</emphasis>', ssml)
        
        # Moderate emphasis (italic)
        ssml = re.sub(r'\*([^*]+)\*', r'<emphasis level="moderate">\1</emphasis>', ssml)
        
        # Reduced emphasis
        ssml = re.sub(r'_([^_]+)_', r'<emphasis level="reduced">\1</emphasis>', ssml)
        
        # Handle phonetic pronunciations using IPA format
        # Format: {{word|IPA}}
        ssml = re.sub(r'\{\{([^|]+)\|([^}]+)\}\}', r'<phoneme alphabet="ipa" ph="\2">\1</phoneme>', ssml)
        
        # Handle specialized vocabulary and acronyms
        # Format: [ACRONYM](expansion)
        ssml = re.sub(r'\[([^\]]+)\]\(([^)]+)\)', r'<say-as interpret-as="characters">\1</say-as> <break time="150ms"/> which stands for \2', ssml)
        
        # Handle numbers with specific interpret-as types
        ssml = re.sub(r'#date\{([^}]+)\}', r'<say-as interpret-as="date" format="mdy">\1</say-as>', ssml)
        ssml = re.sub(r'#time\{([^}]+)\}', r'<say-as interpret-as="time" format="hms12">\1</say-as>', ssml)
        ssml = re.sub(r'#cardinal\{([^}]+)\}', r'<say-as interpret-as="cardinal">\1</say-as>', ssml)
        ssml = re.sub(r'#ordinal\{([^}]+)\}', r'<say-as interpret-as="ordinal">\1</say-as>', ssml)
        
        # Handle voice changes for quoted speech or characters
        # Format: "Character: Speech"
        ssml = re.sub(r'"([^:]+): ([^"]+)"', 
                     lambda m: f'<voice name="{self._get_character_voice(m.group(1))}">{m.group(1)} says: {m.group(2)}</voice>',
                     ssml)
        
        # Add appropriate pausing for paragraph breaks
        ssml = re.sub(r'\n\s*\n', '\n<break time="750ms"/>\n', ssml)
        
        # Clean up any nested or overlapping tags
        ssml = self._clean_overlapping_tags(ssml)
        
        return ssml
        
    def _get_character_voice(self, character_name: str) -> str:
        """
        Map character names to appropriate voices.
        In a real implementation, this would use a more sophisticated mapping system.
        
        Args:
            character_name: Name of the character
            
        Returns:
            Voice name appropriate for the character
        """
        # Simple mapping based on character name
        character_name = character_name.lower().strip()
        
        # Map for different character types to voices
        character_voices = {
            "narrator": "neutral",
            "guide": "professional",
            "child": "child",
            "elder": "old",
            "robot": "robot",
            "alien": "alien"
        }
        
        # Check for keywords in character name
        for keyword, voice in character_voices.items():
            if keyword in character_name:
                return voice
        
        # Default voice selection based on first letter (for demonstration)
        first_char = character_name[0] if character_name else 'a'
        if first_char.lower() in 'abcdefgh':
            return "male1"
        elif first_char.lower() in 'ijklmnop':
            return "female1"
        elif first_char.lower() in 'qrstuvwxyz':
            return "male2"
        else:
            return "neutral"
            
    def _clean_overlapping_tags(self, ssml: str) -> str:
        """
        Clean up any nested or overlapping SSML tags to ensure valid output.
        
        Args:
            ssml: SSML text that might contain invalid nested tags
            
        Returns:
            Cleaned SSML text
        """
        # This is a simplified implementation
        # A real implementation would use an XML parser
        
        # Check for common invalid nesting patterns
        common_tags = ['emphasis', 'say-as', 'phoneme', 'voice', 'prosody']
        
        for tag in common_tags:
            # Find any instances where the same tag is nested
            pattern = f'<{tag}[^>]*>.*?<{tag}[^>]*>'
            matches = re.findall(pattern, ssml)
            
            for match in matches:
                # Get everything up to the second tag start
                second_tag_pos = match.find(f'<{tag}', 1)
                if second_tag_pos > 0:
                    fixed = match[:second_tag_pos] + f'</{tag}>' + match[second_tag_pos:]
                    ssml = ssml.replace(match, fixed)
        
        return ssml
    
    def _generate_scene_audio(self, text: str, scene_id: str, scene_title: str) -> Dict:
        """
        Generate audio for a single scene using Google TTS
        
        Args:
            text: Processed narration text
            scene_id: ID of the scene
            scene_title: Title of the scene
            
        Returns:
            Audio specification for the scene
        """
        if not TTS_AVAILABLE:
            self.logger.warning("TTS engine not available")
            return {
                "audio_file": None,
                "duration": 0,
                "word_timings": [],
                "error": "TTS engine not available"
            }
            
        try:
            # Map voice profile to a supported language code for gTTS
            profile = self.voice_profiles.get(self.voice_id, self.voice_profiles["neutral"])
            lang_code = profile["lang_map"].get(self.language_code, "en")
            
            # Create a unique filename
            audio_filename = f"{scene_id}_{uuid.uuid4().hex[:8]}"
            audio_path = os.path.join(self.output_dir, f"{audio_filename}.mp3")
            
            # Create directories if they don't exist
            os.makedirs(os.path.dirname(audio_path), exist_ok=True)
            
            # Generate audio using gTTS
            start_time = time.time()
            self.logger.info(f"Generating audio for: {scene_title}")
            
            # Set slow parameter based on voice speed
            slow = (self.voice_speed < 0.8)
            
            # Create gTTS object and save to file
            tts = gTTS(text=text, lang=lang_code, slow=slow)
            tts.save(audio_path)
            
            # Verify the file was created
            if not os.path.exists(audio_path):
                raise FileNotFoundError(f"Failed to create audio file at {audio_path}")
            
            # Get audio duration and adjust for speed
            audio_duration = self._get_audio_duration(audio_path)
            
            # If voice_speed is different from 1.0, we need to adjust the audio
            if abs(self.voice_speed - 1.0) > 0.1 and audio_duration > 0:
                # Load with pydub and adjust speed
                try:
                    sound = AudioSegment.from_mp3(audio_path)
                    # For simplicity, we're just changing duration which affects pitch too
                    # A more sophisticated approach would use librosa or similar
                    adjusted_sound = sound._spawn(sound.raw_data, overrides={
                        "frame_rate": int(sound.frame_rate * self.voice_speed)
                    })
                    adjusted_sound = adjusted_sound.set_frame_rate(sound.frame_rate)
                    
                    # Save adjusted audio
                    adjusted_path = os.path.join(self.output_dir, f"{audio_filename}_adjusted.mp3")
                    adjusted_sound.export(adjusted_path, format="mp3")
                    
                    # Update path and duration
                    audio_path = adjusted_path
                    audio_duration = self._get_audio_duration(audio_path)
                except Exception as e:
                    self.logger.warning(f"Could not adjust audio speed: {str(e)}")
            
            # Generate word timings
            word_timings = self._generate_word_timings(text, audio_duration)
            
            generation_time = time.time() - start_time
            self.logger.info(f"Generated audio for scene: {scene_title} ({audio_duration:.1f} seconds) in {generation_time:.1f}s")
            
            audio_spec = {
                "audio_file": audio_path,
                "duration": audio_duration,
                "word_timings": word_timings,
                "format": "mp3",
                "sample_rate": 22050,
                "bit_rate": 192
            }
            
            return audio_spec
            
        except Exception as e:
            self.logger.error(f"Error generating audio: {str(e)}")
            # Return an empty audio spec with error information
            return {
                "audio_file": None,
                "duration": 0,
                "word_timings": [],
                "error": str(e)
            }
    
    def _get_audio_duration(self, audio_path: str) -> float:
        """
        Get the duration of an audio file in seconds
        
        Args:
            audio_path: Path to the audio file
            
        Returns:
            Duration in seconds
        """
        if not os.path.exists(audio_path):
            self.logger.warning(f"Audio file does not exist: {audio_path}")
            return 0.0
            
        try:
            # Try to use pydub to get duration
            audio = AudioSegment.from_mp3(audio_path)
            return len(audio) / 1000.0  # Convert ms to seconds
        except Exception as e:
            self.logger.warning(f"Error using pydub for duration: {str(e)}")
            try:
                # Fallback to pygame
                pygame.mixer.music.load(audio_path)
                pygame.mixer.music.play()
                # Give pygame a moment to process
                time.sleep(0.1)
                # Get the length in seconds
                duration = pygame.mixer.music.get_length()
                pygame.mixer.music.stop()
                return duration
            except Exception as e2:
                self.logger.warning(f"Error using pygame for duration: {str(e2)}")
                # Estimate based on text length (fallback)
                try:
                    with open(audio_path, 'rb') as f:
                        file_size = len(f.read())
                        # Very rough estimate: ~10KB per second for mp3
                        estimated_duration = file_size / 10240.0
                        self.logger.warning(f"Using file size estimate for duration: {estimated_duration:.1f}s")
                        return estimated_duration
                except Exception as e3:
                    self.logger.error(f"Complete failure to determine audio duration: {str(e3)}")
                    return 0.0
    
    def _generate_word_timings(self, text: str, total_duration: float) -> List[Dict]:
        """
        Generate word timing information based on text and audio duration
        
        Args:
            text: The narration text
            total_duration: Total audio duration in seconds
            
        Returns:
            List of word timing dictionaries
        """
        # Split text into words
        words = re.findall(r'\b\w+\b', text)
        
        if not words:
            return []
        
        # Calculate average word duration
        avg_word_duration = total_duration / len(words)
        
        # Generate timing info
        word_timings = []
        current_time = 0
        
        for word in words:
            # Slight randomization in word duration for more natural timing
            word_duration = avg_word_duration * (0.8 + 0.4 * (len(word) / 5))
            
            word_timings.append({
                "word": word,
                "start_time": current_time,
                "end_time": current_time + word_duration
            })
            
            current_time += word_duration
        
        # Scale to match total duration
        scale_factor = total_duration / current_time
        for timing in word_timings:
            timing["start_time"] *= scale_factor
            timing["end_time"] *= scale_factor
        
        return word_timings
    
    def customize_voice(self, voice_id=None, speed=None, pitch=None) -> None:
        """
        Customize voice settings
        
        Args:
            voice_id: New voice identifier
            speed: New speech rate
            pitch: New pitch adjustment
        """
        if voice_id:
            self.voice_id = voice_id
            
        if speed is not None:
            self.voice_speed = max(0.5, min(2.0, speed))
            
        if pitch is not None:
            self.pitch = max(-10, min(10, pitch))
            
        self.logger.info(f"Voice customized: id={self.voice_id}, speed={self.voice_speed}, pitch={self.pitch}")
        
    def get_available_voices(self) -> List[Dict]:
        """
        Get list of available voices
        
        Returns:
            List of voice profile dictionaries
        """
        voices = []
        
        for voice_id, profile in self.voice_profiles.items():
            voices.append({
                "id": voice_id,
                "gender": profile["gender"],
                "age": profile["age"],
                "style": profile["style"],
                "language": self.language_code
            })
            
        return voices
        
    def set_language(self, language_code: str) -> None:
        """
        Set the language for speech synthesis
        
        Args:
            language_code: ISO language code (e.g., en-US, fr-FR)
        """
        self.language_code = language_code
        self.logger.info(f"Language set to {language_code}")
    
    def merge_audio_files(self, audio_specs: List[Dict]) -> str:
        """
        Merge multiple audio files into a single file
        
        Args:
            audio_specs: List of audio specifications
            
        Returns:
            Path to the merged audio file
        """
        if not TTS_AVAILABLE:
            self.logger.warning("Audio libraries not available. Cannot merge files.")
            return ""
            
        try:
            # Get valid audio files
            valid_audio_files = [spec["audio_file"] for spec in audio_specs 
                               if spec.get("audio_file") and os.path.exists(spec.get("audio_file", ""))]
            
            if not valid_audio_files:
                self.logger.warning("No valid audio files to merge")
                return ""
                
            # Create a unique merged filename
            merged_filename = f"merged_{uuid.uuid4().hex[:8]}.mp3"
            merged_path = os.path.join(self.output_dir, merged_filename)
            
            # Merge using pydub
            combined = AudioSegment.empty()
            for audio_file in valid_audio_files:
                sound = AudioSegment.from_mp3(audio_file)
                combined += sound
                
            # Export merged file
            combined.export(merged_path, format="mp3")
            
            total_duration = sum(spec.get("duration", 0) for spec in audio_specs)
            self.logger.info(f"Merged {len(valid_audio_files)} audio files ({total_duration:.1f} seconds)")
            
            return merged_path
            
        except Exception as e:
            self.logger.error(f"Error merging audio files: {str(e)}")
            return "" 

---


## VideoCompiler.py

In [ ]:
"""
Video Compiler Module
------------------
Combines animations and audio into complete educational videos.
Handles synchronization, editing, transitions, and final video production.
"""

import os
import time
import json
import logging
import subprocess
import tempfile
import shutil
from typing import Dict, List, Any, Optional, Union

class VideoCompiler:
    """Compiles animations and audio into complete videos"""
    
    def __init__(self, output_format="mp4", resolution=(1920, 1080), 
                fps=30, quality="high", add_subtitles=True):
        """
        Initialize the video compiler with output preferences
        
        Args:
            output_format: Video format (mp4, webm, mov)
            resolution: Video resolution as (width, height) tuple
            fps: Frames per second
            quality: Output quality (low, medium, high)
            add_subtitles: Whether to add subtitles/captions
        """
        self.output_format = output_format
        self.resolution = resolution
        self.fps = fps
        self.quality = quality
        self.add_subtitles = add_subtitles
        
        # Configure logging
        logging.basicConfig(level=logging.INFO, 
                           format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger("VideoCompiler")
        
        # Quality presets
        self.quality_presets = {
            "low": {
                "bitrate": "1M",
                "preset": "veryfast",
                "audio_bitrate": "96k"
            },
            "medium": {
                "bitrate": "4M",
                "preset": "medium",
                "audio_bitrate": "128k"
            },
            "high": {
                "bitrate": "8M",
                "preset": "slow",
                "audio_bitrate": "192k"
            }
        }
        
        # Check if FFmpeg is installed
        try:
            subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
            self.ffmpeg_available = True
            self.logger.info("FFmpeg is available for video processing")
        except (subprocess.SubprocessError, FileNotFoundError):
            self.ffmpeg_available = False
            self.logger.warning("FFmpeg not found. Falling back to simulation mode.")
    
    def compile_video(self, animation_data: Dict, audio_data: Dict, 
                    output_path: str = None) -> Dict[str, Any]:
        """
        Compile animations and audio into a complete video
        
        Args:
            animation_data: Animation data from AnimationGenerator
            audio_data: Audio data from TextToSpeechGenerator
            output_path: Path for the output video file
            
        Returns:
            Dictionary with video compilation results
        """
        self.logger.info("Starting video compilation")
        
        # Get animation and audio scenes
        animation_scenes = animation_data.get("scenes", [])
        audio_scenes = audio_data.get("scenes", [])
        
        # Check if we have matching scenes
        if len(animation_scenes) != len(audio_scenes):
            self.logger.warning(f"Scene count mismatch: {len(animation_scenes)} animation scenes, {len(audio_scenes)} audio scenes")
        
        # Create output directory if needed
        if output_path:
            os.makedirs(os.path.dirname(os.path.abspath(output_path)), exist_ok=True)
        else:
            # Generate default output path
            temp_dir = tempfile.gettempdir()
            output_path = os.path.join(temp_dir, f"educational_video.{self.output_format}")
        
        # Generate overall video specifications
        video_specs = self._generate_video_specs(animation_data, audio_data)
        
        # Compile individual scenes
        scene_results = []
        for i in range(min(len(animation_scenes), len(audio_scenes))):
            animation_scene = animation_scenes[i]
            audio_scene = audio_scenes[i]
            
            # Compile this scene
            scene_result = self._compile_scene(animation_scene, audio_scene, i)
            scene_results.append(scene_result)
        
        # Combine scenes into final video
        video_result = self._combine_scenes(scene_results, video_specs, output_path)
        
        # Generate subtitles if requested
        if self.add_subtitles:
            subtitle_path = self._generate_subtitles(audio_data, output_path)
            video_result["subtitle_file"] = subtitle_path
        
        self.logger.info(f"Video compilation complete: {output_path}")
        return video_result
    
    def _generate_video_specs(self, animation_data: Dict, audio_data: Dict) -> Dict:
        """Generate video specifications based on input data"""
        # Calculate total duration (use the longer of animation or audio)
        animation_duration = animation_data.get("metadata", {}).get("total_duration", 0)
        audio_duration = audio_data.get("metadata", {}).get("total_duration", 0)
        total_duration = max(animation_duration, audio_duration)
        
        # Get quality settings
        quality_preset = self.quality_presets.get(self.quality, self.quality_presets["medium"])
        
        video_specs = {
            "format": self.output_format,
            "width": self.resolution[0],
            "height": self.resolution[1],
            "fps": self.fps,
            "duration": total_duration,
            "total_frames": int(total_duration * self.fps),
            "quality_settings": quality_preset
        }
        
        return video_specs
    
    def _compile_scene(self, animation_scene: Dict, audio_scene: Dict, scene_index: int) -> Dict:
        """
        Compile a single scene combining animation and audio
        
        Args:
            animation_scene: Animation data for the scene
            audio_scene: Audio data for the scene
            scene_index: Index of the scene
            
        Returns:
            Scene compilation result
        """
        scene_id = animation_scene.get("scene_id", f"scene_{scene_index}")
        title = animation_scene.get("title", f"Scene {scene_index}")
        
        # Get animation specification
        animation_spec = animation_scene.get("animation_spec", {})
        animation_duration = animation_spec.get("total_duration", 0)
        
        # Get audio specification
        audio_spec = audio_scene.get("audio_spec", {})
        audio_duration = audio_spec.get("duration", 0)
        audio_file = audio_spec.get("audio_file", "")
        
        # Determine scene duration (use the longer of animation or audio)
        scene_duration = max(animation_duration, audio_duration)
        
        # Generate temporary output file for this scene
        temp_dir = tempfile.gettempdir()
        scene_output = os.path.join(temp_dir, f"{scene_id}.{self.output_format}")
        
        # Real implementation using FFmpeg to render the animation and combine with audio
        if self.ffmpeg_available and os.path.exists(audio_file):
            try:
                # Get animation frames (assuming they're available as image sequence or video)
                animation_frames = animation_spec.get("animation_frames", "")
                animation_video = animation_spec.get("animation_video", "")
                
                # If we have animation frames (as image sequence)
                if animation_frames and os.path.exists(animation_frames):
                    # Create video from image sequence
                    frame_pattern = os.path.join(animation_frames, "frame_%04d.png")
                    cmd = [
                        "ffmpeg", "-y",
                        "-framerate", str(self.fps),
                        "-i", frame_pattern,
                        "-i", audio_file,
                        "-c:v", "libx264",
                        "-preset", self.quality_presets[self.quality]["preset"],
                        "-b:v", self.quality_presets[self.quality]["bitrate"],
                        "-c:a", "aac",
                        "-b:a", self.quality_presets[self.quality]["audio_bitrate"],
                        "-pix_fmt", "yuv420p",
                        "-shortest",
                        scene_output
                    ]
                    subprocess.run(cmd, check=True)
                
                # If we have animation as video
                elif animation_video and os.path.exists(animation_video):
                    # Combine animation video with audio
                    cmd = [
                        "ffmpeg", "-y",
                        "-i", animation_video,
                        "-i", audio_file,
                        "-c:v", "libx264",
                        "-preset", self.quality_presets[self.quality]["preset"],
                        "-b:v", self.quality_presets[self.quality]["bitrate"],
                        "-c:a", "aac",
                        "-b:a", self.quality_presets[self.quality]["audio_bitrate"],
                        "-shortest",
                        scene_output
                    ]
                    subprocess.run(cmd, check=True)
                
                # If we don't have animation, create a video from a color
                else:
                    # Create a simple color background video with audio
                    cmd = [
                        "ffmpeg", "-y",
                        "-f", "lavfi",
                        "-i", f"color=c=blue:s={self.resolution[0]}x{self.resolution[1]}:r={self.fps}:d={scene_duration}",
                        "-i", audio_file,
                        "-c:v", "libx264",
                        "-preset", self.quality_presets[self.quality]["preset"],
                        "-b:v", self.quality_presets[self.quality]["bitrate"],
                        "-c:a", "aac",
                        "-b:a", self.quality_presets[self.quality]["audio_bitrate"],
                        "-shortest",
                        scene_output
                    ]
                    subprocess.run(cmd, check=True)
                
                self.logger.info(f"Compiled scene with FFmpeg: {title} ({scene_duration:.1f} seconds)")
            
            except subprocess.SubprocessError as e:
                self.logger.error(f"FFmpeg scene compilation failed: {str(e)}")
                # Fallback to simulation
                self._simulate_render_time(scene_duration)
        else:
            # Fallback to simulation if FFmpeg not available or audio file missing
            self._simulate_render_time(scene_duration)
            self.logger.info(f"Simulated scene compilation: {title} ({scene_duration:.1f} seconds)")
        
        scene_result = {
            "scene_id": scene_id,
            "title": title,
            "output_file": scene_output,
            "duration": scene_duration,
            "frames": int(scene_duration * self.fps)
        }
        
        return scene_result
    
    def _combine_scenes(self, scene_results: List[Dict], video_specs: Dict, output_path: str) -> Dict:
        """
        Combine individual scenes into the final video
        
        Args:
            scene_results: List of scene compilation results
            video_specs: Overall video specifications
            output_path: Path for the output video file
            
        Returns:
            Video compilation result
        """
        # Real implementation to concatenate scene videos with FFmpeg
        if self.ffmpeg_available and scene_results:
            try:
                # Create a temporary file listing all input videos
                temp_dir = tempfile.gettempdir()
                concat_file = os.path.join(temp_dir, "concat_list.txt")
                
                with open(concat_file, 'w') as f:
                    for scene in scene_results:
                        if os.path.exists(scene["output_file"]):
                            f.write(f"file '{scene['output_file']}'\n")
                
                # Use FFmpeg's concat demuxer to join the videos
                cmd = [
                    "ffmpeg", "-y",
                    "-f", "concat",
                    "-safe", "0",
                    "-i", concat_file,
                    "-c", "copy",
                    output_path
                ]
                subprocess.run(cmd, check=True)
                
                # Calculate total duration
                total_duration = sum(scene.get("duration", 0) for scene in scene_results)
                
                self.logger.info(f"Combined {len(scene_results)} scenes with FFmpeg: {output_path}")
                
            except (subprocess.SubprocessError, IOError) as e:
                self.logger.error(f"FFmpeg scene combination failed: {str(e)}")
                # Fallback to simulation
                total_duration = sum(scene.get("duration", 0) for scene in scene_results)
                self._simulate_render_time(total_duration * 0.2)
        else:
            # Fallback to simulation if FFmpeg not available
            total_duration = sum(scene.get("duration", 0) for scene in scene_results)
            self._simulate_render_time(total_duration * 0.2)
            self.logger.info(f"Simulated scene combination: {output_path}")
        
        # Generate metadata
        video_result = {
            "output_file": output_path,
            "format": video_specs["format"],
            "resolution": f"{video_specs['width']}x{video_specs['height']}",
            "fps": video_specs["fps"],
            "duration": total_duration,
            "frames": int(total_duration * self.fps),
            "scenes": [scene["scene_id"] for scene in scene_results],
            "quality": self.quality
        }
        
        return video_result
    
    def _generate_subtitles(self, audio_data: Dict, video_path: str) -> str:
        """
        Generate subtitles/captions for the video
        
        Args:
            audio_data: Audio data with word timings
            video_path: Path to the video file
            
        Returns:
            Path to the subtitle file
        """
        # Create subtitle file path based on video path
        subtitle_path = os.path.splitext(video_path)[0] + ".srt"
        
        # Extract word timings from all scenes
        all_word_timings = []
        current_time_offset = 0
        
        for scene in audio_data.get("scenes", []):
            audio_spec = scene.get("audio_spec", {})
            word_timings = audio_spec.get("word_timings", [])
            scene_duration = audio_spec.get("duration", 0)
            
            # Adjust timings with current offset
            adjusted_timings = []
            for timing in word_timings:
                adjusted_timing = timing.copy()
                adjusted_timing["start_time"] += current_time_offset
                adjusted_timing["end_time"] += current_time_offset
                adjusted_timings.append(adjusted_timing)
            
            all_word_timings.extend(adjusted_timings)
            current_time_offset += scene_duration
        
        # Group words into sentences for better subtitle chunking
        sentences = []
        current_sentence = []
        sentence_start_time = 0
        
        for i, timing in enumerate(all_word_timings):
            word = timing["word"]
            
            if not current_sentence:
                sentence_start_time = timing["start_time"]
                
            current_sentence.append(word)
            
            # End the sentence if the word ends with punctuation or it's the last word
            if (word.endswith('.') or word.endswith('!') or word.endswith('?') or 
                i == len(all_word_timings) - 1):
                sentence_text = " ".join(current_sentence)
                sentence_end_time = timing["end_time"]
                
                sentences.append({
                    "text": sentence_text,
                    "start_time": sentence_start_time,
                    "end_time": sentence_end_time
                })
                
                current_sentence = []
        
        # Break sentences into subtitle lines (about 42 chars per line max)
        subtitle_lines = []
        MAX_CHARS_PER_LINE = 42
        
        for i, sentence in enumerate(sentences):
            text = sentence["text"]
            words = text.split()
            
            # If sentence is short enough, use it as is
            if len(text) <= MAX_CHARS_PER_LINE:
                subtitle_lines.append({
                    "index": len(subtitle_lines) + 1,
                    "start_time": sentence["start_time"],
                    "end_time": sentence["end_time"],
                    "text": text
                })
            else:
                # Split into multiple lines
                current_line = []
                current_line_length = 0
                line_start_time = sentence["start_time"]
                
                for j, word in enumerate(words):
                    # If adding this word would exceed the max length, finish the line
                    if current_line_length + len(word) + 1 > MAX_CHARS_PER_LINE and current_line:
                        line_text = " ".join(current_line)
                        # Calculate approximate end time based on position in sentence
                        progress = j / len(words)
                        line_end_time = line_start_time + progress * (sentence["end_time"] - sentence["start_time"])
                        
                        subtitle_lines.append({
                            "index": len(subtitle_lines) + 1,
                            "start_time": line_start_time,
                            "end_time": line_end_time,
                            "text": line_text
                        })
                        
                        current_line = [word]
                        current_line_length = len(word)
                        line_start_time = line_end_time
                    else:
                        current_line.append(word)
                        current_line_length += len(word) + 1  # +1 for space
                
                # Add the last line if there's anything left
                if current_line:
                    line_text = " ".join(current_line)
                    subtitle_lines.append({
                        "index": len(subtitle_lines) + 1,
                        "start_time": line_start_time,
                        "end_time": sentence["end_time"],
                        "text": line_text
                    })
        
        # Write the SRT file in proper format
        with open(subtitle_path, 'w', encoding='utf-8') as f:
            for line in subtitle_lines:
                # Convert time to SRT format (HH:MM:SS,mmm)
                start_h = int(line["start_time"] // 3600)
                start_m = int((line["start_time"] % 3600) // 60)
                start_s = int(line["start_time"] % 60)
                start_ms = int((line["start_time"] * 1000) % 1000)
                
                end_h = int(line["end_time"] // 3600)
                end_m = int((line["end_time"] % 3600) // 60)
                end_s = int(line["end_time"] % 60)
                end_ms = int((line["end_time"] * 1000) % 1000)
                
                # Write SRT entry
                f.write(f"{line['index']}\n")
                f.write(f"{start_h:02d}:{start_m:02d}:{start_s:02d},{start_ms:03d} --> ")
                f.write(f"{end_h:02d}:{end_m:02d}:{end_s:02d},{end_ms:03d}\n")
                f.write(f"{line['text']}\n\n")
        
        self.logger.info(f"Generated SRT subtitles: {subtitle_path} ({len(subtitle_lines)} lines)")
        
        return subtitle_path
    
    def _simulate_render_time(self, duration: float):
        """Simulate rendering time for the given duration"""
        # Simple simulation: 10% of real-time duration
        time.sleep(min(5, duration * 0.1))  # Cap at 5 seconds for demo purposes
    
    def add_background_music(self, video_path: str, music_path: str, volume=0.2) -> str:
        """
        Add background music to a video
        
        Args:
            video_path: Path to the video file
            music_path: Path to the music file
            volume: Volume level for the music (0.0-1.0)
            
        Returns:
            Path to the new video file with music
        """
        # Generate output path
        output_path = os.path.splitext(video_path)[0] + "_with_music" + os.path.splitext(video_path)[1]
        
        # Real implementation using FFmpeg to mix audio tracks
        if self.ffmpeg_available and os.path.exists(video_path) and os.path.exists(music_path):
            try:
                # Get video duration
                cmd = [
                    "ffprobe", 
                    "-v", "error", 
                    "-show_entries", "format=duration", 
                    "-of", "default=noprint_wrappers=1:nokey=1", 
                    video_path
                ]
                result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
                video_duration = float(result.stdout.strip())
                
                # Use FFmpeg to mix the audio tracks
                cmd = [
                    "ffmpeg", "-y",
                    "-i", video_path,
                    "-i", music_path,
                    "-filter_complex", f"[1:a]volume={volume},aloop=loop=-1:size=2e+09[a1];[0:a][a1]amix=inputs=2:duration=first[a]",
                    "-map", "0:v",
                    "-map", "[a]",
                    "-c:v", "copy",
                    "-c:a", "aac",
                    "-b:a", self.quality_presets[self.quality]["audio_bitrate"],
                    "-shortest",
                    output_path
                ]
                subprocess.run(cmd, check=True)
                
                self.logger.info(f"Added background music with FFmpeg: {output_path} (music volume: {volume:.1f})")
                
            except (subprocess.SubprocessError, FileNotFoundError) as e:
                self.logger.error(f"FFmpeg background music addition failed: {str(e)}")
                # Fallback to simulation
                time.sleep(2)
                # Copy the original video as fallback
                if os.path.exists(video_path):
                    shutil.copy2(video_path, output_path)
        else:
            # Simulate processing time
            time.sleep(2)
            self.logger.info(f"Simulated adding background music: {output_path} (music volume: {volume:.1f})")
            # Copy the original video as fallback
            if os.path.exists(video_path):
                shutil.copy2(video_path, output_path)
        
        return output_path
    
    def generate_preview(self, video_path: str, preview_duration=30) -> str:
        """
        Generate a short preview of the video
        
        Args:
            video_path: Path to the video file
            preview_duration: Duration of the preview in seconds
            
        Returns:
            Path to the preview video file
        """
        # Generate output path
        output_path = os.path.splitext(video_path)[0] + "_preview" + os.path.splitext(video_path)[1]
        
        # Real implementation using FFmpeg to extract segment and add watermark
        if self.ffmpeg_available and os.path.exists(video_path):
            try:
                # Create a temporary watermark image with "PREVIEW" text
                watermark_path = os.path.join(tempfile.gettempdir(), "preview_watermark.png")
                
                # Extract segment and add watermark
                cmd = [
                    "ffmpeg", "-y",
                    "-i", video_path,
                    "-t", str(preview_duration),
                    "-vf", f"drawtext=text='PREVIEW':fontcolor=white:fontsize=36:x=(w-text_w)/2:y=h-th-10:box=1:boxcolor=black@0.5:boxborderw=5",
                    "-c:v", "libx264",
                    "-preset", self.quality_presets[self.quality]["preset"],
                    "-c:a", "aac",
                    output_path
                ]
                subprocess.run(cmd, check=True)
                
                self.logger.info(f"Generated video preview with FFmpeg: {output_path} ({preview_duration} seconds)")
                
            except subprocess.SubprocessError as e:
                self.logger.error(f"FFmpeg preview generation failed: {str(e)}")
                # Fallback to simulation
                time.sleep(1)
                # Copy a portion of the original video as fallback
                if os.path.exists(video_path):
                    try:
                        cmd = [
                            "ffmpeg", "-y",
                            "-i", video_path,
                            "-t", str(preview_duration),
                            "-c", "copy",
                            output_path
                        ]
                        subprocess.run(cmd, check=True)
                    except:
                        # Last resort: just copy the file
                        shutil.copy2(video_path, output_path)
        else:
            # Simulate processing time
            time.sleep(1)
            self.logger.info(f"Simulated video preview generation: {output_path} ({preview_duration} seconds)")
            # Copy the original video as fallback if possible
            if os.path.exists(video_path):
                shutil.copy2(video_path, output_path)
        
        return output_path
    
    def set_output_format(self, format="mp4", resolution=(1920, 1080), fps=30, quality="high"):
        """
        Set output format and quality options
        
        Args:
            format: Video format (mp4, webm, mov)
            resolution: Video resolution as (width, height) tuple
            fps: Frames per second
            quality: Output quality (low, medium, high)
        """
        self.output_format = format
        self.resolution = resolution
        self.fps = fps
        self.quality = quality
        
        self.logger.info(f"Output format set: {format}, {resolution[0]}x{resolution[1]}, {fps} fps, {quality} quality")
    
    def export_multiple_formats(self, video_path: str, formats=None) -> List[str]:
        """
        Export the video to multiple formats
        
        Args:
            video_path: Path to the original video file
            formats: List of format specifications
            
        Returns:
            List of paths to the exported video files
        """
        if formats is None:
            formats = [
                {"format": "mp4", "resolution": (1920, 1080), "quality": "high"},
                {"format": "mp4", "resolution": (1280, 720), "quality": "medium"},
                {"format": "webm", "resolution": (1280, 720), "quality": "medium"}
            ]
        
        output_paths = []
        
        # Real implementation using FFmpeg for transcoding
        if self.ffmpeg_available and os.path.exists(video_path):
            for format_spec in formats:
                # Generate output path
                base_name = os.path.splitext(video_path)[0]
                resolution = format_spec.get("resolution", (1920, 1080))
                quality = format_spec.get("quality", "medium")
                format_ext = format_spec.get("format", "mp4")
                quality_preset = self.quality_presets.get(quality, self.quality_presets["medium"])
                
                output_name = f"{base_name}_{resolution[0]}x{resolution[1]}_{quality}.{format_ext}"
                
                try:
                    # Set the video codec based on format
                    if format_ext == "webm":
                        vcodec = "libvpx-vp9"
                        acodec = "libopus"
                    elif format_ext == "mp4":
                        vcodec = "libx264"
                        acodec = "aac"
                    else:
                        vcodec = "libx264"
                        acodec = "aac"
                    
                    # Transcode with FFmpeg
                    cmd = [
                        "ffmpeg", "-y",
                        "-i", video_path,
                        "-c:v", vcodec,
                        "-b:v", quality_preset["bitrate"],
                        "-c:a", acodec,
                        "-b:a", quality_preset["audio_bitrate"],
                        "-s", f"{resolution[0]}x{resolution[1]}",
                        output_name
                    ]
                    
                    subprocess.run(cmd, check=True)
                    output_paths.append(output_name)
                    self.logger.info(f"Exported video format with FFmpeg: {output_name}")
                    
                except subprocess.SubprocessError as e:
                    self.logger.error(f"FFmpeg format conversion failed for {output_name}: {str(e)}")
                    # Skip this format on error
        else:
            # Fallback to simulation if FFmpeg not available
            for format_spec in formats:
                # Generate output path
                base_name = os.path.splitext(video_path)[0]
                resolution = format_spec.get("resolution", (1920, 1080))
                quality = format_spec.get("quality", "medium")
                format_ext = format_spec.get("format", "mp4")
                
                output_name = f"{base_name}_{resolution[0]}x{resolution[1]}_{quality}.{format_ext}"
                
                # Simulate processing time
                time.sleep(1)
                
                # Try a simple copy if the original file exists
                if os.path.exists(video_path):
                    try:
                        shutil.copy2(video_path, output_name)
                        output_paths.append(output_name)
                        self.logger.info(f"Simulated video format export: {output_name}")
                    except:
                        pass
        
        return output_paths 

---


## EnhancedPDFExtractor.py

In [ ]:
"""
Enhanced PDF Content Extractor with Inheritance
----------------------------------------------
Extends the original PDFContentExtractor with improved:
- Text cleaning and noise removal
- Better heading detection
- Content organization
- JSON and TXT output formats
"""
import os
import re
from typing import Dict, List
from PDFContentExtractor import PDFContentExtractor
import logging
import unicodedata
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import tempfile
import io
import pytesseract

class EnhancedPDFExtractor(PDFContentExtractor):
    def __init__(self, tesseract_path=None, math_recognition=True, chem_recognition=True):
        """
        Initialize the enhanced PDF content extractor, inheriting from PDFContentExtractor
        
        Args:
            tesseract_path: Path to Tesseract OCR executable
            math_recognition: Boolean to enable mathematical expression recognition
            chem_recognition: Boolean to enable chemical formula recognition
        """
        # Initialize the parent class
        super().__init__(tesseract_path, math_recognition, chem_recognition)
        
        # Configure logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)
        
        # Additional regex patterns for cleaning
        self.noise_patterns = [
            re.compile(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]'),  # Control characters
            re.compile(r'^\s*$'),  # Empty lines
            re.compile(r'^\s*[.â€¢â– â–¡â—‹â—â—†â—‡â˜…â˜†â™¦â™ â™£â™¥â†’â†â†‘â†“â‡’â‡â‡‘â‡“â‡”â‡•â‡–â‡—â‡˜â‡™]+\s*$'),  # Lines with only bullets/symbols
        ]
        
        # Potential headings detection
        self.heading_patterns = [
            re.compile(r'^[0-9]+\.[0-9.]*\s+[A-Z]'),  # Numbered headings (1.2 Title)
            re.compile(r'^[A-Z][a-z]*\s+[0-9]+\.\s'),  # Chapter/Section headings (Chapter 1.)
            re.compile(r'^[A-Z][A-Z\s]+$'),  # ALL CAPS headings
            re.compile(r'^[A-Z][a-z]+(\s+[A-Z][a-z]+){0,5}$')  # Title Case short lines
        ]
        
        # For filtering out irrelevant short fragments
        self.min_content_length = 10  # Minimum characters for content blocks
        self.min_words_for_content = 3  # Minimum words for content to be considered relevant
        
        # Font analysis parameters
        self.font_heading_threshold = 1.2  # Multiplier above avg font size to be considered heading
        self.font_title_threshold = 1.5   # Multiplier above avg font size to be considered title
        self.font_weight_threshold = 600  # Minimum font weight to be considered bold
    
    def _clean_text_internal(self, text):
        """Internal helper for text cleaning operations"""
        # Normalize unicode characters
        text = unicodedata.normalize('NFKC', text)

        # Replace common problematic characters
        text = text.replace('\u2028', '\n').replace('\u2029', '\n')  # Line/paragraph separators

        # Remove control characters and other noise
        for pattern in self.noise_patterns:
            text = pattern.sub('', text)

        # Fix spacing issues
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
        text = re.sub(r'\n\s*\n+', '\n\n', text)  # Normalize multiple newlines

        # Normalize punctuation spacing
        text = re.sub(r'\s+([.,;:!?)])', r'\1', text)
        text = re.sub(r'([([{])\s+', r'\1', text)

        return text.strip()
    
    def _detect_heading_properties(self, text, font_size=None, avg_font_size=None, is_bold=False, font_stats=None):
        """
        Helper method to determine heading properties for a text block
        
        Args:
            text: The text to analyze
            font_size: Font size of this text (if available)
            avg_font_size: Average font size in the document (if available)
            is_bold: Whether the text is in bold font (if available)
            font_stats: Full font statistics from analyze_document_fonts (if available)
            
        Returns:
            Tuple of (is_heading, heading_level)
        """
        # Short text sanity check
        text = text.strip()
        if not text:
            return False, None
            
        if len(text) > 150:  # Headings shouldn't be too long
            return False, None
        
        # Initialize result and heading level
        is_heading = False
        heading_level = None
        
        # Font size check (if available)
        if font_size and avg_font_size:
            size_ratio = font_size / avg_font_size
            
            # Title/H1
            if size_ratio >= self.font_title_threshold:
                is_heading = True
                heading_level = 1
            # Subtitle/H2
            elif size_ratio >= self.font_heading_threshold:
                is_heading = True
                heading_level = 2
            # Potential section heading if bold and above average size
            elif is_bold and size_ratio >= 1.0:
                is_heading = True
                heading_level = 3
                
        # If we haven't determined it's a heading by font, check patterns
        if not is_heading:
            # Pattern matching for heading-like text
            for pattern in self.heading_patterns:
                if pattern.match(text):
                    is_heading = True
                    # Default to level 3 for pattern-matched headings 
                    # unless we have better information
                    heading_level = heading_level or 3
                    break
                    
            # Additional heuristics for headings
            if not is_heading:
                # Label-like headings ending with colon
                if text.endswith(':') and len(text.split()) < 7:
                    is_heading = True
                    heading_level = heading_level or 4
                
                # Title Case with 2-6 words
                elif len(text) < 50 and text[0].isupper() and not text.endswith('.'):
                    words = text.split()
                    if 2 <= len(words) <= 6 and all(w[0].isupper() for w in words if w[0].isalpha()):
                        is_heading = True
                        heading_level = heading_level or 4
        
        # Check if this text matches any heading candidates in font_stats
        if font_stats and "heading_candidates" in font_stats:
            for candidate in font_stats["heading_candidates"]:
                if text == candidate["text"]:
                    is_heading = True
                    # Use the level from font analysis if available
                    heading_level = candidate["heading_level"]
                    break
                    
        return is_heading, heading_level
    
    def _organize_content_hierarchy(self, blocks, respect_heading_levels=True):
        """
        Helper method to organize content blocks into a hierarchical structure

        Args:
            blocks: List of content blocks with type and text
            respect_heading_levels: Whether to create a hierarchical structure

        Returns:
            Structured content organized by headings
        """
        if not blocks:
            return []

        if not respect_heading_levels:
            # Simple flat structure
            current_heading = None
            structured_content = []

            for block in blocks:
                if block.get("type") == "heading":
                    current_heading = {
                        "heading": block["text"],
                        "content": [],
                        "font_size": block.get("font_size"),
                        "position": block.get("position"),
                        "heading_level": block.get("heading_level", 3)
                    }
                    structured_content.append(current_heading)
                elif current_heading is not None:
                    current_heading["content"].append(block["text"])
                else:
                    # Content without a heading
                    structured_content.append({
                        "heading": None,
                        "content": [block["text"]],
                        "position": block.get("position")
                    })

            return structured_content
        else:
            # Create hierarchical structure based on heading levels
            content_tree = {
                "heading": "Document",
                "heading_level": 0,
                "content": [],
                "children": []
            }

            # Current section pointers at each level
            current_sections = {0: content_tree}

            for block in blocks:
                if block.get("type") == "heading":
                    level = block.get("heading_level", 3)

                    # Create new section
                    new_section = {
                        "heading": block["text"],
                        "heading_level": level,
                        "content": [],
                        "children": [],
                        "font_size": block.get("font_size"),
                        "position": block.get("position")
                    }

                    # Find parent section (closest section with lower level)
                    parent_level = max([l for l in current_sections.keys() if l < level], default=0)
                    parent = current_sections[parent_level]

                    # Add to parent's children
                    parent["children"].append(new_section)

                    # Update current section for this level
                    current_sections[level] = new_section

                    # Clear any deeper levels
                    for deeper_level in list(current_sections.keys()):
                        if deeper_level > level:
                            del current_sections[deeper_level]

                else:
                    # Regular content - add to deepest current section
                    deepest_level = max(current_sections.keys())
                    current_sections[deepest_level]["content"].append(block["text"])

            # Function to flatten the tree for simple output
            def flatten_tree(node, sections=None):
                if sections is None:
                    sections = []

                if node["heading"] != "Document" or node["content"]:
                    sections.append({
                        "heading": node["heading"] if node["heading"] != "Document" else None,
                        "content": node["content"],
                        "heading_level": node["heading_level"],
                        "font_size": node.get("font_size"),
                        "position": node.get("position")
                    })

                for child in node["children"]:
                    flatten_tree(child, sections)

                return sections

            return flatten_tree(content_tree)
        
    def clean_text(self, text):
        """Clean text by removing noise and normalizing content"""
        return self._clean_text_internal(text)
    
    def analyze_document_fonts(self, pdf_doc):
        """
        Analyze fonts across the document to identify common sizes and styles
        
        Args:
            pdf_doc: The fitz PDF document object
            
        Returns:
            Dictionary with font statistics and hierarchical font information
        """
        font_sizes = []
        font_weights = []
        font_details = []
        
        for page_idx, page in enumerate(pdf_doc):
            # Get detailed text information including fonts
            text_dict = page.get_text("dict", flags=11)
            
            for block in text_dict.get("blocks", []):
                if "lines" not in block:
                    continue
                    
                for line in block.get("lines", []):
                    for span in line.get("spans", []):
                        text = span.get("text", "").strip()
                        if not text:
                            continue
                            
                        # Extract font information
                        font_name = span.get("font", "")
                        font_size = span.get("size", 0)
                        flags = span.get("flags", 0)
                        
                        # Determine if it's bold (bit 1 in flags is often used for bold)
                        is_bold = bool(flags & 1)
                        # Determine if it's italic (bit 2 in flags is often used for italic)
                        is_italic = bool(flags & 2)
                        
                        # Estimate font weight (400=normal, 700=bold)
                        font_weight = 700 if is_bold else 400
                        
                        # Store details
                        if font_size > 0:
                            font_sizes.append(font_size)
                            font_weights.append(font_weight)
                            
                            font_details.append({
                                "text": text,
                                "font_name": font_name,
                                "font_size": font_size,
                                "is_bold": is_bold,
                                "is_italic": is_italic,
                                "font_weight": font_weight,
                                "page": page_idx + 1,
                                "bbox": [span.get("bbox", [0, 0, 0, 0])]
                            })
        
        # Compute statistics
        if not font_sizes:
            return {
                "min_size": 0, 
                "max_size": 0, 
                "avg_size": 0, 
                "common_size": 0,
                "font_histogram": {},
                "heading_candidates": []
            }
        
        # Create a histogram of font sizes rounded to nearest 0.5
        font_counts = defaultdict(int)
        for size in font_sizes:
            rounded_size = round(size * 2) / 2
            font_counts[rounded_size] += 1
            
        # Sort by frequency to get most common sizes
        sorted_sizes = sorted(font_counts.items(), key=lambda x: x[1], reverse=True)
        common_size = sorted_sizes[0][0] if sorted_sizes else 0
        
        # Identify potential heading sizes (significantly larger than common size)
        avg_size = sum(font_sizes) / len(font_sizes)
        heading_sizes = [size for size in set(font_sizes) if size >= avg_size * self.font_heading_threshold]
        heading_sizes.sort(reverse=True)  # Sort from largest to smallest
        
        # Identify text that matches heading criteria
        heading_candidates = []
        for detail in font_details:
            # Consider it a heading candidate if:
            # 1. Font size is in heading_sizes list OR
            # 2. Font is bold and size is at least average
            if (detail["font_size"] in heading_sizes or 
                (detail["is_bold"] and detail["font_size"] >= avg_size)):
                
                # Classify heading level based on size
                if detail["font_size"] >= avg_size * self.font_title_threshold:
                    level = 1  # Title/h1
                elif detail["font_size"] >= avg_size * self.font_heading_threshold:
                    level = 2  # Subtitle/h2
                elif detail["is_bold"] and detail["font_size"] >= avg_size:
                    level = 3  # Section/h3
                else:
                    level = 4  # Subsection/h4
                
                heading_candidates.append({
                    "text": detail["text"],
                    "font_size": detail["font_size"],
                    "is_bold": detail["is_bold"],
                    "heading_level": level,
                    "page": detail["page"],
                    "bbox": detail["bbox"]
                })
        
        return {
            "min_size": min(font_sizes),
            "max_size": max(font_sizes),
            "avg_size": avg_size,
            "common_size": common_size,
            "font_histogram": dict(sorted_sizes),
            "heading_candidates": heading_candidates
        }
    
    def is_likely_heading(self, text, font_size=None, avg_font_size=None, is_bold=False, font_stats=None):
        """
        Determine if a text block is likely a heading based on content, font size and style

        Args:
            text: The text to analyze
            font_size: Font size of this text (if available)
            avg_font_size: Average font size in the document (if available)
            is_bold: Whether the text is in bold font (if available)
            font_stats: Full font statistics from analyze_document_fonts (if available)

        Returns:
            Boolean indicating if the text is likely a heading and estimated heading level
        """
        return self._detect_heading_properties(text, font_size, avg_font_size, is_bold, font_stats)

    def is_meaningful_content(self, text, is_heading_candidate=False):
        """Determine if text is meaningful content versus noise"""
        text = text.strip()
        if not text:
            return False

        # Special exception for heading candidates
        if is_heading_candidate:
            # For headings, we just need non-empty text, but still check for problematic characters
            non_ascii = sum(1 for c in text if ord(c) > 127)
            if non_ascii > len(text) * 0.3:  # More than 30% non-ASCII
                return False
            return True

        # For regular content, apply stricter rules
        # Check for minimum length
        if len(text) < self.min_content_length:
            return False

        # Check for minimum words
        words = [w for w in text.split() if w.strip(string.punctuation)]
        if len(words) < self.min_words_for_content:
            return False

        # Check for high proportion of problematic characters
        non_ascii = sum(1 for c in text if ord(c) > 127)
        if non_ascii > len(text) * 0.3:  # More than 30% non-ASCII
            return False

        return True
        
    def _group_content_by_headings(self, page_content, respect_heading_levels=True):
        """
        Group content items under their respective headings

        Args:
            page_content: Dictionary containing blocks of content
            respect_heading_levels: Whether to create a hierarchical structure based on heading levels

        Returns:
            List of content sections organized by headings
        """
        if not page_content.get("blocks"):
            return []

        return self._organize_content_hierarchy(page_content["blocks"], respect_heading_levels)
            
    def extract_content(self, pdf_path, output_dir=None, extract_images=True, extract_tables=True, 
                       extract_equations=True, ocr_quality="high", ocr_language="eng",
                       process_scanned=True, deskew_pages=True, denoise_images=True, 
                       is_scanned=None, contrast_enhancement=False, multi_language_detection=False, **kwargs):
        """
        Override of the parent extract_content method to handle additional parameters
        
        This method accepts all parameters that WorkflowCoordinator passes and forwards them
        to the appropriate extraction method.
        
        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to save extracted images (optional)
            extract_images: Boolean indicating whether to extract images
            extract_tables: Boolean indicating whether to extract tables
            extract_equations: Boolean indicating whether to extract equations (math content)
            ocr_quality: Quality level for OCR ("low", "high", "highest")
            ocr_language: Language code for OCR
            process_scanned: Whether to apply special processing for scanned documents
            deskew_pages: Whether to deskew pages in scanned documents
            denoise_images: Whether to denoise images in scanned documents
            is_scanned: Whether the document is known to be scanned
            contrast_enhancement: Whether to enhance contrast in scanned documents
            multi_language_detection: Whether to detect multiple languages in the document
            **kwargs: Additional keyword arguments
            
        Returns:
            Dictionary containing all extracted content
        """
        # If is_scanned wasn't explicitly provided, detect it
        if is_scanned is None:
            is_scanned = self.is_scanned_document(pdf_path)
        
        # Enable or disable math recognition based on extract_equations parameter
        self.math_recognition = extract_equations
        
        # Configure OCR settings based on quality parameter
        ocr_config = {}
        if ocr_quality == "highest":
            ocr_config = {
                "oem": 1,  # LSTM only
                "psm": 6,  # Assume a single uniform block of text
                "lang": ocr_language
            }
        elif ocr_quality == "high":
            ocr_config = {
                "oem": 3,  # Default, based on what is available
                "psm": 6,  # Assume a single uniform block of text
                "lang": ocr_language
            }
        else:  # low
            ocr_config = {
                "oem": 0,  # Legacy engine only
                "psm": 6,  # Assume a single uniform block of text
                "lang": ocr_language
            }
            
        # Handle multi-language detection
        if multi_language_detection:
            ocr_config["lang"] = "+".join(["eng", "deu", "fra", "spa", "ita"])
        
        # Call the enhanced extraction method with all relevant parameters
        return self.extract_content_with_cleaning(
            pdf_path=pdf_path,
            output_dir=output_dir,
            extract_images=extract_images,
            extract_tables=extract_tables,
            respect_heading_hierarchy=True,
            process_scanned=process_scanned and is_scanned,
            deskew_pages=deskew_pages and is_scanned,
            denoise_images=denoise_images and is_scanned,
            contrast_enhancement=contrast_enhancement and is_scanned,
            ocr_config=ocr_config
        )

    def extract_content_with_cleaning(self, pdf_path, output_dir=None, extract_images=True, extract_tables=True, 
                                     respect_heading_hierarchy=True, process_scanned=False, deskew_pages=False,
                                     denoise_images=False, contrast_enhancement=False, ocr_config=None):
        """
        Enhanced extraction with text cleaning and better structure detection

        This method extends the parent's extract_content method with additional processing:
        - Cleans text to remove noise and unidentified symbols
        - Better detects headings and their relationship to content using font analysis
        - Filters out irrelevant short text fragments
        - Creates hierarchical structure based on heading levels
        - Handles scanned documents with improved OCR and image processing

        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to save extracted images (optional)
            extract_images: Boolean indicating whether to extract images
            extract_tables: Boolean indicating whether to extract tables
            respect_heading_hierarchy: Whether to create hierarchical structure based on heading levels
            process_scanned: Whether to apply special processing for scanned documents
            deskew_pages: Whether to deskew pages in scanned documents
            denoise_images: Whether to denoise images in scanned documents
            contrast_enhancement: Whether to enhance contrast in scanned documents
            ocr_config: Dictionary with OCR configuration parameters

        Returns:
            Dictionary containing all extracted and cleaned content, formatted for analyze_content
        """
        # Process scanned documents with special handling if needed
        if process_scanned:
            # Create a temporary processed PDF with improved quality for extraction
            processed_pdf_path = self._preprocess_scanned_document(
                pdf_path, 
                deskew=deskew_pages,
                denoise=denoise_images,
                enhance_contrast=contrast_enhancement,
                ocr_config=ocr_config
            )
            
            # Use the processed PDF for extraction
            extraction_path = processed_pdf_path
        else:
            extraction_path = pdf_path

        # First use the parent class to extract the raw content
        raw_result = super().extract_content(extraction_path, output_dir, extract_images, extract_tables)

        # Clean up temporary processed PDF if created
        if process_scanned and extraction_path != pdf_path:
            try:
                os.remove(extraction_path)
            except:
                pass  # Ignore errors in cleanup

        # Handle extraction errors
        if "error" in raw_result:
            return raw_result

        # Now enhance and clean the extracted content
        enhanced_result = raw_result.copy()

        try:
            # Reopen the PDF for font analysis
            pdf_doc = fitz.open(pdf_path)

            # Perform detailed font analysis
            self.logger.info("Performing detailed font analysis...")
            font_stats = self.analyze_document_fonts(pdf_doc)
            avg_font_size = font_stats["avg_size"]

            # Add font statistics to the result
            enhanced_result["font_statistics"] = {
                "avg_size": font_stats["avg_size"],
                "common_size": font_stats["common_size"],
                "min_size": font_stats["min_size"],
                "max_size": font_stats["max_size"],
                "size_distribution": font_stats["font_histogram"]
            }

            # Add heading candidates from font analysis
            enhanced_result["detected_headings"] = font_stats["heading_candidates"]

            # Process and clean each page
            enhanced_pages = []
            all_text = []

            # Create content_blocks list that analyze_content expects
            content_blocks = []

            # Create sections dictionary that analyze_content expects
            sections = {}
            current_section_id = None
            current_section_content = []

            for page in raw_result["pages"]:
                page_number = page["page_number"]
                enhanced_page = {
                    "page_number": page_number,
                    "blocks": []
                }

                # Process each text block
                for block in page["blocks"]:
                    # Clean the text
                    cleaned_text = self.clean_text(block["text"])

                    # Extract font information if available
                    font_size = None
                    is_bold = False

                    if "position" in block:
                        # Try to match this block with font information from our analysis
                        block_matches = []
                        block_pos = block["position"]

                        # Look for heading candidates that might match this block
                        for candidate in font_stats["heading_candidates"]:
                            if cleaned_text in candidate["text"] or candidate["text"] in cleaned_text:
                                block_matches.append(candidate)

                        if block_matches:
                            # Use font information from the best match
                            best_match = block_matches[0]  # Simple approach: just take first match
                            font_size = best_match["font_size"]
                            is_bold = best_match["is_bold"]
                        else:
                            # Fallback: estimate font size from position data
                            height = block_pos["y1"] - block_pos["y0"]
                            # Rough heuristic: font size ~ 1/2 of block height for single line
                            font_size = height / 2

                    # First check if this might be a heading
                    is_heading_candidate = self.is_likely_heading(
                        cleaned_text, 
                        font_size, 
                        avg_font_size,
                        is_bold,
                        font_stats
                    )[0]  # Just get the boolean, ignore the heading level for now

                    # Check if it's meaningful content, with special handling for headings
                    if not self.is_meaningful_content(cleaned_text, is_heading_candidate):
                        continue
                    
                    # Now get the full heading info including level
                    is_heading, heading_level = self.is_likely_heading(
                        cleaned_text, 
                        font_size, 
                        avg_font_size,
                        is_bold,
                        font_stats
                    )

                    # Create enhanced block
                    enhanced_block = {
                        "text": cleaned_text,
                        "type": "heading" if is_heading else "content",
                        "position": block.get("position"),
                        "font_size": font_size,
                        "is_bold": is_bold
                    }
    

                    # Add heading level if applicable
                    if is_heading and heading_level:
                        enhanced_block["heading_level"] = heading_level

                    # Keep any special content (math/chemical)
                    if "math_expressions" in block:
                        enhanced_block["math_expressions"] = block["math_expressions"]
                    if "chemical_content" in block:
                        enhanced_block["chemical_content"] = block["chemical_content"]

                    # Add to content_blocks (needed by analyze_content)
                    content_block = {
                        "text": cleaned_text,
                        "type": enhanced_block["type"],
                        "page": page_number,
                        "position": block.get("position")
                    }

                    if is_heading:
                        content_block["heading_level"] = heading_level

                        # Start a new section when we encounter a heading
                        if current_section_id:
                            # Save the previous section
                            sections[current_section_id] = "\n".join(current_section_content)

                        # Create new section ID (simple approach: use heading text)
                        current_section_id = f"section_{len(sections) + 1}_{cleaned_text[:20].replace(' ', '_')}"
                        current_section_content = [cleaned_text]
                    else:
                        # Add content to current section if there is one
                        if current_section_id:
                            current_section_content.append(cleaned_text)

                    # Add special content to content_block if available
                    if "math_expressions" in enhanced_block:
                        content_block["math_expressions"] = enhanced_block["math_expressions"]
                    if "chemical_content" in enhanced_block:
                        content_block["chemical_content"] = enhanced_block["chemical_content"]

                    content_blocks.append(content_block)
                    enhanced_page["blocks"].append(enhanced_block)
                    all_text.append(cleaned_text)

                # Group content by headings
                enhanced_page["structured_content"] = self._group_content_by_headings(
                    enhanced_page, 
                    respect_heading_levels=respect_heading_hierarchy
                )
                enhanced_pages.append(enhanced_page)

            # Save last section if it exists
            if current_section_id and current_section_content:
                sections[current_section_id] = "\n".join(current_section_content)

            # Update the result with enhanced pages
            enhanced_result["pages"] = enhanced_pages
            enhanced_result["text_content"] = "\n\n".join(all_text)

            # Add the expected keys for analyze_content
            enhanced_result["content_blocks"] = content_blocks
            enhanced_result["sections"] = sections

            # Close the document
            pdf_doc.close()

            return enhanced_result

        except Exception as e:
            self.logger.error(f"Error in enhanced extraction: {e}")
            import traceback
            traceback.print_exc()
            # Fall back to the original result if enhancement fails
            return raw_result
    
    def _preprocess_scanned_document(self, pdf_path, deskew=True, denoise=True, enhance_contrast=False, ocr_config=None):
        """
        Preprocess a scanned PDF document to improve extraction quality
        
        Args:
            pdf_path: Path to the PDF file
            deskew: Whether to deskew pages
            denoise: Whether to denoise images
            enhance_contrast: Whether to enhance contrast
            ocr_config: OCR configuration parameters
            
        Returns:
            Path to the processed PDF file
        """
        import cv2
        import numpy as np
        from PIL import Image, ImageEnhance
        import tempfile
        import os
        
        # Create temporary file for the processed PDF
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.pdf')
        output_path = temp_file.name
        temp_file.close()
        
        # Open the input PDF
        doc = fitz.open(pdf_path)
        output_doc = fitz.open()
        
        for page_idx, page in enumerate(doc):
            # Get the page as an image
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            img_np = np.array(img)
            
            # Apply image preprocessing if needed
            if deskew:
                # Convert to grayscale for deskewing
                gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
                
                # Apply threshold to get binary image
                _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                
                # Find all contours
                contours, _ = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                
                # Find the largest contour - assume it's the text block
                if contours:
                    areas = [cv2.contourArea(c) for c in contours]
                    max_index = np.argmax(areas)
                    cnt = contours[max_index]
                    
                    # Find minimum area rectangle
                    rect = cv2.minAreaRect(cnt)
                    angle = rect[2]
                    
                    # Adjust angle - assumes text is horizontal or 90 degrees rotated
                    if angle < -45:
                        angle = 90 + angle
                    
                    # Only deskew if angle is significant
                    if abs(angle) > 0.5:
                        (h, w) = img_np.shape[:2]
                        center = (w // 2, h // 2)
                        M = cv2.getRotationMatrix2D(center, angle, 1.0)
                        img_np = cv2.warpAffine(img_np, M, (w, h), flags=cv2.INTER_CUBIC, 
                                              borderMode=cv2.BORDER_REPLICATE)
            
            # Apply denoising if needed
            if denoise:
                # Use Non-local Means Denoising algorithm
                img_np = cv2.fastNlMeansDenoisingColored(img_np, None, 10, 10, 7, 21)
            
            # Enhance contrast if needed
            if enhance_contrast:
                # Convert back to PIL for easier contrast enhancement
                img = Image.fromarray(img_np)
                enhancer = ImageEnhance.Contrast(img)
                img = enhancer.enhance(1.5)  # Increase contrast by 50%
                img_np = np.array(img)
            
            # Convert back to PIL image
            img = Image.fromarray(img_np)
            
            # Apply OCR if config provided
            if ocr_config:
                text = pytesseract.image_to_string(
                    img, 
                    config=f"-l {ocr_config.get('lang', 'eng')} --oem {ocr_config.get('oem', 3)} --psm {ocr_config.get('psm', 6)}"
                )
                
                # Create a new PDF page
                new_page = output_doc.new_page(width=pix.width, height=pix.height)
                
                # Add the image
                img_bytes = io.BytesIO()
                img.save(img_bytes, format="PNG")
                new_page.insert_image(rect=(0, 0, pix.width, pix.height), stream=img_bytes.getvalue())
                
                # Add the OCR text as invisible text for searchability
                if text:
                    new_page.insert_text((0, 0), text, fontsize=12, color=(0,0,0,0))  # Transparent text
            else:
                # Just add the processed image
                new_page = output_doc.new_page(width=pix.width, height=pix.height)
                img_bytes = io.BytesIO()
                img.save(img_bytes, format="PNG")
                new_page.insert_image(rect=(0, 0, pix.width, pix.height), stream=img_bytes.getvalue())
        
        # Save the processed document
        output_doc.save(output_path)
        output_doc.close()
        doc.close()
        
        return output_path


---


## PDFContentExtractor.py

In [ ]:
"""
Advanced PDF Content Extractor
------------------------------
Extracts content from PDFs with support for:
- Structured document understanding
- Mathematical expressions and equations
- Chemical formulas and reactions
- Tables and figures
- Document metadata
"""

import os
import re
import json
import numpy as np
import fitz  # PyMuPDF
import cv2
from PIL import Image
import io
import pytesseract
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTFigure, LTImage, LTTextBoxHorizontal
import camelot  # For table extraction
from sympy.parsing.latex import parse_latex
import logging
from abc import ABC, abstractmethod
from typing import Dict, List, Any, Optional, Tuple
from enum import Enum

from pipeline.errors.WorkflowErrors import (
    PDFExtractionError, PDFParsingError, OCRError, ContentExtractionError
)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class PDFQuality(Enum):
    """Represents the quality/complexity of a PDF document"""
    SIMPLE = "simple"       # Simple text-based PDF with basic formatting
    STANDARD = "standard"   # Standard PDF with some formatting and basic images
    COMPLEX = "complex"     # Complex PDF with advanced formatting, tables, etc.
    SCANNED = "scanned"     # Primarily scanned content requiring OCR
    PROBLEMATIC = "problematic"  # Problematic PDF with encryption or other issues

class PDFContentExtractor(ABC):
    """
    Base class for PDF content extraction
    Provides common functionality and defines the interface for extractors
    """
    
    def __init__(self, tesseract_path: Optional[str] = None, math_recognition: bool = False,
                chem_recognition: bool = False):
        """
        Initialize the PDF extractor
        
        Args:
            tesseract_path: Path to Tesseract OCR executable
            math_recognition: Whether to enable mathematical expression recognition
            chem_recognition: Whether to enable chemical formula recognition
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.tesseract_path = tesseract_path
        self.math_recognition = math_recognition
        self.chem_recognition = chem_recognition
        
        # Extraction statistics for adaptive processing
        self.extraction_stats = {
            "success_count": 0,
            "failure_count": 0,
            "ocr_count": 0,
            "average_extraction_time": 0,
            "total_extractions": 0
        }
        
        # Configure fallback strategies 
        self.fallback_strategies = [
            self._standard_extraction,
            self._ocr_based_extraction,
            self._hybrid_extraction,
            self._minimal_extraction
        ]
        
        # Initialize OCR configurations for different document qualities
        self._init_ocr_configurations()
            
        # Patterns for identifying mathematical content
        self.math_patterns = {
            'latex_delimiters': re.compile(r'\$(.*?)\$|\\\[(.*?)\\\]|\\\((.*?)\\\)'),
            'subscripts': re.compile(r'[a-zA-Z0-9]_\{[^}]+\}|[a-zA-Z0-9]_[0-9]'),
            'fractions': re.compile(r'\\frac\{[^}]+\}\{[^}]+\}'),
            'integrals': re.compile(r'\\int'),
            'summations': re.compile(r'\\sum'),
            'square_roots': re.compile(r'\\sqrt')
        }
        
        # Patterns for identifying chemical content
        self.chem_patterns = {
            'chemical_formulas': re.compile(r'[A-Z][a-z]?[0-9]*'),
            'reaction_arrows': re.compile(r'->|â†’|âŸ¶|â‡Œ|â‡„|â‡†|â‡‹|â†”'),
            'subscripts': re.compile(r'_\{[^}]+\}|_[0-9]'),
            'superscripts': re.compile(r'\^\{[^}]+\}|\^[0-9\+\-]')
        }
        
    def _init_ocr_configurations(self) -> None:
        """Initialize OCR configurations for different document qualities"""
        # Base OCR configuration
        self.ocr_configs = {
            PDFQuality.SIMPLE: {
                "dpi": 200,
                "psm": 3,  # Fully automatic page segmentation
                "oem": 3,  # Default OCR engine mode
                "preprocess": False,
            },
            PDFQuality.STANDARD: {
                "dpi": 300,
                "psm": 3,
                "oem": 3,
                "preprocess": True,
            },
            PDFQuality.COMPLEX: {
                "dpi": 300,
                "psm": 11,  # Sparse text with OSD
                "oem": 2,  # Legacy + LSTM
                "preprocess": True,
            },
            PDFQuality.SCANNED: {
                "dpi": 400,
                "psm": 3,
                "oem": 2,
                "preprocess": True,
                "denoise": True,
                "deskew": True,
                "contrast_enhance": True,
            },
            PDFQuality.PROBLEMATIC: {
                "dpi": 600,
                "psm": 6,  # Assume a single uniform block of text
                "oem": 0,  # Legacy engine only
                "preprocess": True,
                "denoise": True,
                "deskew": True,
                "contrast_enhance": True,
                "force_ocr": True  # Force OCR even for text-based PDFs
            }
        }
    
    def extract_content(self, pdf_path: str, output_dir: str, 
                      extract_images: bool = True, extract_tables: bool = True,
                      **kwargs) -> Dict[str, Any]:
        """
        Extract content from a PDF file with progressive fallback strategies
        
        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to store extracted content
            extract_images: Whether to extract images
            extract_tables: Whether to extract tables
            **kwargs: Additional extraction options
            
        Returns:
            Dictionary containing extracted content
            
        Raises:
            PDFExtractionError: If extraction fails with all strategies
        """
        # Validate inputs
        if not os.path.exists(pdf_path):
            raise PDFExtractionError(f"PDF file not found: {pdf_path}")
        
        if not os.path.exists(output_dir):
            try:
                os.makedirs(output_dir, exist_ok=True)
            except Exception as e:
                raise PDFExtractionError(f"Failed to create output directory: {str(e)}")
        
        # Analyze PDF quality to determine optimal extraction approach
        pdf_quality = self.analyze_pdf_quality(pdf_path)
        self.logger.info(f"Determined PDF quality as: {pdf_quality.value}")
        
        # Update kwargs with quality-appropriate OCR settings
        if "ocr_quality" in kwargs:
            # Use a hybrid approach - take user settings but adapt based on document
            self._adapt_ocr_settings(kwargs, pdf_quality)
        else:
            # Use quality-determined OCR settings
            for key, value in self.ocr_configs[pdf_quality].items():
                if key not in kwargs:
                    kwargs[key] = value
        
        # Execute extraction with fallback strategies
        extraction_errors = []
        for strategy_index, strategy in enumerate(self.fallback_strategies):
            try:
                self.logger.info(f"Attempting extraction with strategy {strategy_index+1}")
                result = strategy(pdf_path, output_dir, extract_images, extract_tables, **kwargs)
                
                # Validate extraction result
                if self._validate_extraction_result(result):
                    # Update extraction statistics
                    self._update_extraction_stats(True, pdf_quality)
                    return result
                else:
                    extraction_errors.append(f"Strategy {strategy_index+1} produced invalid result")
            except Exception as e:
                self.logger.warning(f"Extraction strategy {strategy_index+1} failed: {str(e)}")
                extraction_errors.append(f"Strategy {strategy_index+1} error: {str(e)}")
        
        # If we reached here, all strategies failed
        self._update_extraction_stats(False, pdf_quality)
        
        # Compile error information for better diagnostics
        error_detail = "\n".join(extraction_errors)
        raise PDFExtractionError(
            f"All extraction strategies failed for {pdf_path}",
            details={
                "pdf_path": pdf_path,
                "pdf_quality": pdf_quality.value,
                "strategies_attempted": len(self.fallback_strategies),
                "errors": extraction_errors
            }
        )
    
    def _adapt_ocr_settings(self, kwargs: Dict[str, Any], pdf_quality: PDFQuality) -> None:
        """
        Adapt OCR settings based on PDF quality and user settings
        
        Args:
            kwargs: User-provided extraction options
            pdf_quality: Determined PDF quality
        """
        # Map user OCR quality setting to internal settings
        user_ocr_quality = kwargs.get("ocr_quality", "standard")
        
        # If PDF is SCANNED or PROBLEMATIC, increase OCR quality
        if pdf_quality in [PDFQuality.SCANNED, PDFQuality.PROBLEMATIC]:
            if user_ocr_quality == "low":
                kwargs["ocr_quality"] = "standard"
            elif user_ocr_quality == "standard":
                kwargs["ocr_quality"] = "high"
            
            # Force deskew and denoise for scanned documents unless explicitly disabled
            if "deskew_pages" not in kwargs:
                kwargs["deskew_pages"] = True
            if "denoise_images" not in kwargs:
                kwargs["denoise_images"] = True
    
    def _update_extraction_stats(self, success: bool, pdf_quality: PDFQuality) -> None:
        """
        Update extraction statistics for adaptive processing
        
        Args:
            success: Whether extraction was successful
            pdf_quality: Quality of the processed PDF
        """
        # Update success/failure counts
        if success:
            self.extraction_stats["success_count"] += 1
        else:
            self.extraction_stats["failure_count"] += 1
        
        # Update OCR count
        if pdf_quality in [PDFQuality.SCANNED, PDFQuality.PROBLEMATIC]:
            self.extraction_stats["ocr_count"] += 1
        
        # Update total extractions
        self.extraction_stats["total_extractions"] += 1
    
    def _validate_extraction_result(self, result: Dict[str, Any]) -> bool:
        """
        Validate extraction result to ensure it contains required data
        
        Args:
            result: Extraction result dictionary
            
        Returns:
            True if result is valid, False otherwise
        """
        # Check for required fields
        if not isinstance(result, dict):
            return False
        
        if "content" not in result or not result["content"]:
            return False
        
        if "pages" not in result or not isinstance(result["pages"], list):
            return False
        
        # Check that we have at least one page with content
        if not any(page.get("text") for page in result["pages"]):
            return False
        
        return True
    
    @abstractmethod
    def analyze_pdf_quality(self, pdf_path: str) -> PDFQuality:
        """
        Analyze PDF to determine its quality and complexity
        
        Args:
            pdf_path: Path to the PDF file
            
        Returns:
            PDFQuality enum indicating the document quality
        """
        pass
    
    @abstractmethod
    def is_scanned_document(self, pdf_path: str) -> bool:
        """
        Determine if the PDF is primarily a scanned document
        
        Args:
            pdf_path: Path to the PDF file
            
        Returns:
            True if document is primarily scanned, False otherwise
        """
        pass
    
    @abstractmethod
    def _standard_extraction(self, pdf_path: str, output_dir: str, 
                           extract_images: bool, extract_tables: bool, 
                           **kwargs) -> Dict[str, Any]:
        """
        Primary extraction strategy using PDF parsing
        
        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to store extracted content
            extract_images: Whether to extract images
            extract_tables: Whether to extract tables
            **kwargs: Additional extraction options
            
        Returns:
            Dictionary containing extracted content
        """
        pass
    
    @abstractmethod
    def _ocr_based_extraction(self, pdf_path: str, output_dir: str, 
                            extract_images: bool, extract_tables: bool, 
                            **kwargs) -> Dict[str, Any]:
        """
        OCR-based extraction strategy for scanned documents
        
        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to store extracted content
            extract_images: Whether to extract images
            extract_tables: Whether to extract tables
            **kwargs: Additional extraction options
            
        Returns:
            Dictionary containing extracted content
        """
        pass
    
    @abstractmethod
    def _hybrid_extraction(self, pdf_path: str, output_dir: str, 
                         extract_images: bool, extract_tables: bool, 
                         **kwargs) -> Dict[str, Any]:
        """
        Hybrid extraction strategy using both parsing and OCR
        
        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to store extracted content
            extract_images: Whether to extract images
            extract_tables: Whether to extract tables
            **kwargs: Additional extraction options
            
        Returns:
            Dictionary containing extracted content
        """
        pass
    
    @abstractmethod
    def _minimal_extraction(self, pdf_path: str, output_dir: str, 
                          extract_images: bool, extract_tables: bool, 
                          **kwargs) -> Dict[str, Any]:
        """
        Minimal extraction strategy for problematic PDFs
        
        Args:
            pdf_path: Path to the PDF file
            output_dir: Directory to store extracted content
            extract_images: Whether to extract images
            extract_tables: Whether to extract tables
            **kwargs: Additional extraction options
            
        Returns:
            Dictionary containing extracted content
        """
        pass
        
    def is_math_content(self, text):
        """Check if text contains mathematical expressions"""
        if not self.math_recognition:
            return False
            
        for pattern_name, pattern in self.math_patterns.items():
            if pattern.search(text):
                return True
        return False
        
    def is_chemical_content(self, text):
        """Check if text contains chemical formulas or reactions"""
        if not self.chem_recognition:
            return False
            
        # Count matches for chemical patterns
        matches = 0
        for pattern_name, pattern in self.chem_patterns.items():
            if pattern.search(text):
                matches += 1
        
        # If multiple patterns match, it's likely chemical content
        return matches >= 2
        
    def process_math_content(self, text):
        """Process and normalize mathematical content"""
        # Convert LaTeX math to a structured representation
        try:
            # Extract LaTeX expressions
            math_expressions = []
            for delim_pattern in [r'\$(.*?)\$', r'\\\[(.*?)\\\]', r'\\\((.*?)\\\)']:
                for match in re.finditer(delim_pattern, text, re.DOTALL):
                    expr = match.group(1)
                    if expr:
                        math_expressions.append({
                            'type': 'math_expression',
                            'latex': expr,
                            'position': match.span()
                        })
                        
            return math_expressions
        except Exception as e:
            logger.warning(f"Error processing math content: {e}")
            return text
            
    def process_chemical_content(self, text):
        """Process chemical formulas and reactions"""
        try:
            # Find reaction arrows
            reactions = []
            for match in self.chem_patterns['reaction_arrows'].finditer(text):
                # Look for chemical components before and after the arrow
                reaction_text = text[max(0, match.start() - 20):min(len(text), match.end() + 20)]
                reactions.append({
                    'type': 'chemical_reaction',
                    'text': reaction_text,
                    'position': match.span()
                })
                
            # Find standalone chemical formulas
            formulas = []
            for match in self.chem_patterns['chemical_formulas'].finditer(text):
                # Only consider it a formula if it matches certain patterns
                formula = match.group(0)
                if len(formula) > 1 and re.search(r'[A-Z][a-z]?[0-9]+', formula):
                    formulas.append({
                        'type': 'chemical_formula',
                        'text': formula,
                        'position': match.span()
                    })
                    
            return {'reactions': reactions, 'formulas': formulas}
        except Exception as e:
            logger.warning(f"Error processing chemical content: {e}")
            return text
            
    def extract_images(self, pdf_doc, output_dir=None):
        """Extract images from PDF with position information"""
        images = []

        for page_idx, page in enumerate(pdf_doc):
            image_list = page.get_images(full=True)

            for img_idx, img_info in enumerate(image_list):
                xref = img_info[0]
                base_image = pdf_doc.extract_image(xref)

                if not base_image:
                    continue

                image_bytes = base_image["image"]
                image_ext = base_image["ext"]

                # Get image position on page - Corrected method usage
                position = None
                for img_rect in page.get_image_rects(img_info):  #  Passing the image info as parameter
                    position = {
                        "x0": img_rect.x0,
                        "y0": img_rect.y0,
                        "x1": img_rect.x1,
                        "y1": img_rect.y1
                    }
                    break
                
                # Save image if output directory provided
                image_filename = None
                if output_dir:
                    os.makedirs(output_dir, exist_ok=True)
                    image_filename = f"page{page_idx+1}_img{img_idx +1}.{image_ext}"
                    image_path = os.path.join(output_dir,   image_filename)

                    with open(image_path, "wb") as img_file:
                        img_file.write(image_bytes)

                # Use OCR to extract any text in the image
                image_text = None
                try:
                    pil_image = Image.open(io.BytesIO(image_bytes))
                    image_text = pytesseract.image_to_string    (pil_image)
                    if not image_text.strip():
                        image_text = None
                except Exception as e:
                    logger.warning(f"OCR failed for image: {e}")

                images.append({
                    "page": page_idx + 1,
                    "index": img_idx + 1,
                    "position": position,
                    "file_name": image_filename,
                    "format": image_ext,
                    "ocr_text": image_text
                })

        return images
    
    def extract_tables(self, pdf_path):
        """Extract tables from PDF using Camelot"""
        tables = []
        
        try:
            # Use Camelot to extract tables
            table_pages = camelot.read_pdf(pdf_path, pages='all', flavor='lattice')
            
            for i, table in enumerate(table_pages):
                tables.append({
                    "page": table.page,
                    "index": i + 1,
                    "data": table.df.to_dict('records'),
                    "accuracy": table.accuracy,
                    "whitespace": table.whitespace,
                    "positions": {
                        "top": table._bbox[1],
                        "bottom": table._bbox[3],
                        "left": table._bbox[0],
                        "right": table._bbox[2]
                    }
                })
        except Exception as e:
            logger.warning(f"Table extraction failed: {e}")
            
        return tables
        
    def extract_document_structure(self, pdf_path):
        """Extract document structure using pdfminer to understand layout"""
        structure = []
        
        for page_layout in extract_pages(pdf_path):
            page_struct = {"type": "page", "elements": []}
            
            # Sort elements by their vertical position (y-coordinate)
            elements = sorted(
                [element for element in page_layout],
                key=lambda e: -e.y0
            )
            
            for element in elements:
                if isinstance(element, LTTextBoxHorizontal):
                    text = element.get_text().strip()
                    if not text:
                        continue
                    
                    # Analyze font size to detect headers
                    font_sizes = []
                    for text_line in element:
                        for char in text_line:
                            if hasattr(char, "size"):
                                font_sizes.append(char.size)
                    
                    avg_font_size = sum(font_sizes) / len(font_sizes) if font_sizes else 0
                    
                    # Detect if this is likely a heading
                    is_heading = False
                    if avg_font_size > 12 and len(text) < 100:  # Simplified heuristic
                        is_heading = True
                    
                    page_struct["elements"].append({
                        "type": "heading" if is_heading else "text",
                        "text": text,
                        "position": {
                            "x0": element.x0,
                            "y0": element.y0,
                            "x1": element.x1,
                            "y1": element.y1
                        },
                        "font_size": avg_font_size if font_sizes else None
                    })
                    
                elif isinstance(element, (LTFigure, LTImage)):
                    page_struct["elements"].append({
                        "type": "figure",
                        "position": {
                            "x0": element.x0,
                            "y0": element.y0,
                            "x1": element.x1,
                            "y1": element.y1
                        }
                    })
            
            structure.append(page_struct)
            
        return structure
                    
    @staticmethod
    def preprocess_pdf_for_extraction(pdf_path, output_path=None):
        """Preprocess PDF to improve extraction quality (optional)"""
        if output_path is None:
            output_path = pdf_path.replace(".pdf", "_processed.pdf")
            
        doc = fitz.open(pdf_path)
        for page in doc:
            # Apply OCR to pages with little text
            if len(page.get_text().strip()) < 100:
                pix = page.get_pixmap()
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                text = pytesseract.image_to_string(img)
                
                # If OCR found text, add it to the page
                if text.strip():
                    page.insert_text((50, 50), text)
                    
        doc.save(output_path)
        doc.close()
        return output_path


---


## test_workflow_coordinator.py

In [ ]:
"""
Test suite for WorkflowCoordinator
----------------------------------
Tests the end-to-end workflow orchestration with mocked dependencies.
"""

import os
import unittest
from unittest.mock import MagicMock, patch, ANY
import tempfile
import json
import shutil
from typing import Dict, Any

# Import the workflow coordinator and related components
from pipeline.WorkflowCoordinator import WorkflowCoordinator
from pipeline.config.ComponentConfig import WorkflowConfig
from pipeline.di.DependencyContainer import DependencyContainer
from pipeline.events.EventSystem import get_event_bus, EventType, reset_event_bus
from pipeline.errors.WorkflowErrors import (
    WorkflowError, PDFExtractionError, ContentAnalysisError,
    MultimediaGenerationError, AdaptiveLearningError
)


class TestWorkflowCoordinator(unittest.TestCase):
    """Test cases for WorkflowCoordinator"""

    def setUp(self):
        """Set up test environment before each test case"""
        # Create temporary directory for outputs
        self.temp_dir = tempfile.mkdtemp(prefix="test_workflow_")
        
        # Create test PDF path (doesn't need to be a real file for mock tests)
        self.test_pdf = os.path.join(self.temp_dir, "test_document.pdf")
        
        # Reset event bus
        reset_event_bus()
        
        # Create test config
        self.test_config = {
            "extraction_options": {
                "extract_images": True,
                "extract_tables": True,
                "extract_equations": True,
                "ocr_quality": "high",
                "ocr_language": "eng"
            },
            "analysis_options": {
                "domain_focus": "science",
                "grade_level": "university",
                "detail_level": "comprehensive"
            },
            "multimedia_options": {
                "script_style": {
                    "tone": "professional",
                    "pacing": "moderate"
                },
                "animation_style": {
                    "style": "modern",
                    "transitions": "smooth"
                },
                "voice_options": {
                    "voice_id": "en-US-neural-1",
                    "speed": 1.0
                }
            },
            "adaptive_options": {
                "create_pathway": True,
                "question_count": 10
            },
            "output_dir": self.temp_dir,
            "async_processing": False
        }
        
        # Create mocks for each dependency
        self.mock_pdf_extractor = MagicMock()
        self.mock_content_analyzer = MagicMock()
        self.mock_multimedia_generator = MagicMock()
        self.mock_adaptive_learning = MagicMock()
        self.mock_student_profile = MagicMock()
        
        # Set up return values for the mocks
        self.mock_pdf_extractor.is_scanned_document.return_value = False
        self.mock_pdf_extractor.extract_content.return_value = {
            "content": "Test content",
            "summary": {"pages": 5, "word_count": 2500},
            "pages": [{"number": 1, "text": "Test page 1"}],
            "images": [{"id": "img1", "caption": "Test image"}],
            "tables": [{"id": "tbl1", "caption": "Test table"}]
        }
        
        self.mock_content_analyzer.analyze_content.return_value = {
            "topic": "Test Topic",
            "summary": {"key_points": ["Point 1", "Point 2"]},
            "concepts": [{"name": "Concept 1", "description": "Description 1"}],
            "structure": {"sections": [{"title": "Section 1", "content": "Content 1"}]}
        }
        
        self.mock_multimedia_generator.generate_video.return_value = {
            "video_file": "test_video.mp4",
            "script_file": "test_script.txt",
            "duration": 120,
            "timeline": [{"time": 0, "action": "intro"}, {"time": 60, "action": "conclusion"}]
        }
        
        self.mock_adaptive_learning.create_learning_pathway.return_value = {
            "learning_path": "test_path_id",
            "questions": [{"id": "q1", "text": "Question 1?"}],
            "concepts": ["concept1", "concept2"]
        }
        
        # Create patch for the DependencyContainer
        self.container_patcher = patch('pipeline.di.DependencyContainer.DependencyContainer')
        self.mock_container_class = self.container_patcher.start()
        self.mock_container = MagicMock()
        self.mock_container_class.return_value = self.mock_container
        
        # Configure container to return our mocks
        self.mock_container.get_component_typed.side_effect = self._mock_get_component

    def _mock_get_component(self, component_type):
        """Return appropriate mock based on requested component type"""
        from pipeline.pdf_extraction.PDFContentExtractor import PDFContentExtractor
        from pipeline.content_analysis.ContentAnalysisPipeline import ContentAnalysisPipeline
        from pipeline.multimedia.MultimediaPipeline import MultimediaGenerationPipeline
        from pipeline.adaptive_learning.AdaptiveLearningManager import AdaptiveLearningManager
        from pipeline.adaptive_learning.StudentProfileManager import StudentProfileManager
        
        # Return the appropriate mock based on the component type
        if component_type == PDFContentExtractor:
            return self.mock_pdf_extractor
        elif component_type == ContentAnalysisPipeline:
            return self.mock_content_analyzer
        elif component_type == MultimediaGenerationPipeline:
            return self.mock_multimedia_generator
        elif component_type == AdaptiveLearningManager:
            return self.mock_adaptive_learning
        elif component_type == StudentProfileManager:
            return self.mock_student_profile
        else:
            return MagicMock()
    
    def tearDown(self):
        """Clean up after each test"""
        # Stop patchers
        self.container_patcher.stop()
        
        # Remove temp directory
        shutil.rmtree(self.temp_dir, ignore_errors=True)
        
        # Reset event bus
        reset_event_bus()
    
    def test_init(self):
        """Test coordinator initialization"""
        coord = WorkflowCoordinator(config_dict=self.test_config, output_dir=self.temp_dir)
        self.assertIsNotNone(coord.config)
        self.assertEqual(coord.config.output_dir, self.temp_dir)
        self.assertTrue(os.path.exists(os.path.join(self.temp_dir, "extracted")))
        self.assertTrue(os.path.exists(os.path.join(self.temp_dir, "analysis")))
        self.assertTrue(os.path.exists(os.path.join(self.temp_dir, "multimedia")))
        self.assertTrue(os.path.exists(os.path.join(self.temp_dir, "feedback")))
        self.assertTrue(os.path.exists(os.path.join(self.temp_dir, "metrics")))
    
    def test_process_pdf_synchronous(self):
        """Test synchronous PDF processing workflow"""
        # Create coordinator with test config
        coord = WorkflowCoordinator(config_dict=self.test_config, output_dir=self.temp_dir)
        
        # Process PDF
        result = coord.process_pdf(pdf_path=self.test_pdf)
        
        # Verify the workflow was executed
        self.mock_pdf_extractor.is_scanned_document.assert_called_once_with(self.test_pdf)
        self.mock_pdf_extractor.extract_content.assert_called_once()
        self.mock_content_analyzer.analyze_content.assert_called_once()
        self.mock_multimedia_generator.generate_video.assert_called_once()
        
        # Verify results
        self.assertEqual(result["status"], "completed")
        self.assertIn("workflow_id", result)
        self.assertIn("performance_metrics", result)
        self.assertIn("extraction", result["performance_metrics"])
        self.assertIn("analysis", result["performance_metrics"])
        self.assertIn("video_generation", result["performance_metrics"])
    
    def test_extraction_error_handling(self):
        """Test error handling during extraction phase"""
        # Set up extraction to fail
        self.mock_pdf_extractor.extract_content.side_effect = PDFExtractionError(
            "Test extraction error", workflow_id=None, details={"test": True}
        )
        
        # Create coordinator with test config
        coord = WorkflowCoordinator(config_dict=self.test_config, output_dir=self.temp_dir)
        
        # Process PDF and expect error
        result = coord.process_pdf(pdf_path=self.test_pdf)
        
        # Verify error status
        self.assertEqual(result["status"], "error")
        self.assertIn("workflow_id", result)
        self.assertIn("error_message", result)
        self.assertIn("extraction error", result["error_message"])
        
        # Verify subsequent steps weren't called
        self.mock_content_analyzer.analyze_content.assert_not_called()
        self.mock_multimedia_generator.generate_video.assert_not_called()
    
    def test_custom_options(self):
        """Test processing with custom options"""
        # Create custom options that override some defaults
        custom_options = {
            "extraction_options": {
                "extract_images": False,
                "ocr_quality": "medium"
            },
            "multimedia_options": {
                "script_style": {
                    "tone": "casual"
                }
            }
        }
        
        # Create coordinator with test config
        coord = WorkflowCoordinator(config_dict=self.test_config, output_dir=self.temp_dir)
        
        # Process PDF with custom options
        result = coord.process_pdf(pdf_path=self.test_pdf, custom_options=custom_options)
        
        # Verify extraction was called with updated options
        call_args = self.mock_pdf_extractor.extract_content.call_args[1]
        self.assertFalse(call_args["extract_images"])
        self.assertEqual(call_args["ocr_quality"], "medium")
        self.assertTrue(call_args["extract_tables"])  # Should still have original value
    
    def test_event_publishing(self):
        """Test that events are published during workflow"""
        # Create event listener
        events = []
        
        def event_listener(event_type, payload):
            events.append((event_type, payload))
        
        # Register listener for all event types
        bus = get_event_bus()
        for event_type in EventType:
            bus.subscribe(event_type, event_listener)
        
        # Create coordinator with test config
        coord = WorkflowCoordinator(config_dict=self.test_config, output_dir=self.temp_dir)
        
        # Process PDF
        result = coord.process_pdf(pdf_path=self.test_pdf)
        
        # Verify events were published
        event_types = [e[0] for e in events]
        self.assertIn(EventType.WORKFLOW_STARTED, event_types)
        self.assertIn(EventType.EXTRACTION_STARTED, event_types)
        self.assertIn(EventType.EXTRACTION_COMPLETED, event_types)
        self.assertIn(EventType.ANALYSIS_STARTED, event_types)
        self.assertIn(EventType.ANALYSIS_COMPLETED, event_types)
        self.assertIn(EventType.VIDEO_GENERATION_STARTED, event_types)
        self.assertIn(EventType.VIDEO_GENERATION_COMPLETED, event_types)
        self.assertIn(EventType.WORKFLOW_COMPLETED, event_types)
    
    def test_student_adaptive_integration(self):
        """Test workflow with student ID for adaptive learning"""
        # Create coordinator with test config
        coord = WorkflowCoordinator(config_dict=self.test_config, output_dir=self.temp_dir)
        
        # Process PDF with student ID
        result = coord.process_pdf(
            pdf_path=self.test_pdf,
            student_id="test_student_123"
        )
        
        # Verify adaptive learning was called with student ID
        self.mock_adaptive_learning.create_learning_pathway.assert_called_once()
        call_args = self.mock_adaptive_learning.create_learning_pathway.call_args[1]
        self.assertEqual(call_args["student_id"], "test_student_123")
        
        # Verify results include adaptive learning
        self.assertIn("adaptive_learning", result)
        self.assertEqual(result["adaptive_learning"]["learning_path"], "test_path_id")


if __name__ == "__main__":
    unittest.main() 

---
